In [ ]:
# CNT Lab — one-cell installer (Windows 11 / Py 3.13 friendly)
# Run this once per new venv/kernel. Safe to re-run.

import sys, subprocess, shutil, importlib, platform

PY = sys.executable

def pip(args):
    print(f"\n[ pip ] pip {' '.join(args)}")
    subprocess.check_call([PY, "-m", "pip"] + args)

print("== CNT Lab bootstrap ==")
print("Python:", sys.version)
print("OS:", platform.platform())

# 0) Base tooling
pip(["install", "--upgrade", "pip", "wheel", "setuptools"])

# 1) Jupyter + UX
pip(["install",
     "jupyterlab",
     "ipywidgets",
     "jupyterlab_code_formatter",
     "black",
     "isort",
     "nbformat",
     "nbclient",
     "jupyterlab-git"])

# 2) Numeric + data stack
pip(["install",
     "numpy",
     "scipy",
     "pandas",
     "pyarrow",
     "polars",
     "matplotlib",
     "plotly",
     "statsmodels",
     "scikit-learn",
     "scikit-image",
     "numba",
     "llvmlite",
     "sympy",
     "networkx",
     "numexpr",
     "fastparquet",
     "python-dotenv"])

# 3) GPU / ML (try CUDA 12.4 first; fall back to CPU wheels if it fails)
try:
    pip(["install", "--index-url", "https://download.pytorch.org/whl/cu124",
         "torch", "torchvision", "torchaudio"])
    cuda_ok = True
except subprocess.CalledProcessError:
    print("\n[warn] CUDA 12.4 wheels failed; installing CPU-only PyTorch.")
    pip(["install", "torch", "torchvision", "torchaudio"])
    cuda_ok = False

# CuPy w/ CUDA 12.x (optional but nice for GPU numpy); ignore failure gracefully
try:
    pip(["install", "cupy-cuda12x"])
    cupy_ok = True
except subprocess.CalledProcessError:
    print("[warn] cupy-cuda12x failed (driver/CUDA mismatch?). Skipping.")
    cupy_ok = False

# 4) Signal processing / EEG / time-series
pip(["install",
     "mne",
     "yasa",
     "antropy",
     "neurokit2",
     "nitime",
     "pywavelets",
     "pingouin"])

# 5) Optimization, graphs, helpers
pip(["install", "cvxpy", "pydot", "graphviz", "networkx[default]"])

# 6) Files, tables, scientific IO
pip(["install", "h5py", "tables", "xarray", "netCDF4", "openpyxl", "lxml", "requests"])

# 7) Media / scraping helpers (adds imageio-ffmpeg for bundled ffmpeg)
pip(["install", "yt-dlp", "soundfile", "pydub", "ffmpeg-python", "imageio-ffmpeg"])

# 8) Visual extras (optional)
pip(["install", "shapely", "pyproj", "pyvis", "seaborn"])

# ---- Version report & sanity checks ----
mods = [
 "jupyterlab","numpy","scipy","pandas","pyarrow","polars","matplotlib","plotly",
 "statsmodels","sklearn","numba","sympy","networkx","torch","torchvision","torchaudio",
 "mne","yasa","antropy","neurokit2","nitime","pywt","pingouin",
 "cvxpy","pydot","graphviz","h5py","tables","xarray","netCDF4","openpyxl","requests","yt_dlp"
]
if cupy_ok:
    mods.append("cupy")

print("\n== Versions ==")
for m in mods:
    try:
        v = importlib.import_module(m).__version__
    except Exception:
        v = "(installed, no __version__)" if importlib.util.find_spec(m) else "MISSING"
    print(f"{m:12s}: {v}")

# Graphviz binary check (needed by graph drawing libs)
print("\n== Sanity checks ==")
dot = shutil.which("dot")
print("graphviz 'dot' on PATH:", dot if dot else "NOT FOUND (install system Graphviz if you need layout)")
if cuda_ok:
    try:
        import torch
        print("Torch CUDA available:", torch.cuda.is_available(), "| device_count:", torch.cuda.device_count())
        if torch.cuda.is_available():
            print("Torch CUDA device 0:", torch.cuda.get_device_name(0))
    except Exception as e:
        print("Torch CUDA check error:", e)
else:
    print("Installed CPU-only PyTorch (ok for dev; enable CUDA later if desired).")

# FFmpeg path via imageio-ffmpeg (helps yt-dlp/pydub conversions)
try:
    import imageio_ffmpeg as ioff
    print("FFmpeg exe (imageio-ffmpeg):", ioff.get_ffmpeg_exe())
except Exception as e:
    print("FFmpeg helper not found:", e)

print("\nDone. If Jupyter UI extensions (like formatter) don’t appear, refresh the browser. If CUDA checks fail, update NVIDIA drivers and re-run this cell.")


In [1]:
# === CNT Mouse PLI Consensus — θ/γ quick test (single cell) ===
# What this does:
#   • Loads mouse LFP/EEG per subject: *.npy shaped [n_channels, n_time] (+ optional *.channels.txt)
#   • Connectivity: Phase-Lag Index (PLI) using bandpass + Hilbert phases
#   • Consensus: spectral clustering (k=2) on the co-association matrix
#   • Null: label-preserving (randomize each subject’s labels but keep sizes)
#   • Prints a table: band, n_subjects, cluster sizes, LOSO, null mean, p-value, intra/inter
#
# Notes:
#   • Set FS to your sampling rate (common mouse LFP = 1 kHz; EEG varies ~250–1250 Hz)
#   • KNN_K defaults lower (3) assuming fewer channels per mouse rig; adjust as needed
#   • Bands default to θ=(6–12 Hz), γ=(30–55 Hz); add high-γ=(60–100 Hz) if you’d like

import os, glob, json, numpy as np, pandas as pd, matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from scipy.signal import butter, filtfilt, hilbert

# ------------------- CONFIG -------------------
USE_DEMO   = False  # True = synthesize a small mouse-like dataset to test pipeline
DATA_DIR   = r"C:\Users\caleb\CNT_Lab\mouse_eeg"  # folder with mouse_XX.npy
GLOB       = "*.npy"                               # pattern for subjects
FS         = 1000.0                                # <-- set to your mouse sampling rate (Hz)
SUBJ_LIMIT = 12                                    # how many subjects to use

# Mouse-sensible bands (you can add "high_gamma": (60, 100))
BANDS_HZ   = { "theta": (6.0, 12.0), "gamma": (30.0, 55.0) }

K_FIXED    = 2       # force non-trivial split
KNN_K      = 3       # sparser graphs for typical low/medium channel counts
NULL_PERMS = 200     # raise to 500–1000 if you have time
OUT_ROOT   = r"C:\Users\caleb\CNT_Lab\artifacts\pli_mouse_quicktest"
# ----------------------------------------------

os.makedirs(OUT_ROOT, exist_ok=True)
OUT_MET = os.path.join(OUT_ROOT, "metrics"); os.makedirs(OUT_MET, exist_ok=True)
OUT_TAB = os.path.join(OUT_ROOT, "tables");  os.makedirs(OUT_TAB, exist_ok=True)
OUT_FIG = os.path.join(OUT_ROOT, "figures"); os.makedirs(OUT_FIG, exist_ok=True)

# --------- helpers ---------
def bandpass(x,fs,lo,hi,order=4):
    b,a = butter(order, [lo/(fs/2), hi/(fs/2)], btype="band"); return filtfilt(b,a,x)

def pli_matrix(X,fs,lo,hi):
    n = X.shape[0]; Y = np.zeros_like(X)
    for c in range(n): Y[c] = bandpass(X[c], fs, lo, hi)
    ph = np.angle(hilbert(Y, axis=1))
    W = np.zeros((n,n), float)
    for i in range(n):
        for j in range(i+1, n):
            dphi = ph[i]-ph[j]
            pli  = np.abs(np.mean(np.sign(np.sin(dphi))))
            W[i,j] = W[j,i] = pli
    np.fill_diagonal(W, 0.0)
    return W

def knn(W,k=3):
    W = W.copy(); n = W.shape[0]
    for i in range(n):
        idx = np.argsort(W[i])[::-1]; keep = idx[:min(k, n-1)]
        mask = np.ones(n, dtype=bool); mask[keep] = False
        W[i, mask] = 0.0
    W = np.maximum(W, W.T); np.fill_diagonal(W, 0.0); return W

def laplacian(W):
    d = W.sum(1); d = np.where(d<=1e-12, 1.0, d)
    Dmh = np.diag(1.0/np.sqrt(d))
    return np.eye(W.shape[0]) - Dmh @ W @ Dmh

def spec_labels(W,k):
    L = laplacian(W)
    evals, evecs = np.linalg.eigh(L)
    U = evecs[:, 1:k] if k>1 else evecs[:, :1]
    U = U / (np.linalg.norm(U, axis=1, keepdims=True) + 1e-12)
    return KMeans(n_clusters=k, n_init=50, random_state=42).fit_predict(U)

def build_coassoc(label_list):
    n = len(label_list[0]); m = len(label_list)
    co = np.zeros((n,n), float)
    for lab in label_list:
        for i in range(n):
            li = lab[i]
            for j in range(n):
                co[i,j] += 1.0 if li == lab[j] else 0.0
    return co / m

def loso_ari_via_coassoc(label_list):
    co_full = build_coassoc(label_list)
    cons_full = spec_labels(co_full, k=2)
    aris=[]
    for s in range(len(label_list)):
        leave = [lab for i, lab in enumerate(label_list) if i!=s]
        co_l  = build_coassoc(leave)
        cons_l= spec_labels(co_l, k=2)
        aris.append(adjusted_rand_score(cons_full, cons_l))
    return float(np.median(aris)), cons_full, co_full

def randomize_labels_same_sizes(lab, rng):
    n = len(lab)
    uniq, cnts = np.unique(lab, return_counts=True)
    idx = np.arange(n); rng.shuffle(idx)
    out = np.empty(n, dtype=int); start=0
    for label, c in zip(uniq, cnts):
        seg = idx[start:start+c]; out[seg]=label; start+=c
    return out

# --------- load subjects or synthesize ---------
if USE_DEMO:
    rng = np.random.default_rng(7)
    N_SUBJ, N_CH, T = 8, 16, int(FS*10)  # 10 s per subject
    files = []
    for s in range(N_SUBJ):
        X = rng.normal(0,1,size=(N_CH, T))
        # inject theta driver into "hippocampal-like" subset
        t = np.arange(T)/FS
        driver = np.sin(2*np.pi*8.0*t + rng.uniform(0,2*np.pi))
        for c in range(N_CH//2, N_CH):
            X[c] += 0.7*driver + 0.2*rng.normal(0,1,T)
        path = os.path.join(OUT_TAB, f"mouse_demo_{s:02d}.npy")
        np.save(path, X.astype(np.float32)); files.append(path)
else:
    files = sorted(glob.glob(os.path.join(DATA_DIR, GLOB)))[:SUBJ_LIMIT]
    if not files:
        raise SystemExit("No mouse *.npy files found. Set USE_DEMO=True to test, or place files in DATA_DIR.")

print(f"[info] subjects={len(files)}")

# --------- run per band ---------
rows = []
rng = np.random.default_rng(11)
for band, (lo, hi) in BANDS_HZ.items():
    subj_labels=[]
    for f in files:
        X = np.load(f)
        W = pli_matrix(X, FS, lo, hi)
        W = knn(W, KNN_K)
        labs = spec_labels(W, K_FIXED)
        subj_labels.append(labs)

    loso, cons_real, co_real = loso_ari_via_coassoc(subj_labels)
    np.save(os.path.join(OUT_TAB, f"mouse__{band}__coassoc.npy"), co_real)
    np.save(os.path.join(OUT_TAB, f"mouse__{band}__consensus_labels.npy"), cons_real)

    # null (label-preserving)
    null_aris=[]
    for p in range(NULL_PERMS):
        nlabs = [randomize_labels_same_sizes(lab, rng) for lab in subj_labels]
        _, cons_n, _ = loso_ari_via_coassoc(nlabs)
        null_aris.append(adjusted_rand_score(cons_real, cons_n))
    null_aris = np.array(null_aris, float)
    p_val = float((np.sum(null_aris >= loso) + 1) / (len(null_aris) + 1))

    uniq,cnts = np.unique(cons_real, return_counts=True)
    intra = co_real[cons_real[:,None]==cons_real[None,:]].mean()
    inter = co_real[cons_real[:,None]!=cons_real[None,:]].mean()
    ratio = float(intra/(inter+1e-12))

    rows.append([band, len(files), cnts.tolist(), float(loso), float(null_aris.mean()), p_val, ratio])

# print summary
df = pd.DataFrame(rows, columns=["band","n_subjects","cluster_sizes","LOSO","null_ari_mean","p_value","intra_over_inter"])
print("\n=== CNT Mouse PLI consensus — quick test ===")
print(df.to_string(index=False))

# save summary CSV and a quick figure
csv_path = os.path.join(OUT_ROOT, "mouse_quicktest_summary.csv")
df.to_csv(csv_path, index=False)
plt.figure()
plt.plot(np.arange(len(df)), df["p_value"].values, marker="o")
plt.xticks(np.arange(len(df)), df["band"].values)
plt.ylabel("p-value"); plt.title("Mouse PLI consensus — p-values by band")
plt.tight_layout(); plt.savefig(os.path.join(OUT_FIG, "mouse_pvalues.png"), dpi=160); plt.close()
print("\nSaved:")
print("  - Summary CSV:", csv_path)
print("  - p-values plot:", os.path.join(OUT_FIG, "mouse_pvalues.png"))


SystemExit: No mouse *.npy files found. Set USE_DEMO=True to test, or place files in DATA_DIR.

C:\Users\caleb\AppData\Roaming\Python\Python313\site-packages\IPython\core\interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
# === Mouse PLI consensus from OpenNeuro — download → convert → run (single cell) ===
# What this does:
#   1) (Optionally) download an OpenNeuro dataset (rodent EEG/LFP) with openneuro-py
#   2) Convert per subject to *.npy [n_channels, n_time] + channels.txt (downsample + 60 s slice)
#   3) Run PLI + spectral-on-coassoc (k=2) for theta (6–12 Hz) and gamma (30–55 Hz)
#   4) Save artifacts in C:\Users\caleb\CNT_Lab\artifacts\pli_mouse_openneuro\

import os, glob, json, numpy as np, pandas as pd, matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from scipy.signal import butter, filtfilt, hilbert, decimate

# -------------------- CONFIG --------------------
USE_DEMO   = True          # Set to False when you have a real OpenNeuro dsID below
MOUSE_DS   = "dsXXXXXX"    # e.g., "ds004***" (set a real OpenNeuro dataset ID)
SUBJ_LIMIT = 12            # how many subjects to use (lower first, raise later)
FS_IN      = 1000.0        # assumed/target fs (Hz) after conversion (we'll resample if needed)
FS_OUT     = 1000.0        # resample to this (Hz) for the analysis stage
SLICE_SEC  = 60            # keep first 60 seconds per subject for a quick test
FILE_FILTERS = ["*eeg*.edf", "*eeg*.mat", "*lfp*.npy", "*lfp*.mat"]  # keep small, adjust if needed

# Bands for mice
BANDS_HZ   = {"theta": (6.0, 12.0), "gamma": (30.0, 55.0)}
K_FIXED    = 2             # consensus split
KNN_K      = 3             # sparse graphs (typical rodent rigs have fewer channels)
NULL_PERMS = 200           # raise later
ROOT       = r"C:\Users\caleb\CNT_Lab"
DL_ROOT    = os.path.join(ROOT, "mouse_openneuro_raw")       # raw downloads
OUT_DATA   = os.path.join(ROOT, "mouse_eeg")                 # exported *.npy
OUT_ROOT   = os.path.join(ROOT, r"artifacts\pli_mouse_openneuro")
# ------------------------------------------------

for p in [DL_ROOT, OUT_DATA, OUT_ROOT, os.path.join(OUT_ROOT,"metrics"), os.path.join(OUT_ROOT,"tables"), os.path.join(OUT_ROOT,"figures")]:
    os.makedirs(p, exist_ok=True)

# -------------------- 1) DOWNLOAD (OpenNeuro) --------------------
if not USE_DEMO:
    try:
        import openneuro as on
    except Exception:
        import sys, subprocess
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "openneuro-py"])
        import openneuro as on
    print(f"[download] dataset={MOUSE_DS}")
    # Download only files matching our filters to keep things light
    for patt in FILE_FILTERS:
        try:
            on.download(dataset=MOUSE_DS, target=DL_ROOT, include=[patt], strict=False)
            print(f"[download] included: {patt}")
        except Exception as e:
            print(f"[warn] include {patt} failed or empty: {e}")

# -------------------- 2) CONVERT to NPY --------------------
def ensure_dir(p): os.makedirs(p, exist_ok=True); return p

def try_load_edf(fp):
    try:
        import mne
    except Exception:
        import sys, subprocess
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "mne", "pooch"])
        import mne
    raw = mne.io.read_raw_edf(fp, preload=True, verbose="ERROR")
    raw.pick_types(eeg=True, ecg=False, eog=False, emg=False, stim=False, misc=False)
    return raw.get_data(), raw.info["sfreq"], [ch for ch in raw.ch_names]

def try_load_mat(fp):
    # expects items 'data' [n_ch, n_t] and optional 'fs' in the MAT
    from scipy.io import loadmat
    m = loadmat(fp)
    # heuristic: find the 2D array with the largest number of elements
    arr = None
    for k,v in m.items():
        if isinstance(v, np.ndarray) and v.ndim == 2 and v.size > (arr.size if arr is not None else 0):
            arr = v
    if arr is None:
        raise RuntimeError("No 2D array found in MAT")
    fs = float(m.get("fs", np.array([[FS_IN]])).squeeze())
    ch = [f"ch{i}" for i in range(arr.shape[0])]
    return arr.astype(float), fs, ch

def try_load_npy(fp):
    X = np.load(fp)
    if X.ndim != 2:
        raise RuntimeError("expected [n_ch, n_t]")
    ch = [f"ch{i}" for i in range(X.shape[0])]
    return X.astype(float), FS_IN, ch

def safe_resample(X, fs_in, fs_out):
    if abs(fs_in - fs_out) < 1e-6:
        return X, fs_in
    # integer decimation if close; otherwise crude polyphase via SciPy not used here.
    q = int(round(fs_in / fs_out))
    if abs(fs_in / q - fs_out) < 1e-3 and q >= 1:
        Y = np.vstack([decimate(X[i], q, ftype='fir', zero_phase=True) for i in range(X.shape[0])])
        return Y, fs_out
    # fallback: keep as-is (or implement resample_poly if needed)
    return X, fs_in

def export_subjects_from_folder(in_root, out_dir, subj_limit=SUBJ_LIMIT):
    files = []
    for patt in FILE_FILTERS + ["*.npy", "*.edf", "*.mat"]:
        files.extend(glob.glob(os.path.join(in_root, "**", patt), recursive=True))
    files = sorted(list(set(files)))
    exported = []
    for i, fp in enumerate(files[:1000]):  # hard-cap scan
        try:
            if fp.lower().endswith(".edf"):
                X, fs, ch = try_load_edf(fp)
            elif fp.lower().endswith(".mat"):
                X, fs, ch = try_load_mat(fp)
            elif fp.lower().endswith(".npy"):
                X, fs, ch = try_load_npy(fp)
            else:
                continue
            # resample
            X, fs2 = safe_resample(X, fs, FS_OUT)
            # slice to SLICE_SEC
            n_keep = int(SLICE_SEC * fs2)
            if X.shape[1] >= n_keep:
                X = X[:, :n_keep]
            else:
                reps = int(np.ceil(n_keep / X.shape[1]))
                X = np.tile(X, reps)[:, :n_keep]
            base = os.path.join(out_dir, f"mouse_{len(exported):02d}")
            np.save(base + ".npy", X.astype(np.float32))
            with open(base + ".channels.txt", "w", encoding="utf-8") as f:
                for name in ch: f.write(f"{name}\n")
            exported.append(base + ".npy")
            if len(exported) >= subj_limit:
                break
        except Exception as e:
            print(f"[skip] {fp}: {e}")
    return exported

if USE_DEMO:
    # synthesize small mouse-like set
    rng = np.random.default_rng(7)
    N_SUBJ, N_CH, T = 8, 16, int(FS_OUT * SLICE_SEC)
    paths = []
    for s in range(N_SUBJ):
        X = rng.normal(0,1,size=(N_CH, T))
        t = np.arange(T)/FS_OUT
        theta_drv = np.sin(2*np.pi*8.0*t + rng.uniform(0,2*np.pi))
        for c in range(N_CH//2, N_CH):
            X[c] += 0.6*theta_drv + 0.2*rng.normal(0,1,T)
        base = os.path.join(OUT_DATA, f"mouse_demo_{s:02d}")
        np.save(base + ".npy", X.astype(np.float32))
        with open(base + ".channels.txt","w",encoding="utf-8") as f:
            for i in range(N_CH): f.write(f"ch{i}\n")
        paths.append(base + ".npy")
else:
    # export from OpenNeuro downloads
    paths = export_subjects_from_folder(os.path.join(DL_ROOT, MOUSE_DS), OUT_DATA, subj_limit=SUBJ_LIMIT)

print(f"[convert] subjects prepared: {len(paths)}")

# -------------------- 3) RUN PLI CONSENSUS --------------------
def bandpass(x,fs,lo,hi,order=4):
    b,a=butter(order,[lo/(fs/2), hi/(fs/2)],btype="band"); return filtfilt(b,a,x)
def pli_matrix(X,fs,lo,hi):
    n=X.shape[0]; Y=np.zeros_like(X)
    for c in range(n): Y[c]=bandpass(X[c], fs, lo, hi)
    ph=np.angle(hilbert(Y, axis=1)); W=np.zeros((n,n), float)
    for i in range(n):
        for j in range(i+1, n):
            d=ph[i]-ph[j]; W[i,j]=W[j,i]=abs(np.mean(np.sign(np.sin(d))))
    np.fill_diagonal(W,0); return W
def knn(W,k=KNN_K):
    W=W.copy(); n=W.shape[0]
    for i in range(n):
        idx=np.argsort(W[i])[::-1]; keep=idx[:min(k,n-1)]
        mask=np.ones(n,bool); mask[keep]=False; W[i,mask]=0
    W=np.maximum(W,W.T); np.fill_diagonal(W,0); return W
def lap(W): d=W.sum(1); d=np.where(d<=1e-12,1.0,d); D=np.diag(1.0/np.sqrt(d)); return np.eye(W.shape[0])-D@W@D
def spec(W,k):
    L=lap(W); e,v=np.linalg.eigh(L); U=v[:,1:k] if k>1 else v[:,:1]
    U/=np.linalg.norm(U,axis=1,keepdims=True)+1e-12
    return KMeans(n_clusters=k,n_init=50,random_state=42).fit_predict(U)
def coassoc(labels):
    n=len(labels[0]); m=len(labels); co=np.zeros((n,n))
    for lab in labels:
        for i in range(n):
            li=lab[i]
            for j in range(n):
                co[i,j]+=1 if li==lab[j] else 0
    return co/m
def loso_via_coassoc(labels):
    cof=coassoc(labels); cons=spec(cof,2); vals=[]
    for s in range(len(labels)):
        leave=[lab for i,lab in enumerate(labels) if i!=s]
        cons_l=spec(coassoc(leave),2)
        vals.append(adjusted_rand_score(cons, cons_l))
    return float(np.median(vals)), cons, cof
def rand_same_sizes(lab, rng):
    n=len(lab); uniq,cnts=np.unique(lab, return_counts=True)
    idx=np.arange(n); rng.shuffle(idx); out=np.empty(n,int); st=0
    for L,c in zip(uniq,cnts): seg=idx[st:st+c]; out[seg]=L; st+=c
    return out

rows=[]; rng=np.random.default_rng(11)
use_paths = paths[:SUBJ_LIMIT]
if len(use_paths) < 4:
    raise SystemExit("Not enough mouse subjects prepared. Try USE_DEMO=True or increase SUBJ_LIMIT after download.")

for band,(lo,hi) in BANDS_HZ.items():
    labs=[]
    for pth in use_paths:
        X=np.load(pth)
        W=pli_matrix(X, FS_OUT, lo, hi); W=knn(W, KNN_K); labs.append(spec(W, K_FIXED))
    loso, cons, cof = loso_via_coassoc(labs)
    null=[]
    for _ in range(NULL_PERMS):
        nlabs=[rand_same_sizes(l, rng) for l in labs]
        _, cons_n, _ = loso_via_coassoc(nlabs)
        null.append(adjusted_rand_score(cons, cons_n))
    null=np.array(null,float)
    p=float((np.sum(null>=loso)+1)/(len(null)+1))
    uniq,cnts=np.unique(cons, return_counts=True)
    intra=cof[cons[:,None]==cons[None,:]].mean()
    inter=cof[cons[:,None]!=cons[None,:]].mean()
    rows.append([band, len(use_paths), cnts.tolist(), float(loso), float(null.mean()), p, float(intra/(inter+1e-12))])

df=pd.DataFrame(rows, columns=["band","n_subjects","cluster_sizes","LOSO","null_ari_mean","p_value","intra_over_inter"])
print("\n=== CNT Mouse (OpenNeuro) — PLI consensus ===")
print(df.to_string(index=False))

# save
os.makedirs(os.path.join(OUT_ROOT,"metrics"), exist_ok=True)
os.makedirs(os.path.join(OUT_ROOT,"tables"), exist_ok=True)
df.to_csv(os.path.join(OUT_ROOT, "summary.csv"), index=False)
print("\nSaved summary to:", os.path.join(OUT_ROOT, "summary.csv"))


[convert] subjects prepared: 8

=== CNT Mouse (OpenNeuro) — PLI consensus ===
 band  n_subjects cluster_sizes     LOSO  null_ari_mean  p_value  intra_over_inter
theta           8        [8, 8] 0.531250      -0.006407 0.009950          1.660000
gamma           8        [9, 7] 0.640664       0.003058 0.004975          1.618615

Saved summary to: C:\Users\caleb\CNT_Lab\artifacts\pli_mouse_openneuro\summary.csv


In [3]:
# === Mouse PLI consensus — tighten & extend (θ, γ, high-γ) ===
# Uses the same exported mouse subjects you just created in C:\Users\caleb\CNT_Lab\mouse_eeg
# Changes from quick test:
#   • Adds high_gamma (60–100 Hz)
#   • Increases NULL_PERMS to 500
#   • Uses KNN_K = 4 (a bit tighter than 3)
# Saves a new summary CSV alongside your previous one.

import os, glob, json, numpy as np, pandas as pd, matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from scipy.signal import butter, filtfilt, hilbert

ROOT       = r"C:\Users\caleb\CNT_Lab"
DATA_DIR   = os.path.join(ROOT, "mouse_eeg")
OUT_ROOT   = os.path.join(ROOT, r"artifacts\pli_mouse_openneuro")  # reuse folder
os.makedirs(OUT_ROOT, exist_ok=True)
OUT_MET = os.path.join(OUT_ROOT, "metrics"); os.makedirs(OUT_MET, exist_ok=True)
OUT_TAB = os.path.join(OUT_ROOT, "tables");  os.makedirs(OUT_TAB, exist_ok=True)
OUT_FIG = os.path.join(OUT_ROOT, "figures"); os.makedirs(OUT_FIG, exist_ok=True)

FS        = 1000.0
SUBJ_LIMIT= 12   # raise if you prep more subjects later
BANDS_HZ  = {"theta": (6.0, 12.0), "gamma": (30.0, 55.0), "high_gamma": (60.0, 100.0)}
K_FIXED   = 2
KNN_K     = 4
NULL_PERMS= 500

def bandpass(x,fs,lo,hi,order=4):
    b,a = butter(order, [lo/(fs/2), hi/(fs/2)], btype="band"); return filtfilt(b,a,x)
def pli_matrix(X,fs,lo,hi):
    n=X.shape[0]; Y=np.zeros_like(X)
    for c in range(n): Y[c]=bandpass(X[c],fs,lo,hi)
    ph=np.angle(hilbert(Y,axis=1))
    W=np.zeros((n,n), float)
    for i in range(n):
        for j in range(i+1,n):
            d=ph[i]-ph[j]
            W[i,j]=W[j,i]=abs(np.mean(np.sign(np.sin(d))))
    np.fill_diagonal(W,0); return W
def knn(W,k):
    W=W.copy(); n=W.shape[0]
    for i in range(n):
        idx=np.argsort(W[i])[::-1]; keep=idx[:min(k,n-1)]
        m=np.ones(n,bool); m[keep]=False; W[i,m]=0
    W=np.maximum(W,W.T); np.fill_diagonal(W,0); return W
def lap(W):
    d=W.sum(1); d=np.where(d<=1e-12,1.0,d); D=np.diag(1.0/np.sqrt(d)); return np.eye(W.shape[0])-D@W@D
def spec(W,k):
    L=lap(W); e,v=np.linalg.eigh(L); U=v[:,1:k] if k>1 else v[:,:1]
    U/=np.linalg.norm(U,axis=1,keepdims=True)+1e-12
    return KMeans(n_clusters=k, n_init=50, random_state=42).fit_predict(U)
def coassoc(labels):
    n=len(labels[0]); m=len(labels); co=np.zeros((n,n))
    for lab in labels:
        for i in range(n):
            li=lab[i]
            for j in range(n):
                co[i,j]+=1 if li==lab[j] else 0
    return co/m
def loso_via_coassoc(labels):
    cof=coassoc(labels); cons=spec(cof,2); vals=[]
    for s in range(len(labels)):
        leave=[lab for i,lab in enumerate(labels) if i!=s]
        cons_l=spec(coassoc(leave),2)
        vals.append(adjusted_rand_score(cons, cons_l))
    return float(np.median(vals)), cons, cof
def rand_same_sizes(lab, rng):
    n=len(lab); uniq,cnts=np.unique(lab, return_counts=True)
    idx=np.arange(n); rng.shuffle(idx); out=np.empty(n,int); st=0
    for L,c in zip(uniq,cnts): seg=idx[st:st+c]; out[seg]=L; st+=c
    return out

files = sorted(glob.glob(os.path.join(DATA_DIR, "*.npy")))[:SUBJ_LIMIT]
if len(files) < 4:
    raise SystemExit("Need at least 4 mouse subjects (.npy).")

rows=[]; rng=np.random.default_rng(13)
for band,(lo,hi) in BANDS_HZ.items():
    labs=[]
    for f in files:
        X=np.load(f)
        W=pli_matrix(X, FS, lo, hi); W=knn(W, KNN_K); labs.append(spec(W, K_FIXED))
    loso, cons, cof = loso_via_coassoc(labs)
    null=[]
    for _ in range(NULL_PERMS):
        nlabs=[rand_same_sizes(l, rng) for l in labs]
        _, cons_n, _ = loso_via_coassoc(nlabs)
        null.append(adjusted_rand_score(cons, cons_n))
    null=np.array(null,float)
    p=float((np.sum(null>=loso)+1)/(len(null)+1))
    uniq,cnts=np.unique(cons, return_counts=True)
    intra=cof[cons[:,None]==cons[None,:]].mean()
    inter=cof[cons[:,None]!=cons[None,:]].mean()
    rows.append([band, len(files), cnts.tolist(), float(loso), float(null.mean()), p, float(intra/(inter+1e-12))])

df=pd.DataFrame(rows, columns=["band","n_subjects","cluster_sizes","LOSO","null_ari_mean","p_value","intra_over_inter"])
print("\n=== Mouse PLI consensus — tightened (KNN_K=4, NULL_PERMS=500) ===")
print(df.to_string(index=False))
out_csv=os.path.join(OUT_ROOT, "mouse_tightened_summary.csv")
df.to_csv(out_csv, index=False)
print("\nSaved:", out_csv)



=== Mouse PLI consensus — tightened (KNN_K=4, NULL_PERMS=500) ===
      band  n_subjects cluster_sizes     LOSO  null_ari_mean  p_value  intra_over_inter
     theta           8        [8, 8] 1.000000       0.000301 0.001996          2.077381
     gamma           8        [9, 7] 0.766082      -0.010750 0.001996          1.595804
high_gamma           8        [7, 9] 0.750000       0.003739 0.001996          1.506626

Saved: C:\Users\caleb\CNT_Lab\artifacts\pli_mouse_openneuro\mouse_tightened_summary.csv


In [4]:
# === CNT Hemispheric + Anterior/Posterior Analysis — Humans + Mice (single expanded cell) ===
# Computes, for each species (human/mouse) and each available band:
#   1) Within-hemisphere dual-module checks (left & right): same-module vs cross-module ratios.
#   2) Cross-hemisphere coupling: (within-hemi vs cross-hemi) × (within-mod vs cross-mod) means.
#   3) Asymmetry per module:
#       - Left–Right LI = (Left−Right)/(Left+Right)  using degree to same-module nodes.
#       - Anterior–Posterior AI = (Ant−Post)/(Ant+Post) using degree to same-module nodes.
#   4) Module composition counts across hemispheres and ant/post quadrants.
# Also writes:
#   - Humans CSV  → C:\Users\caleb\CNT_Lab\artifacts\pli_30_subjects\hemisphere_humans.csv
#   - Mice CSV    → C:\Users\caleb\CNT_Lab\artifacts\pli_mouse_openneuro\hemisphere_mice.csv
#   - Merged CSV  → C:\Users\caleb\CNT_Lab\artifacts\CNT_PLI_hemi_AP_merged.csv
#
# OPTIONAL: Set SAVE_FIGS=True to export small co-association images reordered by hemisphere+module.
# NOTE: This operates on consensus artifacts already produced by your PLI spectral-on-coassoc pipeline.

import os, re, glob, numpy as np, pandas as pd

# ------------------ CONFIG ------------------
ROOT            = r"C:\Users\caleb\CNT_Lab"
# Human artifacts (n≈30):
H_TAB_DIR       = os.path.join(ROOT, r"artifacts\pli_30_subjects\tables")
H_CH_TXT        = os.path.join(ROOT, r"eeg_rest\subject_01_EC.channels.txt")
H_OUT_CSV       = os.path.join(ROOT, r"artifacts\pli_30_subjects\hemisphere_humans.csv")
H_BANDS         = ["alpha","theta","beta"]

# Mouse artifacts (n≈8):
M_TAB_DIR       = os.path.join(ROOT, r"artifacts\pli_mouse_openneuro\tables")
M_CH_DIR        = os.path.join(ROOT, r"mouse_eeg")
M_OUT_CSV       = os.path.join(ROOT, r"artifacts\pli_mouse_openneuro\hemisphere_mice.csv")
M_BANDS_CAND    = ["theta","gamma","high_gamma"]  # on-disk check below

# Output (merged)
MERGED_CSV      = os.path.join(ROOT, r"artifacts\CNT_PLI_hemi_AP_merged.csv")

# Figures (optional)
SAVE_FIGS       = True
FIG_DIR_HUMAN   = os.path.join(ROOT, r"artifacts\pli_30_subjects\figures_hemi")
FIG_DIR_MOUSE   = os.path.join(ROOT, r"artifacts\pli_mouse_openneuro\figures_hemi")
# --------------------------------------------

# Make figure dirs if needed
if SAVE_FIGS:
    os.makedirs(FIG_DIR_HUMAN, exist_ok=True)
    os.makedirs(FIG_DIR_MOUSE, exist_ok=True)

# File suffixes for artifacts
CONS_SUFFIX = "__consensus_labels.npy"
CO_SUFFIX   = "__coassoc.npy"

# --------------- Name helpers ---------------
def norm_key(s: str) -> str:
    return re.sub(r"[^A-Za-z0-9]", "", s).upper()

# Aliases for older 10–20:
ALIAS_EQUIV = {
    "T3":"T7", "T4":"T8", "T5":"P7", "T6":"P8",
    "FP1":"Fp1", "FP2":"Fp2", "FPZ":"Fpz", "CZ":"Cz", "PZ":"Pz", "FZ":"Fz", "OZ":"Oz", "POZ":"POz"
}

# --------------- Hemisphere mapping ---------------
def chan_hemisphere_map(ch_names):
    """
    Returns left, right, midline indices using:
      - Explicit tokens: '_L', '-L', '(L)', 'LEFT'  → Left
                         '_R', '-R', '(R)', 'RIGHT' → Right
      - 10–20 rule: odd=Left (Fp1,F3,...), even=Right (Fp2,F4,...), 'Z' midline (Fz,Cz,Pz,Oz)
    Unknown → midline (conservative).
    """
    L, R, Z = [], [], []
    for i, ch in enumerate(ch_names):
        raw = ch
        # alias expansion for matching
        alias = ALIAS_EQUIV.get(raw.upper(), raw)
        key = norm_key(alias)

        # Explicit L/R tags first
        if re.search(r"(^|[_\-\(])L($|[_\-\)])", raw, flags=re.I) or re.search(r"LEFT", raw, flags=re.I):
            L.append(i); continue
        if re.search(r"(^|[_\-\(])R($|[_\-\)])", raw, flags=re.I) or re.search(r"RIGHT", raw, flags=re.I):
            R.append(i); continue

        # Midline 'Z' near end
        if re.search(r"[A-Za-z]Z$", alias, flags=re.I):
            Z.append(i); continue

        # 10–20 odd/even trailing digit
        m = re.search(r"(\d+)$", alias)
        if m:
            try:
                d = int(m.group(1))
                (L if d % 2 == 1 else R).append(i)
                continue
            except:
                pass

        # If unknown but contains 'Z' anywhere → midline
        if 'Z' in key:
            Z.append(i)
        else:
            # conservative default
            Z.append(i)
    return np.array(L, int), np.array(R, int), np.array(Z, int)

# --------------- Anterior / Posterior mapping ---------------
ANT_PREFIXES = ("FP","AF","F","FC")
MID_PREFIXES = ("C",)      # central strip
POST_PREFIXES= ("CP","P","PO","O","TP","FT")  # TP/FT skew posterior/anterior depending on lab; included as weak indicators

def chan_AP_map(ch_names):
    """
    Rough anterior vs posterior using common 10–20 prefixes.
    Returns A (anterior), P (posterior), C (central) indices.
    Unknowns go to central to avoid bias.
    """
    A, P, C = [], [], []
    for i, ch in enumerate(ch_names):
        alias = ALIAS_EQUIV.get(ch.upper(), ch)
        key   = alias.upper()
        # Normalize like 'AF3', 'Pz', etc.
        pref = re.match(r"[A-Za-z]+", key)
        pref = pref.group(0) if pref else ""
        # Assign by prefix
        if any(pref.startswith(px) for px in ANT_PREFIXES):
            A.append(i)
        elif any(pref.startswith(px) for px in POST_PREFIXES):
            P.append(i)
        elif any(pref.startswith(px) for px in MID_PREFIXES):
            C.append(i)
        else:
            C.append(i)
    return np.array(A,int), np.array(P,int), np.array(C,int)

# --------------- Metric helpers ---------------
def mean_safe(x):
    return float(np.nan) if x.size == 0 else float(x.mean())

def within_region_dualmodule_stats(co, labels, idx):
    """Inside a region (e.g., left or right), compute same- vs cross-module means & ratio."""
    if idx.size < 3:  # not enough points
        return np.nan, np.nan, np.nan
    sub = np.ix_(idx, idx)
    co_r = co[sub]
    lab  = labels[idx]
    same = lab[:,None] == lab[None,:]
    diff = ~same
    np.fill_diagonal(same, False)
    np.fill_diagonal(diff, False)
    intra_same = mean_safe(co_r[same])
    intra_diff = mean_safe(co_r[diff])
    ratio = intra_same / (intra_diff + 1e-12)
    return intra_same, intra_diff, float(ratio)

def cross_hemi_coupling_table(co, labels, L, R):
    """
    2×2 means for (within-hemi vs cross-hemi) × (within-module vs cross-module).
    """
    n = len(labels)
    Lmask = np.zeros((n,n), bool); Lmask[np.ix_(L,L)] = True
    Rmask = np.zeros((n,n), bool); Rmask[np.ix_(R,R)] = True
    within_hemi = Lmask | Rmask
    cross_hemi  = np.zeros((n,n), bool); cross_hemi[np.ix_(L,R)] = True; cross_hemi[np.ix_(R,L)] = True

    same_mod = labels[:,None] == labels[None,:]
    diff_mod = ~same_mod
    for m in (within_hemi, cross_hemi, same_mod, diff_mod):
        np.fill_diagonal(m, False)

    return {
        "WH_WM": mean_safe(co[within_hemi & same_mod]),
        "WH_CM": mean_safe(co[within_hemi & diff_mod]),
        "CH_WM": mean_safe(co[cross_hemi  & same_mod]),
        "CH_CM": mean_safe(co[cross_hemi  & diff_mod]),
    }

def degree_to_same_module(co, labels, module_id):
    """Return per-channel degree to nodes in the same module."""
    mask = (labels == module_id)
    return co[:, mask].sum(axis=1)

def LI_LR(co, labels, L, R, module_id):
    """Left–Right lateralization index for a module."""
    deg = degree_to_same_module(co, labels, module_id)
    Lsum = float(deg[L].sum()) if L.size else 0.0
    Rsum = float(deg[R].sum()) if R.size else 0.0
    return (Lsum - Rsum) / (Lsum + Rsum + 1e-12)

def AI_AP(co, labels, A, P, module_id):
    """Anterior–Posterior asymmetry index for a module."""
    deg = degree_to_same_module(co, labels, module_id)
    Asum = float(deg[A].sum()) if A.size else 0.0
    Psum = float(deg[P].sum()) if P.size else 0.0
    return (Asum - Psum) / (Asum + Psum + 1e-12)

def module_composition_counts(labels, L, R, Z, A, P, C):
    out = {}
    for m in sorted(np.unique(labels)):
        out[f"cnt_L_mod{m}"] = int((labels[L]==m).sum()) if L.size else 0
        out[f"cnt_R_mod{m}"] = int((labels[R]==m).sum()) if R.size else 0
        out[f"cnt_Z_mod{m}"] = int((labels[Z]==m).sum()) if Z.size else 0
        out[f"cnt_A_mod{m}"] = int((labels[A]==m).sum()) if A.size else 0
        out[f"cnt_P_mod{m}"] = int((labels[P]==m).sum()) if P.size else 0
        out[f"cnt_C_mod{m}"] = int((labels[C]==m).sum()) if C.size else 0
    return out

# --------------- Channel loaders ---------------
def load_human_channels():
    if os.path.exists(H_CH_TXT):
        with open(H_CH_TXT, "r", encoding="utf-8") as f:
            return [ln.strip() for ln in f if ln.strip()]
    return [f"ch{i}" for i in range(64)]

def load_mouse_channels():
    cands = sorted(glob.glob(os.path.join(M_CH_DIR, "*.channels.txt")))
    if cands:
        with open(cands[0], "r", encoding="utf-8") as f:
            return [ln.strip() for ln in f if ln.strip()]
    return [f"ch{i}" for i in range(32)]

# --------------- Analysis core ---------------
def analyze_species(species, bands, tab_dir, stem_prefix, ch_names, fig_dir=None):
    """
    species: 'human' or 'mouse'
    bands  : iterable of band names to process
    tab_dir: folder with coassoc / labels npy
    stem_prefix: e.g., "band__" (human) or "mouse__" (mouse)
    ch_names: list of channel names
    fig_dir: optional folder to save small ordered coassoc matrices
    """
    L, R, Z = chan_hemisphere_map(ch_names)
    A, P, C = chan_AP_map(ch_names)

    if L.size + R.size == 0:
        print(f"[{species}] WARNING: no left/right mapping; skipping.")
        return pd.DataFrame()

    rows = []
    for b in bands:
        cons_fp = os.path.join(tab_dir, f"{stem_prefix}{b}{CONS_SUFFIX}")
        co_fp   = os.path.join(tab_dir, f"{stem_prefix}{b}{CO_SUFFIX}")
        if not (os.path.exists(cons_fp) and os.path.exists(co_fp)):
            continue

        labels = np.load(cons_fp)
        co     = np.load(co_fp)
        nch    = len(labels)

        # 1) Within-hemisphere dual-module checks
        l_same, l_diff, l_ratio = within_region_dualmodule_stats(co, labels, L)
        r_same, r_diff, r_ratio = within_region_dualmodule_stats(co, labels, R)

        # 2) Cross-hemisphere coupling
        cross_tbl = cross_hemi_coupling_table(co, labels, L, R)

        # 3) Asymmetry indices per module (assume binary modules {0,1})
        uniq_mods = sorted(np.unique(labels))
        # Left-Right LI
        li_mod = {f"LI_LR_mod{m}": LI_LR(co, labels, L, R, m) for m in uniq_mods}
        # Anterior-Posterior AI
        ai_mod = {f"AI_AP_mod{m}": AI_AP(co, labels, A, P, m) for m in uniq_mods}

        # 4) Module composition counts by hemi & A/P/C
        comp = module_composition_counts(labels, L, R, Z, A, P, C)

        # Optional small figure: reorder by [L then R] and within each by module 0 then 1
        if fig_dir is not None:
            try:
                # build order: Left(mod0,mod1) + Right(mod0,mod1) + then include Z in the end
                order = []
                for hemi_idx in (L, R, Z):
                    for m in uniq_mods:
                        order.extend(list(np.where((np.isin(np.arange(nch), hemi_idx)) & (labels==m))[0]))
                order = np.array(order, int)
                from matplotlib import pyplot as plt
                plt.figure()
                plt.imshow(co[np.ix_(order,order)], aspect='auto')
                plt.title(f"{species} {b}: coassoc [L/R/Z × mod0/mod1]")
                plt.colorbar()
                plt.tight_layout()
                fig_path = os.path.join(fig_dir, f"{species}__{b}__coassoc_ordered.png")
                plt.savefig(fig_path, dpi=140)
                plt.close()
            except Exception as e:
                print(f"[{species}:{b}] fig generation failed: {e}")

        row = {
            "species": species,
            "band": b,
            "n_channels": nch,
            "n_left": int(L.size),
            "n_right": int(R.size),
            "n_mid": int(Z.size),
            "n_ant": int(A.size),
            "n_post": int(P.size),
            "n_cent": int(C.size),

            # within-hemisphere dual-module checks
            "left_intra_same": l_same,
            "left_intra_diff": l_diff,
            "left_intra_ratio": l_ratio,
            "right_intra_same": r_same,
            "right_intra_diff": r_diff,
            "right_intra_ratio": r_ratio,

            # cross-hemisphere coupling
            "WH_WM": cross_tbl["WH_WM"],
            "WH_CM": cross_tbl["WH_CM"],
            "CH_WM": cross_tbl["CH_WM"],
            "CH_CM": cross_tbl["CH_CM"],
        }
        row.update(li_mod)
        row.update(ai_mod)
        row.update(comp)
        rows.append(row)

    return pd.DataFrame(rows)

# --------------- RUN: Humans ---------------
# Load channels
h_ch = load_human_channels()
# Analyze
if os.path.isdir(H_TAB_DIR):
    df_h = analyze_species(
        species="human",
        bands=H_BANDS,
        tab_dir=H_TAB_DIR,
        stem_prefix="band__",
        ch_names=h_ch,
        fig_dir=(FIG_DIR_HUMAN if SAVE_FIGS else None)
    )
else:
    df_h = pd.DataFrame()
    print("[human] tables folder not found; skipping.")

# Save
if not df_h.empty:
    df_h.to_csv(H_OUT_CSV, index=False)
    print("\n=== Humans: Hemispheric/AP metrics (saved) ===")
    cols_print_h = [
        "band","n_channels","n_left","n_right","n_ant","n_post",
        "left_intra_ratio","right_intra_ratio",
        "WH_WM","WH_CM","CH_WM","CH_CM",
        "LI_LR_mod0","LI_LR_mod1","AI_AP_mod0","AI_AP_mod1"
    ]
    print(df_h[ [c for c in cols_print_h if c in df_h.columns] ].to_string(index=False))
    if SAVE_FIGS:
        print("Figures →", FIG_DIR_HUMAN)

# --------------- RUN: Mice ---------------
# Determine available mouse bands
mbands = []
for b in M_BANDS_CAND:
    if os.path.exists(os.path.join(M_TAB_DIR, f"mouse__{b}{CONS_SUFFIX}")) and \
       os.path.exists(os.path.join(M_TAB_DIR, f"mouse__{b}{CO_SUFFIX}")):
        mbands.append(b)

m_ch = load_mouse_channels()
if os.path.isdir(M_TAB_DIR) and len(mbands):
    df_m = analyze_species(
        species="mouse",
        bands=mbands,
        tab_dir=M_TAB_DIR,
        stem_prefix="mouse__",
        ch_names=m_ch,
        fig_dir=(FIG_DIR_MOUSE if SAVE_FIGS else None)
    )
else:
    df_m = pd.DataFrame()
    print("[mouse] tables folder not found or no bands present; skipping.")

# Save
if not df_m.empty:
    df_m.to_csv(M_OUT_CSV, index=False)
    print("\n=== Mice: Hemispheric/AP metrics (saved) ===")
    cols_print_m = [
        "band","n_channels","n_left","n_right","n_ant","n_post",
        "left_intra_ratio","right_intra_ratio",
        "WH_WM","WH_CM","CH_WM","CH_CM",
        "LI_LR_mod0","LI_LR_mod1","AI_AP_mod0","AI_AP_mod1"
    ]
    print(df_m[ [c for c in cols_print_m if c in df_m.columns] ].to_string(index=False))
    if SAVE_FIGS:
        print("Figures →", FIG_DIR_MOUSE)

# --------------- MERGE & SAVE ---------------
frames = []
if not df_h.empty: frames.append(df_h.assign(dataset="human"))
if not df_m.empty: frames.append(df_m.assign(dataset="mouse"))
if frames:
    merged = pd.concat(frames, ignore_index=True)
    os.makedirs(os.path.dirname(MERGED_CSV), exist_ok=True)
    merged.to_csv(MERGED_CSV, index=False)
    print("\nMerged CSV →", MERGED_CSV)
else:
    print("\nNo data frames to merge; check paths and bands.")


[human] WARNING: no left/right mapping; skipping.
[mouse] tables folder not found or no bands present; skipping.

No data frames to merge; check paths and bands.


In [5]:
# === CNT Hemi + A/P: Full Prep + Analysis (Humans + Mice) — single cell ===
# Does:
#  A) HUMANS: ensure channel names (pulls EEGBCI S001 R02 if needed), use existing coassoc/labels.
#  B) MICE:  if mouse consensus files missing, download+convert (OpenNeuro) OR synth demo, then build consensus.
#  C) Run Hemispheric + Anterior/Posterior analysis for both. Save CSVs and (optional) ordered coassoc figures.

import os, re, glob, json, numpy as np, pandas as pd

# ------------------ CONFIG ------------------
ROOT            = r"C:\Users\caleb\CNT_Lab"

# Humans (n≈30) — expects you've run the 30-subject sweep already
H_TAB_DIR       = os.path.join(ROOT, r"artifacts\pli_30_subjects\tables")
H_CH_TXT        = os.path.join(ROOT, r"eeg_rest\subject_01_EC.channels.txt")
H_BANDS         = ["alpha","theta","beta"]
H_OUT_CSV       = os.path.join(ROOT, r"artifacts\pli_30_subjects\hemisphere_humans.csv")

# Mice — will auto-prepare if missing
M_TAB_DIR       = os.path.join(ROOT, r"artifacts\pli_mouse_openneuro\tables")
M_MET_DIR       = os.path.join(ROOT, r"artifacts\pli_mouse_openneuro\metrics")
M_FIG_DIR       = os.path.join(ROOT, r"artifacts\pli_mouse_openneuro\figures")
M_DATA_DIR      = os.path.join(ROOT, r"mouse_eeg")
M_RAW_DIR       = os.path.join(ROOT, r"mouse_openneuro_raw")
M_BANDS_TARGET  = ["theta","gamma","high_gamma"]
M_OUT_CSV       = os.path.join(ROOT, r"artifacts\pli_mouse_openneuro\hemisphere_mice.csv")

# OpenNeuro (set dsID if you want a real dataset; otherwise demo synthesizes 8 subjects)
USE_DEMO        = True            # False to actually download from OpenNeuro
MOUSE_DS        = "dsXXXXXX"      # e.g., "ds004***" when you have a real dsID
FILE_FILTERS    = ["*eeg*.edf", "*eeg*.mat", "*lfp*.npy", "*lfp*.mat"]

# Analysis params
SAVE_FIGS       = True
FIG_DIR_HUMAN   = os.path.join(ROOT, r"artifacts\pli_30_subjects\figures_hemi")
FIG_DIR_MOUSE   = os.path.join(ROOT, r"artifacts\pli_mouse_openneuro\figures_hemi")

# Mouse PLI settings (used only if we need to build mouse consensus now)
FS        = 1000.0
SLICE_SEC = 60
K_FIXED   = 2
KNN_K     = 4
NULL_PERMS= 300
BANDS_HZ  = {"theta": (6.0, 12.0), "gamma": (30.0, 55.0), "high_gamma": (60.0, 100.0)}

# ------------------ UTILITIES ------------------
def ensure(p): os.makedirs(p, exist_ok=True); return p
if SAVE_FIGS:
    ensure(FIG_DIR_HUMAN); ensure(FIG_DIR_MOUSE)
ensure(M_TAB_DIR); ensure(M_MET_DIR); ensure(M_FIG_DIR); ensure(M_DATA_DIR)

CONS_SUFFIX = "__consensus_labels.npy"
CO_SUFFIX   = "__coassoc.npy"

def norm_key(s: str) -> str:
    return re.sub(r"[^A-Za-z0-9]", "", s).upper()

ALIAS_EQUIV = {
    "T3":"T7", "T4":"T8", "T5":"P7", "T6":"P8",
    "FP1":"Fp1", "FP2":"Fp2", "FPZ":"Fpz", "CZ":"Cz", "PZ":"Pz", "FZ":"Fz", "OZ":"Oz", "POZ":"POz"
}

def chan_hemisphere_map(ch_names):
    L, R, Z = [], [], []
    for i, ch in enumerate(ch_names):
        alias = ALIAS_EQUIV.get(ch.upper(), ch)
        # explicit tags
        if re.search(r"(^|[_\-\(])L($|[_\-\)])", ch, flags=re.I) or re.search(r"LEFT", ch, flags=re.I):
            L.append(i); continue
        if re.search(r"(^|[_\-\(])R($|[_\-\)])", ch, flags=re.I) or re.search(r"RIGHT", ch, flags=re.I):
            R.append(i); continue
        # midline Z
        if re.search(r"[A-Za-z]Z$", alias, flags=re.I):
            Z.append(i); continue
        # 10-20 odd/even
        m = re.search(r"(\d+)$", alias)
        if m:
            try:
                d=int(m.group(1)); (L if d%2==1 else R).append(i); continue
            except: pass
        # unknown → midline
        if 'Z' in alias.upper(): Z.append(i)
        else: Z.append(i)
    return np.array(L,int), np.array(R,int), np.array(Z,int)

ANT_PREFIXES = ("FP","AF","F","FC")
MID_PREFIXES = ("C",)
POST_PREFIXES= ("CP","P","PO","O","TP","FT")

def chan_AP_map(ch_names):
    A, P, C = [], [], []
    for i, ch in enumerate(ch_names):
        alias = ALIAS_EQUIV.get(ch.upper(), ch).upper()
        pref = re.match(r"[A-Za-z]+", alias)
        pref = pref.group(0) if pref else ""
        if any(pref.startswith(px) for px in ANT_PREFIXES): A.append(i)
        elif any(pref.startswith(px) for px in POST_PREFIXES): P.append(i)
        elif any(pref.startswith(px) for px in MID_PREFIXES): C.append(i)
        else: C.append(i)
    return np.array(A,int), np.array(P,int), np.array(C,int)

def mean_safe(x):
    return float(np.nan) if x.size==0 else float(x.mean())

def within_region_dualmodule_stats(co, labels, idx):
    if idx.size<3: return np.nan, np.nan, np.nan
    sub=np.ix_(idx,idx); co_r=co[sub]; lab=labels[idx]
    same=lab[:,None]==lab[None,:]; diff=~same
    np.fill_diagonal(same,False); np.fill_diagonal(diff,False)
    intra_same=mean_safe(co_r[same]); intra_diff=mean_safe(co_r[diff])
    return intra_same, intra_diff, float(intra_same/(intra_diff+1e-12))

def cross_hemi_coupling_table(co, labels, L, R):
    n=len(labels)
    Lmask=np.zeros((n,n),bool); Lmask[np.ix_(L,L)]=True
    Rmask=np.zeros((n,n),bool); Rmask[np.ix_(R,R)]=True
    within=Lmask|Rmask
    cross=np.zeros((n,n),bool); cross[np.ix_(L,R)]=True; cross[np.ix_(R,L)]=True
    same=labels[:,None]==labels[None,:]; diff=~same
    for m in (within,cross,same,diff): np.fill_diagonal(m,False)
    return {"WH_WM":mean_safe(co[within&same]),"WH_CM":mean_safe(co[within&diff]),
            "CH_WM":mean_safe(co[cross&same]), "CH_CM":mean_safe(co[cross&diff])}

def degree_to_same_module(co, labels, m_id):
    mask=(labels==m_id); return co[:,mask].sum(axis=1)

def LI_LR(co, labels, L, R, m_id):
    deg=degree_to_same_module(co,labels,m_id)
    Lsum=float(deg[L].sum()) if L.size else 0.0
    Rsum=float(deg[R].sum()) if R.size else 0.0
    return (Lsum-Rsum)/(Lsum+Rsum+1e-12)

def AI_AP(co, labels, A, P, m_id):
    deg=degree_to_same_module(co,labels,m_id)
    Asum=float(deg[A].sum()) if A.size else 0.0
    Psum=float(deg[P].sum()) if P.size else 0.0
    return (Asum-Psum)/(Asum+Psum+1e-12)

def module_composition_counts(labels, L, R, Z, A, P, C):
    out={}
    for m in sorted(np.unique(labels)):
        out[f"cnt_L_mod{m}"]=int((labels[L]==m).sum()) if L.size else 0
        out[f"cnt_R_mod{m}"]=int((labels[R]==m).sum()) if R.size else 0
        out[f"cnt_Z_mod{m}"]=int((labels[Z]==m).sum()) if Z.size else 0
        out[f"cnt_A_mod{m}"]=int((labels[A]==m).sum()) if A.size else 0
        out[f"cnt_P_mod{m}"]=int((labels[P]==m).sum()) if P.size else 0
        out[f"cnt_C_mod{m}"]=int((labels[C]==m).sum()) if C.size else 0
    return out

# ------------------ A) HUMANS: ensure channel names ------------------
def ensure_human_channels():
    if os.path.exists(H_CH_TXT):
        with open(H_CH_TXT,"r",encoding="utf-8") as f:
            chs=[ln.strip() for ln in f if ln.strip()]
        L,R,Z=chan_hemisphere_map(chs)
        if L.size+R.size>0:
            print("[human] channel names present and mappable.")
            return chs
        else:
            print("[human] channels present but unmappable — will refresh from EEGBCI.")
    # try fetch one EEGBCI run to get canonical names
    try:
        import mne
    except Exception:
        import sys, subprocess
        subprocess.check_call([sys.executable,"-m","pip","install","-q","mne","pooch"])
        import mne
    try:
        # S001 run 2 (EC)
        try:
            fpaths=mne.datasets.eegbci.load_data(subjects=[1], runs=[2], update_path=True, verbose="ERROR")
        except TypeError:
            fpaths=mne.datasets.eegbci.load_data(subject=1, runs=[2], update_path=True, verbose="ERROR")
        raw=mne.io.read_raw_edf(fpaths[0], preload=True, verbose="ERROR")
        raw.pick_types(eeg=True, stim=False, eog=False, ecg=False, emg=False, misc=False)
        chs=list(raw.ch_names)
        os.makedirs(os.path.dirname(H_CH_TXT), exist_ok=True)
        with open(H_CH_TXT,"w",encoding="utf-8") as f:
            for c in chs: f.write(c+"\n")
        print("[human] wrote channel names to:", H_CH_TXT)
        return chs
    except Exception as e:
        print("[human] could not fetch EEGBCI channel names; using generic labels.", e)
        chs=[f"ch{i}" for i in range(64)]
        with open(H_CH_TXT,"w",encoding="utf-8") as f:
            for c in chs: f.write(c+"\n")
        return chs

# ------------------ B) MICE: prepare consensus if missing ------------------
def mouse_need_build():
    present=[]
    for b in M_BANDS_TARGET:
        if os.path.exists(os.path.join(M_TAB_DIR,f"mouse__{b}{CONS_SUFFIX}")) and \
           os.path.exists(os.path.join(M_TAB_DIR,f"mouse__{b}{CO_SUFFIX}")):
            present.append(b)
    missing=[b for b in M_BANDS_TARGET if b not in present]
    return missing

def bandpass(x,fs,lo,hi,order=4):
    from scipy.signal import butter,filtfilt
    b,a=butter(order,[lo/(fs/2), hi/(fs/2)],btype="band"); return filtfilt(b,a,x)

def pli_matrix(X,fs,lo,hi):
    from scipy.signal import hilbert
    n=X.shape[0]; Y=np.zeros_like(X)
    for c in range(n): Y[c]=bandpass(X[c],fs,lo,hi)
    ph=np.angle(hilbert(Y,axis=1))
    W=np.zeros((n,n),float)
    for i in range(n):
        for j in range(i+1,n):
            d=ph[i]-ph[j]; W[i,j]=W[j,i]=abs(np.mean(np.sign(np.sin(d))))
    np.fill_diagonal(W,0); return W

def knn(W,k):
    W=W.copy(); n=W.shape[0]
    for i in range(n):
        idx=np.argsort(W[i])[::-1]; keep=idx[:min(k,n-1)]
        m=np.ones(n,bool); m[keep]=False; W[i,m]=0
    W=np.maximum(W,W.T); np.fill_diagonal(W,0); return W

def lap(W):
    d=W.sum(1); d=np.where(d<=1e-12,1.0,d); D=np.diag(1.0/np.sqrt(d)); return np.eye(W.shape[0])-D@W@D

def spec(W,k):
    from sklearn.cluster import KMeans
    e,v=np.linalg.eigh(lap(W)); U=v[:,1:k] if k>1 else v[:,:1]
    U/=np.linalg.norm(U,axis=1,keepdims=True)+1e-12
    return KMeans(n_clusters=k,n_init=50,random_state=42).fit_predict(U)

def coassoc(labels):
    n=len(labels[0]); m=len(labels); co=np.zeros((n,n),float)
    for lab in labels:
        for i in range(n):
            li=lab[i]
            for j in range(n):
                co[i,j]+=1 if li==lab[j] else 0
    return co/m

def loso_via_coassoc(labels):
    cof=coassoc(labels); cons=spec(cof,2); vals=[]
    from sklearn.metrics import adjusted_rand_score
    for s in range(len(labels)):
        leave=[lab for i,lab in enumerate(labels) if i!=s]
        cons_l=spec(coassoc(leave),2)
        vals.append(adjusted_rand_score(cons,cons_l))
    return float(np.median(vals)), cons, cof

def rand_same_sizes(lab, rng):
    n=len(lab); uniq,cnts=np.unique(lab, return_counts=True)
    idx=np.arange(n); rng.shuffle(idx); out=np.empty(n,int); st=0
    for L,c in zip(uniq,cnts): seg=idx[st:st+c]; out[seg]=L; st+=c
    return out

def prepare_mouse_data():
    paths=sorted(glob.glob(os.path.join(M_DATA_DIR,"*.npy")))
    if len(paths)>=4:
        print("[mouse] found NPY subjects:", len(paths))
        return paths
    if not USE_DEMO:
        # Try to download from OpenNeuro
        try:
            import openneuro as on
        except Exception:
            import sys, subprocess
            subprocess.check_call([sys.executable,"-m","pip","install","-q","openneuro-py"])
            import openneuro as on
        print(f"[mouse] downloading OpenNeuro {MOUSE_DS} (filtered)…")
        for patt in FILE_FILTERS:
            try:
                on.download(dataset=MOUSE_DS, target=M_RAW_DIR, include=[patt], strict=False)
                print("[mouse] included:", patt)
            except Exception as e:
                print("[mouse] include failed:", patt, e)
        # Convert a few files to NPY
        paths = export_subjects_from_folder(os.path.join(M_RAW_DIR, MOUSE_DS), M_DATA_DIR, subj_limit=12)
        return paths
    else:
        # synth demo 8 subjects × 16 ch × 60 s
        rng=np.random.default_rng(7)
        N_SUBJ,N_CH,T = 8,16,int(FS*SLICE_SEC)
        paths=[]
        for s in range(N_SUBJ):
            X=rng.normal(0,1,size=(N_CH,T))
            t=np.arange(T)/FS
            theta=np.sin(2*np.pi*8.0*t + rng.uniform(0,2*np.pi))
            for c in range(N_CH//2,N_CH):
                X[c]+=0.6*theta + 0.2*rng.normal(0,1,T)
            base=os.path.join(M_DATA_DIR,f"mouse_demo_{s:02d}")
            np.save(base+".npy",X.astype(np.float32))
            with open(base+".channels.txt","w",encoding="utf-8") as f:
                for i in range(N_CH): f.write(f"ch{i}\n")
            paths.append(base+".npy")
        print("[mouse] synthesized demo subjects:", len(paths))
        return paths

def export_subjects_from_folder(in_root, out_dir, subj_limit=12):
    from scipy.io import loadmat
    from scipy.signal import decimate
    def try_load_edf(fp):
        try:
            import mne
        except Exception:
            import sys, subprocess
            subprocess.check_call([sys.executable,"-m","pip","install","-q","mne","pooch"])
            import mne
        raw=mne.io.read_raw_edf(fp, preload=True, verbose="ERROR")
        raw.pick_types(eeg=True, stim=False, eog=False, ecg=False, emg=False, misc=False)
        return raw.get_data(), float(raw.info["sfreq"]), list(raw.ch_names)
    def try_load_mat(fp):
        m=loadmat(fp); arr=None
        for k,v in m.items():
            if isinstance(v,np.ndarray) and v.ndim==2 and (arr is None or v.size>arr.size): arr=v
        if arr is None: raise RuntimeError("No 2D array in MAT")
        fs=float(m.get("fs", np.array([[FS]])).squeeze())
        return arr.astype(float), fs, [f"ch{i}" for i in range(arr.shape[0])]
    def try_load_npy(fp):
        X=np.load(fp); 
        if X.ndim!=2: raise RuntimeError("expected [n_ch,n_t]")
        return X.astype(float), FS, [f"ch{i}" for i in range(X.shape[0])]
    files=[]
    for patt in FILE_FILTERS+["*.npy","*.edf","*.mat"]:
        files+=glob.glob(os.path.join(in_root,"**",patt), recursive=True)
    files=sorted(list(set(files)))
    exported=[]
    for fp in files:
        try:
            if fp.lower().endswith(".edf"):
                X,fs,ch=try_load_edf(fp)
            elif fp.lower().endswith(".mat"):
                X,fs,ch=try_load_mat(fp)
            elif fp.lower().endswith(".npy"):
                X,fs,ch=try_load_npy(fp)
            else:
                continue
            # resample down by integer factor if close
            from math import isclose
            from scipy.signal import decimate
            if not np.isclose(fs, FS, atol=1e-6):
                q=int(round(fs/FS))
                if q>=1 and abs(fs/q-FS)<1e-3:
                    X=np.vstack([decimate(X[i], q, ftype='fir', zero_phase=True) for i in range(X.shape[0])])
                    fs=FS
            # trim/tile to SLICE_SEC
            n_keep=int(SLICE_SEC*fs)
            if X.shape[1]>=n_keep: X=X[:,:n_keep]
            else: 
                reps=int(np.ceil(n_keep/X.shape[1])); X=np.tile(X,reps)[:,:n_keep]
            base=os.path.join(out_dir,f"mouse_{len(exported):02d}")
            np.save(base+".npy",X.astype(np.float32))
            with open(base+".channels.txt","w",encoding="utf-8") as f:
                for c in ch: f.write(c+"\n")
            exported.append(base+".npy")
            if len(exported)>=subj_limit: break
        except Exception as e:
            print("[mouse] skip", fp, e)
    return exported

def build_mouse_consensus_if_missing():
    missing = mouse_need_build()
    if not missing:
        print("[mouse] consensus present for", M_BANDS_TARGET)
        return
    # ensure data
    paths = prepare_mouse_data()
    files = sorted(paths)[:max(8, len(paths))]  # use at least a handful
    if len(files)<4:
        print("[mouse] not enough data to build consensus."); return
    # per band consensus
    from sklearn.metrics import adjusted_rand_score
    rng=np.random.default_rng(13)
    for band, (lo,hi) in BANDS_HZ.items():
        if band not in missing: 
            continue
        labs=[]
        for f in files:
            X=np.load(f); W=pli_matrix(X,FS,lo,hi); W=knn(W,KNN_K); labs.append(spec(W,K_FIXED))
        loso, cons, cof = loso_via_coassoc(labs)
        # save
        np.save(os.path.join(M_TAB_DIR,f"mouse__{band}{CONS_SUFFIX}"), cons)
        np.save(os.path.join(M_TAB_DIR,f"mouse__{band}{CO_SUFFIX}"), cof)
        # null for completeness (optional metric dump)
        null=[]
        for _ in range(NULL_PERMS):
            nlabs=[rand_same_sizes(l, rng) for l in labs]
            _, cons_n, _ = loso_via_coassoc(nlabs)
            null.append(adjusted_rand_score(cons, cons_n))
        null=np.array(null,float)
        p=float((np.sum(null>=loso)+1)/(len(null)+1))
        met={"band":band,"n_subjects":len(files),"LOSO":float(loso),"null_mean":float(null.mean()),"p_value":p}
        ensure(M_MET_DIR)
        with open(os.path.join(M_MET_DIR,f"mouse__{band}__metrics.json"),"w",encoding="utf-8") as f:
            json.dump(met,f,indent=2)
        print(f"[mouse] built {band}: LOSO={loso:.3f}, p≈{p:.4f}")

# ------------------ C) ANALYSIS CORE ------------------
def analyze_species(species, bands, tab_dir, stem_prefix, ch_names, fig_dir=None):
    L,R,Z = chan_hemisphere_map(ch_names)
    A,P,C = chan_AP_map(ch_names)
    if L.size+R.size==0:
        print(f"[{species}] WARNING: no L/R mapping; skipping.")
        return pd.DataFrame()
    rows=[]
    for b in bands:
        cons_fp=os.path.join(tab_dir,f"{stem_prefix}{b}{CONS_SUFFIX}")
        co_fp  =os.path.join(tab_dir,f"{stem_prefix}{b}{CO_SUFFIX}")
        if not (os.path.exists(cons_fp) and os.path.exists(co_fp)): 
            continue
        labels=np.load(cons_fp); co=np.load(co_fp); nch=len(labels)
        # within L/R
        l_same,l_diff,l_ratio=within_region_dualmodule_stats(co,labels,L)
        r_same,r_diff,r_ratio=within_region_dualmodule_stats(co,labels,R)
        # cross hemi matrix
        cross_tbl=cross_hemi_coupling_table(co,labels,L,R)
        # asym L/R & A/P per module
        uniq_mods=sorted(np.unique(labels))
        li_mod={f"LI_LR_mod{m}": LI_LR(co,labels,L,R,m) for m in uniq_mods}
        ai_mod={f"AI_AP_mod{m}": AI_AP(co,labels,A,P,m) for m in uniq_mods}
        comp = module_composition_counts(labels,L,R,Z,A,P,C)
        # optional figure: reorder by [L(mod0,mod1) | R(mod0,mod1) | Z(mod0,mod1)]
        if fig_dir:
            try:
                order=[]
                for hemi_idx in (L,R,Z):
                    for m in uniq_mods:
                        order.extend(list(np.where((np.isin(np.arange(nch),hemi_idx))&(labels==m))[0]))
                order=np.array(order,int)
                import matplotlib.pyplot as plt
                plt.figure()
                plt.imshow(co[np.ix_(order,order)], aspect='auto')
                plt.title(f"{species} {b}: coassoc [L/R/Z × mod0/mod1]"); plt.colorbar(); plt.tight_layout()
                figp=os.path.join(fig_dir,f"{species}__{b}__coassoc_ordered.png")
                plt.savefig(figp,dpi=140); plt.close()
            except Exception as e:
                print(f"[{species}:{b}] fig failed:", e)
        row={"species":species,"band":b,"n_channels":nch,"n_left":int(L.size),"n_right":int(R.size),
             "n_mid":int(Z.size),"n_ant":int(A.size),"n_post":int(P.size),"n_cent":int(C.size),
             "left_intra_same":l_same,"left_intra_diff":l_diff,"left_intra_ratio":l_ratio,
             "right_intra_same":r_same,"right_intra_diff":r_diff,"right_intra_ratio":r_ratio,
             "WH_WM":cross_tbl["WH_WM"],"WH_CM":cross_tbl["WH_CM"],"CH_WM":cross_tbl["CH_WM"],"CH_CM":cross_tbl["CH_CM"]}
        row.update(li_mod); row.update(ai_mod); row.update(comp); rows.append(row)
    return pd.DataFrame(rows)

# ------------------ RUN SEQUENCE ------------------
# Humans: make sure channel names exist & are mappable
h_ch = ensure_human_channels()

# Mice: if consensus missing, build it (downloads or demo if needed)
build_mouse_consensus_if_missing()

# Humans analysis
if os.path.isdir(H_TAB_DIR):
    df_h = analyze_species("human", H_BANDS, H_TAB_DIR, "band__", h_ch, FIG_DIR_HUMAN if SAVE_FIGS else None)
else:
    df_h = pd.DataFrame(); print("[human] tables folder not found; skipping.")

if not df_h.empty:
    ensure(os.path.dirname(H_OUT_CSV)); df_h.to_csv(H_OUT_CSV, index=False)
    print("\n=== Humans: Hemispheric/AP metrics (saved) ===")
    cols_h = ["band","n_channels","n_left","n_right","n_ant","n_post",
              "left_intra_ratio","right_intra_ratio","WH_WM","WH_CM","CH_WM","CH_CM",
              "LI_LR_mod0","LI_LR_mod1","AI_AP_mod0","AI_AP_mod1"]
    print(df_h[[c for c in cols_h if c in df_h.columns]].to_string(index=False))
    if SAVE_FIGS: print("Human figs →", FIG_DIR_HUMAN)

# Mice bands actually present
mbands=[]
for b in M_BANDS_TARGET:
    if os.path.exists(os.path.join(M_TAB_DIR,f"mouse__{b}{CONS_SUFFIX}")) and \
       os.path.exists(os.path.join(M_TAB_DIR,f"mouse__{b}{CO_SUFFIX}")):
        mbands.append(b)

# Mouse channels: take first channels.txt if present; else generic
m_ch_files=sorted(glob.glob(os.path.join(M_DATA_DIR,"*.channels.txt")))
if m_ch_files:
    with open(m_ch_files[0],"r",encoding="utf-8") as f:
        m_ch=[ln.strip() for ln in f if ln.strip()]
else:
    m_ch=[f"ch{i}" for i in range(32)]

if os.path.isdir(M_TAB_DIR) and mbands:
    df_m = analyze_species("mouse", mbands, M_TAB_DIR, "mouse__", m_ch, FIG_DIR_MOUSE if SAVE_FIGS else None)
else:
    df_m = pd.DataFrame(); print("[mouse] no mouse tables present after build; skipping.")

if not df_m.empty:
    ensure(os.path.dirname(M_OUT_CSV)); df_m.to_csv(M_OUT_CSV, index=False)
    print("\n=== Mice: Hemispheric/AP metrics (saved) ===")
    cols_m = ["band","n_channels","n_left","n_right","n_ant","n_post",
              "left_intra_ratio","right_intra_ratio","WH_WM","WH_CM","CH_WM","CH_CM",
              "LI_LR_mod0","LI_LR_mod1","AI_AP_mod0","AI_AP_mod1"]
    print(df_m[[c for c in cols_m if c in df_m.columns]].to_string(index=False))
    if SAVE_FIGS: print("Mouse figs →", FIG_DIR_MOUSE)

# Merge
frames=[]
if not df_h.empty: frames.append(df_h.assign(dataset="human"))
if not df_m.empty: frames.append(df_m.assign(dataset="mouse"))
if frames:
    merged=pd.concat(frames, ignore_index=True)
    merged_csv=os.path.join(ROOT, r"artifacts\CNT_PLI_hemi_AP_merged.csv")
    ensure(os.path.dirname(merged_csv)); merged.to_csv(merged_csv, index=False)
    print("\nMerged CSV →", merged_csv)
else:
    print("\nNo data frames to merge; check paths and that builds completed.")


[human] channels present but unmappable — will refresh from EEGBCI.
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
[human] wrote channel names to: C:\Users\caleb\CNT_Lab\eeg_rest\subject_01_EC.channels.txt
[mouse] found NPY subjects: 8
[mouse] built theta: LOSO=1.000, p≈0.0033
[mouse] built gamma: LOSO=0.766, p≈0.0033
[mouse] built high_gamma: LOSO=0.750, p≈0.0033
[human] WARNING: no L/R mapping; skipping.

=== Mice: Hemispheric/AP metrics (saved) ===
      band  n_channels  n_left  n_right  n_ant  n_post  left_intra_ratio  right_intra_ratio    WH_WM    WH_CM    CH_WM    CH_CM  LI_LR_mod0  LI_LR_mod1  AI_AP_mod0  AI_AP_mod1
     theta          16       8        8      0       0          1.969981           1.846154 0.644231 0.337500 0.629167 0.319853    0.107692   -0.081712         0.0         0.0
     gamma          16       8        8      0       0          1.440789           1.384615 0.547414 0.384259 0.602679 0.399306    0.065574   -0.036530         0.

In [6]:
# === FIX: Human channel mapping + Hemi/AP analysis (updates merged CSV) ===
import os, re, glob, numpy as np, pandas as pd

ROOT      = r"C:\Users\caleb\CNT_Lab"
H_TAB_DIR = os.path.join(ROOT, r"artifacts\pli_30_subjects\tables")
H_CH_TXT  = os.path.join(ROOT, r"eeg_rest\subject_01_EC.channels.txt")
H_OUT_CSV = os.path.join(ROOT, r"artifacts\pli_30_subjects\hemisphere_humans.csv")
H_BANDS   = ["alpha","theta","beta"]

M_TAB_DIR = os.path.join(ROOT, r"artifacts\pli_mouse_openneuro\tables")
M_OUT_CSV = os.path.join(ROOT, r"artifacts\pli_mouse_openneuro\hemisphere_mice.csv")
MERGED    = os.path.join(ROOT, r"artifacts\CNT_PLI_hemi_AP_merged.csv")

SAVE_FIGS     = True
FIG_DIR_HUMAN = os.path.join(ROOT, r"artifacts\pli_30_subjects\figures_hemi_fix")
os.makedirs(FIG_DIR_HUMAN, exist_ok=True)

CONS_SUFFIX = "__consensus_labels.npy"
CO_SUFFIX   = "__coassoc.npy"

# --- Canonical left/right lists (backup safety net) ---
LEFT_CANON = set(map(str.upper, [
    "Fp1","AF7","AF3","F7","F5","F3","F1","FT7","FC5","FC3","FC1","T7","C5","C3","C1",
    "TP7","CP5","CP3","CP1","P7","P5","P3","P1","PO7","PO3","O1"
]))
RIGHT_CANON = set(map(str.upper, [
    "Fp2","AF8","AF4","F8","F6","F4","F2","FT8","FC6","FC4","FC2","T8","C6","C4","C2",
    "TP8","CP6","CP4","CP2","P8","P6","P4","P2","PO8","PO4","O2"
]))
MID_CANON = set(map(str.upper, ["Fpz","AFz","Fz","FCz","Cz","CPz","Pz","POz","Oz"]))

# --- Clean & normalize channel names ---
def clean_label(x: str) -> str:
    y = x.strip()
    # strip common vendor tokens
    y = re.sub(r"(?i)^(EEG|MEG|EOG|ECG|EMG)[\s_\-]+", "", y)
    y = re.sub(r"(?i)[\s_\-]*(REF|AV|AVERAGE|AVG|M1|M2)$", "", y)
    # remove spaces/dashes/periods
    y = re.sub(r"[ \-\.]+", "", y)
    # common alias fixes
    y = y.replace("FP", "Fp").replace("AF", "AF").replace("FC","FC").replace("CP","CP").replace("PO","PO")
    # Standardize case: first letters keep case, digits keep
    return y

def load_human_channels_clean():
    if not os.path.exists(H_CH_TXT):
        raise SystemExit("Human channel file missing. Re-run the 30-subject sweep (it writes channels).")
    with open(H_CH_TXT, "r", encoding="utf-8") as f:
        raw = [ln.strip() for ln in f if ln.strip()]
    lab = [clean_label(x) for x in raw]
    return raw, lab

def hemi_map_from_labels(clean_labels):
    L, R, Z = [], [], []
    for i, ch in enumerate(clean_labels):
        up = ch.upper()
        # backup canonical lists first
        if up in LEFT_CANON:  L.append(i); continue
        if up in RIGHT_CANON: R.append(i); continue
        if up in MID_CANON:   Z.append(i); continue

        # explicit L/R tokens (rare after cleaning, but keep)
        if re.search(r"(^|[_\-\(])L($|[_\-\)])", ch, flags=re.I) or re.search(r"LEFT", ch, flags=re.I):
            L.append(i); continue
        if re.search(r"(^|[_\-\(])R($|[_\-\)])", ch, flags=re.I) or re.search(r"RIGHT", ch, flags=re.I):
            R.append(i); continue

        # 10–20 odd/even; 'z' midline
        if re.search(r"[A-Za-z]z$", ch): Z.append(i); continue
        m = re.search(r"(\d+)$", ch)
        if m:
            try:
                d = int(m.group(1))
                (L if d % 2 == 1 else R).append(i); continue
            except: pass

        # fallthrough → midline (conservative)
        Z.append(i)
    return np.array(L,int), np.array(R,int), np.array(Z,int)

# --- A/P map (rough) ---
ANT_PREFIXES = ("Fp","AF","F","FC")
MID_PREFIXES = ("C",)
POST_PREFIXES= ("CP","P","PO","O","TP","FT")
def ap_map_from_labels(clean_labels):
    A, P, C = [], [], []
    for i, ch in enumerate(clean_labels):
        pref = re.match(r"[A-Za-z]+", ch)
        pref = pref.group(0) if pref else ""
        if any(pref.startswith(px) for px in ANT_PREFIXES): A.append(i)
        elif any(pref.startswith(px) for px in POST_PREFIXES): P.append(i)
        elif any(pref.startswith(px) for px in MID_PREFIXES): C.append(i)
        else: C.append(i)
    return np.array(A,int), np.array(P,int), np.array(C,int)

def mean_safe(x):
    return float(np.nan) if x.size==0 else float(x.mean())

def within_region_dualmodule_stats(co, labels, idx):
    if idx.size < 3: return np.nan, np.nan, np.nan
    sub = np.ix_(idx, idx); co_r = co[sub]; lab = labels[idx]
    same = lab[:,None]==lab[None,:]; diff = ~same
    np.fill_diagonal(same,False); np.fill_diagonal(diff,False)
    intra_same = mean_safe(co_r[same]); intra_diff = mean_safe(co_r[diff])
    return intra_same, intra_diff, float(intra_same/(intra_diff+1e-12))

def cross_hemi_coupling_table(co, labels, L, R):
    n = len(labels)
    Lmask=np.zeros((n,n),bool); Lmask[np.ix_(L,L)] = True
    Rmask=np.zeros((n,n),bool); Rmask[np.ix_(R,R)] = True
    within = Lmask | Rmask
    cross  = np.zeros((n,n),bool); cross[np.ix_(L,R)]=True; cross[np.ix_(R,L)]=True
    same   = labels[:,None]==labels[None,:]; diff = ~same
    for m in (within,cross,same,diff): np.fill_diagonal(m,False)
    return {
        "WH_WM": mean_safe(co[within & same]),
        "WH_CM": mean_safe(co[within & diff]),
        "CH_WM": mean_safe(co[cross  & same]),
        "CH_CM": mean_safe(co[cross  & diff]),
    }

def degree_to_same_module(co, labels, m_id):
    mask=(labels==m_id); return co[:,mask].sum(axis=1)
def LI_LR(co, labels, L, R, m_id):
    deg = degree_to_same_module(co,labels,m_id)
    Lsum=float(deg[L].sum()) if L.size else 0.0
    Rsum=float(deg[R].sum()) if R.size else 0.0
    return (Lsum-Rsum)/(Lsum+Rsum+1e-12)
def AI_AP(co, labels, A, P, m_id):
    deg = degree_to_same_module(co,labels,m_id)
    Asum=float(deg[A].sum()) if A.size else 0.0
    Psum=float(deg[P].sum()) if P.size else 0.0
    return (Asum-Psum)/(Asum+Psum+1e-12)

def module_composition_counts(labels, L, R, Z, A, P, C):
    out={}
    for m in sorted(np.unique(labels)):
        out[f"cnt_L_mod{m}"]=int((labels[L]==m).sum()) if L.size else 0
        out[f"cnt_R_mod{m}"]=int((labels[R]==m).sum()) if R.size else 0
        out[f"cnt_Z_mod{m}"]=int((labels[Z]==m).sum()) if Z.size else 0
        out[f"cnt_A_mod{m}"]=int((labels[A]==m).sum()) if A.size else 0
        out[f"cnt_P_mod{m}"]=int((labels[P]==m).sum()) if P.size else 0
        out[f"cnt_C_mod{m}"]=int((labels[C]==m).sum()) if C.size else 0
    return out

# --- Human analysis ---
def analyze_human():
    if not os.path.isdir(H_TAB_DIR):
        print("[human] tables folder not found; aborting human analysis.")
        return pd.DataFrame()
    raw, lab = load_human_channels_clean()
    L,R,Z = hemi_map_from_labels(lab)
    if L.size+R.size == 0:
        print("[human] still no L/R mapping after cleaning — will attempt index-based split as last resort.")
        # fallback split by index (first half left, second half right) to let you inspect quickly
        n = len(lab); L = np.arange(0, n//2); R = np.arange(n//2, n); Z = np.array([], int)
    A,P,C = ap_map_from_labels(lab)

    rows=[]
    for b in H_BANDS:
        cons_fp = os.path.join(H_TAB_DIR, f"band__{b}{CONS_SUFFIX}")
        co_fp   = os.path.join(H_TAB_DIR, f"band__{b}{CO_SUFFIX}")
        if not (os.path.exists(cons_fp) and os.path.exists(co_fp)): 
            continue
        labels = np.load(cons_fp); co = np.load(co_fp); nch = len(labels)

        l_same,l_diff,l_ratio = within_region_dualmodule_stats(co, labels, L)
        r_same,r_diff,r_ratio = within_region_dualmodule_stats(co, labels, R)

        cross_tbl = cross_hemi_coupling_table(co, labels, L, R)

        uniq = sorted(np.unique(labels))
        li_mod = {f"LI_LR_mod{m}": LI_LR(co, labels, L, R, m) for m in uniq}
        ai_mod = {f"AI_AP_mod{m}": AI_AP(co, labels, A, P, m) for m in uniq}
        comp   = module_composition_counts(labels, L, R, Z, A, P, C)

        # small fig: reorder coassoc by [L mod0, L mod1, R mod0, R mod1, Z mod0, Z mod1]
        if SAVE_FIGS:
            try:
                order=[]
                for hemi in (L,R,Z):
                    for m in uniq:
                        order.extend(list(np.where((np.isin(np.arange(nch),hemi))&(labels==m))[0]))
                order = np.array(order, int)
                import matplotlib.pyplot as plt
                plt.figure()
                plt.imshow(co[np.ix_(order, order)], aspect='auto')
                plt.title(f"human {b}: coassoc [L/R/Z × mod0/mod1]"); plt.colorbar(); plt.tight_layout()
                fp = os.path.join(FIG_DIR_HUMAN, f"human__{b}__coassoc_ordered.png")
                plt.savefig(fp, dpi=140); plt.close()
            except Exception as e:
                print(f"[human:{b}] fig failed:", e)

        row = {
            "species":"human","band":b,"n_channels":nch,
            "n_left":int(L.size),"n_right":int(R.size),"n_mid":int(Z.size),
            "n_ant":int(A.size),"n_post":int(P.size),"n_cent":int(C.size),
            "left_intra_same":l_same,"left_intra_diff":l_diff,"left_intra_ratio":l_ratio,
            "right_intra_same":r_same,"right_intra_diff":r_diff,"right_intra_ratio":r_ratio,
            "WH_WM":cross_tbl["WH_WM"],"WH_CM":cross_tbl["WH_CM"],"CH_WM":cross_tbl["CH_WM"],"CH_CM":cross_tbl["CH_CM"],
        }
        row.update(li_mod); row.update(ai_mod); row.update(comp); rows.append(row)
    return pd.DataFrame(rows)

df_h = analyze_human()
if not df_h.empty:
    df_h.to_csv(H_OUT_CSV, index=False)
    print("\n=== Humans (fixed): Hemispheric/AP metrics (saved) ===")
    cols = ["band","n_channels","n_left","n_right","n_ant","n_post",
            "left_intra_ratio","right_intra_ratio","WH_WM","WH_CM","CH_WM","CH_CM",
            "LI_LR_mod0","LI_LR_mod1","AI_AP_mod0","AI_AP_mod1"]
    print(df_h[[c for c in cols if c in df_h.columns]].to_string(index=False))
    if SAVE_FIGS: print("Human figs →", FIG_DIR_HUMAN)
else:
    print("[human] no bands processed — check that human consensus files exist.")

# Update merged CSV if mouse results exist
frames=[]
if os.path.exists(H_OUT_CSV):
    frames.append(pd.read_csv(H_OUT_CSV).assign(dataset="human"))
if os.path.exists(M_OUT_CSV):
    frames.append(pd.read_csv(M_OUT_CSV).assign(dataset="mouse"))

if frames:
    merged = pd.concat(frames, ignore_index=True)
    os.makedirs(os.path.dirname(MERGED), exist_ok=True)
    merged.to_csv(MERGED, index=False)
    print("\nMerged CSV →", MERGED)
else:
    print("\nMerged CSV not written (no species CSVs available).")



=== Humans (fixed): Hemispheric/AP metrics (saved) ===
 band  n_channels  n_left  n_right  n_ant  n_post  left_intra_ratio  right_intra_ratio    WH_WM    WH_CM    CH_WM    CH_CM  LI_LR_mod0  LI_LR_mod1  AI_AP_mod0  AI_AP_mod1
alpha          64      27       27     21      17          2.270072           2.129487 0.768217 0.348976 0.660854 0.257274    0.043745   -0.039997    0.592513   -0.424026
theta          64      27       27     21      17          1.683705           1.641144 0.773547 0.465642 0.554859 0.261050   -0.067393    0.016287    0.440015   -0.257249
 beta          64      27       27     21      17          4.490455           5.039549 0.891420 0.187821 0.811233 0.114194    0.043345   -0.022706    0.821387   -0.705677
Human figs → C:\Users\caleb\CNT_Lab\artifacts\pli_30_subjects\figures_hemi_fix

Merged CSV → C:\Users\caleb\CNT_Lab\artifacts\CNT_PLI_hemi_AP_merged.csv


In [7]:
# === CNT Human EC vs EO — PLI Consensus + Hemi/AP in one cell ===
# What this does:
#   1) Ensure EO (run 1) NPY for subjects 1..30; EC (run 2) assumed present from your previous run.
#   2) Build PLI spectral-on-coassoc (k=2) consensus for α/θ/β in EC and EO (skips if files exist).
#   3) Clean & map channels (10–20 + aliases) → Left/Right/Midline and Anterior/Posterior.
#   4) Compute Hemi/AP metrics for EC and EO, then output a side-by-side comparison with deltas.
#
# Outputs:
#   EC consensus/tables (already exist):  C:\Users\caleb\CNT_Lab\artifacts\pli_30_subjects\...
#   EO consensus/tables (new):            C:\Users\caleb\CNT_Lab\artifacts\pli_30_subjects_EO\...
#   Hemi/AP metrics EC:                   ...\pli_30_subjects\hemisphere_humans.csv
#   Hemi/AP metrics EO:                   ...\pli_30_subjects_EO\hemisphere_humans.csv
#   EC vs EO comparison table:            C:\Users\caleb\CNT_Lab\artifacts\CNT_PLI_hemi_AP_EC_EO_compare.csv
#   (Optional) ordered co-association figures in ...\figures_hemi_*

import os, re, glob, json, numpy as np, pandas as pd

# ------------------ CONFIG ------------------
ROOT      = r"C:\Users\caleb\CNT_Lab"
SUBJECTS  = list(range(1,31))
FS_OUT    = 250.0
DURATION_S= 60

# Bands and consensus params
BANDS_HZ  = {"alpha": (8.0,13.0), "theta": (4.0,8.0), "beta": (13.0,30.0)}
K_FIXED   = 2
KNN_K     = 6
NULL_PERMS= 300

# Paths
DATA_DIR  = os.path.join(ROOT, "eeg_rest")  # subject_##_{EC|EO}.npy live here
# EC artifacts (assumed built)
EC_DIR    = os.path.join(ROOT, r"artifacts\pli_30_subjects")
EC_TAB    = os.path.join(EC_DIR, "tables")
EC_MET    = os.path.join(EC_DIR, "metrics")
# EO artifacts (will build here)
EO_DIR    = os.path.join(ROOT, r"artifacts\pli_30_subjects_EO")
EO_TAB    = os.path.join(EO_DIR, "tables")
EO_MET    = os.path.join(EO_DIR, "metrics")

# Channel file (we'll clean/map from here)
H_CH_TXT  = os.path.join(ROOT, r"eeg_rest\subject_01_EC.channels.txt")

# Optional figures
SAVE_FIGS    = True
EC_FIG_HEMI  = os.path.join(EC_DIR, "figures_hemi")
EO_FIG_HEMI  = os.path.join(EO_DIR, "figures_hemi")
for p in [DATA_DIR, EC_TAB, EC_MET, EO_TAB, EO_MET]:
    os.makedirs(p, exist_ok=True)
if SAVE_FIGS:
    os.makedirs(EC_FIG_HEMI, exist_ok=True)
    os.makedirs(EO_FIG_HEMI, exist_ok=True)

# Output comparison CSV
OUT_COMPARE = os.path.join(ROOT, r"artifacts\CNT_PLI_hemi_AP_EC_EO_compare.csv")

# ------------------ NPY EXPORT: EO if missing ------------------
def ensure_eo_npys():
    try:
        import mne
    except Exception:
        import sys, subprocess
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "mne", "pooch"])
        import mne

    exported = []
    for s in SUBJECTS:
        base = os.path.join(DATA_DIR, f"subject_{s:02d}_EO.npy")
        if os.path.exists(base):
            exported.append((s, "exists")); continue
        # fetch run 1 (eyes-open)
        try:
            try:
                fpaths = mne.datasets.eegbci.load_data(subjects=[s], runs=[1], update_path=True, verbose="ERROR")
            except TypeError:
                fpaths = mne.datasets.eegbci.load_data(subject=s, runs=[1], update_path=True, verbose="ERROR")
            raws=[]
            for fp in fpaths:
                raw=mne.io.read_raw_edf(fp, preload=True, verbose="ERROR")
                raw.pick_types(eeg=True, stim=False, eog=False, ecg=False, emg=False, misc=False)
                raws.append(raw)
            if not raws:
                exported.append((s, "no_raw")); continue
            raw = mne.concatenate_raws(raws, verbose="ERROR")
            # montage + filter + resample
            try:
                raw.set_montage("standard_1020", on_missing="ignore", match_case=False, verbose="ERROR")
            except Exception:
                pass
            raw.filter(1.0, 45.0, fir_design="firwin", verbose="ERROR")
            raw.resample(FS_OUT, npad="auto", verbose="ERROR")
            n_keep = int(DURATION_S * raw.info["sfreq"])
            X = raw.get_data(picks="eeg")
            if X.shape[1] >= n_keep:
                X = X[:, :n_keep]
            else:
                reps = int(np.ceil(n_keep / X.shape[1])); X = np.tile(X, reps)[:, :n_keep]
            ch_names = mne.pick_info(raw.info, mne.pick_types(raw.info, eeg=True)).ch_names
            base_out = os.path.join(DATA_DIR, f"subject_{s:02d}_EO")
            np.save(base_out + ".npy", X.astype(np.float32))
            with open(base_out + ".channels.txt", "w", encoding="utf-8") as f:
                for ch in ch_names: f.write(ch + "\n")
            exported.append((s, "ok"))
        except Exception as e:
            exported.append((s, f"error: {e}"))
    print("[EO export]", exported[:8], "..." if len(exported)>8 else "")

# ------------------ PLI CONSENSUS CORE ------------------
from scipy.signal import butter, filtfilt, hilbert
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

def bandpass(x,fs,lo,hi,order=4):
    b,a = butter(order, [lo/(fs_out/2), hi/(fs_out/2)], btype="band")
    return filtfilt(b,a,x)

def pli_matrix(X, fs_out, lo, hi):
    n=X.shape[0]; Y=np.zeros_like(X)
    b,a = butter(4, [lo/(fs_out/2), hi/(fs_out/2)], btype="band")
    for c in range(n):
        Y[c] = filtfilt(b,a,X[c])
    ph = np.angle(hilbert(Y, axis=1))
    W  = np.zeros((n,n), float)
    for i in range(n):
        for j in range(i+1, n):
            dphi = ph[i]-ph[j]
            pli  = np.abs(np.mean(np.sign(np.sin(dphi))))
            W[i,j] = W[j,i] = pli
    np.fill_diagonal(W, 0.0)
    return W

def knn(W,k=6):
    W=W.copy(); n=W.shape[0]
    for i in range(n):
        idx=np.argsort(W[i])[::-1]; keep=idx[:k]
        mask=np.ones(n,bool); mask[keep]=False; W[i,mask]=0
    W=np.maximum(W,W.T); np.fill_diagonal(W,0); return W

def lap(W):
    d=W.sum(1); d=np.where(d<=1e-12,1.0,d); D=np.diag(1.0/np.sqrt(d))
    return np.eye(W.shape[0]) - D@W@D

def spec_labels(W,k=2):
    e,v = np.linalg.eigh(lap(W))
    U = v[:,1:k] if k>1 else v[:,:1]
    U/= (np.linalg.norm(U, axis=1, keepdims=True) + 1e-12)
    return KMeans(n_clusters=k, n_init=50, random_state=42).fit_predict(U)

def coassoc(labels):
    n=len(labels[0]); m=len(labels); co=np.zeros((n,n), float)
    for lab in labels:
        for i in range(n):
            li=lab[i]
            for j in range(n):
                co[i,j]+=1.0 if li==lab[j] else 0.0
    return co/m

def loso_via_coassoc(labels):
    cof = coassoc(labels)
    cons= spec_labels(cof, k=2)
    vals=[]
    for s in range(len(labels)):
        leave=[lab for i,lab in enumerate(labels) if i!=s]
        cons_l= spec_labels(coassoc(leave), k=2)
        vals.append(adjusted_rand_score(cons, cons_l))
    return float(np.median(vals)), cons, cof

def rand_same_sizes(lab, rng):
    n=len(lab); uniq,cnts=np.unique(lab, return_counts=True)
    idx=np.arange(n); rng.shuffle(idx)
    out=np.empty(n,int); st=0
    for L,c in zip(uniq,cnts):
        seg=idx[st:st+c]; out[seg]=L; st+=c
    return out

def build_consensus_for_condition(tag_dir, tag_tab, condition_tag):
    """
    condition_tag: 'EC' or 'EO'
    Writes: band__{band}__consensus_labels.npy + band__{band}__coassoc.npy under tag_tab.
    """
    files = [os.path.join(DATA_DIR, f"subject_{s:02d}_{condition_tag}.npy") for s in SUBJECTS if os.path.exists(os.path.join(DATA_DIR, f"subject_{s:02d}_{condition_tag}.npy"))]
    if len(files) < 10:
        print(f"[{condition_tag}] WARNING: only {len(files)} NPY found.")
    rng=np.random.default_rng(7)
    for band,(lo,hi) in BANDS_HZ.items():
        # skip if already present
        cons_fp=os.path.join(tag_tab, f"band__{band}__consensus_labels.npy")
        co_fp  =os.path.join(tag_tab, f"band__{band}__coassoc.npy")
        if os.path.exists(cons_fp) and os.path.exists(co_fp):
            continue
        # per-subject labels
        subj_labels=[]
        for f in files:
            X=np.load(f); W=pli_matrix(X, FS_OUT, lo, hi); W=knn(W, KNN_K)
            labs=spec_labels(W, k=K_FIXED)
            subj_labels.append(labs)
        if not subj_labels:
            print(f"[{condition_tag}] no subjects for {band}")
            continue
        loso, cons, cof = loso_via_coassoc(subj_labels)
        np.save(cons_fp, cons); np.save(co_fp, cof)
        # null just for log
        null=[]
        for _ in range(NULL_PERMS):
            nlabs=[rand_same_sizes(l, rng) for l in subj_labels]
            _, cons_n, _ = loso_via_coassoc(nlabs)
            null.append(adjusted_rand_score(cons, cons_n))
        null=np.array(null,float)
        p=float((np.sum(null>=loso)+1)/(len(null)+1))
        met={"band":band,"n_subjects":len(files),"LOSO":float(loso),"null_mean":float(null.mean()),"p_value":p}
        with open(os.path.join(tag_dir,"metrics", f"band__{band}__metrics.json"),"w",encoding="utf-8") as f:
            json.dump(met,f,indent=2)
        print(f"[{condition_tag}] {band}: LOSO={loso:.3f}, p≈{p:.4f} (saved)")

# ------------------ Channel cleaning + mapping ------------------
LEFT_CANON = set(map(str.upper, [
    "Fp1","AF7","AF3","F7","F5","F3","F1","FT7","FC5","FC3","FC1","T7","C5","C3","C1",
    "TP7","CP5","CP3","CP1","P7","P5","P3","P1","PO7","PO3","O1"
]))
RIGHT_CANON = set(map(str.upper, [
    "Fp2","AF8","AF4","F8","F6","F4","F2","FT8","FC6","FC4","FC2","T8","C6","C4","C2",
    "TP8","CP6","CP4","CP2","P8","P6","P4","P2","PO8","PO4","O2"
]))
MID_CANON = set(map(str.upper, ["Fpz","AFz","Fz","FCz","Cz","CPz","Pz","POz","Oz"]))

def clean_label(x: str) -> str:
    y = x.strip()
    y = re.sub(r"(?i)^(EEG|MEG|EOG|ECG|EMG)[\s_\-]+", "", y)
    y = re.sub(r"(?i)[\s_\-]*(REF|AV|AVERAGE|AVG|M1|M2)$", "", y)
    y = re.sub(r"[ \-\.]+", "", y)
    y = y.replace("FP","Fp")
    return y

def load_human_channels_clean():
    if not os.path.exists(H_CH_TXT):
        raise SystemExit("Missing human channel file: " + H_CH_TXT)
    with open(H_CH_TXT,"r",encoding="utf-8") as f:
        raw=[ln.strip() for ln in f if ln.strip()]
    lab=[clean_label(x) for x in raw]
    return lab

ANT_PREFIXES=("Fp","AF","F","FC"); MID_PREFIXES=("C",); POST_PREFIXES=("CP","P","PO","O","TP","FT")

def hemi_map_from_labels(clean_labels):
    L,R,Z=[],[],[]
    for i,ch in enumerate(clean_labels):
        up=ch.upper()
        if up in LEFT_CANON:  L.append(i); continue
        if up in RIGHT_CANON: R.append(i); continue
        if up in MID_CANON:   Z.append(i); continue
        if re.search(r"[A-Za-z]z$", ch): Z.append(i); continue
        m=re.search(r"(\d+)$", ch)
        if m:
            try:
                d=int(m.group(1)); (L if d%2==1 else R).append(i); continue
            except: pass
        Z.append(i)
    return np.array(L,int), np.array(R,int), np.array(Z,int)

def ap_map_from_labels(clean_labels):
    A,P,C=[],[],[]
    for i,ch in enumerate(clean_labels):
        pref=re.match(r"[A-Za-z]+", ch); pref=pref.group(0) if pref else ""
        if any(pref.startswith(px) for px in ANT_PREFIXES): A.append(i)
        elif any(pref.startswith(px) for px in POST_PREFIXES): P.append(i)
        elif any(pref.startswith(px) for px in MID_PREFIXES): C.append(i)
        else: C.append(i)
    return np.array(A,int), np.array(P,int), np.array(C,int)

# ------------------ Hemi/AP metrics ------------------
def mean_safe(x): return float(np.nan) if x.size==0 else float(x.mean())

def within_region_dualmodule_stats(co, labels, idx):
    if idx.size<3: return np.nan, np.nan, np.nan
    sub=np.ix_(idx,idx); co_r=co[sub]; lab=labels[idx]
    same=lab[:,None]==lab[None,:]; diff=~same
    np.fill_diagonal(same,False); np.fill_diagonal(diff,False)
    intra_same=mean_safe(co_r[same]); intra_diff=mean_safe(co_r[diff])
    return intra_same, intra_diff, float(intra_same/(intra_diff+1e-12))

def cross_hemi_coupling_table(co, labels, L, R):
    n=len(labels)
    Lmask=np.zeros((n,n),bool); Lmask[np.ix_(L,L)]=True
    Rmask=np.zeros((n,n),bool); Rmask[np.ix_(R,R)]=True
    within=Lmask|Rmask
    cross=np.zeros((n,n),bool); cross[np.ix_(L,R)]=True; cross[np.ix_(R,L)]=True
    same=labels[:,None]==labels[None,:]; diff=~same
    for m in (within,cross,same,diff): np.fill_diagonal(m,False)
    return {"WH_WM":mean_safe(co[within&same]),"WH_CM":mean_safe(co[within&diff]),
            "CH_WM":mean_safe(co[cross&same]), "CH_CM":mean_safe(co[cross&diff])}

def degree_to_same_module(co, labels, m_id):
    mask=(labels==m_id); return co[:,mask].sum(axis=1)
def LI_LR(co, labels, L, R, m_id):
    deg=degree_to_same_module(co,labels,m_id)
    Lsum=float(deg[L].sum()) if L.size else 0.0
    Rsum=float(deg[R].sum()) if R.size else 0.0
    return (Lsum-Rsum)/(Lsum+Rsum+1e-12)
def AI_AP(co, labels, A, P, m_id):
    deg=degree_to_same_module(co,labels,m_id)
    Asum=float(deg[A].sum()) if A.size else 0.0
    Psum=float(deg[P].sum()) if P.size else 0.0
    return (Asum-Psum)/(Asum+Psum+1e-12)

def module_composition_counts(labels, L, R, Z, A, P, C):
    out={}
    for m in sorted(np.unique(labels)):
        out[f"cnt_L_mod{m}"]=int((labels[L]==m).sum()) if L.size else 0
        out[f"cnt_R_mod{m}"]=int((labels[R]==m).sum()) if R.size else 0
        out[f"cnt_Z_mod{m}"]=int((labels[Z]==m).sum()) if Z.size else 0
        out[f"cnt_A_mod{m}"]=int((labels[A]==m).sum()) if A.size else 0
        out[f"cnt_P_mod{m}"]=int((labels[P]==m).sum()) if P.size else 0
        out[f"cnt_C_mod{m}"]=int((labels[C]==m).sum()) if C.size else 0
    return out

def hemi_ap_for_condition(tag_dir, tag_tab, figures_dir):
    # load channels
    lab = load_human_channels_clean()
    L,R,Z = hemi_map_from_labels(lab)
    A,P,C = ap_map_from_labels(lab)
    rows=[]
    for b in BANDS_HZ.keys():
        cons_fp=os.path.join(tag_tab, f"band__{b}__consensus_labels.npy")
        co_fp  =os.path.join(tag_tab, f"band__{b}__coassoc.npy")
        if not (os.path.exists(cons_fp) and os.path.exists(co_fp)):
            continue
        labels=np.load(cons_fp); co=np.load(co_fp); nch=len(labels)
        l_same,l_diff,l_ratio=within_region_dualmodule_stats(co,labels,L)
        r_same,r_diff,r_ratio=within_region_dualmodule_stats(co,labels,R)
        cross_tbl=cross_hemi_coupling_table(co,labels,L,R)
        uniq=sorted(np.unique(labels))
        li_mod={f"LI_LR_mod{m}": LI_LR(co,labels,L,R,m) for m in uniq}
        ai_mod={f"AI_AP_mod{m}": AI_AP(co,labels,A,P,m) for m in uniq}
        comp=module_composition_counts(labels,L,R,Z,A,P,C)
        if SAVE_FIGS:
            try:
                order=[]
                for hemi in (L,R,Z):
                    for m in uniq:
                        order.extend(list(np.where((np.isin(np.arange(nch),hemi))&(labels==m))[0]))
                order=np.array(order,int)
                import matplotlib.pyplot as plt
                plt.figure()
                plt.imshow(co[np.ix_(order,order)], aspect='auto')
                plt.title(f"{os.path.basename(tag_dir)} {b}: coassoc [L/R/Z × mod0/mod1]"); plt.colorbar(); plt.tight_layout()
                fp=os.path.join(figures_dir, f"{os.path.basename(tag_dir)}__{b}__coassoc_ordered.png")
                plt.savefig(fp, dpi=140); plt.close()
            except Exception as e:
                print(f"[{os.path.basename(tag_dir)}:{b}] fig failed:", e)
        row={"band":b,"n_channels":nch,
             "n_left":int(L.size),"n_right":int(R.size),"n_mid":int(Z.size),
             "n_ant":int(A.size),"n_post":int(P.size),"n_cent":int(C.size),
             "left_intra_ratio":l_ratio,"right_intra_ratio":r_ratio,
             "WH_WM":cross_tbl["WH_WM"],"WH_CM":cross_tbl["WH_CM"],"CH_WM":cross_tbl["CH_WM"],"CH_CM":cross_tbl["CH_CM"]}
        row.update(li_mod); row.update(ai_mod); row.update(comp); rows.append(row)
    return pd.DataFrame(rows)

# ------------------ RUN PIPELINE ------------------
# 1) Ensure EO NPYs
ensure_eo_npys()

# 2) Build EC/EO consensus (skip EC if exists)
fs_out = FS_OUT  # used inside bandpass
build_consensus_for_condition(EC_DIR, EC_TAB, "EC")  # will skip if files already exist
build_consensus_for_condition(EO_DIR, EO_TAB, "EO")

# 3) Hemi/AP metrics for EC and EO
df_ec = hemi_ap_for_condition(EC_DIR, EC_TAB, EC_FIG_HEMI)
df_eo = hemi_ap_for_condition(EO_DIR, EO_TAB, EO_FIG_HEMI)

# Save per-condition CSVs
if not df_ec.empty:
    out_ec = os.path.join(EC_DIR, "hemisphere_humans.csv"); df_ec.to_csv(out_ec, index=False)
    print("Saved EC hemi/AP →", out_ec)
if not df_eo.empty:
    out_eo = os.path.join(EO_DIR, "hemisphere_humans.csv"); df_eo.to_csv(out_eo, index=False)
    print("Saved EO hemi/AP →", out_eo)

# 4) Build comparison table (EC vs EO) with deltas
if not df_ec.empty and not df_eo.empty:
    # join on band
    join_cols = ["band","n_channels","n_left","n_right","n_ant","n_post","n_cent"]
    metric_cols = ["left_intra_ratio","right_intra_ratio","WH_WM","WH_CM","CH_WM","CH_CM",
                   "LI_LR_mod0","LI_LR_mod1","AI_AP_mod0","AI_AP_mod1"]
    # rename for merge
    ec = df_ec[join_cols + metric_cols].copy()
    eo = df_eo[join_cols + metric_cols].copy()
    ec = ec.rename(columns={c: f"{c}_EC" for c in metric_cols})
    eo = eo.rename(columns={c: f"{c}_EO" for c in metric_cols})
    cmp = pd.merge(ec, eo, on=join_cols, how="inner")
    # add deltas (EC - EO)
    for c in metric_cols:
        cmp[f"Δ_{c}"] = cmp[f"{c}_EC"] - cmp[f"{c}_EO"]
    cmp.to_csv(OUT_COMPARE, index=False)
    print("\n=== EC vs EO comparison (key metrics + Δ) ===")
    print(cmp[["band",
               "left_intra_ratio_EC","left_intra_ratio_EO","Δ_left_intra_ratio",
               "right_intra_ratio_EC","right_intra_ratio_EO","Δ_right_intra_ratio",
               "WH_WM_EC","WH_WM_EO","Δ_WH_WM",
               "CH_WM_EC","CH_WM_EO","Δ_CH_WM",
               "LI_LR_mod0_EC","LI_LR_mod0_EO","Δ_LI_LR_mod0",
               "AI_AP_mod0_EC","AI_AP_mod0_EO","Δ_AI_AP_mod0"]].to_string(index=False))
    print("\nSaved comparison CSV →", OUT_COMPARE)
else:
    print("\nComparison not written (one condition missing).")


NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


...

NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
[EO export] [(1, 'ok'), (2, 'ok'), (3, 'ok'), (4, 'ok'), (5, 'ok'), (6, 'ok'), (7, 'ok'), (8, 'ok')] ...
[EO] alpha: LOSO=1.000, p≈0.0033 (saved)
[EO] theta: LOSO=1.000, p≈0.0033 (saved)
[EO] beta: LOSO=1.000, p≈0.0033 (saved)
Saved EC hemi/AP → C:\Users\caleb\CNT_Lab\artifacts\pli_30_subjects\hemisphere_humans.csv
Saved EO hemi/AP → C:\Users\caleb\CNT_Lab\artifacts\pli_30_subjects_EO\hemisphere_humans.csv

=== EC vs EO comparison (key metrics + Δ) ===
 band  left_intra_ratio_EC  left_intra_ratio_EO  Δ_left_intra_ratio  right_intra_ratio_EC  right_intra_ratio_EO  Δ_right_intra_ratio  WH_WM_EC  WH_WM_EO   Δ_WH_WM  CH_WM_EC  CH_WM_EO   Δ_CH_WM  LI_LR_mod0_EC  LI_LR_mod0_EO  Δ_LI_LR_mod0  AI_AP_mod0_EC  AI_AP_mod0_EO  Δ_AI_AP_mod0
alpha             2.270072             2.557772           -0.287700              2.129487              3.331607            -1.202120  0.768217  0.804118 -0.035901  0.660854  0.730783 -0

In [8]:
# === EC vs EO — Subject-level paired metrics + bootstrap CIs + scalp plots (single cell) ===
# What this cell does:
#   A) For each subject (1..30) and each band (alpha/theta/beta), compute per-subject metrics under EC and EO:
#        - left_intra_ratio, right_intra_ratio
#        - WH_WM, CH_WM  (within-hemi / cross-hemi & within-module)
#        - AI_AP_mod0    (anterior - posterior asymmetry for module 0)
#      Then compute paired EC–EO deltas, bootstrap 95% CIs, and p-values (two-sided, mean delta).
#   B) Draw fast scalp "two-cluster" maps for EC and EO consensus labels for each band (no MNE).
#
# Inputs (assumed present from your prior runs and the previous EO cell):
#   NPY data:  C:\Users\caleb\CNT_Lab\eeg_rest\subject_##_{EC|EO}.npy
#   Channels:  C:\Users\caleb\CNT_Lab\eeg_rest\subject_01_EC.channels.txt
#   EC consensus: C:\Users\caleb\CNT_Lab\artifacts\pli_30_subjects\tables\band__{band}__consensus_labels.npy
#                 C:\Users\caleb\CNT_Lab\artifacts\pli_30_subjects\tables\band__{band}__coassoc.npy
#   EO consensus: C:\Users\caleb\CNT_Lab\artifacts\pli_30_subjects_EO\tables\band__{band}__consensus_labels.npy
#
# Outputs:
#   C:\Users\caleb\CNT_Lab\artifacts\CNT_PLI_EC_EO_paired_bootstrap.csv
#   C:\Users\caleb\CNT_Lab\artifacts\pli_30_subjects\figures_ec_eo_scalp\scalp__{band}__{EC|EO}.png

import os, re, glob, json, numpy as np, pandas as pd, matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt, hilbert

# ------------------- CONFIG -------------------
ROOT      = r"C:\Users\caleb\CNT_Lab"
DATA_DIR  = os.path.join(ROOT, "eeg_rest")
SUBJECTS  = list(range(1,31))
FS        = 250.0
BANDS_HZ  = {"alpha": (8.0,13.0), "theta": (4.0,8.0), "beta": (13.0,30.0)}
K_FIXED   = 2
KNN_K     = 6
BOOT_B    = 2000

EC_DIR, EO_DIR = (os.path.join(ROOT, r"artifacts\pli_30_subjects"),
                  os.path.join(ROOT, r"artifacts\pli_30_subjects_EO"))
EC_TAB, EO_TAB = (os.path.join(EC_DIR, "tables"),
                  os.path.join(EO_DIR, "tables"))

CH_TXT   = os.path.join(ROOT, r"eeg_rest\subject_01_EC.channels.txt")
OUT_CSV  = os.path.join(ROOT, r"artifacts\CNT_PLI_EC_EO_paired_bootstrap.csv")
SCALP_DIR= os.path.join(ROOT, r"artifacts\pli_30_subjects\figures_ec_eo_scalp")
os.makedirs(SCALP_DIR, exist_ok=True)

# ------------------- Channel handling -------------------
def clean_label(x: str) -> str:
    y = x.strip()
    y = re.sub(r"(?i)^(EEG|MEG|EOG|ECG|EMG)[\s_\-]+", "", y)
    y = re.sub(r"(?i)[\s_\-]*(REF|AV|AVERAGE|AVG|M1|M2)$", "", y)
    y = re.sub(r"[ \-\.]+", "", y)
    y = y.replace("FP","Fp")
    return y

def load_channels():
    if not os.path.exists(CH_TXT):
        raise SystemExit("Channel file missing: " + CH_TXT)
    with open(CH_TXT, "r", encoding="utf-8") as f:
        ch = [clean_label(ln.strip()) for ln in f if ln.strip()]
    return ch

ch_names = load_channels()

# Hemisphere map
LEFT_CANON = set(map(str.upper, [
    "Fp1","AF7","AF3","F7","F5","F3","F1","FT7","FC5","FC3","FC1","T7","C5","C3","C1",
    "TP7","CP5","CP3","CP1","P7","P5","P3","P1","PO7","PO3","O1"
]))
RIGHT_CANON = set(map(str.upper, [
    "Fp2","AF8","AF4","F8","F6","F4","F2","FT8","FC6","FC4","FC2","T8","C6","C4","C2",
    "TP8","CP6","CP4","CP2","P8","P6","P4","P2","PO8","PO4","O2"
]))
MID_CANON = set(map(str.upper, ["Fpz","AFz","Fz","FCz","Cz","CPz","Pz","POz","Oz"]))

def hemi_map(labels):
    L,R,Z = [],[],[]
    for i, ch in enumerate(labels):
        up = ch.upper()
        if up in LEFT_CANON:  L.append(i); continue
        if up in RIGHT_CANON: R.append(i); continue
        if up in MID_CANON:   Z.append(i); continue
        if re.search(r"[A-Za-z]z$", ch): Z.append(i); continue
        m = re.search(r"(\d+)$", ch)
        if m:
            try:
                d=int(m.group(1)); (L if d%2==1 else R).append(i); continue
            except: pass
        Z.append(i)
    return np.array(L,int), np.array(R,int), np.array(Z,int)

# Anterior/Posterior map (for AI_AP_mod0)
ANT_PREFIXES = ("Fp","AF","F","FC"); MID_PREFIXES=("C",); POST_PREFIXES=("CP","P","PO","O","TP","FT")
def ap_map(labels):
    A,P,C = [],[],[]
    for i, ch in enumerate(labels):
        pref=re.match(r"[A-Za-z]+", ch); pref=pref.group(0) if pref else ""
        if any(pref.startswith(px) for px in ANT_PREFIXES): A.append(i)
        elif any(pref.startswith(px) for px in POST_PREFIXES): P.append(i)
        elif any(pref.startswith(px) for px in MID_PREFIXES): C.append(i)
        else: C.append(i)
    return np.array(A,int), np.array(P,int), np.array(C,int)

L_idx, R_idx, Z_idx = hemi_map(ch_names)
A_idx, P_idx, C_idx = ap_map(ch_names)

# ------------------- Spectral clustering / metrics -------------------
from sklearn.cluster import KMeans

def bandpass(x, fs, lo, hi, order=4):
    b,a = butter(order, [lo/(fs/2), hi/(fs/2)], btype="band")
    return filtfilt(b,a,x)

def pli_matrix(X, fs, lo, hi):
    n = X.shape[0]
    b,a = butter(4, [lo/(fs/2), hi/(fs/2)], btype="band")
    Y = np.zeros_like(X)
    for c in range(n):
        Y[c] = filtfilt(b,a,X[c])
    ph = np.angle(hilbert(Y, axis=1))
    W  = np.zeros((n,n), float)
    for i in range(n):
        for j in range(i+1, n):
            dphi = ph[i]-ph[j]
            pli  = np.abs(np.mean(np.sign(np.sin(dphi))))
            W[i,j] = W[j,i] = pli
    np.fill_diagonal(W, 0.0)
    return W

def knn(W,k):
    W = W.copy(); n = W.shape[0]
    for i in range(n):
        idx = np.argsort(W[i])[::-1]; keep = idx[:k]
        mask = np.ones(n, bool); mask[keep] = False
        W[i,mask] = 0.0
    W = np.maximum(W,W.T); np.fill_diagonal(W, 0.0)
    return W

def lap(W):
    d=W.sum(1); d=np.where(d<=1e-12,1.0,d); D=np.diag(1.0/np.sqrt(d))
    return np.eye(W.shape[0]) - D@W@D

def spec_labels(W, k=2):
    e,v = np.linalg.eigh(lap(W))
    U   = v[:,1:k] if k>1 else v[:,:1]
    U   = U / (np.linalg.norm(U, axis=1, keepdims=True) + 1e-12)
    return KMeans(n_clusters=k, n_init=50, random_state=42).fit_predict(U)

def mean_safe(x): return float(np.nan) if x.size==0 else float(x.mean())

def within_region_ratio(W, labels, idx):
    """Within a region (e.g., left/right), ratio of same-module mean to cross-module mean using W."""
    if idx.size < 3: return np.nan
    sub = np.ix_(idx,idx); Wr = W[sub]; lab = labels[idx]
    same = lab[:,None] == lab[None,:]; diff = ~same
    np.fill_diagonal(same, False); np.fill_diagonal(diff, False)
    m_same = mean_safe(Wr[same]); m_diff = mean_safe(Wr[diff])
    return float(m_same / (m_diff + 1e-12))

def hemi_module_means(W, labels, L, R):
    """WH_WM and CH_WM (within-hemisphere within-module; cross-hemisphere within-module) from W."""
    n = len(labels)
    Lmask = np.zeros((n,n), bool); Lmask[np.ix_(L,L)] = True
    Rmask = np.zeros((n,n), bool); Rmask[np.ix_(R,R)] = True
    within = Lmask | Rmask
    cross  = np.zeros((n,n), bool); cross[np.ix_(L,R)] = True; cross[np.ix_(R,L)] = True
    same   = labels[:,None] == labels[None,:]
    for m in (within, cross, same):
        np.fill_diagonal(m, False)
    WH_WM = mean_safe(W[within & same])
    CH_WM = mean_safe(W[cross  & same])
    return WH_WM, CH_WM

def AI_AP_mod0(W, labels, A, P):
    """Anterior-Posterior asymmetry for module 0 using degree-to-same-module."""
    m = 0
    mask = (labels == m); deg = W[:,mask].sum(axis=1)
    Asum = float(deg[A].sum()) if A.size else 0.0
    Psum = float(deg[P].sum()) if P.size else 0.0
    return (Asum - Psum) / (Asum + Psum + 1e-12)

# ------------------- Per-subject metric extractor -------------------
def subject_metrics(subject_id, cond_tag, band):
    """Compute per-subject metrics for one subject, condition (EC/EO), and band."""
    f = os.path.join(DATA_DIR, f"subject_{subject_id:02d}_{cond_tag}.npy")
    if not os.path.exists(f):
        return None
    X = np.load(f)
    lo,hi = BANDS_HZ[band]
    W  = pli_matrix(X, FS, lo, hi)
    W  = knn(W, KNN_K)
    lbl= spec_labels(W, k=K_FIXED)
    # Metrics:
    l_ratio = within_region_ratio(W, lbl, L_idx)
    r_ratio = within_region_ratio(W, lbl, R_idx)
    WH_WM, CH_WM = hemi_module_means(W, lbl, L_idx, R_idx)
    ai_ap0 = AI_AP_mod0(W, lbl, A_idx, P_idx)
    return {"subject": subject_id, "cond": cond_tag, "band": band,
            "left_intra_ratio": l_ratio, "right_intra_ratio": r_ratio,
            "WH_WM": WH_WM, "CH_WM": CH_WM, "AI_AP_mod0": ai_ap0}

# ------------------- Build paired deltas + bootstrap -------------------
def bootstrap_ci_p(deltas, B=2000, two_sided=True, seed=7):
    """Bootstrap mean delta CI and p-value (two-sided) against 0."""
    deltas = np.array([d for d in deltas if np.isfinite(d)])
    if deltas.size == 0:
        return np.nan, (np.nan, np.nan), np.nan
    rng = np.random.default_rng(seed)
    means = []
    n = len(deltas)
    for _ in range(B):
        samp = deltas[rng.integers(0, n, size=n)]
        means.append(np.mean(samp))
    means = np.array(means)
    ci_lo, ci_hi = np.percentile(means, [2.5, 97.5])
    # two-sided p: proportion of bootstrap means with |mean| >= |obs_mean|
    obs = float(np.mean(deltas))
    p = float((np.sum(np.abs(means) >= abs(obs)) + 1) / (B + 1))
    return obs, (float(ci_lo), float(ci_hi)), p

rows = []
for band in BANDS_HZ.keys():
    # collect per-subject metrics
    recs = []
    for s in SUBJECTS:
        m_ec = subject_metrics(s, "EC", band)
        m_eo = subject_metrics(s, "EO", band)
        if m_ec and m_eo:
            recs.append((m_ec, m_eo))
    if not recs:
        continue
    # build paired deltas
    metrics = ["left_intra_ratio","right_intra_ratio","WH_WM","CH_WM","AI_AP_mod0"]
    for met in metrics:
        deltas = [r[0][met] - r[1][met] for r in recs]  # EC - EO
        obs, (ci_lo, ci_hi), p = bootstrap_ci_p(deltas, B=BOOT_B)
        rows.append({"band": band, "metric": met, "N_pairs": len(deltas),
                     "delta_mean": obs, "ci_2.5": ci_lo, "ci_97.5": ci_hi, "p_boot": p})

df_boot = pd.DataFrame(rows)
df_boot.to_csv(OUT_CSV, index=False)
print("=== EC - EO (paired, subject-level) — bootstrap CIs & p-values ===")
print(df_boot.to_string(index=False))
print("Saved:", OUT_CSV)

# ------------------- Fast scalp plots: EC vs EO consensus labels -------------------
# Minimal 10–20 2D coords for plotting (no interpolation)
canon_xy = {
 "Fp1":(-0.5, 0.95), "Fpz":(0.0, 0.98), "Fp2":(0.5, 0.95),
 "AF7":(-0.65, 0.75), "AF3":(-0.35, 0.78), "AFz":(0.0, 0.80), "AF4":(0.35,0.78), "AF8":(0.65,0.75),
 "F7":(-0.8, 0.55), "F5":(-0.55,0.58), "F3":(-0.35,0.60), "F1":(-0.15,0.62), "Fz":(0.0,0.65),
 "F2":(0.15,0.62), "F4":(0.35,0.60), "F6":(0.55,0.58), "F8":(0.8,0.55),
 "FT7":(-0.9, 0.35), "FC5":(-0.6,0.40), "FC3":(-0.4,0.42), "FC1":(-0.2,0.44), "FCz":(0.0,0.45),
 "FC2":(0.2,0.44), "FC4":(0.4,0.42), "FC6":(0.6,0.40), "FT8":(0.9,0.35),
 "T7":(-1.0, 0.05), "C5":(-0.6,0.05), "C3":(-0.4,0.05), "C1":(-0.2,0.05), "Cz":(0.0,0.05),
 "C2":(0.2,0.05), "C4":(0.4,0.05), "C6":(0.6,0.05), "T8":(1.0,0.05),
 "TP7":(-0.9,-0.25), "CP5":(-0.6,-0.25), "CP3":(-0.4,-0.25), "CP1":(-0.2,-0.25), "CPz":(0.0,-0.25),
 "CP2":(0.2,-0.25), "CP4":(0.4,-0.25), "CP6":(0.6,-0.25), "TP8":(0.9,-0.25),
 "P7":(-0.8,-0.50), "P5":(-0.55,-0.50), "P3":(-0.35,-0.50), "P1":(-0.15,-0.50), "Pz":(0.0,-0.52),
 "P2":(0.15,-0.50), "P4":(0.35,-0.50), "P6":(0.55,-0.50), "P8":(0.8,-0.50),
 "PO7":(-0.65,-0.70), "PO3":(-0.35,-0.70), "POz":(0.0,-0.72), "PO4":(0.35,-0.70), "PO8":(0.65,-0.70),
 "O1":(-0.4,-0.90), "Oz":(0.0,-0.92), "O2":(0.4,-0.90)
}
def coords_for_channels(names):
    out=[]; idx=[]
    canon = {k.upper():k for k in canon_xy.keys()}
    alias = {"T3":"T7","T4":"T8","T5":"P7","T6":"P8","FP1":"Fp1","FP2":"Fp2","FPZ":"Fpz","OZ":"Oz","CZ":"Cz","PZ":"Pz","FZ":"Fz","POZ":"POz"}
    for i,ch in enumerate(names):
        up = ch.upper()
        key = alias.get(up, ch)
        keyu= key.upper()
        if keyu in canon:
            out.append(canon_xy[canon[keyu]]); idx.append(i)
    return np.array(out,float), np.array(idx,int)

def draw_head(ax):
    head = plt.Circle((0,0), 1.03, fill=False, linewidth=2)
    nose = plt.Polygon([[-0.12,1.03],[0,1.15],[0.12,1.03]], fill=False)
    ax.add_patch(head); ax.add_patch(nose)
    ax.set_xlim(-1.15,1.15); ax.set_ylim(-1.1,1.2)
    ax.set_aspect("equal"); ax.axis("off")

def plot_scalp(cons_labels, names, title, out_png):
    pts, keep = coords_for_channels(names)
    labs = np.array(cons_labels)[keep]
    fig = plt.figure(figsize=(6,6)); ax = fig.add_subplot(111); draw_head(ax)
    m0 = labs==0; m1 = labs==1
    ax.scatter(pts[m0,0], pts[m0,1], s=70, label="Cluster 0")
    ax.scatter(pts[m1,0], pts[m1,1], s=70, marker="s", label="Cluster 1")
    for name in ["Fpz","Fz","Cz","Pz","Oz"]:
        if name in canon_xy:
            x,y = canon_xy[name]; ax.text(x,y+0.03,name,ha="center",va="bottom",fontsize=9)
    ax.legend(loc="upper right"); ax.set_title(title)
    fig.tight_layout(); fig.savefig(out_png,dpi=160); plt.close(fig)

for band in BANDS_HZ.keys():
    # EC
    cons_ec = os.path.join(EC_TAB, f"band__{band}__consensus_labels.npy")
    if os.path.exists(cons_ec):
        plot_scalp(np.load(cons_ec), ch_names,
                   f"EC consensus clusters | {band}", 
                   os.path.join(SCALP_DIR, f"scalp__{band}__EC.png"))
    # EO
    cons_eo = os.path.join(EO_TAB, f"band__{band}__consensus_labels.npy")
    if os.path.exists(cons_eo):
        plot_scalp(np.load(cons_eo), ch_names,
                   f"EO consensus clusters | {band}", 
                   os.path.join(SCALP_DIR, f"scalp__{band}__EO.png"))

print("Scalp plots →", SCALP_DIR)


=== EC - EO (paired, subject-level) — bootstrap CIs & p-values ===
 band            metric  N_pairs  delta_mean    ci_2.5   ci_97.5   p_boot
alpha  left_intra_ratio       30    3.695037 -1.147616 11.750398 0.468766
alpha right_intra_ratio       30    1.019468 -7.511263 12.953784 0.847076
alpha             WH_WM       30    0.019686  0.010083  0.030510 0.483258
alpha             CH_WM       30    0.016207  0.008887  0.024559 0.498251
alpha        AI_AP_mod0       30   -0.125969 -0.355893  0.099395 0.502249
theta  left_intra_ratio       30   -2.207807 -4.221322 -0.738394 0.470265
theta right_intra_ratio       30    1.234547 -1.153567  3.902559 0.494753
theta             WH_WM       30   -0.008086 -0.012853 -0.003888 0.506247
theta             CH_WM       30   -0.002711 -0.007172  0.001351 0.488256
theta        AI_AP_mod0       30   -0.140331 -0.288054  0.002500 0.485757
 beta  left_intra_ratio       30   -0.237323 -2.095473  1.063375 0.771614
 beta right_intra_ratio       30    0.728343 

In [9]:
# === FIX: EC–EO paired stats with proper permutation p-values (sign-flip) ===
# Recomputes per-subject EC–EO deltas for {left/right_intra_ratio, WH_WM, CH_WM, AI_AP_mod0}
# and outputs corrected two-sided permutation p-values + bootstrap 95% CIs.

import os, re, glob, json, numpy as np, pandas as pd
from scipy.signal import butter, filtfilt, hilbert
from sklearn.cluster import KMeans

ROOT      = r"C:\Users\caleb\CNT_Lab"
DATA_DIR  = os.path.join(ROOT, "eeg_rest")
SUBJECTS  = list(range(1,31))
FS        = 250.0
BANDS_HZ  = {"alpha": (8.0,13.0), "theta": (4.0,8.0), "beta": (13.0,30.0)}
K_FIXED   = 2
KNN_K     = 6
BOOT_B    = 2000
PERM_B    = 10000

OUT_CSV   = os.path.join(ROOT, r"artifacts\CNT_PLI_EC_EO_paired_perm.csv")
CH_TXT    = os.path.join(ROOT, r"eeg_rest\subject_01_EC.channels.txt")

def clean_label(x: str) -> str:
    y = x.strip()
    y = re.sub(r"(?i)^(EEG|MEG|EOG|ECG|EMG)[\s_\-]+", "", y)
    y = re.sub(r"(?i)[\s_\-]*(REF|AV|AVERAGE|AVG|M1|M2)$", "", y)
    y = re.sub(r"[ \-\.]+", "", y)
    y = y.replace("FP","Fp")
    return y

def load_channels():
    if not os.path.exists(CH_TXT):
        raise SystemExit("Channel file missing: " + CH_TXT)
    with open(CH_TXT, "r", encoding="utf-8") as f:
        return [clean_label(ln.strip()) for ln in f if ln.strip()]

ch_names = load_channels()

LEFT_CANON = set(map(str.upper, [
    "Fp1","AF7","AF3","F7","F5","F3","F1","FT7","FC5","FC3","FC1","T7","C5","C3","C1",
    "TP7","CP5","CP3","CP1","P7","P5","P3","P1","PO7","PO3","O1"
]))
RIGHT_CANON = set(map(str.upper, [
    "Fp2","AF8","AF4","F8","F6","F4","F2","FT8","FC6","FC4","FC2","T8","C6","C4","C2",
    "TP8","CP6","CP4","CP2","P8","P6","P4","P2","PO8","PO4","O2"
]))
MID_CANON = set(map(str.upper, ["Fpz","AFz","Fz","FCz","Cz","CPz","Pz","POz","Oz"]))

def hemi_map(labels):
    L,R,Z=[],[],[]
    for i,ch in enumerate(labels):
        up=ch.upper()
        if up in LEFT_CANON:  L.append(i); continue
        if up in RIGHT_CANON: R.append(i); continue
        if up in MID_CANON:   Z.append(i); continue
        if re.search(r"[A-Za-z]z$", ch): Z.append(i); continue
        m=re.search(r"(\d+)$", ch)
        if m:
            try:
                d=int(m.group(1)); (L if d%2==1 else R).append(i); continue
            except: pass
        Z.append(i)
    return np.array(L,int), np.array(R,int), np.array(Z,int)

ANT_PREFIXES=("Fp","AF","F","FC"); MID_PREFIXES=("C",); POST_PREFIXES=("CP","P","PO","O","TP","FT")
def ap_map(labels):
    A,P,C=[],[],[]
    for i,ch in enumerate(labels):
        pref=re.match(r"[A-Za-z]+", ch); pref=pref.group(0) if pref else ""
        if any(pref.startswith(px) for px in ANT_PREFIXES): A.append(i)
        elif any(pref.startswith(px) for px in POST_PREFIXES): P.append(i)
        elif any(pref.startswith(px) for px in MID_PREFIXES): C.append(i)
        else: C.append(i)
    return np.array(A,int), np.array(P,int), np.array(C,int)

L_idx, R_idx, Z_idx = hemi_map(ch_names)
A_idx, P_idx, C_idx = ap_map(ch_names)

def bandpass(x, fs, lo, hi, order=4):
    b,a = butter(order, [lo/(fs/2), hi/(fs/2)], btype="band")
    return filtfilt(b,a,x)

def pli_matrix(X, fs, lo, hi):
    n = X.shape[0]
    b,a = butter(4, [lo/(fs/2), hi/(fs/2)], btype="band")
    Y = np.zeros_like(X)
    for c in range(n):
        Y[c] = filtfilt(b,a,X[c])
    ph = np.angle(hilbert(Y, axis=1))
    W  = np.zeros((n,n), float)
    for i in range(n):
        for j in range(i+1, n):
            dphi = ph[i]-ph[j]
            pli  = np.abs(np.mean(np.sign(np.sin(dphi))))
            W[i,j] = W[j,i] = pli
    np.fill_diagonal(W, 0.0)
    return W

def knn(W,k):
    W=W.copy(); n=W.shape[0]
    for i in range(n):
        idx=np.argsort(W[i])[::-1]; keep=idx[:k]
        mask=np.ones(n,bool); mask[keep]=False; W[i,mask]=0
    W=np.maximum(W,W.T); np.fill_diagonal(W,0); return W

def lap(W):
    d=W.sum(1); d=np.where(d<=1e-12,1.0,d); D=np.diag(1.0/np.sqrt(d))
    return np.eye(W.shape[0]) - D@W@D

def spec_labels(W, k=2):
    e,v = np.linalg.eigh(lap(W))
    U   = v[:,1:k] if k>1 else v[:,:1]
    U   = U / (np.linalg.norm(U, axis=1, keepdims=True) + 1e-12)
    return KMeans(n_clusters=k, n_init=50, random_state=42).fit_predict(U)

def mean_safe(x): return float(np.nan) if x.size==0 else float(x.mean())

def within_region_ratio(W, labels, idx):
    if idx.size<3: return np.nan
    sub=np.ix_(idx,idx); Wr=W[sub]; lab=labels[idx]
    same=lab[:,None]==lab[None,:]; diff=~same
    np.fill_diagonal(same,False); np.fill_diagonal(diff,False)
    m_same=mean_safe(Wr[same]); m_diff=mean_safe(Wr[diff])
    return float(m_same/(m_diff+1e-12))

def hemi_module_means(W, labels, L, R):
    n=len(labels)
    Lmask=np.zeros((n,n),bool); Lmask[np.ix_(L,L)]=True
    Rmask=np.zeros((n,n),bool); Rmask[np.ix_(R,R)]=True
    within=Lmask|Rmask
    cross =np.zeros((n,n),bool); cross[np.ix_(L,R)]=True; cross[np.ix_(R,L)]=True
    same  =labels[:,None]==labels[None,:]
    for m in (within,cross,same): np.fill_diagonal(m,False)
    WH_WM=mean_safe(W[within&same]); CH_WM=mean_safe(W[cross&same])
    return WH_WM, CH_WM

def AI_AP_mod0(W, labels, A, P):
    m=0; mask=(labels==m); deg=W[:,mask].sum(axis=1)
    Asum=float(deg[A].sum()) if A.size else 0.0
    Psum=float(deg[P].sum()) if P.size else 0.0
    return (Asum-Psum)/(Asum+Psum+1e-12)

def subject_metrics(subject_id, cond_tag, band):
    f = os.path.join(DATA_DIR, f"subject_{subject_id:02d}_{cond_tag}.npy")
    if not os.path.exists(f): return None
    X = np.load(f)
    lo,hi=BANDS_HZ[band]
    W  = pli_matrix(X, FS, lo, hi)
    W  = knn(W, KNN_K)
    lbl= spec_labels(W, k=K_FIXED)
    l_ratio = within_region_ratio(W, lbl, L_idx)
    r_ratio = within_region_ratio(W, lbl, R_idx)
    WH_WM, CH_WM = hemi_module_means(W, lbl, L_idx, R_idx)
    ai_ap0 = AI_AP_mod0(W, lbl, A_idx, P_idx)
    return {"subject": subject_id, "cond": cond_tag, "band": band,
            "left_intra_ratio": l_ratio, "right_intra_ratio": r_ratio,
            "WH_WM": WH_WM, "CH_WM": CH_WM, "AI_AP_mod0": ai_ap0}

def bootstrap_ci(deltas, B=2000, seed=7):
    rng=np.random.default_rng(seed)
    deltas=np.array([d for d in deltas if np.isfinite(d)])
    if deltas.size==0: return np.nan, np.nan
    means=[]
    n=len(deltas)
    for _ in range(B):
        samp=deltas[rng.integers(0,n,size=n)]
        means.append(np.mean(samp))
    lo,hi=np.percentile(means,[2.5,97.5])
    return float(lo), float(hi)

def perm_pvalue(deltas, B=10000, seed=11):
    rng=np.random.default_rng(seed)
    deltas=np.array([d for d in deltas if np.isfinite(d)])
    if deltas.size==0: return np.nan
    obs=np.mean(deltas)
    # sign-flip under null: EC and EO labels are exchangeable → delta signs random
    cnt=1  # +1 for obs
    for _ in range(B):
        signs=np.where(rng.random(len(deltas))<0.5, 1.0, -1.0)
        perm=np.mean(deltas*signs)
        if abs(perm) >= abs(obs): cnt+=1
    return float(cnt/(B+1))

rows=[]
for band in BANDS_HZ.keys():
    recs=[]
    for s in SUBJECTS:
        ec=subject_metrics(s,"EC",band)
        eo=subject_metrics(s,"EO",band)
        if ec and eo: recs.append((ec,eo))
    if not recs: continue
    metrics=["left_intra_ratio","right_intra_ratio","WH_WM","CH_WM","AI_AP_mod0"]
    for met in metrics:
        deltas=[r[0][met]-r[1][met] for r in recs]  # EC-EO
        dmean=float(np.mean([d for d in deltas if np.isfinite(d)])) if len(deltas) else np.nan
        ci_lo,ci_hi=bootstrap_ci(deltas, B=BOOT_B)
        p_perm=perm_pvalue(deltas, B=PERM_B)
        rows.append({"band":band,"metric":met,"N_pairs":len(deltas),
                     "delta_mean":dmean,"ci_2.5":ci_lo,"ci_97.5":ci_hi,"p_perm":p_perm})

df = pd.DataFrame(rows)
df.to_csv(OUT_CSV, index=False)
print("=== EC - EO (paired) — corrected permutation p-values ===")
print(df.to_string(index=False))
print("Saved:", OUT_CSV)


=== EC - EO (paired) — corrected permutation p-values ===
 band            metric  N_pairs  delta_mean    ci_2.5   ci_97.5   p_perm
alpha  left_intra_ratio       30    3.695037 -1.147616 11.750398 0.383762
alpha right_intra_ratio       30    1.019468 -7.511263 12.953784 0.891211
alpha             WH_WM       30    0.019686  0.010083  0.030510 0.000200
alpha             CH_WM       30    0.016207  0.008887  0.024559 0.000100
alpha        AI_AP_mod0       30   -0.125969 -0.355893  0.099395 0.280572
theta  left_intra_ratio       30   -2.207807 -4.221322 -0.738394 0.004200
theta right_intra_ratio       30    1.234547 -1.153567  3.902559 0.364464
theta             WH_WM       30   -0.008086 -0.012853 -0.003888 0.001200
theta             CH_WM       30   -0.002711 -0.007172  0.001351 0.219978
theta        AI_AP_mod0       30   -0.140331 -0.288054  0.002500 0.062394
 beta  left_intra_ratio       30   -0.237323 -2.095473  1.063375 0.904310
 beta right_intra_ratio       30    0.728343 -1.087263

In [10]:
# === Compose 1-page EC vs EO PDF (paired stats + scalp maps) ===
import os, pandas as pd, matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

ROOT      = r"C:\Users\caleb\CNT_Lab"
CSV_STATS = os.path.join(ROOT, r"artifacts\CNT_PLI_EC_EO_paired_perm.csv")
SCALP_DIR = os.path.join(ROOT, r"artifacts\pli_30_subjects\figures_ec_eo_scalp")
OUT_PDF   = os.path.join(ROOT, r"artifacts\CNT_PLI_EC_EO_summary.pdf")

# Load stats
df = pd.read_csv(CSV_STATS) if os.path.exists(CSV_STATS) else pd.DataFrame()

# Prepare page
fig = plt.figure(figsize=(11, 8.5))  # landscape letter

# Title
ax_t = fig.add_axes([0.05, 0.90, 0.90, 0.08]); ax_t.axis("off")
ax_t.text(0.5, 0.5, "CNT — Eyes-Closed vs Eyes-Open (Subject-level PLI; paired EC–EO)",
          ha="center", va="center", fontsize=16, weight="bold")

# Table (compact: only key rows with p_perm < 0.05 + alpha WH_WM/CH_WM rows)
keep = []
if not df.empty:
    for band in ["alpha","theta","beta"]:
        dsub = df[df["band"]==band].copy()
        # always show WH_WM and CH_WM for all bands
        keep.append(dsub[dsub["metric"]=="WH_WM"])
        keep.append(dsub[dsub["metric"]=="CH_WM"])
        # include significant others (p_perm<0.05)
        keep.append(dsub[(dsub["metric"]!="WH_WM") & (dsub["metric"]!="CH_WM") & (dsub["p_perm"]<0.05)])
    show = pd.concat(keep, ignore_index=True) if keep else df
    show = show[["band","metric","N_pairs","delta_mean","ci_2.5","ci_97.5","p_perm"]]
    show["delta_mean"] = show["delta_mean"].map(lambda x: f"{x:.3f}")
    show["ci_2.5"]     = show["ci_2.5"].map(lambda x: f"{x:.3f}")
    show["ci_97.5"]    = show["ci_97.5"].map(lambda x: f"{x:.3f}")
    show["p_perm"]     = show["p_perm"].map(lambda x: f"{x:.4f}")
else:
    show = pd.DataFrame([{"band":"—","metric":"No data","N_pairs":"","delta_mean":"","ci_2.5":"","ci_97.5":"","p_perm":""}])

ax_tab = fig.add_axes([0.05, 0.58, 0.90, 0.28]); ax_tab.axis("off")
table = ax_tab.table(cellText=show.values, colLabels=show.columns, loc="center")
table.auto_set_font_size(False); table.set_fontsize(8); table.scale(1.0, 1.3)

# Scalp strip: for each band, EC (left) vs EO (right)
slots = {
    "alpha": [0.05, 0.05, 0.28, 0.45],
    "theta": [0.36, 0.05, 0.28, 0.45],
    "beta" : [0.67, 0.05, 0.28, 0.45],
}
for band, rect in slots.items():
    ax = fig.add_axes(rect); ax.axis("off")
    # assemble side-by-side: EC then EO
    p_ec = os.path.join(SCALP_DIR, f"scalp__{band}__EC.png")
    p_eo = os.path.join(SCALP_DIR, f"scalp__{band}__EO.png")
    imgs = []
    if os.path.exists(p_ec): imgs.append(("EC", plt.imread(p_ec)))
    if os.path.exists(p_eo): imgs.append(("EO", plt.imread(p_eo)))
    if imgs:
        # stack horizontally with minimal gutters
        total_w = sum(img.shape[1] for _,img in imgs)
        max_h   = max(img.shape[0] for _,img in imgs)
        strip = 255*np.ones((max_h, total_w, 4), dtype=np.uint8)
        x=0
        for tag, img in imgs:
            h,w = img.shape[0], img.shape[1]
            strip[:h, x:x+w, :img.shape[2]] = img
            # label
            ax.text((x+w/2)/total_w, 1.02, f"{band.upper()} {tag}", transform=ax.transAxes,
                    ha="center", va="bottom", fontsize=10)
            x += w
        ax.imshow(strip); ax.set_title(f"{band} — EC vs EO")
    else:
        ax.text(0.5, 0.5, f"{band} scalp images missing", ha="center", va="center", fontsize=10)

# Save
pp = PdfPages(OUT_PDF); pp.savefig(fig, dpi=200); pp.close(); plt.close(fig)
print("Saved 1-page PDF:", OUT_PDF)


Saved 1-page PDF: C:\Users\caleb\CNT_Lab\artifacts\CNT_PLI_EC_EO_summary.pdf


In [11]:
# === EC vs EO prediction from PLI field metrics (subject-level) ===
# Uses per-subject PLI features (alpha/theta/beta) to classify EC vs EO.
# Outputs: AUC per band + permutation p-values, CSV + quick plot.

import os, re, glob, json, numpy as np, pandas as pd, matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt, hilbert
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import roc_auc_score

ROOT      = r"C:\Users\caleb\CNT_Lab"
DATA_DIR  = os.path.join(ROOT, "eeg_rest")
SUBJECTS  = list(range(1,31))
FS        = 250.0
BANDS_HZ  = {"alpha": (8.0,13.0), "theta": (4.0,8.0), "beta": (13.0,30.0)}
K_FIXED   = 2
KNN_K     = 6

OUT_DIR   = os.path.join(ROOT, "artifacts", "pli_ec_eo_prediction")
os.makedirs(OUT_DIR, exist_ok=True)
OUT_CSV   = os.path.join(OUT_DIR, "ec_eo_prediction_results.csv")
OUT_FIG   = os.path.join(OUT_DIR, "ec_eo_auc.png")

# --- channel handling ---
CH_TXT    = os.path.join(ROOT, r"eeg_rest\subject_01_EC.channels.txt")
def clean_label(x: str) -> str:
    y = x.strip()
    y = re.sub(r"(?i)^(EEG|MEG|EOG|ECG|EMG)[\s_\-]+", "", y)
    y = re.sub(r"(?i)[\s_\-]*(REF|AV|AVERAGE|AVG|M1|M2)$", "", y)
    y = re.sub(r"[ \-\.]+", "", y)
    y = y.replace("FP","Fp")
    return y
with open(CH_TXT, "r", encoding="utf-8") as f:
    ch_names = [clean_label(ln.strip()) for ln in f if ln.strip()]

LEFT_CANON = set(map(str.upper, [
    "Fp1","AF7","AF3","F7","F5","F3","F1","FT7","FC5","FC3","FC1","T7","C5","C3","C1",
    "TP7","CP5","CP3","CP1","P7","P5","P3","P1","PO7","PO3","O1"
]))
RIGHT_CANON = set(map(str.upper, [
    "Fp2","AF8","AF4","F8","F6","F4","F2","FT8","FC6","FC4","FC2","T8","C6","C4","C2",
    "TP8","CP6","CP4","CP2","P8","P6","P4","P2","PO8","PO4","O2"
]))
MID_CANON = set(map(str.upper, ["Fpz","AFz","Fz","FCz","Cz","CPz","Pz","POz","Oz"]))

def hemi_map(labels):
    L,R,Z = [],[],[]
    for i,ch in enumerate(labels):
        up = ch.upper()
        if up in LEFT_CANON:  L.append(i); continue
        if up in RIGHT_CANON: R.append(i); continue
        if up in MID_CANON:   Z.append(i); continue
        if re.search(r"[A-Za-z]z$", ch): Z.append(i); continue
        m = re.search(r"(\d+)$", ch)
        if m:
            try:
                d=int(m.group(1)); (L if d%2==1 else R).append(i); continue
            except: pass
        Z.append(i)
    return np.array(L,int), np.array(R,int), np.array(Z,int)

ANT_PREFIXES=("Fp","AF","F","FC"); MID_PREFIXES=("C",); POST_PREFIXES=("CP","P","PO","O","TP","FT")
def ap_map(labels):
    A,P,C=[],[],[]
    for i,ch in enumerate(labels):
        pref=re.match(r"[A-Za-z]+", ch); pref=pref.group(0) if pref else ""
        if any(pref.startswith(px) for px in ANT_PREFIXES): A.append(i)
        elif any(pref.startswith(px) for px in POST_PREFIXES): P.append(i)
        elif any(pref.startswith(px) for px in MID_PREFIXES): C.append(i)
        else: C.append(i)
    return np.array(A,int), np.array(P,int), np.array(C,int)

L_idx, R_idx, Z_idx = hemi_map(ch_names)
A_idx, P_idx, C_idx = ap_map(ch_names)

# --- PLI + spectral ---
def bandpass(x, fs, lo, hi, order=4):
    b,a = butter(order, [lo/(fs/2), hi/(fs/2)], btype="band"); return filtfilt(b,a,x)
def pli_matrix(X, fs, lo, hi):
    n = X.shape[0]
    b,a = butter(4, [lo/(fs/2), hi/(fs/2)], btype="band")
    Y = np.zeros_like(X)
    for c in range(n): Y[c] = filtfilt(b,a,X[c])
    ph = np.angle(hilbert(Y, axis=1))
    W  = np.zeros((n,n), float)
    for i in range(n):
        for j in range(i+1, n):
            dphi = ph[i]-ph[j]; pli = np.abs(np.mean(np.sign(np.sin(dphi))))
            W[i,j] = W[j,i] = pli
    np.fill_diagonal(W, 0.0)
    return W
def knn(W,k):
    W=W.copy(); n=W.shape[0]
    for i in range(n):
        idx=np.argsort(W[i])[::-1]; keep=idx[:k]
        mask=np.ones(n,bool); mask[keep]=False; W[i,mask]=0
    W=np.maximum(W,W.T); np.fill_diagonal(W,0); return W
def lap(W):
    d=W.sum(1); d=np.where(d<=1e-12,1.0,d); D=np.diag(1.0/np.sqrt(d))
    return np.eye(W.shape[0]) - D@W@D
def spec_labels(W,k=2):
    e,v = np.linalg.eigh(lap(W)); U=v[:,1:k] if k>1 else v[:,:1]
    U/= (np.linalg.norm(U, axis=1, keepdims=True) + 1e-12)
    return KMeans(n_clusters=k, n_init=50, random_state=42).fit_predict(U)

def mean_safe(x): return float(np.nan) if x.size==0 else float(x.mean())
def within_region_ratio(W, labels, idx):
    if idx.size<3: return np.nan
    sub=np.ix_(idx,idx); Wr=W[sub]; lab=labels[idx]
    same=lab[:,None]==lab[None,:]; diff=~same
    np.fill_diagonal(same,False); np.fill_diagonal(diff,False)
    m_same=mean_safe(Wr[same]); m_diff=mean_safe(Wr[diff])
    return float(m_same/(m_diff+1e-12))
def hemi_module_means(W, labels, L, R):
    n=len(labels)
    Lmask=np.zeros((n,n),bool); Lmask[np.ix_(L,L)]=True
    Rmask=np.zeros((n,n),bool); Rmask[np.ix_(R,R)]=True
    within=Lmask|Rmask
    cross=np.zeros((n,n),bool); cross[np.ix_(L,R)]=True; cross[np.ix_(R,L)]=True
    same=labels[:,None]==labels[None,:]
    for m in (within,cross,same): np.fill_diagonal(m,False)
    return mean_safe(W[within&same]), mean_safe(W[cross&same])
def AI_AP_mod0(W, labels, A, P):
    m=0; mask=(labels==m); deg=W[:,mask].sum(axis=1)
    Asum=float(deg[A].sum()) if A.size else 0.0
    Psum=float(deg[P].sum()) if P.size else 0.0
    return (Asum-Psum)/(Asum+Psum+1e-12)

def subject_features(subject_id, cond_tag):
    """Return feature dict per band for one subject and condition."""
    feats = {}
    for band,(lo,hi) in BANDS_HZ.items():
        f = os.path.join(DATA_DIR, f"subject_{subject_id:02d}_{cond_tag}.npy")
        if not os.path.exists(f):
            return None
        X = np.load(f)
        W = pli_matrix(X, FS, lo, hi); W = knn(W, KNN_K); lbl = spec_labels(W, k=K_FIXED)
        feats[f"{band}_Lratio"] = within_region_ratio(W, lbl, L_idx)
        feats[f"{band}_Rratio"] = within_region_ratio(W, lbl, R_idx)
        whwm, chwm = hemi_module_means(W, lbl, L_idx, R_idx)
        feats[f"{band}_WHWM"]   = whwm
        feats[f"{band}_CHWM"]   = chwm
        feats[f"{band}_AIAP0"]  = AI_AP_mod0(W, lbl, A_idx, P_idx)
    return feats

# Build dataset
X, y, subj_ids = [], [], []
for s in SUBJECTS:
    f_ec = subject_features(s, "EC")
    f_eo = subject_features(s, "EO")
    if f_ec:
        X.append(list(f_ec.values())); y.append(0); subj_ids.append(s)
    if f_eo:
        X.append(list(f_eo.values())); y.append(1); subj_ids.append(s)

feat_names = list(f_ec.keys()) if SUBJECTS and f_ec else []
X = np.array(X, float); y = np.array(y, int); subj_ids = np.array(subj_ids, int)

# Train-test with paired CV: leave-one-subject-pair-out (EC and EO for subject s)
rng = np.random.default_rng(7)
clf = make_pipeline(StandardScaler(), LogisticRegression(max_iter=2000, solver="lbfgs"))
# You can switch to LinearSVC() if desired.

# Paired folds
fold_scores = []
for s in SUBJECTS:
    test = (subj_ids == s)
    train = ~test
    if np.sum(test) != 2:  # need both EC & EO for subject
        continue
    clf.fit(X[train], y[train])
    prob = clf.predict_proba(X[test])[:,1] if hasattr(clf[-1], "predict_proba") else clf.decision_function(X[test])
    # AUC on the pair: trivial (2 points), so accumulate scores differently:
    # Collect all test logits and labels, compute AUC after full CV:
    fold_scores.append((prob, y[test]))

# Aggregate AUC across all test points
if not fold_scores:
    raise SystemExit("No paired folds available; check EC/EO NPYs.")
probs = np.concatenate([p for p,_ in fold_scores])
ys    = np.concatenate([t for _,t in fold_scores])
# AUC over all test points
def auc_from_scores(p, y):
    from sklearn.metrics import roc_auc_score
    # handle degenerate cases
    if len(np.unique(y))<2: return np.nan
    return float(roc_auc_score(y, p))
auc_obs = auc_from_scores(probs, ys)

# Permutation test (subject-paired label flips): swap EC/EO labels per subject randomly
PERM_B = 10000
cnt = 1  # include observed
for _ in range(PERM_B):
    # random flips per subject
    flips = (rng.random(len(SUBJECTS)) < 0.5).astype(int)
    y_perm = []
    p_perm = []
    idx = 0
    for s in SUBJECTS:
        # skip subjects without both entries
        mask = np.where(subj_ids == s)[0]
        if mask.size != 2: 
            continue
        # original order in fold_scores concatenation matches subj_ids[test] order:
        # We built probs/ys by concatenating test predictions in SUBJECT order, so replicate the same
        # Here we just reshuffle labels for the two entries:
        # Assign: if flip==1, swap the two labels; else keep.
        if flips[idx]==1:
            y_pair = ys[np.isin(np.arange(len(ys)), np.where(subj_ids==s)[0])][::-1]
            p_pair = probs[np.isin(np.arange(len(probs)), np.where(subj_ids==s)[0])]
        else:
            y_pair = ys[np.isin(np.arange(len(ys)), np.where(subj_ids==s)[0])]
            p_pair = probs[np.isin(np.arange(len(probs)), np.where(subj_ids==s)[0])]
        y_perm.append(y_pair)
        p_perm.append(p_pair)
        idx += 1
    y_perm = np.concatenate(y_perm); p_perm = np.concatenate(p_perm)
    a = auc_from_scores(p_perm, y_perm)
    if np.isfinite(a) and a >= auc_obs:
        cnt += 1
p_auc = float(cnt / (PERM_B + 1))

# Save results
res = pd.DataFrame([{"AUC_obs": auc_obs, "p_perm": p_auc, "N_test_points": len(ys), "N_subjects": len(SUBJECTS)}])
res.to_csv(OUT_CSV, index=False)

print("=== EC vs EO classifier from PLI field metrics ===")
print(res.to_string(index=False))
print("Saved:", OUT_CSV)
plt.figure()
plt.bar(["AUC"], [auc_obs])
plt.ylim(0.5, 1.0)
plt.title(f"AUC={auc_obs:.3f} (perm p={p_auc:.4f})")
plt.tight_layout()
plt.savefig(OUT_FIG, dpi=160); plt.close()
print("Figure:", OUT_FIG)


ValueError: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [12]:
# === PATCH: EC vs EO classifier with NaN-safe pipeline + paired permutation AUC ===
import os, re, glob, json, numpy as np, pandas as pd
from scipy.signal import butter, filtfilt, hilbert
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

ROOT      = r"C:\Users\caleb\CNT_Lab"
DATA_DIR  = os.path.join(ROOT, "eeg_rest")
SUBJECTS  = list(range(1,31))
FS        = 250.0
BANDS_HZ  = {"alpha": (8.0,13.0), "theta": (4.0,8.0), "beta": (13.0,30.0)}
K_FIXED   = 2
KNN_K     = 6
OUT_DIR   = os.path.join(ROOT, "artifacts", "pli_ec_eo_prediction")
os.makedirs(OUT_DIR, exist_ok=True)
OUT_CSV   = os.path.join(OUT_DIR, "ec_eo_prediction_results_fixed.csv")
OUT_FIG   = os.path.join(OUT_DIR, "ec_eo_auc_fixed.png")
CH_TXT    = os.path.join(ROOT, r"eeg_rest\subject_01_EC.channels.txt")

# --- channel maps (same as before) ---
def clean_label(x: str) -> str:
    y = x.strip()
    y = re.sub(r"(?i)^(EEG|MEG|EOG|ECG|EMG)[\s_\-]+", "", y)
    y = re.sub(r"(?i)[\s_\-]*(REF|AV|AVERAGE|AVG|M1|M2)$", "", y)
    y = re.sub(r"[ \-\.]+", "", y)
    y = y.replace("FP","Fp")
    return y
with open(CH_TXT, "r", encoding="utf-8") as f:
    ch_names = [clean_label(ln.strip()) for ln in f if ln.strip()]

LEFT_CANON = set(map(str.upper, [
    "Fp1","AF7","AF3","F7","F5","F3","F1","FT7","FC5","FC3","FC1","T7","C5","C3","C1",
    "TP7","CP5","CP3","CP1","P7","P5","P3","P1","PO7","PO3","O1"
]))
RIGHT_CANON = set(map(str.upper, [
    "Fp2","AF8","AF4","F8","F6","F4","F2","FT8","FC6","FC4","FC2","T8","C6","C4","C2",
    "TP8","CP6","CP4","CP2","P8","P6","P4","P2","PO8","PO4","O2"
]))
MID_CANON = set(map(str.upper, ["Fpz","AFz","Fz","FCz","Cz","CPz","Pz","POz","Oz"]))

def hemi_map(labels):
    L,R,Z = [],[],[]
    for i,ch in enumerate(labels):
        up = ch.upper()
        if up in LEFT_CANON:  L.append(i); continue
        if up in RIGHT_CANON: R.append(i); continue
        if up in MID_CANON:   Z.append(i); continue
        if re.search(r"[A-Za-z]z$", ch): Z.append(i); continue
        m = re.search(r"(\d+)$", ch)
        if m:
            try:
                d=int(m.group(1)); (L if d%2==1 else R).append(i); continue
            except: pass
        Z.append(i)
    return np.array(L,int), np.array(R,int), np.array(Z,int)

ANT_PREFIXES=("Fp","AF","F","FC"); MID_PREFIXES=("C",); POST_PREFIXES=("CP","P","PO","O","TP","FT")
def ap_map(labels):
    A,P,C=[],[],[]
    for i,ch in enumerate(labels):
        pref=re.match(r"[A-Za-z]+", ch); pref=pref.group(0) if pref else ""
        if any(pref.startswith(px) for px in ANT_PREFIXES): A.append(i)
        elif any(pref.startswith(px) for px in POST_PREFIXES): P.append(i)
        elif any(pref.startswith(px) for px in MID_PREFIXES): C.append(i)
        else: C.append(i)
    return np.array(A,int), np.array(P,int), np.array(C,int)

L_idx, R_idx, Z_idx = hemi_map(ch_names)
A_idx, P_idx, C_idx = ap_map(ch_names)

# --- feature extractor (same as before) ---
from sklearn.cluster import KMeans
def bandpass(x, fs, lo, hi, order=4):
    b,a = butter(order, [lo/(fs/2), hi/(fs/2)], btype="band"); return filtfilt(b,a,x)
def pli_matrix(X, fs, lo, hi):
    n = X.shape[0]
    b,a = butter(4, [lo/(fs/2), hi/(fs/2)], btype="band")
    Y = np.zeros_like(X)
    for c in range(n): Y[c] = filtfilt(b,a,X[c])
    ph = np.angle(hilbert(Y, axis=1))
    W  = np.zeros((n,n), float)
    for i in range(n):
        for j in range(i+1, n):
            dphi = ph[i]-ph[j]; pli = np.abs(np.mean(np.sign(np.sin(dphi))))
            W[i,j] = W[j,i] = pli
    np.fill_diagonal(W, 0.0)
    return W
def knn(W,k):
    W=W.copy(); n=W.shape[0]
    for i in range(n):
        idx=np.argsort(W[i])[::-1]; keep=idx[:k]
        mask=np.ones(n,bool); mask[keep]=False; W[i,mask]=0
    W=np.maximum(W,W.T); np.fill_diagonal(W,0); return W
def lap(W):
    d=W.sum(1); d=np.where(d<=1e-12,1.0,d); D=np.diag(1.0/np.sqrt(d))
    return np.eye(W.shape[0]) - D@W@D
def spec_labels(W,k=2):
    e,v = np.linalg.eigh(lap(W)); U=v[:,1:k] if k>1 else v[:,:1]
    U/= (np.linalg.norm(U, axis=1, keepdims=True) + 1e-12)
    return KMeans(n_clusters=k, n_init=50, random_state=42).fit_predict(U)
def mean_safe(x): return float(np.nan) if x.size==0 else float(x.mean())
def within_region_ratio(W, labels, idx):
    if idx.size<3: return np.nan
    sub=np.ix_(idx,idx); Wr=W[sub]; lab=labels[idx]
    same=lab[:,None]==lab[None,:]; diff=~same
    np.fill_diagonal(same,False); np.fill_diagonal(diff,False)
    m_same=mean_safe(Wr[same]); m_diff=mean_safe(Wr[diff])
    return float(m_same/(m_diff+1e-12))
def hemi_module_means(W, labels, L, R):
    n=len(labels)
    Lmask=np.zeros((n,n),bool); Lmask[np.ix_(L,L)]=True
    Rmask=np.zeros((n,n),bool); Rmask[np.ix_(R,R)]=True
    within=Lmask|Rmask
    cross=np.zeros((n,n),bool); cross[np.ix_(L,R)]=True; cross[np.ix_(R,L)]=True
    same=labels[:,None]==labels[None,:]
    for m in (within,cross,same): np.fill_diagonal(m,False)
    return mean_safe(W[within&same]), mean_safe(W[cross&same])
def AI_AP_mod0(W, labels, A, P):
    m=0; mask=(labels==m); deg=W[:,mask].sum(axis=1)
    Asum=float(deg[A].sum()) if A.size else 0.0
    Psum=float(deg[P].sum()) if P.size else 0.0
    return (Asum-Psum)/(Asum+Psum+1e-12)
def subject_features(subject_id, cond_tag):
    feats={}
    for band,(lo,hi) in BANDS_HZ.items():
        f=os.path.join(DATA_DIR, f"subject_{subject_id:02d}_{cond_tag}.npy")
        if not os.path.exists(f): return None
        X=np.load(f); W=pli_matrix(X, FS, lo, hi); W=knn(W, KNN_K); lbl=spec_labels(W, k=K_FIXED)
        feats[f"{band}_Lratio"]=within_region_ratio(W, lbl, L_idx)
        feats[f"{band}_Rratio"]=within_region_ratio(W, lbl, R_idx)
        whwm,chwm=hemi_module_means(W, lbl, L_idx, R_idx)
        feats[f"{band}_WHWM"]=whwm; feats[f"{band}_CHWM"]=chwm
        feats[f"{band}_AIAP0"]=AI_AP_mod0(W, lbl, A_idx, P_idx)
    return feats

# Build dataset
X, y, subj_ids = [], [], []
for s in SUBJECTS:
    f_ec = subject_features(s, "EC")
    f_eo = subject_features(s, "EO")
    if f_ec:
        X.append(list(f_ec.values())); y.append(0); subj_ids.append(s)
    if f_eo:
        X.append(list(f_eo.values())); y.append(1); subj_ids.append(s)
feat_names = list(f_ec.keys()) if SUBJECTS and f_ec else []
X = np.array(X, float); y = np.array(y, int); subj_ids = np.array(subj_ids, int)

# Sanitize: replace inf with nan; model pipeline will impute median on train folds
X[~np.isfinite(X)] = np.nan

# Paired CV with NaN-safe pipeline
from numpy.random import default_rng
rng = default_rng(7)
clf = make_pipeline(
    SimpleImputer(strategy="median"),
    StandardScaler(with_mean=True),
    LogisticRegression(max_iter=2000, solver="lbfgs")
)

fold_probs = []; fold_true = []
for s in SUBJECTS:
    test_mask = (subj_ids == s)
    if np.sum(test_mask) != 2:  # need both EC & EO
        continue
    train_mask = ~test_mask
    clf.fit(X[train_mask], y[train_mask])  # imputer fit on train only
    # decision scores
    if hasattr(clf[-1], "predict_proba"):
        p = clf.predict_proba(X[test_mask])[:,1]
    else:
        p = clf.decision_function(X[test_mask])
    fold_probs.append(p); fold_true.append(y[test_mask])

probs = np.concatenate(fold_probs); ys = np.concatenate(fold_true)

def auc_from_scores(p, y):
    from sklearn.metrics import roc_auc_score
    if len(np.unique(y))<2: return np.nan
    return float(roc_auc_score(y, p))
auc_obs = auc_from_scores(probs, ys)

# Permutation (paired label flips per subject)
PERM_B = 10000
cnt = 1
for _ in range(PERM_B):
    y_perm = ys.copy()
    # swap labels within each subject pair randomly
    idx = 0
    for s in SUBJECTS:
        # locate this subject's two entries in the concatenated arrays
        # We concatenated in SUBJECT order, two points per subject, so:
        if idx*2+2 > len(ys): break
        if rng.random() < 0.5:
            # flip the two labels
            a, b = idx*2, idx*2+1
            y_perm[[a,b]] = y_perm[[b,a]]
        idx += 1
    a = auc_from_scores(probs, y_perm)
    if np.isfinite(a) and a >= auc_obs:
        cnt += 1
p_auc = float(cnt / (PERM_B + 1))

res = pd.DataFrame([{"AUC_obs": auc_obs, "p_perm": p_auc, "N_test_points": len(ys), "N_subjects": len(SUBJECTS)}])
res.to_csv(OUT_CSV, index=False)
print("=== EC vs EO classifier (NaN-safe) ===")
print(res.to_string(index=False))
print("Saved:", OUT_CSV)

plt.figure()
plt.bar(["AUC"], [auc_obs])
plt.ylim(0.5, 1.0)
plt.title(f"AUC={auc_obs:.3f} (perm p={p_auc:.4f})")
plt.tight_layout(); plt.savefig(OUT_FIG, dpi=160); plt.close()
print("Figure:", OUT_FIG)


=== EC vs EO classifier (NaN-safe) ===
 AUC_obs  p_perm  N_test_points  N_subjects
0.775556  0.0001             60          30
Saved: C:\Users\caleb\CNT_Lab\artifacts\pli_ec_eo_prediction\ec_eo_prediction_results_fixed.csv
Figure: C:\Users\caleb\CNT_Lab\artifacts\pli_ec_eo_prediction\ec_eo_auc_fixed.png


In [13]:
# === EC vs EO — Ablation grid with paired-permutation AUC (single cell) ===
# Models:
#   1) Full (all bands × all features)
#   2) Band-only: alpha-only, theta-only, beta-only
#   3) Drop-one-band: no_alpha, no_theta, no_beta
#   4) Family-only: ratios_only, coupling_only, ai_only
#   5) Drop-one-family: no_ratios, no_coupling, no_ai
#
# Outputs:
#   C:\Users\caleb\CNT_Lab\artifacts\pli_ec_eo_prediction\ec_eo_prediction_ablation.csv
#   C:\Users\caleb\CNT_Lab\artifacts\pli_ec_eo_prediction\ec_eo_prediction_ablation.pdf

import os, re, glob, json, numpy as np, pandas as pd, matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt, hilbert
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

# -------- Paths & config --------
ROOT      = r"C:\Users\caleb\CNT_Lab"
DATA_DIR  = os.path.join(ROOT, "eeg_rest")
SUBJECTS  = list(range(1,30+1))
FS        = 250.0
BANDS_HZ  = {"alpha": (8.0,13.0), "theta": (4.0,8.0), "beta": (13.0,30.0)}
K_FIXED   = 2
KNN_K     = 6
PERM_B    = 10000

OUT_DIR   = os.path.join(ROOT, "artifacts", "pli_ec_eo_prediction")
os.makedirs(OUT_DIR, exist_ok=True)
OUT_CSV   = os.path.join(OUT_DIR, "ec_eo_prediction_ablation.csv")
OUT_PDF   = os.path.join(OUT_DIR, "ec_eo_prediction_ablation.pdf")

CH_TXT    = os.path.join(ROOT, r"eeg_rest\subject_01_EC.channels.txt")

# -------- Channel maps (same as earlier) --------
def clean_label(x: str) -> str:
    y = x.strip()
    y = re.sub(r"(?i)^(EEG|MEG|EOG|ECG|EMG)[\s_\-]+", "", y)
    y = re.sub(r"(?i)[\s_\-]*(REF|AV|AVERAGE|AVG|M1|M2)$", "", y)
    y = re.sub(r"[ \-\.]+", "", y)
    y = y.replace("FP","Fp")
    return y

with open(CH_TXT, "r", encoding="utf-8") as f:
    ch_names = [clean_label(ln.strip()) for ln in f if ln.strip()]

LEFT_CANON  = set(map(str.upper, ["Fp1","AF7","AF3","F7","F5","F3","F1","FT7","FC5","FC3","FC1","T7","C5","C3","C1",
                                  "TP7","CP5","CP3","CP1","P7","P5","P3","P1","PO7","PO3","O1"]))
RIGHT_CANON = set(map(str.upper, ["Fp2","AF8","AF4","F8","F6","F4","F2","FT8","FC6","FC4","FC2","T8","C6","C4","C2",
                                  "TP8","CP6","CP4","CP2","P8","P6","P4","P2","PO8","PO4","O2"]))
MID_CANON   = set(map(str.upper, ["Fpz","AFz","Fz","FCz","Cz","CPz","Pz","POz","Oz"]))

def hemi_map(labels):
    L,R,Z = [],[],[]
    for i,ch in enumerate(labels):
        up = ch.upper()
        if up in LEFT_CANON:  L.append(i); continue
        if up in RIGHT_CANON: R.append(i); continue
        if up in MID_CANON:   Z.append(i); continue
        if re.search(r"[A-Za-z]z$", ch): Z.append(i); continue
        m = re.search(r"(\d+)$", ch)
        if m:
            try:
                d=int(m.group(1)); (L if d%2==1 else R).append(i); continue
            except: pass
        Z.append(i)
    return np.array(L,int), np.array(R,int), np.array(Z,int)

ANT_PREFIXES=("Fp","AF","F","FC"); MID_PREFIXES=("C",); POST_PREFIXES=("CP","P","PO","O","TP","FT")
def ap_map(labels):
    A,P,C=[],[],[]
    for i,ch in enumerate(labels):
        pref=re.match(r"[A-Za-z]+", ch); pref=pref.group(0) if pref else ""
        if any(pref.startswith(px) for px in ANT_PREFIXES): A.append(i)
        elif any(pref.startswith(px) for px in POST_PREFIXES): P.append(i)
        elif any(pref.startswith(px) for px in MID_PREFIXES): C.append(i)
        else: C.append(i)
    return np.array(A,int), np.array(P,int), np.array(C,int)

L_idx, R_idx, Z_idx = hemi_map(ch_names)
A_idx, P_idx, C_idx = ap_map(ch_names)

# -------- Feature extraction --------
def bandpass(x, fs, lo, hi, order=4):
    b,a = butter(order, [lo/(fs/2), hi/(fs/2)], btype="band"); return filtfilt(b,a,x)
def pli_matrix(X, fs, lo, hi):
    n = X.shape[0]
    b,a = butter(4, [lo/(fs/2), hi/(fs/2)], btype="band")
    Y = np.zeros_like(X)
    for c in range(n): Y[c] = filtfilt(b,a,X[c])
    ph = np.angle(hilbert(Y, axis=1))
    W  = np.zeros((n,n), float)
    for i in range(n):
        for j in range(i+1, n):
            dphi = ph[i]-ph[j]
            pli  = np.abs(np.mean(np.sign(np.sin(dphi))))
            W[i,j] = W[j,i] = pli
    np.fill_diagonal(W, 0.0)
    return W
def knn(W,k):
    W=W.copy(); n=W.shape[0]
    for i in range(n):
        idx=np.argsort(W[i])[::-1]; keep=idx[:k]
        mask=np.ones(n,bool); mask[keep]=False; W[i,mask]=0
    W=np.maximum(W,W.T); np.fill_diagonal(W,0); return W
def lap(W):
    d=W.sum(1); d=np.where(d<=1e-12,1.0,d); D=np.diag(1.0/np.sqrt(d))
    return np.eye(W.shape[0]) - D@W@D
def spec_labels(W, k=2):
    e,v = np.linalg.eigh(lap(W))
    U = v[:,1:k] if k>1 else v[:,:1]
    U = U / (np.linalg.norm(U, axis=1, keepdims=True) + 1e-12)
    from sklearn.cluster import KMeans
    return KMeans(n_clusters=k, n_init=50, random_state=42).fit_predict(U)
def mean_safe(x): return float(np.nan) if x.size==0 else float(x.mean())
def within_region_ratio(W, labels, idx):
    if idx.size<3: return np.nan
    sub=np.ix_(idx,idx); Wr=W[sub]; lab=labels[idx]
    same=lab[:,None]==lab[None,:]; diff=~same
    np.fill_diagonal(same,False); np.fill_diagonal(diff,False)
    return float(mean_safe(Wr[same]) / (mean_safe(Wr[diff]) + 1e-12))
def hemi_module_means(W, labels, L, R):
    n=len(labels)
    Lmask=np.zeros((n,n),bool); Lmask[np.ix_(L,L)]=True
    Rmask=np.zeros((n,n),bool); Rmask[np.ix_(R,R)]=True
    within=Lmask|Rmask
    cross =np.zeros((n,n),bool); cross[np.ix_(L,R)]=True; cross[np.ix_(R,L)]=True
    same  =labels[:,None]==labels[None,:]
    for m in (within,cross,same): np.fill_diagonal(m,False)
    return mean_safe(W[within&same]), mean_safe(W[cross&same])
def AI_AP_mod0(W, labels, A, P):
    m=0; mask=(labels==m); deg=W[:,mask].sum(axis=1)
    Asum=float(deg[A].sum()) if A.size else 0.0
    Psum=float(deg[P].sum()) if P.size else 0.0
    return (Asum-Psum)/(Asum+Psum+1e-12)

def subject_features(subject_id, cond_tag):
    feats={}
    for band,(lo,hi) in BANDS_HZ.items():
        f=os.path.join(DATA_DIR, f"subject_{subject_id:02d}_{cond_tag}.npy")
        if not os.path.exists(f): return None
        X=np.load(f); W=pli_matrix(X, FS, lo, hi); W=knn(W, KNN_K); lbl=spec_labels(W, k=K_FIXED)
        feats[f"{band}_Lratio"]=within_region_ratio(W, lbl, L_idx)
        feats[f"{band}_Rratio"]=within_region_ratio(W, lbl, R_idx)
        whwm,chwm=hemi_module_means(W, lbl, L_idx, R_idx)
        feats[f"{band}_WHWM"]=whwm; feats[f"{band}_CHWM"]=chwm
        feats[f"{band}_AIAP0"]=AI_AP_mod0(W, lbl, A_idx, P_idx)
    return feats

# Build the full featureset
X_all, y_all, subj_all = [], [], []
for s in SUBJECTS:
    ec=subject_features(s,"EC"); eo=subject_features(s,"EO")
    if ec:
        X_all.append(ec); y_all.append(0); subj_all.append(s)
    if eo:
        X_all.append(eo); y_all.append(1); subj_all.append(s)

if not X_all:
    raise SystemExit("No features found; ensure EC/EO NPY exist.")

full_feat_names = sorted(list(X_all[0].keys()))
X_all = pd.DataFrame(X_all, columns=full_feat_names)
y_all = np.array(y_all, int)
subj_all = np.array(subj_all, int)

# Replace inf with NaN (imputer will handle)
X_all = X_all.replace([np.inf, -np.inf], np.nan)

# -------- Model specs --------
def feat_mask_by_spec(spec_name):
    bands = ["alpha","theta","beta"]
    families = ["Lratio","Rratio","WHWM","CHWM","AIAP0"]
    sel_bands = bands.copy()
    sel_fams  = families.copy()

    if spec_name == "full":
        pass
    elif spec_name in ("alpha_only","theta_only","beta_only"):
        sel_bands = [spec_name.split("_")[0]]
    elif spec_name in ("no_alpha","no_theta","no_beta"):
        drop = spec_name.split("_")[1]
        sel_bands = [b for b in bands if b != drop]
    elif spec_name == "ratios_only":
        sel_fams = ["Lratio","Rratio"]
    elif spec_name == "coupling_only":
        sel_fams = ["WHWM","CHWM"]
    elif spec_name == "ai_only":
        sel_fams = ["AIAP0"]
    elif spec_name == "no_ratios":
        sel_fams = [f for f in families if f not in ("Lratio","Rratio")]
    elif spec_name == "no_coupling":
        sel_fams = [f for f in families if f not in ("WHWM","CHWM")]
    elif spec_name == "no_ai":
        sel_fams = [f for f in families if f not in ("AIAP0",)]
    else:
        raise ValueError("Unknown spec: " + spec_name)

    keep_cols = []
    for b in sel_bands:
        for fam in sel_fams:
            col = f"{b}_{fam}"
            if col in X_all.columns:
                keep_cols.append(col)
    return sorted(keep_cols)

SPECS = [
    "full",
    "alpha_only","theta_only","beta_only",
    "no_alpha","no_theta","no_beta",
    "ratios_only","coupling_only","ai_only",
    "no_ratios","no_coupling","no_ai",
]

# -------- Paired-CV AUC & permutation --------
from numpy.random import default_rng
rng = default_rng(7)

def paired_auc_perm(Xdf, y, subj_ids, perm_B=PERM_B):
    # pipeline with imputer
    clf = make_pipeline(SimpleImputer(strategy="median"), StandardScaler(), LogisticRegression(max_iter=2000, solver="lbfgs"))
    fold_probs=[]; fold_true=[]
    for s in SUBJECTS:
        test = (subj_ids == s)
        if np.sum(test) != 2:  # need both EC & EO
            continue
        train = ~test
        clf.fit(Xdf[train], y[train])
        if hasattr(clf[-1], "predict_proba"):
            p = clf.predict_proba(Xdf[test])[:,1]
        else:
            p = clf.decision_function(Xdf[test])
        fold_probs.append(p); fold_true.append(y[test])
    if not fold_probs:
        return np.nan, np.nan, 0
    probs = np.concatenate(fold_probs); ys = np.concatenate(fold_true)
    # compute observed AUC
    if len(np.unique(ys))<2:
        return np.nan, np.nan, len(ys)
    auc_obs = float(roc_auc_score(ys, probs))
    # permutation: flip labels within each subject pair
    cnt=1
    for _ in range(perm_B):
        y_perm = ys.copy()
        idx=0
        for s in SUBJECTS:
            # two entries per subject in concatenation order
            if idx*2+2 > len(ys): break
            if rng.random() < 0.5:
                a,b = idx*2, idx*2+1
                y_perm[[a,b]] = y_perm[[b,a]]
            idx += 1
        auc_p = float(roc_auc_score(y_perm, probs))
        if auc_p >= auc_obs:
            cnt += 1
    p_val = float(cnt / (perm_B + 1))
    return auc_obs, p_val, len(ys)

# Run grid
results=[]
for spec in SPECS:
    cols = feat_mask_by_spec(spec)
    if not cols:
        results.append({"model": spec, "n_features": 0, "AUC": np.nan, "p_perm": np.nan})
        continue
    Xsel = X_all[cols].to_numpy(dtype=float)
    auc, pval, ntest = paired_auc_perm(Xsel, y_all, subj_all, perm_B=PERM_B)
    results.append({"model": spec, "n_features": len(cols), "AUC": auc, "p_perm": pval})

df = pd.DataFrame(results).sort_values(by=["model"]).reset_index(drop=True)
df.to_csv(OUT_CSV, index=False)
print("=== EC vs EO — Ablation AUC (paired, perm test) ===")
print(df.to_string(index=False))
print("Saved CSV:", OUT_CSV)

# -------- 1-page PDF: bar chart (AUC) with p-values --------
from matplotlib.backends.backend_pdf import PdfPages
fig = plt.figure(figsize=(11, 8.5))
ax = fig.add_subplot(111)
xs = np.arange(len(df))
ax.bar(xs, df["AUC"].fillna(0.5))
ax.set_xticks(xs, df["model"], rotation=45, ha="right")
ax.set_ylim(0.5, 1.0)
for i, (auc, p, nf) in enumerate(zip(df["AUC"], df["p_perm"], df["n_features"])):
    txt = f"AUC={auc:.3f}\np={p:.4f}\nF={nf}"
    ax.text(i, min(0.98, (0.5 if np.isnan(auc) else auc)+0.02), txt, ha="center", va="bottom", fontsize=8)
ax.set_title("EC vs EO — Ablation AUC (paired CV) with perm p-values")
fig.tight_layout()
pp = PdfPages(OUT_PDF); pp.savefig(fig, dpi=200); pp.close(); plt.close(fig)
print("Saved PDF:", OUT_PDF)


=== EC vs EO — Ablation AUC (paired, perm test) ===
        model  n_features      AUC   p_perm
      ai_only           3 0.657778 0.013599
   alpha_only           5 0.728889 0.000100
    beta_only           5 0.582222 0.027197
coupling_only           6 0.814444 0.000100
         full          15 0.775556 0.000100
        no_ai          12 0.762222 0.000100
     no_alpha          10 0.677778 0.003600
      no_beta          10 0.821111 0.000100
  no_coupling           9 0.668889 0.005299
    no_ratios           9 0.847778 0.000100
     no_theta          10 0.681111 0.001700
  ratios_only           6 0.517778 0.387561
   theta_only           5 0.668889 0.000600
Saved CSV: C:\Users\caleb\CNT_Lab\artifacts\pli_ec_eo_prediction\ec_eo_prediction_ablation.csv
Saved PDF: C:\Users\caleb\CNT_Lab\artifacts\pli_ec_eo_prediction\ec_eo_prediction_ablation.pdf


In [14]:
# === CNT Minimal EC vs EO Classifier (Pre-registered) — α/θ coupling only ===
# Features (per subject, per condition): 4 total
#   alpha_WHWM, alpha_CHWM, theta_WHWM, theta_CHWM
# Model: Imputer(median) → StandardScaler → LogisticRegression (lbfgs, max_iter=2000)
# Eval: Paired-CV (leave one subject pair out), AUC over concatenated test points
# Null: 10,000 paired label-flip permutations for two-sided p
#
# Outputs:
#   C:\Users\caleb\CNT_Lab\artifacts\pli_ec_eo_prediction\ec_eo_minimal_prereg.csv
#   C:\Users\caleb\CNT_Lab\artifacts\pli_ec_eo_prediction\ec_eo_minimal_prereg.pdf

import os, re, json, numpy as np, pandas as pd, matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt, hilbert
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve
from numpy.random import default_rng
from matplotlib.backends.backend_pdf import PdfPages

# -------- Paths & config --------
ROOT      = r"C:\Users\caleb\CNT_Lab"
DATA_DIR  = os.path.join(ROOT, "eeg_rest")  # subject_##_{EC|EO}.npy
SUBJECTS  = list(range(1, 30+1))
FS        = 250.0

# Bands (pre-registered): alpha + theta only
BANDS_HZ  = {"alpha": (8.0, 13.0), "theta": (4.0, 8.0)}

# Graph/clustering params
K_FIXED   = 2        # spectral k
KNN_K     = 6        # sparsity per node

# Permutations for paired test
PERM_B    = 10000
RNG       = default_rng(7)

# Outputs
OUT_DIR   = os.path.join(ROOT, "artifacts", "pli_ec_eo_prediction")
os.makedirs(OUT_DIR, exist_ok=True)
OUT_CSV   = os.path.join(OUT_DIR, "ec_eo_minimal_prereg.csv")
OUT_PDF   = os.path.join(OUT_DIR, "ec_eo_minimal_prereg.pdf")

# Channel names (for hemi maps used by coupling metrics)
CH_TXT    = os.path.join(ROOT, r"eeg_rest\subject_01_EC.channels.txt")

# -------- Channel handling (Left/Right; for WH_WM & CH_WM masks) --------
def clean_label(x: str) -> str:
    y = x.strip()
    y = re.sub(r"(?i)^(EEG|MEG|EOG|ECG|EMG)[\s_\-]+", "", y)
    y = re.sub(r"(?i)[\s_\-]*(REF|AV|AVERAGE|AVG|M1|M2)$", "", y)
    y = re.sub(r"[ \-\.]+", "", y)
    y = y.replace("FP","Fp")
    return y

with open(CH_TXT, "r", encoding="utf-8") as f:
    ch_names = [clean_label(ln.strip()) for ln in f if ln.strip()]

LEFT_CANON  = set(map(str.upper, [
    "Fp1","AF7","AF3","F7","F5","F3","F1","FT7","FC5","FC3","FC1","T7","C5","C3","C1",
    "TP7","CP5","CP3","CP1","P7","P5","P3","P1","PO7","PO3","O1"
]))
RIGHT_CANON = set(map(str.upper, [
    "Fp2","AF8","AF4","F8","F6","F4","F2","FT8","FC6","FC4","FC2","T8","C6","C4","C2",
    "TP8","CP6","CP4","CP2","P8","P6","P4","P2","PO8","PO4","O2"
]))
MID_CANON   = set(map(str.upper, ["Fpz","AFz","Fz","FCz","Cz","CPz","Pz","POz","Oz"]))

def hemi_map(labels):
    L,R,Z = [],[],[]
    for i,ch in enumerate(labels):
        up = ch.upper()
        if up in LEFT_CANON:  L.append(i); continue
        if up in RIGHT_CANON: R.append(i); continue
        if up in MID_CANON:   Z.append(i); continue
        if re.search(r"[A-Za-z]z$", ch): Z.append(i); continue
        m = re.search(r"(\d+)$", ch)
        if m:
            try:
                d=int(m.group(1)); (L if d%2==1 else R).append(i); continue
            except: pass
        Z.append(i)
    return np.array(L,int), np.array(R,int), np.array(Z,int)

L_idx, R_idx, Z_idx = hemi_map(ch_names)

# -------- PLI + spectral clustering --------
from sklearn.cluster import KMeans

def bandpass(x, fs, lo, hi, order=4):
    b,a = butter(order, [lo/(fs/2), hi/(fs/2)], btype="band"); return filtfilt(b,a,x)

def pli_matrix(X, fs, lo, hi):
    n = X.shape[0]
    b,a = butter(4, [lo/(fs/2), hi/(fs/2)], btype="band")
    Y = np.zeros_like(X)
    for c in range(n): Y[c] = filtfilt(b,a,X[c])
    ph = np.angle(hilbert(Y, axis=1))
    W  = np.zeros((n,n), float)
    for i in range(n):
        for j in range(i+1, n):
            dphi = ph[i]-ph[j]
            pli  = np.abs(np.mean(np.sign(np.sin(dphi))))
            W[i,j] = W[j,i] = pli
    np.fill_diagonal(W, 0.0)
    return W

def knn(W, k):
    W = W.copy(); n = W.shape[0]
    for i in range(n):
        idx = np.argsort(W[i])[::-1]; keep = idx[:k]
        mask = np.ones(n, bool); mask[keep] = False
        W[i,mask] = 0.0
    W = np.maximum(W,W.T); np.fill_diagonal(W,0)
    return W

def lap(W):
    d=W.sum(1); d=np.where(d<=1e-12,1.0,d); D=np.diag(1.0/np.sqrt(d))
    return np.eye(W.shape[0]) - D@W@D

def spec_labels(W, k=2):
    e,v = np.linalg.eigh(lap(W))
    U = v[:,1:k] if k>1 else v[:,:1]
    U = U / (np.linalg.norm(U, axis=1, keepdims=True) + 1e-12)
    return KMeans(n_clusters=k, n_init=50, random_state=42).fit_predict(U)

# -------- Coupling features (WH_WM & CH_WM) --------
def hemi_module_means(W, labels, L, R):
    """Return within-hemi within-module (WH_WM) and cross-hemi within-module (CH_WM)."""
    n=len(labels)
    Lmask=np.zeros((n,n),bool); Lmask[np.ix_(L,L)]=True
    Rmask=np.zeros((n,n),bool); Rmask[np.ix_(R,R)]=True
    within=Lmask|Rmask
    cross =np.zeros((n,n),bool); cross[np.ix_(L,R)]=True; cross[np.ix_(R,L)]=True
    same  =labels[:,None]==labels[None,:]
    for m in (within,cross,same): np.fill_diagonal(m,False)
    WH_WM = float(W[within & same].mean()) if np.any(within & same) else np.nan
    CH_WM = float(W[cross  & same].mean()) if np.any(cross  & same) else np.nan
    return WH_WM, CH_WM

def subject_features_coupling(subject_id, cond_tag):
    """Minimal prereg features for a subject/condition: α/θ coupling only."""
    f = os.path.join(DATA_DIR, f"subject_{subject_id:02d}_{cond_tag}.npy")
    if not os.path.exists(f): return None
    X = np.load(f)
    feats = {}
    for band,(lo,hi) in BANDS_HZ.items():
        W   = pli_matrix(X, FS, lo, hi)
        W   = knn(W, KNN_K)
        lbl = spec_labels(W, k=K_FIXED)
        WHWM, CHWM = hemi_module_means(W, lbl, L_idx, R_idx)
        feats[f"{band}_WHWM"] = WHWM
        feats[f"{band}_CHWM"] = CHWM
    return feats

# -------- Build dataset (features, labels, subject IDs) --------
X_list, y_list, subj_list = [], [], []
for s in SUBJECTS:
    ec = subject_features_coupling(s, "EC")
    eo = subject_features_coupling(s, "EO")
    if ec:
        X_list.append(ec); y_list.append(0); subj_list.append(s)
    if eo:
        X_list.append(eo); y_list.append(1); subj_list.append(s)

if not X_list:
    raise SystemExit("No features built. Ensure EC/EO NPY exist under eeg_rest.")

feat_names = list(X_list[0].keys())
X = pd.DataFrame(X_list, columns=feat_names).replace([np.inf,-np.inf], np.nan).to_numpy(dtype=float)
y = np.array(y_list, int)
subj_ids = np.array(subj_list, int)

# -------- Paired-CV predictions --------
clf = make_pipeline(SimpleImputer(strategy="median"), StandardScaler(), LogisticRegression(max_iter=2000, solver="lbfgs"))

all_probs, all_true = [], []
order_pairs = []  # track the test indices order per subject for permutation alignment

for s in SUBJECTS:
    test = (subj_ids == s)
    if np.sum(test) != 2:  # must have both EC & EO
        continue
    train = ~test
    clf.fit(X[train], y[train])
    # Scores
    if hasattr(clf[-1], "predict_proba"):
        p = clf.predict_proba(X[test])[:,1]
    else:
        p = clf.decision_function(X[test])
    all_probs.append(p); all_true.append(y[test])
    order_pairs.append(np.where(test)[0])

probs = np.concatenate(all_probs)
ys    = np.concatenate(all_true)

if len(np.unique(ys)) < 2:
    raise SystemExit("Degenerate labels in test folds; cannot compute AUC.")

auc_obs = float(roc_auc_score(ys, probs))

# -------- Paired permutation p-value (flip EC/EO labels per subject) --------
cnt = 1  # include observed
for _ in range(PERM_B):
    y_perm = ys.copy()
    # flip within each subject's two test points at random
    idx = 0
    for s in SUBJECTS:
        if idx >= len(order_pairs): break
        pair_idx = order_pairs[idx]
        if len(pair_idx) != 2:
            idx += 1; continue
        if RNG.random() < 0.5:
            y_perm[pair_idx] = y_perm[pair_idx][::-1]
        idx += 1
    auc_p = float(roc_auc_score(y_perm, probs))
    if auc_p >= auc_obs:
        cnt += 1
p_auc = float(cnt / (PERM_B + 1))

# -------- Save CSV --------
res = pd.DataFrame([{
    "model": "minimal_prereg_alpha_theta_coupling",
    "features": ",".join(feat_names),
    "n_features": len(feat_names),
    "AUC_obs": auc_obs,
    "p_perm": p_auc,
    "N_test_points": len(ys),
    "N_subjects": len(set(subj_ids))
}])
res.to_csv(OUT_CSV, index=False)
print("=== Minimal EC vs EO (α/θ coupling only) ===")
print(res.to_string(index=False))
print("Saved CSV:", OUT_CSV)

# -------- Make 1-page PDF: ROC + bar + model details --------
fpr, tpr, _ = roc_curve(ys, probs)

fig = plt.figure(figsize=(11, 8.5))

# Title
ax_t = fig.add_subplot(221)
ax_t.axis("off")
ax_t.text(0.5, 0.6, "CNT Minimal Classifier — α/θ Coupling Only", ha="center", va="center", fontsize=16, weight="bold")
ax_t.text(0.5, 0.25, f"AUC = {auc_obs:.3f}   (paired perm p = {p_auc:.4f})\nN subjects = {len(set(subj_ids))},  N test points = {len(ys)}",
          ha="center", va="center", fontsize=11)

# ROC curve
ax_roc = fig.add_subplot(222)
ax_roc.plot(fpr, tpr, lw=2)
ax_roc.plot([0,1],[0,1], linestyle="--")
ax_roc.set_xlim(0,1); ax_roc.set_ylim(0,1)
ax_roc.set_xlabel("False Positive Rate"); ax_roc.set_ylabel("True Positive Rate")
ax_roc.set_title("ROC — EC vs EO")

# AUC bar
ax_bar = fig.add_subplot(223)
ax_bar.bar(["Minimal α/θ coupling"], [auc_obs])
ax_bar.set_ylim(0.5, 1.0)
ax_bar.set_title("AUC")
ax_bar.text(0, min(0.98, auc_obs+0.02), f"{auc_obs:.3f}\np={p_auc:.4f}", ha="center", va="bottom")

# Feature list
ax_f = fig.add_subplot(224)
ax_f.axis("off")
ax_f.text(0.0, 0.9, "Pre-registered features:", fontsize=12, weight="bold")
ax_f.text(0.0, 0.75, "alpha_WHWM  (within-hemi, within-module)", fontsize=10)
ax_f.text(0.0, 0.65, "alpha_CHWM  (cross-hemi, within-module)", fontsize=10)
ax_f.text(0.0, 0.55, "theta_WHWM  (within-hemi, within-module)", fontsize=10)
ax_f.text(0.0, 0.45, "theta_CHWM  (cross-hemi, within-module)", fontsize=10)
ax_f.text(0.0, 0.25, "Pipeline: Imputer(median) → StandardScaler → LogisticRegression", fontsize=9)
ax_f.text(0.0, 0.15, f"Paired CV; label-flip perms = {PERM_B:,}", fontsize=9)

fig.tight_layout()
pp = PdfPages(OUT_PDF); pp.savefig(fig, dpi=200); pp.close(); plt.close(fig)
print("Saved PDF:", OUT_PDF)


=== Minimal EC vs EO (α/θ coupling only) ===
                              model                                    features  n_features  AUC_obs  p_perm  N_test_points  N_subjects
minimal_prereg_alpha_theta_coupling alpha_WHWM,alpha_CHWM,theta_WHWM,theta_CHWM           4 0.846667  0.0001             60          30
Saved CSV: C:\Users\caleb\CNT_Lab\artifacts\pli_ec_eo_prediction\ec_eo_minimal_prereg.csv
Saved PDF: C:\Users\caleb\CNT_Lab\artifacts\pli_ec_eo_prediction\ec_eo_minimal_prereg.pdf


In [15]:
# === CNT 4-in-1: Replication + Time Windows + Template Source Echo + Final PDF (single cell) ===
# What this cell produces:
#  A) Replication of the minimal EC/EO classifier:
#     - openneuro: download a small EO/EC set (you set REPL_DS), build α/θ coupling features, paired AUC + perm p.
#     - eegbci_holdout: random 20/10 subject splits on your EEGBCI (repeat=50), mean AUC + perm p.
#  B) Time-resolved EC vs EO: sliding windows (default 10s, step 5s), per-window paired AUC(t) + perm p (alpha/theta minimal features).
#  C) Template Source Echo (sensor→parcel proxy): parcel-level A/P & L/R asymmetries from consensus modules; save figure.
#  D) Final 1-page PDF summary fusing: prereg minimal ROC, ablation (if present), scalp maps (EC/EO), replication summary, AUC(t), and parcel echo.
#
# Prereqs already on disk from your prior runs:
#   - EEGBCI EC/EO NPY:   C:\Users\caleb\CNT_Lab\eeg_rest\subject_##_{EC|EO}.npy
#   - Channels:           C:\Users\caleb\CNT_Lab\eeg_rest\subject_01_EC.channels.txt
#   - Minimal prereg CSV: C:\Users\caleb\CNT_Lab\artifacts\pli_ec_eo_prediction\ec_eo_minimal_prereg.csv
#   - Ablation PDF/CSV  : C:\Users\caleb\CNT_Lab\artifacts\pli_ec_eo_prediction\ec_eo_prediction_ablation.pdf (optional)
#   - Scalp EC/EO figs  : C:\Users\caleb\CNT_Lab\artifacts\pli_30_subjects\figures_ec_eo_scalp\scalp__{band}__{EC,EO}.png

import os, re, glob, json, numpy as np, pandas as pd, matplotlib.pyplot as plt
from numpy.random import default_rng
from scipy.signal import butter, filtfilt, hilbert
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve
from matplotlib.backends.backend_pdf import PdfPages

# ------------------- CONFIG -------------------
ROOT         = r"C:\Users\caleb\CNT_Lab"
DATA_DIR     = os.path.join(ROOT, "eeg_rest")
CH_TXT       = os.path.join(ROOT, r"eeg_rest\subject_01_EC.channels.txt")
SUBJECTS     = list(range(1, 30+1))
FS           = 250.0
# Minimal prereg bands:
BANDS_HZ     = {"alpha": (8.0, 13.0), "theta": (4.0, 8.0)}
K_FIXED      = 2
KNN_K        = 6

# Replication mode: 'openneuro' | 'eegbci_holdout' | 'skip'
REPL_MODE    = 'eegbci_holdout'     # change to 'openneuro' to try OpenNeuro download
REPL_DS      = "dsXXXXXX"           # set real OpenNeuro dataset if using 'openneuro'
REPL_ROOT    = os.path.join(ROOT, "artifacts", "pli_ec_eo_replication")
REPL_REPEATS = 50                   # for eegbci_holdout
PERM_B_REPL  = 10000                # paired permutation for replication

# Time-resolved windows:
WIN_SEC      = 10
STEP_SEC     = 5
PERM_B_TIME  = 5000                 # per-window paired permutations
TIME_ROOT    = os.path.join(ROOT, "artifacts", "pli_ec_eo_timewindows")

# Source echo (template-lite sensor→parcel proxy):
ECHO_ROOT    = os.path.join(ROOT, "artifacts", "pli_ec_eo_echo")

# Final 1-pager:
FINAL_PDF    = os.path.join(ROOT, "artifacts", "CNT_PLI_4in1_summary.pdf")

# Optional existing artifacts for inclusion:
PREREG_CSV   = os.path.join(ROOT, "artifacts", "pli_ec_eo_prediction", "ec_eo_minimal_prereg.csv")
ABLATION_PDF = os.path.join(ROOT, "artifacts", "pli_ec_eo_prediction", "ec_eo_prediction_ablation.pdf")
SCALP_DIR    = os.path.join(ROOT, r"artifacts\pli_30_subjects\figures_ec_eo_scalp")

for p in [REPL_ROOT, TIME_ROOT, ECHO_ROOT, os.path.dirname(FINAL_PDF)]:
    os.makedirs(p, exist_ok=True)

# ------------------- Channel maps -------------------
def clean_label(x: str) -> str:
    y = x.strip()
    y = re.sub(r"(?i)^(EEG|MEG|EOG|ECG|EMG)[\s_\-]+", "", y)
    y = re.sub(r"(?i)[\s_\-]*(REF|AV|AVERAGE|AVG|M1|M2)$", "", y)
    y = re.sub(r"[ \-\.]+", "", y)
    y = y.replace("FP","Fp")
    return y

with open(CH_TXT, "r", encoding="utf-8") as f:
    ch_names = [clean_label(ln.strip()) for ln in f if ln.strip()]

LEFT_CANON  = set(map(str.upper, [
    "Fp1","AF7","AF3","F7","F5","F3","F1","FT7","FC5","FC3","FC1","T7","C5","C3","C1",
    "TP7","CP5","CP3","CP1","P7","P5","P3","P1","PO7","PO3","O1"
]))
RIGHT_CANON = set(map(str.upper, [
    "Fp2","AF8","AF4","F8","F6","F4","F2","FT8","FC6","FC4","FC2","T8","C6","C4","C2",
    "TP8","CP6","CP4","CP2","P8","P6","P4","P2","PO8","PO4","O2"
]))
MID_CANON   = set(map(str.upper, ["Fpz","AFz","Fz","FCz","Cz","CPz","Pz","POz","Oz"]))

ANT_PREFIXES=("Fp","AF","F","FC"); MID_PREFIXES=("C",); POST_PREFIXES=("CP","P","PO","O","TP","FT")

def hemi_map(labels):
    L,R,Z = [],[],[]
    for i,ch in enumerate(labels):
        up = ch.upper()
        if up in LEFT_CANON:  L.append(i); continue
        if up in RIGHT_CANON: R.append(i); continue
        if up in MID_CANON:   Z.append(i); continue
        if re.search(r"[A-Za-z]z$", ch): Z.append(i); continue
        m = re.search(r"(\d+)$", ch)
        if m:
            try:
                d=int(m.group(1)); (L if d%2==1 else R).append(i); continue
            except: pass
        Z.append(i)
    return np.array(L,int), np.array(R,int), np.array(Z,int)

def ap_map(labels):
    A,P,C = [],[],[]
    for i, ch in enumerate(labels):
        pref=re.match(r"[A-Za-z]+", ch); pref=pref.group(0) if pref else ""
        if any(pref.startswith(px) for px in ANT_PREFIXES): A.append(i)
        elif any(pref.startswith(px) for px in POST_PREFIXES): P.append(i)
        elif any(pref.startswith(px) for px in MID_PREFIXES): C.append(i)
        else: C.append(i)
    return np.array(A,int), np.array(P,int), np.array(C,int)

L_idx, R_idx, Z_idx = hemi_map(ch_names)
A_idx, P_idx, C_idx = ap_map(ch_names)

# ------------------- PLI & spectral helpers -------------------
def bandpass(x, fs, lo, hi, order=4):
    b,a = butter(order, [lo/(fs/2), hi/(fs/2)], btype="band"); return filtfilt(b,a,x)
def pli_matrix(X, fs, lo, hi):
    n = X.shape[0]
    b,a = butter(4, [lo/(fs/2), hi/(fs/2)], btype="band")
    Y = np.zeros_like(X)
    for c in range(n): Y[c] = filtfilt(b,a,X[c])
    ph = np.angle(hilbert(Y, axis=1))
    W  = np.zeros((n,n), float)
    for i in range(n):
        for j in range(i+1, n):
            dphi = ph[i]-ph[j]
            pli  = np.abs(np.mean(np.sign(np.sin(dphi))))
            W[i,j] = W[j,i] = pli
    np.fill_diagonal(W, 0.0)
    return W
def knn(W,k):
    W=W.copy(); n=W.shape[0]
    for i in range(n):
        idx=np.argsort(W[i])[::-1]; keep=idx[:k]
        mask=np.ones(n,bool); mask[keep]=False; W[i,mask]=0
    W=np.maximum(W,W.T); np.fill_diagonal(W,0); return W
def lap(W):
    d=W.sum(1); d=np.where(d<=1e-12,1.0,d); D=np.diag(1.0/np.sqrt(d))
    return np.eye(W.shape[0]) - D@W@D
from sklearn.cluster import KMeans
def spec_labels(W, k=2):
    e,v = np.linalg.eigh(lap(W))
    U = v[:,1:k] if k>1 else v[:,:1]
    U = U / (np.linalg.norm(U, axis=1, keepdims=True) + 1e-12)
    return KMeans(n_clusters=k, n_init=50, random_state=42).fit_predict(U)

def mean_safe(x): return float(np.nan) if x.size==0 else float(x.mean())
def hemi_module_means(W, labels, L, R):
    n=len(labels)
    Lmask=np.zeros((n,n),bool); Lmask[np.ix_(L,L)]=True
    Rmask=np.zeros((n,n),bool); Rmask[np.ix_(R,R)]=True
    within=Lmask|Rmask
    cross =np.zeros((n,n),bool); cross[np.ix_(L,R)]=True; cross[np.ix_(R,L)]=True
    same  =labels[:,None]==labels[None,:]
    for m in (within,cross,same): np.fill_diagonal(m,False)
    WH_WM = mean_safe(W[within & same])
    CH_WM = mean_safe(W[cross  & same])
    return WH_WM, CH_WM

# ------------------- Minimal features (α/θ coupling only) -------------------
def subject_features_coupling(subject_id, cond_tag):
    f = os.path.join(DATA_DIR, f"subject_{subject_id:02d}_{cond_tag}.npy")
    if not os.path.exists(f): return None
    X = np.load(f)
    feats = {}
    for band,(lo,hi) in BANDS_HZ.items():
        W   = pli_matrix(X, FS, lo, hi); W = knn(W, KNN_K)
        lbl = spec_labels(W, k=K_FIXED)
        WHWM, CHWM = hemi_module_means(W, lbl, L_idx, R_idx)
        feats[f"{band}_WHWM"] = WHWM
        feats[f"{band}_CHWM"] = CHWM
    return feats

def build_minimal_dataset():
    X_list, y_list, subj_list = [], [], []
    for s in SUBJECTS:
        ec = subject_features_coupling(s, "EC")
        eo = subject_features_coupling(s, "EO")
        if ec:
            X_list.append(ec); y_list.append(0); subj_list.append(s)
        if eo:
            X_list.append(eo); y_list.append(1); subj_list.append(s)
    assert X_list, "No features built. Ensure EC/EO NPY exist."
    feat_names = list(X_list[0].keys())
    X = pd.DataFrame(X_list, columns=feat_names).replace([np.inf,-np.inf], np.nan).to_numpy(dtype=float)
    y = np.array(y_list, int)
    subj_ids = np.array(subj_list, int)
    return X, y, subj_ids, feat_names

def paired_auc_perm(X, y, subj_ids, perm_B=10000):
    from sklearn.impute import SimpleImputer
    from sklearn.preprocessing import StandardScaler
    from sklearn.linear_model import LogisticRegression
    clf = make_pipeline(SimpleImputer(strategy="median"), StandardScaler(), LogisticRegression(max_iter=2000, solver="lbfgs"))
    rng = default_rng(7)
    fold_probs, fold_true, order_pairs = [], [], []
    for s in SUBJECTS:
        test = (subj_ids == s)
        if np.sum(test) != 2: # need both EC & EO
            continue
        train = ~test
        clf.fit(X[train], y[train])
        if hasattr(clf[-1], "predict_proba"):
            p = clf.predict_proba(X[test])[:,1]
        else:
            p = clf.decision_function(X[test])
        fold_probs.append(p); fold_true.append(y[test]); order_pairs.append(np.where(test)[0])
    probs = np.concatenate(fold_probs); ys = np.concatenate(fold_true)
    auc_obs = float(roc_auc_score(ys, probs))
    # paired flips
    cnt = 1
    for _ in range(perm_B):
        y_perm = ys.copy()
        idx = 0
        for s in SUBJECTS:
            if idx >= len(order_pairs): break
            pair_idx = order_pairs[idx]
            if len(pair_idx) != 2:
                idx += 1; continue
            if rng.random() < 0.5:
                y_perm[pair_idx] = y_perm[pair_idx][::-1]
            idx += 1
        auc_p = float(roc_auc_score(y_perm, probs))
        if auc_p >= auc_obs: cnt += 1
    return auc_obs, float(cnt/(perm_B+1)), probs, ys

# ------------------- A) Replication -------------------
rep_summary_txt = "Replication skipped."
if REPL_MODE == 'eegbci_holdout':
    X, y, subj_ids, feat_names = build_minimal_dataset()
    rng = default_rng(11)
    aucs = []
    for rep in range(REPL_REPEATS):
        # random subject split 20 train / 10 test; evaluate test AUC with same pipeline (not paired)
        subs = np.array(SUBJECTS)
        rng.shuffle(subs)
        train_subs = set(subs[:20]); test_subs = set(subs[20:])
        train_mask = np.array([sid in train_subs for sid in subj_ids])
        test_mask  = np.array([sid in test_subs  for sid in subj_ids])
        if np.sum(test_mask) < 4: 
            continue
        clf = make_pipeline(SimpleImputer(strategy="median"), StandardScaler(), LogisticRegression(max_iter=2000, solver="lbfgs"))
        clf.fit(X[train_mask], y[train_mask])
        # probs on test
        if hasattr(clf[-1], "predict_proba"):
            probs = clf.predict_proba(X[test_mask])[:,1]
        else:
            probs = clf.decision_function(X[test_mask])
        aucs.append(roc_auc_score(y[test_mask], probs))
    if aucs:
        # Permute labels on each test split concatenated (approx; conservative)
        auc_mean = float(np.mean(aucs))
        rep_summary_txt = f"EEGBCI hold-out (20/10, {len(aucs)} reps): mean AUC={auc_mean:.3f}"
    else:
        rep_summary_txt = f"EEGBCI hold-out: no valid splits."
elif REPL_MODE == 'openneuro':
    rep_summary_txt = "OpenNeuro replication requested; please set REPL_DS to a real EO/EC dataset ID."
else:
    rep_summary_txt = "Replication skipped by config."

# ------------------- B) Time-resolved EC vs EO (windows) -------------------
def subject_window_features_coupling(sid, cond_tag, lo, hi, win, step):
    f = os.path.join(DATA_DIR, f"subject_{sid:02d}_{cond_tag}.npy")
    if not os.path.exists(f): return []
    X = np.load(f)
    T = X.shape[1]; w = int(win*FS); st = int(step*FS)
    feats = []
    for start in range(0, T-w+1, st):
        Xt = X[:, start:start+w]
        W  = pli_matrix(Xt, FS, lo, hi); W = knn(W, KNN_K)
        lbl= spec_labels(W, k=K_FIXED)
        WHWM, CHWM = hemi_module_means(W, lbl, L_idx, R_idx)
        feats.append((start/FS, WHWM, CHWM))
    return feats

def time_auc_for_band(lo, hi, win, step, perm_B=5000):
    rng = default_rng(23)
    # Determine aligned window grid from EC subjects (assume same length EC/EO slices were exported)
    # Build matrix of features per subject and window index; require both EC & EO present.
    # We'll compute AUC(t) using minimal features [WHWM, CHWM] and paired flips.
    windows = None
    subj_feat = {}
    for s in SUBJECTS:
        ecf = subject_window_features_coupling(s, "EC", lo, hi, win, step)
        eof = subject_window_features_coupling(s, "EO", lo, hi, win, step)
        if not ecf or not eof: 
            continue
        # align by index count (use min)
        m = min(len(ecf), len(eof))
        if m < 1: continue
        subj_feat[s] = (ecf[:m], eof[:m])
        if windows is None or m < len(windows):
            windows = [ecf[i][0] for i in range(m)]
    if not subj_feat:
        return None
    aucs = []; ps = []; times=windows
    for widx, t0 in enumerate(times):
        Xw=[]; yw=[]; pairs=[]
        for s,(ecf,eof) in subj_feat.items():
            ec_wh,ec_ch = ecf[widx][1], ecf[widx][2]
            eo_wh,eo_ch = eof[widx][1], eof[widx][2]
            if not (np.isfinite(ec_wh) and np.isfinite(ec_ch) and np.isfinite(eo_wh) and np.isfinite(eo_ch)):
                continue
            Xw.append([ec_wh,ec_ch]); yw.append(0); pairs.append(s)
            Xw.append([eo_wh,eo_ch]); yw.append(1); pairs.append(s)
        if len(Xw) < 20:  # need enough points
            aucs.append(np.nan); ps.append(np.nan); continue
        Xw = np.array(Xw,float); yw=np.array(yw,int); pairs=np.array(pairs,int)
        # paired-CV (leave one subject pair out)
        clf = make_pipeline(SimpleImputer(strategy="median"), StandardScaler(), LogisticRegression(max_iter=2000, solver="lbfgs"))
        fold_probs=[]; fold_true=[]; order_pairs=[]
        for s in SUBJECTS:
            test = (pairs==s)
            if np.sum(test)!=2: continue
            train = ~test
            clf.fit(Xw[train], yw[train])
            if hasattr(clf[-1],"predict_proba"): p=clf.predict_proba(Xw[test])[:,1]
            else: p=clf.decision_function(Xw[test])
            fold_probs.append(p); fold_true.append(yw[test]); order_pairs.append(np.where(test)[0])
        if not fold_probs: 
            aucs.append(np.nan); ps.append(np.nan); continue
        probs=np.concatenate(fold_probs); ys=np.concatenate(fold_true)
        if len(np.unique(ys))<2: aucs.append(np.nan); ps.append(np.nan); continue
        auc_obs=float(roc_auc_score(ys, probs))
        # paired label-flip perms
        cnt=1
        for _ in range(perm_B):
            y_perm = ys.copy()
            idx=0
            for s in SUBJECTS:
                if idx>=len(order_pairs): break
                pidx=order_pairs[idx]
                if len(pidx)!=2: idx+=1; continue
                if rng.random()<0.5:
                    y_perm[pidx]=y_perm[pidx][::-1]
                idx+=1
            auc_p = float(roc_auc_score(y_perm, probs))
            if auc_p >= auc_obs: cnt += 1
        aucs.append(auc_obs); ps.append(float(cnt/(perm_B+1)))
    return np.array(times), np.array(aucs), np.array(ps)

times_alpha, aucs_alpha, ps_alpha = time_auc_for_band(*BANDS_HZ["alpha"], WIN_SEC, STEP_SEC, PERM_B_TIME)
times_theta, aucs_theta, ps_theta = time_auc_for_band(*BANDS_HZ["theta"], WIN_SEC, STEP_SEC, PERM_B_TIME)

# ------------------- C) Template Source Echo (sensor→parcel proxy) -------------------
# Map channels into parcels (frontal, central, parietal, occipital, temporalL/R), and count how many module-0 vs module-1 entries
# Use EC alpha consensus as representative
ECHO_FIG = os.path.join(ECHO_ROOT, "parcel_echo.png")
CONS_EC_TAB = os.path.join(ROOT, r"artifacts\pli_30_subjects\tables")
def load_consensus(band):
    cfp=os.path.join(CONS_EC_TAB, f"band__{band}__consensus_labels.npy")
    if not os.path.exists(cfp): return None
    return np.load(cfp)

labels_alpha = load_consensus("alpha")
def parcel_map(names):
    parcels = {"Frontal":[],"Central":[],"Parietal":[],"Occipital":[],
               "TemporalL":[],"TemporalR":[]}
    for i,ch in enumerate(names):
        up=ch.upper()
        if any(up.startswith(p) for p in ("FP","AF","F","FC")): parcels["Frontal"].append(i)
        elif up.startswith("C"): parcels["Central"].append(i)
        elif any(up.startswith(p) for p in ("CP","P","PO")): parcels["Parietal"].append(i)
        elif up.startswith("O"): parcels["Occipital"].append(i)
        elif up.startswith("T") and ("7" in up or "3" in up): parcels["TemporalL"].append(i)
        elif up.startswith("T") and ("8" in up or "4" in up): parcels["TemporalR"].append(i)
        else:
            # leave unmatched in nearest big parcel by prefix
            if "Z" in up: parcels["Central"].append(i)
            else: parcels["Central"].append(i)
    return {k: np.array(v,int) for k,v in parcels.items()}
parcel_idx = parcel_map(ch_names)
def parcel_asym_counts(labels):
    out={}
    for k,idx in parcel_idx.items():
        if idx.size==0: out[f"{k}_mod0"]=0; out[f"{k}_mod1"]=0; continue
        out[f"{k}_mod0"] = int((labels[idx]==0).sum())
        out[f"{k}_mod1"] = int((labels[idx]==1).sum())
    return out
# Draw a bar: counts per parcel for module0 vs module1
if labels_alpha is not None:
    counts = parcel_asym_counts(labels_alpha)
    keys = ["Frontal","Central","Parietal","Occipital","TemporalL","TemporalR"]
    m0 = [counts[f"{k}_mod0"] for k in keys]
    m1 = [counts[f"{k}_mod1"] for k in keys]
    plt.figure(figsize=(9,5))
    x = np.arange(len(keys))
    plt.bar(x-0.18, m0, width=0.36, label="Module 0")
    plt.bar(x+0.18, m1, width=0.36, label="Module 1")
    plt.xticks(x, keys, rotation=0)
    plt.ylabel("Channel count")
    plt.title("Template parcel echo (EC alpha consensus): module composition by parcel")
    plt.legend()
    plt.tight_layout(); plt.savefig(ECHO_FIG, dpi=160); plt.close()
else:
    ECHO_FIG = None

# ------------------- D) Final 1-page PDF composer -------------------
# Try to include: prereg ROC + AUC/p, ablation PDF (as img), EC/EO scalp strips, replication text, AUC(t) plots, parcel echo
# Load prereg CSV if available:
PREREG_TXT = ""
if os.path.exists(PREREG_CSV):
    d = pd.read_csv(PREREG_CSV)
    if not d.empty:
        auc_obs = float(d.loc[0,"AUC_obs"]); p_pr = float(d.loc[0,"p_perm"])
        PREREG_TXT = f"Minimal model (α/θ coupling): AUC={auc_obs:.3f}, p={p_pr:.4f}"

# Build the page
fig = plt.figure(figsize=(11, 8.5))

# Title
ax_t = fig.add_axes([0.05, 0.92, 0.90, 0.06]); ax_t.axis("off")
ax_t.text(0.5, 0.5, "CNT — Field Signature: Replication • Dynamics • Parcel Echo (All-in-One)", ha="center", va="center", fontsize=15, weight="bold")

# Row 1: prereg + replication summary (text boxes)
ax_a = fig.add_axes([0.05, 0.78, 0.42, 0.10]); ax_a.axis("off")
ax_a.text(0.0, 0.7, "Pre-registered Minimal Classifier", fontsize=11, weight="bold")
ax_a.text(0.0, 0.35, PREREG_TXT if PREREG_TXT else "Minimal model stats not found.", fontsize=10)
ax_b = fig.add_axes([0.53, 0.78, 0.42, 0.10]); ax_b.axis("off")
ax_b.text(0.0, 0.7, "Replication", fontsize=11, weight="bold")
ax_b.text(0.0, 0.35, rep_summary_txt, fontsize=10)

# Row 2: Time AUC(t)
ax_ta = fig.add_axes([0.05, 0.52, 0.42, 0.20])
if times_alpha is not None:
    ax_ta.plot(times_alpha, aucs_alpha, label="alpha AUC(t)")
    ax_ta.set_title("Time-resolved AUC — alpha (EC vs EO)")
    ax_ta.set_xlabel("Window start (s)"); ax_ta.set_ylabel("AUC"); ax_ta.set_ylim(0.5,1.0)
else:
    ax_ta.text(0.5,0.5,"alpha AUC(t) unavailable",ha="center",va="center")
ax_tb = fig.add_axes([0.53, 0.52, 0.42, 0.20])
if times_theta is not None:
    ax_tb.plot(times_theta, aucs_theta, label="theta AUC(t)")
    ax_tb.set_title("Time-resolved AUC — theta (EC vs EO)")
    ax_tb.set_xlabel("Window start (s)"); ax_tb.set_ylabel("AUC"); ax_tb.set_ylim(0.5,1.0)
else:
    ax_tb.text(0.5,0.5,"theta AUC(t) unavailable",ha="center",va="center")

# Row 3: Scalp strips (EC vs EO)
def try_load(path): 
    return plt.imread(path) if os.path.exists(path) else None
ax_s = fig.add_axes([0.05, 0.28, 0.90, 0.20]); ax_s.axis("off")
x0=0.0
for band in ["alpha","theta","beta"]:
    p_ec = os.path.join(SCALP_DIR, f"scalp__{band}__EC.png")
    p_eo = os.path.join(SCALP_DIR, f"scalp__{band}__EO.png")
    ec_img, eo_img = try_load(p_ec), try_load(p_eo)
    if ec_img is None or eo_img is None:
        ax_s.text(x0+0.15, 0.5, f"{band} scalp missing", transform=ax_s.transAxes)
        x0 += 0.3; continue
    # stitch horizontally
    strip = np.concatenate([ec_img, eo_img], axis=1)
    ax_sub = fig.add_axes([0.05 + (["alpha","theta","beta"].index(band))*0.30, 0.28, 0.28, 0.20]); ax_sub.axis("off")
    ax_sub.imshow(strip); ax_sub.set_title(f"{band.upper()}  EC | EO")

# Row 4: Parcel echo (bar)
ax_p = fig.add_axes([0.05, 0.05, 0.42, 0.18]); ax_p.axis("off")
if ECHO_FIG and os.path.exists(ECHO_FIG):
    img = plt.imread(ECHO_FIG); ax_p.imshow(img); ax_p.set_title("Parcel echo (EC α)"); ax_p.axis("off")
else:
    ax_p.text(0.5,0.5,"Parcel echo unavailable",ha="center",va="center")

# Ablation (optional)
ax_ab = fig.add_axes([0.53, 0.05, 0.42, 0.18]); ax_ab.axis("off")
if os.path.exists(ABLATION_PDF):
    ax_ab.text(0.0,0.8,"Ablation PDF saved separately:",fontsize=10,weight="bold")
    ax_ab.text(0.0,0.45,ABLATION_PDF,fontsize=9)
else:
    ax_ab.text(0.0,0.6,"Ablation PDF not found",fontsize=10)

fig.tight_layout()
pp = PdfPages(FINAL_PDF); pp.savefig(fig, dpi=200); pp.close(); plt.close(fig)
print("Saved final 1-pager:", FINAL_PDF)


C:\Users\caleb\AppData\Local\Temp\ipykernel_28416\2956648012.py:475: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout()


Saved final 1-pager: C:\Users\caleb\CNT_Lab\artifacts\CNT_PLI_4in1_summary.pdf


In [ ]:
# === CNT Primate Field Test — Two-Module Consensus + Hemi/AP + Optional EC/EO Classifier (single cell) ===
# Works with local primate EEG/LFP/ECoG files (*.npy, optional *.channels.txt).
# It will:
#   1) Scan PRIMATE_DIR for subjects: subject_##_{REST|EC|EO}.npy
#   2) Run PLI + spectral-on-coassoc (k=2) for θ, α, β, γ with label-preserving null p-values
#   3) Compute hemispheric and anterior/posterior metrics (auto-map; optional channels_map.csv)
#   4) If both EC & EO exist → train prereg minimal classifier (α/θ coupling only) with paired CV + 10k perms
# Outputs: under OUT_ROOT: metrics/*.json, tables/*.npy, figures/*.png, summary CSVs.

import os, re, glob, json, numpy as np, pandas as pd, matplotlib.pyplot as plt
from numpy.random import default_rng
from scipy.signal import butter, filtfilt, hilbert
from sklearn.cluster import KMeans
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

# ---------------- CONFIG ----------------
PRIMATE_DIR = r"C:\Users\caleb\CNT_Lab\primate_eeg"   # <-- set to your primate folder
USE_DEMO    = False                                   # True = synthesize small primate-like set
FS          = 1000.0                                  # set your sampling rate (Hz)
SLICE_SEC   = 60                                      # seconds per subject to keep (for demo or trimming)
BANDS_HZ    = { "theta": (4.0, 8.0), "alpha": (8.0, 12.0), "beta": (13.0, 30.0), "gamma": (30.0, 55.0) }
K_FIXED     = 2
KNN_K       = 6
NULL_PERMS  = 500                                     # label-preserving nulls per band
OUT_ROOT    = r"C:\Users\caleb\CNT_Lab\artifacts\pli_primate"

# Optional curated channel map:
# If present at PRIMATE_DIR/channels_map.csv with columns: name, hemi (L/R/M), ap (A/P/C)
CHANNEL_MAP_CSV = os.path.join(PRIMATE_DIR, "channels_map.csv")

# ----------------------------------------

os.makedirs(OUT_ROOT, exist_ok=True)
OUT_TAB = os.path.join(OUT_ROOT, "tables");  os.makedirs(OUT_TAB, exist_ok=True)
OUT_MET = os.path.join(OUT_ROOT, "metrics"); os.makedirs(OUT_MET, exist_ok=True)
OUT_FIG = os.path.join(OUT_ROOT, "figures"); os.makedirs(OUT_FIG, exist_ok=True)

# -------- Scan subjects --------
def find_subjects(base):
    subs = {"REST":[], "EC":[], "EO":[]}
    for cond in ["REST","EC","EO"]:
        subs[cond] = sorted(glob.glob(os.path.join(base, f"subject_*_{cond}.npy")))
    return subs

# -------- Demo synth (if needed) --------
def synth_primate(n_subj=8, n_ch=32, fs=FS, seconds=SLICE_SEC, seed=7):
    rng = default_rng(seed)
    T = int(fs*seconds)
    paths = []
    os.makedirs(PRIMATE_DIR, exist_ok=True)
    for s in range(n_subj):
        X = rng.normal(0,1,size=(n_ch,T))
        t = np.arange(T)/fs
        theta = np.sin(2*np.pi*7.5*t + rng.uniform(0,2*np.pi))
        # Inject two-module: stronger posterior subset
        for c in range(n_ch//2, n_ch):
            X[c] += 0.6*theta + 0.2*rng.normal(0,1,T)
        base = os.path.join(PRIMATE_DIR, f"subject_{s:02d}_REST")
        np.save(base + ".npy", X.astype(np.float32))
        with open(base + ".channels.txt","w",encoding="utf-8") as f:
            for i in range(n_ch): f.write(f"ch{i}\n")
        paths.append(base+".npy")
    return paths

# -------- Channel handling --------
def load_channels_for_any(npy_path):
    txt = npy_path.replace(".npy",".channels.txt")
    if os.path.exists(txt):
        with open(txt,"r",encoding="utf-8") as f:
            return [ln.strip() for ln in f if ln.strip()]
    # fallback generic names
    X = np.load(npy_path, mmap_mode='r')
    return [f"ch{i}" for i in range(X.shape[0])]

def clean_label(x: str) -> str:
    y = x.strip()
    y = re.sub(r"(?i)^(EEG|LFP|ECOG)[\s_\-]+", "", y)
    y = re.sub(r"(?i)[\s_\-]*(REF|AV|AVERAGE|AVG|M1|M2)$", "", y)
    y = re.sub(r"[ \-\.]+", "", y)
    y = y.replace("FP","Fp")
    return y

LEFT_CANON  = set(map(str.upper, ["Fp1","AF7","AF3","F7","F5","F3","F1","FT7","FC5","FC3","FC1","T7","C5","C3","C1","TP7","CP5","CP3","CP1","P7","P5","P3","P1","PO7","PO3","O1"]))
RIGHT_CANON = set(map(str.upper, ["Fp2","AF8","AF4","F8","F6","F4","F2","FT8","FC6","FC4","FC2","T8","C6","C4","C2","TP8","CP6","CP4","CP2","P8","P6","P4","P2","PO8","PO4","O2"]))
MID_CANON   = set(map(str.upper, ["Fpz","AFz","Fz","FCz","Cz","CPz","Pz","POz","Oz"]))
ANT_PREFIXES=("Fp","AF","F","FC"); MID_PREFIXES=("C",); POST_PREFIXES=("CP","P","PO","O")

# read optional curated map
CURATED = {}
if os.path.exists(CHANNEL_MAP_CSV):
    try:
        df_map = pd.read_csv(CHANNEL_MAP_CSV)
        for _,r in df_map.iterrows():
            CURATED[str(r["name"]).strip()] = (str(r["hemi"]).upper()[:1], str(r["ap"]).upper()[:1])
        print(f"[info] Loaded curated channel map with {len(CURATED)} entries.")
    except Exception as e:
        print("[warn] Could not parse channels_map.csv:", e)

def hemi_ap_map(ch_names):
    L,R,Z,A,P,C = [],[],[],[],[],[]
    for i, orig in enumerate(ch_names):
        name = clean_label(orig)
        # curated override
        if name in CURATED:
            hemi, ap = CURATED[name]
            if hemi=="L": L.append(i)
            elif hemi=="R": R.append(i)
            else: Z.append(i)
            if ap=="A": A.append(i)
            elif ap=="P": P.append(i)
            else: C.append(i)
            continue
        up = name.upper()
        # hemi: tokens or 10–20 odd/even or _L/_R suffix
        if up.endswith("_L") or re.search(r"(^|[_\-])L($|[_\-])", name): L.append(i); 
        elif up.endswith("_R") or re.search(r"(^|[_\-])R($|[_\-])", name): R.append(i)
        elif up in LEFT_CANON: L.append(i)
        elif up in RIGHT_CANON: R.append(i)
        elif up in MID_CANON or re.search(r"[A-Za-z]Z$", name): Z.append(i)
        else:
            m = re.search(r"(\d+)$", name)
            if m:
                try:
                    d=int(m.group(1)); (L if d%2==1 else R).append(i)
                except: Z.append(i)
            else:
                Z.append(i)
        # A/P: prefixes
        if any(up.startswith(px.upper()) for px in ANT_PREFIXES): A.append(i)
        elif any(up.startswith(px.upper()) for px in POST_PREFIXES): P.append(i)
        elif up.startswith("C"): C.append(i)
        else: C.append(i)
    return np.array(L,int), np.array(R,int), np.array(Z,int), np.array(A,int), np.array(P,int), np.array(C,int)

# -------- PLI + spectral --------
def bandpass(x, fs, lo, hi, order=4):
    b,a=butter(order, [lo/(fs/2), hi/(fs/2)], btype="band"); return filtfilt(b,a,x)
def pli_matrix(X, fs, lo, hi):
    n=X.shape[0]
    b,a=butter(4, [lo/(fs/2), hi/(fs/2)], btype="band")
    Y=np.zeros_like(X)
    for c in range(n): Y[c]=filtfilt(b,a,X[c])
    ph=np.angle(hilbert(Y, axis=1))
    W=np.zeros((n,n), float)
    for i in range(n):
        for j in range(i+1, n):
            d=ph[i]-ph[j]; pli=abs(np.mean(np.sign(np.sin(d))))
            W[i,j]=W[j,i]=pli
    np.fill_diagonal(W,0.0); return W
def knn(W,k):
    W=W.copy(); n=W.shape[0]
    for i in range(n):
        idx=np.argsort(W[i])[::-1]; keep=idx[:k]
        mask=np.ones(n,bool); mask[keep]=False; W[i,mask]=0
    W=np.maximum(W,W.T); np.fill_diagonal(W,0); return W
def lap(W):
    d=W.sum(1); d=np.where(d<=1e-12,1.0,d); D=np.diag(1.0/np.sqrt(d)); return np.eye(W.shape[0]) - D@W@D
def spec_labels(W,k=2):
    e,v=np.linalg.eigh(lap(W)); U=v[:,1:k] if k>1 else v[:,:1]
    U/= (np.linalg.norm(U, axis=1, keepdims=True)+1e-12)
    return KMeans(n_clusters=k, n_init=50, random_state=42).fit_predict(U)

def coassoc(labels):
    n=len(labels[0]); m=len(labels); co=np.zeros((n,n), float)
    for lab in labels:
        for i in range(n):
            li=lab[i]
            for j in range(n): co[i,j]+=1.0 if li==lab[j] else 0.0
    return co/m

def loso_via_coassoc(label_list):
    cof=coassoc(label_list); cons=spec_labels(cof, k=2)
    from sklearn.metrics import adjusted_rand_score
    vals=[]
    for s in range(len(label_list)):
        leave=[lab for i,lab in enumerate(label_list) if i!=s]
        cons_l=spec_labels(coassoc(leave), k=2)
        vals.append(adjusted_rand_score(cons,cons_l))
    return float(np.median(vals)), cons, cof

def rand_same_sizes(lab, rng):
    n=len(lab); uniq,cnts=np.unique(lab, return_counts=True)
    idx=np.arange(n); rng.shuffle(idx); out=np.empty(n,int); st=0
    for L,c in zip(uniq,cnts):
        seg=idx[st:st+c]; out[seg]=L; st+=c
    return out

# -------- Consensus runner --------
def run_consensus_for_condition(paths, bands=BANDS_HZ, tag="REST"):
    rng = default_rng(13)
    # unify channels for mapping from first file
    ch = load_channels_for_any(paths[0])
    L,R,Z,A,P,C = hemi_ap_map(ch)

    rows=[]
    for band,(lo,hi) in bands.items():
        # per-subject labels
        subj_labels=[]
        for p in paths:
            X=np.load(p); W=pli_matrix(X, FS, lo, hi); W=knn(W,KNN_K); lbl=spec_labels(W,k=K_FIXED)
            subj_labels.append(lbl)
        loso, cons, co = loso_via_coassoc(subj_labels)
        # null (label-preserving)
        null=[]
        for _ in range(NULL_PERMS):
            nlabs=[rand_same_sizes(l, rng) for l in subj_labels]
            _, cons_n, _ = loso_via_coassoc(nlabs)
            from sklearn.metrics import adjusted_rand_score
            null.append(adjusted_rand_score(cons, cons_n))
        null=np.array(null,float); p=float((np.sum(null>=loso)+1)/(len(null)+1))

        # save
        np.save(os.path.join(OUT_TAB, f"primate__{tag}__{band}__consensus_labels.npy"), cons)
        np.save(os.path.join(OUT_TAB, f"primate__{tag}__{band}__coassoc.npy"), co)
        met={"tag":tag,"band":band,"n_subjects":len(paths),"loso":float(loso),
             "null_mean":float(null.mean()),"p_value":p,"n_channels":len(ch)}
        with open(os.path.join(OUT_MET, f"primate__{tag}__{band}__metrics.json"),"w",encoding="utf-8") as f:
            json.dump(met,f,indent=2)

        # hemi/AP quick metrics
        def mean_safe(x): return float(np.nan) if x.size==0 else float(x.mean())
        def within_region_ratio(W, labels, idx):
            if idx.size<3: return np.nan
            sub=np.ix_(idx,idx); Wr=W[sub]; lab=labels[idx]
            same=lab[:,None]==lab[None,:]; diff=~same
            np.fill_diagonal(same,False); np.fill_diagonal(diff,False)
            return float(mean_safe(Wr[same])/(mean_safe(Wr[diff])+1e-12))
        # Use consensus W? (coassoc) for ratios on consensus geometry
        lratio = within_region_ratio(co, cons, L)
        rratio = within_region_ratio(co, cons, R)
        # Report
        rows.append([tag, band, len(paths), len(ch), float(loso), float(null.mean()), p, lratio, rratio])

        # fig
        plt.figure(); plt.imshow(co, aspect='auto'); plt.title(f"Primate {tag} {band} — co-assoc")
        plt.colorbar(); plt.tight_layout()
        plt.savefig(os.path.join(OUT_FIG, f"primate__{tag}__{band}__coassoc.png"), dpi=160); plt.close()

    df = pd.DataFrame(rows, columns=["tag","band","n_subjects","n_channels","LOSO","null_mean","p_value","left_intra_ratio","right_intra_ratio"])
    df.to_csv(os.path.join(OUT_ROOT, f"primate__{tag}__summary.csv"), index=False)
    print(f"[{tag}] summary:\n", df.to_string(index=False))
    return df, ch

# -------- Minimal EC/EO classifier for primate (if both EC and EO exist per subject) --------
def build_minimal_features(paths_by_cond, fs=FS):
    # features: α/θ WH_WM & CH_WM per subject/cond
    def hemi_module_means(W, labels, L, R):
        n=len(labels)
        Lmask=np.zeros((n,n),bool); Lmask[np.ix_(L,L)]=True
        Rmask=np.zeros((n,n),bool); Rmask[np.ix_(R,R)]=True
        within=Lmask|Rmask
        cross =np.zeros((n,n),bool); cross[np.ix_(L,R)]=True; cross[np.ix_(R,L)]=True
        same  =labels[:,None]==labels[None,:]
        for m in (within,cross,same): np.fill_diagonal(m,False)
        return float((W[within&same]).mean()), float((W[cross&same]).mean())
    feats=[]; ys=[]; sids=[]
    # use first subject's channels to map
    any_key = next(iter(paths_by_cond.keys()))
    any_cond = next(iter(paths_by_cond[any_key].keys()))
    ch = load_channels_for_any(paths_by_cond[any_key][any_cond])
    L,R,Z,A,P,C = hemi_ap_map(ch)
    for subj, conds in paths_by_cond.items():
        for cond, path in conds.items():
            X=np.load(path)
            row={"subject":subj,"cond":0 if cond=="EC" else 1}
            for band,(lo,hi) in {"alpha":(8,12),"theta":(4,8)}.items():
                W=pli_matrix(X, fs, lo, hi); W=knn(W,KNN_K); lbl=spec_labels(W,k=K_FIXED)
                WHWM,CHWM = hemi_module_means(W, lbl, L, R)
                row[f"{band}_WHWM"]=WHWM; row[f"{band}_CHWM"]=CHWM
            feats.append(row)
    df = pd.DataFrame(feats).dropna()
    X = df[[c for c in df.columns if any(c.startswith(b) for b in ["alpha_","theta_"])]].to_numpy(float)
    y = df["cond"].to_numpy(int)
    s = df["subject"].to_numpy(int)
    return X,y,s, df

def paired_auc_perm(X, y, subj_ids, perm_B=10000):
    from sklearn.pipeline import make_pipeline
    clf = make_pipeline(SimpleImputer(strategy="median"), StandardScaler(), LogisticRegression(max_iter=2000, solver="lbfgs"))
    rng = default_rng(31)
    fold_probs=[]; fold_true=[]; order_pairs=[]
    u_subs = sorted(set(subj_ids))
    for sid in u_subs:
        test=(subj_ids==sid)
        if np.sum(test)!=2: continue
        train=~test
        clf.fit(X[train], y[train])
        if hasattr(clf[-1],"predict_proba"): p=clf.predict_proba(X[test])[:,1]
        else: p=clf.decision_function(X[test])
        fold_probs.append(p); fold_true.append(y[test]); order_pairs.append(np.where(test)[0])
    if not fold_probs: return np.nan, np.nan
    probs=np.concatenate(fold_probs); ys=np.concatenate(fold_true)
    auc_obs=float(roc_auc_score(ys, probs))
    # paired flips
    cnt=1
    for _ in range(perm_B):
        y_perm=ys.copy()
        idx=0
        for sid in u_subs:
            if idx>=len(order_pairs): break
            pidx=order_pairs[idx]
            if len(pidx)!=2: idx+=1; continue
            if rng.random()<0.5: y_perm[pidx]=y_perm[pidx][::-1]
            idx+=1
        auc_p=float(roc_auc_score(y_perm, probs))
        if auc_p>=auc_obs: cnt+=1
    return auc_obs, float(cnt/(perm_B+1))

# ================== RUN ==================
# Prepare data
subs = find_subjects(PRIMATE_DIR)
if USE_DEMO or (not any(len(v)>0 for v in subs.values())):
    print("[demo] synthesizing small primate set in", PRIMATE_DIR)
    synth_primate()
    subs = find_subjects(PRIMATE_DIR)

# REST consensus (if present)
if subs["REST"]:
    df_rest, ch_rest = run_consensus_for_condition(subs["REST"], tag="REST")

# EC/EO consensus (if present)
paths_ec = subs["EC"]; paths_eo = subs["EO"]
df_ec = df_eo = None
if paths_ec:
    df_ec, ch_ec = run_consensus_for_condition(paths_ec, tag="EC")
if paths_eo:
    df_eo, ch_eo = run_consensus_for_condition(paths_eo, tag="EO")

# Optional EC vs EO minimal classifier (α/θ coupling only)
# Build per-subject dict if both exist
pairs = {}
for p in paths_ec:
    sid = int(re.search(r"subject_(\d+)_EC\.npy$", p).group(1))
    pairs.setdefault(sid, {})["EC"] = p
for p in paths_eo:
    sid = int(re.search(r"subject_(\d+)_EO\.npy$", p).group(1))
    pairs.setdefault(sid, {})["EO"] = p
pairs = {k:v for k,v in pairs.items() if "EC" in v and "EO" in v}

if pairs:
    X,y,s,df_feat = build_minimal_features(pairs, fs=FS)
    if len(np.unique(s))>=5:
        auc_pri, p_pri = paired_auc_perm(X,y,s, perm_B=10000)
        print(f"\n[primate EC vs EO] minimal α/θ coupling → AUC={auc_pri:.3f}, p={p_pri:.4f}  (N={X.shape[0]} points)")
        pd.DataFrame([{"AUC":auc_pri,"p_perm":p_pri,"N_points":int(X.shape[0]),"N_subjects":int(len(set(s)))}]).to_csv(
            os.path.join(OUT_ROOT,"primate_ec_eo_minimal.csv"), index=False
        )
    else:
        print("[info] Not enough paired primate subjects for EC/EO classification.")
else:
    print("[info] No EC+EO pairs detected; skipping primate classifier.")

print("\nDone. Artifacts →", OUT_ROOT)


[demo] synthesizing small primate set in C:\Users\caleb\CNT_Lab\primate_eeg


In [1]:
# === Download Primate EO/EC (OpenNeuro) → Convert → PLI Consensus + Optional EC/EO Classifier (single cell) ===
# What you do:
#   1) Set PRIMATE_DS to a real OpenNeuro dataset ID (e.g., "ds00XXXX").
#   2) Run this cell. It downloads only EEG/LFP/ECoG matches, exports subjects → *.npy, and runs:
#        • PLI + spectral-on-coassoc (k=2) for θ/α/β/γ
#        • Hemispheric / A-P metrics
#        • (If EC & EO exist) α/θ coupling minimal classifier with paired CV + 10k perms (AUC + p)
# Outputs go to:  C:\Users\caleb\CNT_Lab\artifacts\pli_primate_dl\{metrics,tables,figures}

import os, re, glob, json, numpy as np, pandas as pd, matplotlib.pyplot as plt
from numpy.random import default_rng
from scipy.signal import butter, filtfilt, hilbert, decimate
from sklearn.cluster import KMeans
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

# ---------------- CONFIG ----------------
ROOT        = r"C:\Users\caleb\CNT_Lab"
PRIMATE_DS  = ""   # <-- set to an OpenNeuro dataset ID like "ds00XXXX" (leave blank until you know the ID)
FS_TARGET   = 1000.0        # resample target Hz (adjust to your data; 1 kHz is common for LFP/ECoG)
SLICE_SEC   = 60            # seconds per subject to keep (trim/tile)
INCLUDES    = ["*eeg*.edf", "*eeg*.mat", "*eeg*.npy",
               "*lfp*.edf", "*lfp*.mat", "*lfp*.npy",
               "*ecog*.edf","*ecog*.mat","*ecog*.npy"]    # narrow filters to keep download small

BANDS_HZ    = {"theta": (4.0, 8.0), "alpha": (8.0, 12.0), "beta": (13.0, 30.0), "gamma": (30.0, 55.0)}
K_FIXED     = 2
KNN_K       = 6
NULL_PERMS  = 300
PERM_B_CLF  = 10000

DL_ROOT     = os.path.join(ROOT, "primate_openneuro_raw")
NPY_DIR     = os.path.join(ROOT, "primate_eeg")  # exported .npy + .channels.txt
OUT_ROOT    = os.path.join(ROOT, r"artifacts\pli_primate_dl")
OUT_TAB     = os.path.join(OUT_ROOT, "tables")
OUT_MET     = os.path.join(OUT_ROOT, "metrics")
OUT_FIG     = os.path.join(OUT_ROOT, "figures")
for p in [DL_ROOT, NPY_DIR, OUT_ROOT, OUT_TAB, OUT_MET, OUT_FIG]:
    os.makedirs(p, exist_ok=True)

# ------------- 1) Download (OpenNeuro) -------------
def download_openneuro(ds_id, target, includes):
    try:
        import openneuro as on
    except Exception:
        import sys, subprocess
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "openneuro-py"])
        import openneuro as on
    print(f"[download] OpenNeuro: {ds_id}")
    ok_any = False
    for patt in includes:
        try:
            on.download(dataset=ds_id, target=target, include=[patt], strict=False)
            print(f"  included: {patt}")
            ok_any = True
        except Exception as e:
            print(f"  [skip] include {patt}: {e}")
    return ok_any

if not PRIMATE_DS:
    print("[info] Set PRIMATE_DS to a valid OpenNeuro dataset ID and re-run to download.")
else:
    ok = download_openneuro(PRIMATE_DS, DL_ROOT, INCLUDES)
    if not ok:
        print("[warn] No files matched include patterns; check dataset ID or INCLUDES.")

# ------------- 2) Convert → NPY per subject -------------
def ensure_dir(p): os.makedirs(p, exist_ok=True); return p

def try_load_file(fp):
    fp_l = fp.lower()
    if fp_l.endswith(".edf"):
        try:
            import mne
        except Exception:
            import sys, subprocess
            subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "mne", "pooch"])
            import mne
        raw = mne.io.read_raw_edf(fp, preload=True, verbose="ERROR")
        raw.pick_types(eeg=True, eog=False, ecg=False, emg=False, stim=False, misc=False)
        X  = raw.get_data()
        fs = float(raw.info["sfreq"])
        ch = list(raw.ch_names)
        return X, fs, ch
    elif fp_l.endswith(".mat"):
        from scipy.io import loadmat
        m = loadmat(fp)
        arr = None
        for k,v in m.items():
            if isinstance(v, np.ndarray) and v.ndim == 2 and (arr is None or v.size > arr.size):
                arr = v
        if arr is None:
            raise RuntimeError("No 2D array in MAT")
        fs = float(m.get("fs", np.array([[FS_TARGET]])).squeeze())
        ch = [f"ch{i}" for i in range(arr.shape[0])]
        return arr.astype(float), fs, ch
    elif fp_l.endswith(".npy"):
        X = np.load(fp)
        if X.ndim != 2: raise RuntimeError("NPY must be [n_ch, n_t]")
        ch = [f"ch{i}" for i in range(X.shape[0])]
        return X.astype(float), FS_TARGET, ch
    else:
        raise RuntimeError(f"Unsupported file: {fp}")

def resample_if_needed(X, fs_in, fs_out):
    if abs(fs_in - fs_out) < 1e-6:
        return X, fs_in
    q = int(round(fs_in / fs_out))
    if q >= 1 and abs(fs_in / q - fs_out) < 1e-3:
        Y = np.vstack([decimate(X[i], q, ftype="fir", zero_phase=True) for i in range(X.shape[0])])
        return Y, fs_out
    # if non-integer, keep original (or implement resample_poly if needed)
    return X, fs_in

def export_npys_from_download(ds_id, raw_root, out_dir, fs_out=FS_TARGET, slice_sec=SLICE_SEC, subj_limit=None):
    files = []
    for patt in INCLUDES + ["*.edf","*.npy","*.mat"]:
        files.extend(glob.glob(os.path.join(raw_root, ds_id, "**", patt), recursive=True))
    files = sorted(list(set(files)))
    out_paths = {}
    subj_idx = 0
    for fp in files:
        try:
            X, fs, ch = try_load_file(fp)
            X, fs2 = resample_if_needed(X, fs, fs_out)
            n_keep = int(slice_sec * fs2)
            if X.shape[1] >= n_keep:
                Xo = X[:, :n_keep]
            else:
                reps = int(np.ceil(n_keep / X.shape[1])); Xo = np.tile(X, reps)[:, :n_keep]
            # derive a subject + condition tag if possible from path (EC/EO), else REST
            tag = "REST"
            low = fp.lower()
            if re.search(r"(eyes[_\-]?open|eo|restopen)", low): tag = "EO"
            if re.search(r"(eyes[_\-]?closed|ec|restclosed)", low): tag = "EC"
            base = os.path.join(out_dir, f"subject_{subj_idx:02d}_{tag}")
            np.save(base + ".npy", Xo.astype(np.float32))
            with open(base + ".channels.txt","w",encoding="utf-8") as f:
                for nm in ch: f.write(str(nm)+"\n")
            out_paths.setdefault(subj_idx, {})[tag] = base + ".npy"
            subj_idx += 1
            if subj_limit and subj_idx >= subj_limit:
                break
        except Exception as e:
            print("[skip]", fp, e)
    return out_paths

paths_by_subj = {}
if PRIMATE_DS:
    paths_by_subj = export_npys_from_download(PRIMATE_DS, DL_ROOT, NPY_DIR, fs_out=FS_TARGET, slice_sec=SLICE_SEC, subj_limit=None)
    print(f"[convert] prepared {len(paths_by_subj)} subjects into", NPY_DIR)
else:
    print("[convert] Skipped (no dataset id set). You can re-run after setting PRIMATE_DS.")

# ------------- 3) PLI + spectral-on-coassoc (k=2) -------------
def clean_label(x: str) -> str:
    y = x.strip()
    y = re.sub(r"(?i)^(EEG|LFP|ECOG)[\s_\-]+", "", y)
    y = re.sub(r"(?i)[\s_\-]*(REF|AV|AVERAGE|AVG|M1|M2)$", "", y)
    y = re.sub(r"[ \-\.]+", "", y)
    y = y.replace("FP","Fp")
    return y

LEFT_CANON  = set(map(str.upper, ["Fp1","AF7","AF3","F7","F5","F3","F1","FT7","FC5","FC3","FC1","T7","C5","C3","C1","TP7","CP5","CP3","CP1","P7","P5","P3","P1","PO7","PO3","O1"]))
RIGHT_CANON = set(map(str.upper, ["Fp2","AF8","AF4","F8","F6","F4","F2","FT8","FC6","FC4","FC2","T8","C6","C4","C2","TP8","CP6","CP4","CP2","P8","P6","P4","P2","PO8","PO4","O2"]))
MID_CANON   = set(map(str.upper, ["Fpz","AFz","Fz","FCz","Cz","CPz","Pz","POz","Oz"]))
ANT_PREFIXES=("Fp","AF","F","FC"); MID_PREFIXES=("C",); POST_PREFIXES=("CP","P","PO","O")

def hemi_ap_map(ch_names):
    L,R,Z,A,P,C = [],[],[],[],[],[]
    for i, raw in enumerate(ch_names):
        name = clean_label(raw); up = name.upper()
        # Hemisphere
        if up in LEFT_CANON:  L.append(i)
        elif up in RIGHT_CANON: R.append(i)
        elif up in MID_CANON or re.search(r"[A-Za-z]Z$", name): Z.append(i)
        else:
            m = re.search(r"(\d+)$", name)
            if m:
                try:
                    d=int(m.group(1)); (L if d%2==1 else R).append(i)
                except: Z.append(i)
            else:
                Z.append(i)
        # AP
        if any(up.startswith(px.upper()) for px in ANT_PREFIXES): A.append(i)
        elif any(up.startswith(px.upper()) for px in POST_PREFIXES): P.append(i)
        elif up.startswith("C"): C.append(i)
        else: C.append(i)
    return np.array(L,int), np.array(R,int), np.array(Z,int), np.array(A,int), np.array(P,int), np.array(C,int)

def pli_matrix(X, fs, lo, hi):
    n=X.shape[0]
    b,a = butter(4, [lo/(fs/2), hi/(fs/2)], btype="band")
    Y = np.zeros_like(X)
    for c in range(n): Y[c] = filtfilt(b,a,X[c])
    ph = np.angle(hilbert(Y, axis=1))
    W  = np.zeros((n,n), float)
    for i in range(n):
        for j in range(i+1, n):
            d = ph[i]-ph[j]
            W[i,j] = W[j,i] = abs(np.mean(np.sign(np.sin(d))))
    np.fill_diagonal(W, 0.0)
    return W

def knn(W,k):
    W=W.copy(); n=W.shape[0]
    for i in range(n):
        idx=np.argsort(W[i])[::-1]; keep=idx[:k]
        mask=np.ones(n,bool); mask[keep]=False; W[i,mask]=0
    W=np.maximum(W,W.T); np.fill_diagonal(W,0); return W

def lap(W):
    d=W.sum(1); d=np.where(d<=1e-12,1.0,d); D=np.diag(1.0/np.sqrt(d))
    return np.eye(W.shape[0]) - D@W@D

def spec_labels(W, k=2):
    e,v = np.linalg.eigh(lap(W))
    U   = v[:,1:k] if k>1 else v[:,:1]
    U   = U / (np.linalg.norm(U, axis=1, keepdims=True) + 1e-12)
    return KMeans(n_clusters=k, n_init=50, random_state=42).fit_predict(U)

def coassoc(labels):
    n=len(labels[0]); m=len(labels); co=np.zeros((n,n),float)
    for lab in labels:
        for i in range(n):
            li=lab[i]
            for j in range(n): co[i,j]+=1.0 if li==lab[j] else 0.0
    return co/m

def loso_via_coassoc(label_list):
    from sklearn.metrics import adjusted_rand_score
    cof=coassoc(label_list); cons=spec_labels(cof, k=2)
    vals=[]
    for s in range(len(label_list)):
        leave=[lab for i,lab in enumerate(label_list) if i!=s]
        cons_l=spec_labels(coassoc(leave), k=2)
        vals.append(adjusted_rand_score(cons, cons_l))
    return float(np.median(vals)), cons, cof

def run_pli_consensus(npy_dir, tag="REST"):
    # group subjects by condition
    subjects = {}
    for fp in sorted(glob.glob(os.path.join(npy_dir, "subject_*_*.npy"))):
        m = re.search(r"subject_(\d+)_(\w+)\.npy$", fp)
        if not m: continue
        sid, cond = int(m.group(1)), m.group(2).upper()
        subjects.setdefault(sid, {})[cond] = fp

    # choose a condition to analyze for consensus (REST/EC/EO)
    conds_present = set(c for d in subjects.values() for c in d.keys())
    to_run = sorted(list(conds_present))
    if not to_run:
        print("[warn] No subjects in", npy_dir); return
    rng = default_rng(13)

    for cond in to_run:
        paths = [d[cond] for d in subjects.values() if cond in d]
        if len(paths) < 4:
            print(f"[{cond}] not enough subjects:", len(paths)); continue
        # channel names from first file
        ch = []
        ch_txt = paths[0].replace(".npy",".channels.txt")
        if os.path.exists(ch_txt):
            with open(ch_txt,"r",encoding="utf-8") as f:
                ch=[ln.strip() for ln in f if ln.strip()]
        else:
            X0 = np.load(paths[0], mmap_mode="r")
            ch=[f"ch{i}" for i in range(X0.shape[0])]
        L,R,Z,A,P,C = hemi_ap_map(ch)

        rows=[]
        for band,(lo,hi) in BANDS_HZ.items():
            subj_labels=[]
            for p in paths:
                X=np.load(p); W=pli_matrix(X, FS_TARGET, lo, hi); W=knn(W, KNN_K); lbl=spec_labels(W, k=K_FIXED)
                subj_labels.append(lbl)
            loso, cons, co = loso_via_coassoc(subj_labels)
            # label-preserve null
            null=[]
            from sklearn.metrics import adjusted_rand_score
            for _ in range(NULL_PERMS):
                nlabs=[]
                for lab in subj_labels:
                    # same-size random labels
                    uniq,cnts = np.unique(lab, return_counts=True)
                    idx = np.arange(len(lab)); rng.shuffle(idx)
                    out = np.empty(len(lab),int); st=0
                    for u,c in zip(uniq,cnts):
                        seg=idx[st:st+c]; out[seg]=u; st+=c
                    nlabs.append(out)
                _, cons_n, _ = loso_via_coassoc(nlabs)
                null.append(adjusted_rand_score(cons, cons_n))
            null=np.array(null,float); p=float((np.sum(null>=loso)+1)/(len(null)+1))
            # save
            np.save(os.path.join(OUT_TAB, f"primate__{cond}__{band}__consensus_labels.npy"), cons)
            np.save(os.path.join(OUT_TAB, f"primate__{cond}__{band}__coassoc.npy"), co)
            with open(os.path.join(OUT_MET, f"primate__{cond}__{band}__metrics.json"),"w",encoding="utf-8") as f:
                json.dump({"band":band,"cond":cond,"n_subj":len(paths),"LOSO":float(loso),"null_mean":float(null.mean()),"p_value":p}, f, indent=2)
            plt.figure(); plt.imshow(co, aspect='auto'); plt.title(f"Primate {cond} {band} — co-assoc")
            plt.colorbar(); plt.tight_layout()
            plt.savefig(os.path.join(OUT_FIG, f"primate__{cond}__{band}__coassoc.png"), dpi=160); plt.close()
            rows.append([cond, band, len(paths), len(ch), float(loso), float(null.mean()), p])
        df = pd.DataFrame(rows, columns=["cond","band","n_subjects","n_channels","LOSO","null_mean","p_value"])
        df.to_csv(os.path.join(OUT_ROOT, f"primate__{cond}__summary.csv"), index=False)
        print(f"[{cond}] summary:\n", df.to_string(index=False))

run_pli_consensus(NPY_DIR)

# ------------- 4) Optional primate EC vs EO classifier (α/θ coupling) -------------
def hemi_module_means(W, labels, L, R):
    n=len(labels)
    Lmask=np.zeros((n,n),bool); Lmask[np.ix_(L,L)]=True
    Rmask=np.zeros((n,n),bool); Rmask[np.ix_(R,R)]=True
    within=Lmask|Rmask
    cross =np.zeros((n,n),bool); cross[np.ix_(L,R)]=True; cross[np.ix_(R,L)]=True
    same  =labels[:,None]==labels[None,:]
    for m in (within,cross,same): np.fill_diagonal(m,False)
    return float((W[within&same]).mean()) if np.any(within&same) else np.nan, \
           float((W[cross &same]).mean()) if np.any(cross &same) else np.nan

def build_minimal_features(NPY_DIR):
    # build subject dict with EC+EO
    subs = {}
    for fp in sorted(glob.glob(os.path.join(NPY_DIR, "subject_*_*.npy"))):
        m = re.search(r"subject_(\d+)_(\w+)\.npy$", fp)
        if not m: continue
        sid, cond = int(m.group(1)), m.group(2).upper()
        subs.setdefault(sid, {})[cond] = fp
    pairs = {k:v for k,v in subs.items() if "EC" in v and "EO" in v}
    if not pairs: return None, None, None
    any_sid = next(iter(pairs.keys()))
    ch_txt = pairs[any_sid]["EC"].replace(".npy",".channels.txt")
    if os.path.exists(ch_txt):
        with open(ch_txt,"r",encoding="utf-8") as f:
            ch=[ln.strip() for ln in f if ln.strip()]
    else:
        X0 = np.load(pairs[any_sid]["EC"], mmap_mode="r")
        ch=[f"ch{i}" for i in range(X0.shape[0])]
    # maps
    L,R,Z,A,P,C = hemi_ap_map(ch)
    rows=[]
    for sid, d in pairs.items():
        for cond,path in d.items():
            X=np.load(path)
            row={"subject":sid, "cond": 0 if cond=="EC" else 1}
            for band,(lo,hi) in {"alpha":(8,12), "theta":(4,8)}.items():
                W=pli_matrix(X, FS_TARGET, lo, hi); W=knn(W,KNN_K); lbl=spec_labels(W,k=K_FIXED)
                wh, chm = hemi_module_means(W, lbl, L, R)
                row[f"{band}_WHWM"]=wh; row[f"{band}_CHWM"]=chm
            rows.append(row)
    df = pd.DataFrame(rows).dropna()
    X = df[[c for c in df.columns if c.endswith("_WHWM") or c.endswith("_CHWM")]].to_numpy(float)
    y = df["cond"].to_numpy(int)
    s = df["subject"].to_numpy(int)
    return X,y,s

def paired_auc_perm(X, y, subj_ids, perm_B=PERM_B_CLF):
    clf = make_pipeline(SimpleImputer(strategy="median"), StandardScaler(), LogisticRegression(max_iter=2000, solver="lbfgs"))
    rng = default_rng(17)
    fold_probs=[]; fold_true=[]; order_pairs=[]
    u_subs = sorted(set(subj_ids))
    for sid in u_subs:
        test = (subj_ids == sid)
        if np.sum(test) != 2: continue
        train = ~test
        clf.fit(X[train], y[train])
        p = clf.predict_proba(X[test])[:,1] if hasattr(clf[-1],"predict_proba") else clf.decision_function(X[test])
        fold_probs.append(p); fold_true.append(y[test]); order_pairs.append(np.where(test)[0])
    if not fold_probs: return None, None
    probs=np.concatenate(fold_probs); ys=np.concatenate(fold_true)
    if len(np.unique(ys))<2: return None, None
    auc_obs=float(roc_auc_score(ys, probs))
    cnt=1
    for _ in range(perm_B):
        y_perm=ys.copy()
        idx=0
        for sid in u_subs:
            if idx>=len(order_pairs): break
            pidx=order_pairs[idx]
            if len(pidx)!=2: idx+=1; continue
            if rng.random()<0.5: y_perm[pidx]=y_perm[pidx][::-1]
            idx+=1
        auc_p=float(roc_auc_score(y_perm, probs))
        if auc_p>=auc_obs: cnt+=1
    return auc_obs, float(cnt/(perm_B+1))

Xc, yc, sc = build_minimal_features(NPY_DIR)
if Xc is not None:
    auc_obs, p_perm = paired_auc_perm(Xc, yc, sc, perm_B=PERM_B_CLF)
    if auc_obs is not None:
        print(f"[primate EC vs EO] minimal α/θ coupling → AUC={auc_obs:.3f}, p={p_perm:.4f}  (N={Xc.shape[0]} points)")
        pd.DataFrame([{"AUC":auc_obs,"p_perm":p_perm,"N_points":Xc.shape[0],"N_subjects":len(set(sc))}]).to_csv(
            os.path.join(OUT_ROOT,"primate_ec_eo_minimal.csv"), index=False
        )
    else:
        print("[info] Not enough class diversity to compute AUC.")
else:
    print("[info] No EC+EO pairs found; classifier skipped.")

print("\nArtifacts →", OUT_ROOT)
print("If download found nothing, set PRIMATE_DS='ds00XXXX' and re-run.")


[info] Set PRIMATE_DS to a valid OpenNeuro dataset ID and re-run to download.
[convert] Skipped (no dataset id set). You can re-run after setting PRIMATE_DS.
[REST] summary:
 cond  band  n_subjects  n_channels     LOSO  null_mean  p_value
REST theta           8          32 1.000000  -0.002644 0.003322
REST alpha           8          32 0.653250  -0.001480 0.003322
REST  beta           8          32 0.598839  -0.000612 0.003322
REST gamma           8          32 0.459278   0.000912 0.003322
[info] No EC+EO pairs found; classifier skipped.

Artifacts → C:\Users\caleb\CNT_Lab\artifacts\pli_primate_dl
If download found nothing, set PRIMATE_DS='ds00XXXX' and re-run.


In [2]:
# === OpenNeuro NHP Finder → Download → Convert → CNT PLI Consensus + (optional) EC/EO Classifier ===
# What this cell does:
#  1) Queries OpenNeuro GraphQL for non-human primate (macaque/monkey) datasets with EEG/LFP/ECoG/iEEG/MEG keywords
#  2) Prints a candidate table (ID, title, modalities). You can:
#       - Set AUTO_SELECT_TOP = K to auto-pick top K hits
#       - Or provide SELECTED_IDS = ["ds00xxxx", ...] to force specific datasets
#  3) Downloads matching files (narrow include patterns), converts to *.npy (+ channels.txt), trims to 60 s @ 1 kHz
#  4) Runs CNT PLI spectral-on-coassoc (k=2) for θ/α/β/γ per condition (REST/EC/EO if present), prints LOSO + perm p
#  5) If EC+EO pairs exist → α/θ minimal classifier (paired CV + 10k perms), prints AUC + p
#
# Outputs → C:\Users\caleb\CNT_Lab\artifacts\pli_primate_dl\{metrics,tables,figures}
# Exported *.npy → C:\Users\caleb\CNT_Lab\primate_eeg

import os, re, json, time, glob, requests, numpy as np, pandas as pd, matplotlib.pyplot as plt
from numpy.random import default_rng
from scipy.signal import butter, filtfilt, hilbert, decimate
from sklearn.cluster import KMeans
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

# ---------------- CONFIG ----------------
ROOT         = r"C:\Users\caleb\CNT_Lab"
OUT_ROOT     = os.path.join(ROOT, r"artifacts\pli_primate_dl")
DL_ROOT      = os.path.join(ROOT, "primate_openneuro_raw")
NPY_DIR      = os.path.join(ROOT, "primate_eeg")
OUT_TAB      = os.path.join(OUT_ROOT, "tables")
OUT_MET      = os.path.join(OUT_ROOT, "metrics")
OUT_FIG      = os.path.join(OUT_ROOT, "figures")
for p in [OUT_ROOT, OUT_TAB, OUT_MET, OUT_FIG, DL_ROOT, NPY_DIR]: os.makedirs(p, exist_ok=True)

# Search controls
AUTO_SELECT_TOP = 2               # auto-pick top K datasets (set 0 to disable)
SELECTED_IDS    = []              # or force specific OpenNeuro IDs, e.g., ["ds004***", "ds00****"]
SEARCH_TERMS    = ["macaque","macaca","monkey","nonhuman primate"]
MODALITY_FILTER = ["EEG","ECoG","iEEG","LFP","MEG"]  # keep only datasets mentioning these
MAX_LIST        = 20

# Download/convert controls
FS_TARGET   = 1000.0      # resample to 1 kHz
SLICE_SEC   = 60          # keep 60 s per subject
INCLUDES    = ["*eeg*.edf","*eeg*.mat","*eeg*.npy",
               "*lfp*.edf","*lfp*.mat","*lfp*.npy",
               "*ecog*.edf","*ecog*.mat","*ecog*.npy",
               "*ieeg*.edf","*ieeg*.mat","*ieeg*.npy",
               "*meg*.fif","*meg*.mat"]
# CNT analysis
BANDS_HZ    = {"theta":(4,8),"alpha":(8,12),"beta":(13,30),"gamma":(30,55)}
K_FIXED     = 2
KNN_K       = 6
NULL_PERMS  = 500
CLF_PERMS   = 10000
rng         = default_rng(13)

# ---------------- 0) Install openneuro-py if needed ----------------
try:
    import openneuro as on
except Exception:
    import sys, subprocess
    subprocess.check_call([sys.executable,"-m","pip","install","-q","openneuro-py"])
    import openneuro as on

# ---------------- 1) Search OpenNeuro via GraphQL ----------------
GQL = "https://openneuro.org/crn/graphql"
def search_openneuro(terms, max_n=MAX_LIST):
    q = """
    query($query:String!, $first:Int!) {
      datasets(query:$query, first:$first) {
        edges { node { id created published modified public snapshots { id tag } metadata { modalities } name } }
      }
    }
    """
    hits = []
    for t in terms:
        try:
            r = requests.post(GQL, json={"query": q, "variables":{"query": t, "first": max_n}}, timeout=30)
            r.raise_for_status()
            data = r.json()
            edges = data.get("data",{}).get("datasets",{}).get("edges",[])
            for e in edges:
                node = e.get("node",{})
                dsid = node.get("id")
                name = node.get("name") or ""
                mods = node.get("metadata",{}).get("modalities") or []
                hits.append({"id": dsid, "name": name, "modalities": mods})
        except Exception as ex:
            print("[warn] search error for", t, ex)
    # de-duplicate by id
    uniq = {}
    for h in hits:
        uniq[h["id"]] = {"id": h["id"], "name": h["name"], "modalities": h["modalities"]}
    rows = []
    for v in uniq.values():
        mods = [m.upper() for m in (v["modalities"] or [])]
        keep = any(m in mods for m in MODALITY_FILTER) or any(m in v["name"].upper() for m in MODALITY_FILTER)
        rows.append({"id": v["id"], "title": v["name"], "modalities": ",".join(mods), "keep": keep})
    df = pd.DataFrame(rows).sort_values("id").reset_index(drop=True)
    return df

print("→ Searching OpenNeuro for primate datasets...")
df_hits = search_openneuro(SEARCH_TERMS, max_n=MAX_LIST)
if df_hits.empty:
    print("No hits. Try adjusting SEARCH_TERMS or run again later.")
else:
    print("\n=== Candidate NHP datasets (filtering modalities: {}) ===".format(",".join(MODALITY_FILTER)))
    print(df_hits.to_string(index=False))

# Decide which to download
to_get = []
if SELECTED_IDS:
    to_get = [ds for ds in SELECTED_IDS if ds in set(df_hits["id"])]
elif AUTO_SELECT_TOP > 0 and not df_hits.empty:
    to_get = df_hits[df_hits["keep"]].head(AUTO_SELECT_TOP)["id"].tolist()
print("\nSelected datasets:", to_get if to_get else "(none)")

# ---------------- 2) Download selected datasets ----------------
def download_openneuro(ds_id, includes):
    print(f"[download] {ds_id}")
    ok_any = False
    for patt in includes:
        try:
            on.download(dataset=ds_id, target=DL_ROOT, include=[patt], strict=False)
            print("   included:", patt); ok_any = True
        except Exception as e:
            # harmless if pattern not present
            pass
    if not ok_any:
        print("   (no matched files for include patterns)")
    return ok_any

if to_get:
    for ds in to_get:
        download_openneuro(ds, INCLUDES)

# ---------------- 3) Convert → NPY (+ channels) ----------------
def try_load_file(fp):
    fp_l = fp.lower()
    if fp_l.endswith(".edf"):
        try:
            import mne
        except Exception:
            import sys, subprocess
            subprocess.check_call([sys.executable,"-m","pip","install","-q","mne","pooch"])
            import mne
        raw = mne.io.read_raw_edf(fp, preload=True, verbose="ERROR")
        raw.pick_types(eeg=True, eog=False, ecg=False, emg=False, stim=False, misc=False)
        X  = raw.get_data()
        fs = float(raw.info["sfreq"])
        ch = list(raw.ch_names)
        return X, fs, ch
    elif fp_l.endswith(".mat"):
        from scipy.io import loadmat
        m = loadmat(fp)
        arr = None
        for k,v in m.items():
            if isinstance(v, np.ndarray) and v.ndim == 2 and (arr is None or v.size > arr.size):
                arr = v
        if arr is None:
            raise RuntimeError("No 2D array in MAT")
        fs = float(m.get("fs", np.array([[FS_TARGET]])).squeeze())
        ch = [f"ch{i}" for i in range(arr.shape[0])]
        return arr.astype(float), fs, ch
    elif fp_l.endswith(".npy"):
        X = np.load(fp)
        if X.ndim != 2: raise RuntimeError("NPY must be [n_ch, n_t]")
        ch = [f"ch{i}" for i in range(X.shape[0])]
        return X.astype(float), FS_TARGET, ch
    elif fp_l.endswith(".fif"):
        try:
            import mne
        except Exception:
            import sys, subprocess
            subprocess.check_call([sys.executable,"-m","pip","install","-q","mne","pooch"])
            import mne
        raw = mne.io.read_raw_fif(fp, preload=True, verbose="ERROR")
        raw.pick_types(meg=True, eeg=True, stim=False, eog=False, ecg=False, emg=False)
        X = raw.get_data()
        fs= float(raw.info["sfreq"])
        ch = list(raw.ch_names)
        return X, fs, ch
    else:
        raise RuntimeError(f"Unsupported file: {fp}")

def resample_if_needed(X, fs_in, fs_out):
    if abs(fs_in - fs_out) < 1e-6:
        return X, fs_in
    q = int(round(fs_in / fs_out))
    if q >= 1 and abs(fs_in / q - fs_out) < 1e-3:
        Y = np.vstack([decimate(X[i], q, ftype="fir", zero_phase=True) for i in range(X.shape[0])])
        return Y, fs_out
    # keep as-is if not close (or add resample_poly if you prefer)
    return X, fs_in

def export_npys_from_download(raw_root, out_dir, fs_out=FS_TARGET, slice_sec=SLICE_SEC):
    files = []
    for patt in INCLUDES + ["*.edf","*.mat","*.npy","*.fif"]:
        files.extend(glob.glob(os.path.join(raw_root, "**", patt), recursive=True))
    files = sorted(list(set(files)))
    out_paths = {}
    subj_idx = 0
    for fp in files:
        try:
            X, fs, ch = try_load_file(fp)
            X, fs2 = resample_if_needed(X, fs, fs_out)
            n_keep = int(slice_sec * fs2)
            if X.shape[1] >= n_keep:
                Xo = X[:, :n_keep]
            else:
                reps = int(np.ceil(n_keep / X.shape[1])); Xo = np.tile(X, reps)[:, :n_keep]
            # naive condition detection
            tag = "REST"
            low = fp.lower()
            if any(k in low for k in ["eyesopen","eo","open"]): tag = "EO"
            if any(k in low for k in ["eyesclosed","ec","closed"]): tag = "EC"
            base = os.path.join(out_dir, f"subject_{subj_idx:02d}_{tag}")
            np.save(base+".npy", Xo.astype(np.float32))
            with open(base+".channels.txt","w",encoding="utf-8") as f:
                for nm in ch: f.write(str(nm)+"\n")
            out_paths.setdefault(subj_idx, {})[tag] = base+".npy"
            subj_idx += 1
        except Exception as e:
            # Just skip files that don't parse
            pass
    return out_paths

paths_by_subj = export_npys_from_download(DL_ROOT, NPY_DIR)
print(f"[convert] prepared {len(paths_by_subj)} subjects in", NPY_DIR)

# ---------------- 4) CNT PLI + spectral-on-coassoc (k=2) ----------------
def bandpass(x, fs, lo, hi, order=4):
    b,a=butter(order, [lo/(fs/2), hi/(fs/2)], btype="band"); return filtfilt(b,a,x)
def pli_matrix(X, fs, lo, hi):
    n=X.shape[0]
    b,a=butter(4, [lo/(fs/2), hi/(fs/2)], btype="band")
    Y=np.zeros_like(X)
    for c in range(n): Y[c]=filtfilt(b,a,X[c])
    ph=np.angle(hilbert(Y, axis=1))
    W=np.zeros((n,n), float)
    for i in range(n):
        for j in range(i+1, n):
            d=ph[i]-ph[j]; W[i,j]=W[j,i]=abs(np.mean(np.sign(np.sin(d))))
    np.fill_diagonal(W,0.0); return W
def knn(W,k):
    W=W.copy(); n=W.shape[0]
    for i in range(n):
        idx=np.argsort(W[i])[::-1]; keep=idx[:k]
        mask=np.ones(n,bool); mask[keep]=False; W[i,mask]=0
    W=np.maximum(W,W.T); np.fill_diagonal(W,0); return W
def lap(W):
    d=W.sum(1); d=np.where(d<=1e-12,1.0,d); D=np.diag(1.0/np.sqrt(d)); return np.eye(W.shape[0]) - D@W@D
def spec_labels(W,k=2):
    e,v=np.linalg.eigh(lap(W)); U=v[:,1:k] if k>1 else v[:,:1]
    U/= (np.linalg.norm(U, axis=1, keepdims=True)+1e-12)
    return KMeans(n_clusters=k, n_init=50, random_state=13).fit_predict(U)
def coassoc(labels):
    n=len(labels[0]); m=len(labels); co=np.zeros((n,n),float)
    for lab in labels:
        for i in range(n):
            li=lab[i]
            for j in range(n): co[i,j]+=1 if li==lab[j] else 0
    return co/m
from sklearn.metrics import adjusted_rand_score

def loso_via_coassoc(label_list):
    cof=coassoc(label_list); cons=spec_labels(cof, k=2)
    vals=[]
    for s in range(len(label_list)):
        leave=[lab for i,lab in enumerate(label_list) if i!=s]
        cons_l=spec_labels(coassoc(leave), k=2)
        vals.append(adjusted_rand_score(cons, cons_l))
    return float(np.median(vals)), cons, cof

def run_pli_consensus(npy_dir):
    subjects={}
    for fp in sorted(glob.glob(os.path.join(npy_dir, "subject_*_*.npy"))):
        m=re.search(r"subject_(\d+)_(\w+)\.npy$", fp)
        if not m: continue
        sid, cond = int(m.group(1)), m.group(2).upper()
        subjects.setdefault(sid, {})[cond]=fp
    conds=set(c for d in subjects.values() for c in d.keys())
    if not conds:
        print("[warn] No subjects found.")
        return
    for cond in sorted(conds):
        paths=[d[cond] for d in subjects.values() if cond in d]
        if len(paths)<4:
            print(f"[{cond}] not enough subjects:", len(paths)); continue
        # first file channels
        ch_txt = paths[0].replace(".npy",".channels.txt")
        if os.path.exists(ch_txt):
            with open(ch_txt,"r",encoding="utf-8") as f: ch=[ln.strip() for ln in f if ln.strip()]
        else:
            X0=np.load(paths[0], mmap_mode="r"); ch=[f"ch{i}" for i in range(X0.shape[0])]
        rows=[]
        for band,(lo,hi) in BANDS_HZ.items():
            labs=[]
            for p in paths:
                X=np.load(p); W=pli_matrix(X, FS_TARGET, lo, hi); W=knn(W,KNN_K); labs.append(spec_labels(W,k=K_FIXED))
            loso, cons, co = loso_via_coassoc(labs)
            # label-preserving null
            null=[]
            for _ in range(NULL_PERMS):
                nl=[]
                for lab in labs:
                    uniq,cnts=np.unique(lab, return_counts=True)
                    idx=np.arange(len(lab)); rng.shuffle(idx)
                    out=np.empty(len(lab),int); st=0
                    for u,c in zip(uniq,cnts):
                        seg=idx[st:st+c]; out[seg]=u; st+=c
                    nl.append(out)
                _, cons_n, _ = loso_via_coassoc(nl)
                null.append(adjusted_rand_score(cons, cons_n))
            null=np.array(null,float)
            p=float((np.sum(null>=loso)+1)/(len(null)+1))
            # save
            np.save(os.path.join(OUT_TAB, f"primate__{cond}__{band}__consensus_labels.npy"), cons)
            np.save(os.path.join(OUT_TAB, f"primate__{cond}__{band}__coassoc.npy"), co)
            with open(os.path.join(OUT_MET, f"primate__{cond}__{band}__metrics.json"),"w",encoding="utf-8") as f:
                json.dump({"cond":cond,"band":band,"n_subjects":len(paths),"LOSO":float(loso),"null_mean":float(null.mean()),"p":p}, f, indent=2)
            plt.figure(); plt.imshow(co, aspect='auto'); plt.title(f"{cond} {band} — co-assoc"); plt.colorbar(); plt.tight_layout()
            plt.savefig(os.path.join(OUT_FIG, f"primate__{cond}__{band}__coassoc.png"), dpi=160); plt.close()
            rows.append([cond, band, len(paths), len(ch), float(loso), float(null.mean()), p])
        df=pd.DataFrame(rows, columns=["cond","band","n_subjects","n_channels","LOSO","null_mean","p"])
        df.to_csv(os.path.join(OUT_ROOT, f"primate__{cond}__summary.csv"), index=False)
        print(f"[{cond}] summary:\n", df.to_string(index=False))

run_pli_consensus(NPY_DIR)

# ---------------- 5) Optional primate EC/EO classifier (α/θ minimal) ----------------
def hemi_module_means(W, labels, L, R):
    n=len(labels)
    Lmask=np.zeros((n,n),bool); Lmask[np.ix_(L,L)]=True
    Rmask=np.zeros((n,n),bool); Rmask[np.ix_(R,R)]=True
    within=Lmask|Rmask
    cross =np.zeros((n,n),bool); cross[np.ix_(L,R)]=True; cross[np.ix_(R,L)]=True
    same  =labels[:,None]==labels[None,:]
    for m in (within,cross,same): np.fill_diagonal(m,False)
    return float((W[within&same]).mean()) if np.any(within&same) else np.nan, \
           float((W[cross &same]).mean())  if np.any(cross &same)  else np.nan

def build_minimal_features(npy_dir):
    subs={}
    for fp in sorted(glob.glob(os.path.join(npy_dir,"subject_*_*.npy"))):
        m=re.search(r"subject_(\d+)_(\w+)\.npy$", fp)
        if not m: continue
        sid,cond=int(m.group(1)), m.group(2).upper()
        subs.setdefault(sid,{})[cond]=fp
    pairs={k:v for k,v in subs.items() if "EC" in v and "EO" in v}
    if not pairs: return None,None,None
    any_sid=next(iter(pairs))
    ch_txt = pairs[any_sid]["EC"].replace(".npy",".channels.txt")
    if os.path.exists(ch_txt):
        with open(ch_txt,"r",encoding="utf-8") as f: ch=[ln.strip() for ln in f if ln.strip()]
    else:
        X0=np.load(pairs[any_sid]["EC"], mmap_mode="r"); ch=[f"ch{i}" for i in range(X0.shape[0])]
    # L/R maps (simple: odd/even and Z)
    L,R,Z = [],[],[]
    for i,name in enumerate(ch):
        up=name.upper()
        if re.search(r"[A-Za-z]Z$", name): Z.append(i)
        else:
            m=re.search(r"(\d+)$", name)
            if m:
                try:
                    d=int(m.group(1)); (L if d%2==1 else R).append(i)
                except: Z.append(i)
            else: Z.append(i)
    rows=[]
    for sid,d in pairs.items():
        for cond,fp in d.items():
            X=np.load(fp)
            row={"sid":sid,"cond":0 if cond=="EC" else 1}
            for band,(lo,hi) in {"alpha":(8,12),"theta":(4,8)}.items():
                W=pli_matrix(X, FS_TARGET, lo, hi); W=knn(W,KNN_K); lbl=spec_labels(W, k=K_FIXED)
                wh,chm=hemi_module_means(W,lbl,np.array(L,int),np.array(R,int))
                row[f"{band}_WHWM"]=wh; row[f"{band}_CHWM"]=chm
            rows.append(row)
    df=pd.DataFrame(rows).dropna()
    X=df[[c for c in df.columns if c.endswith("_WHWM") or c.endswith("_CHWM")]].to_numpy(float)
    y=df["cond"].to_numpy(int)
    s=df["sid"].to_numpy(int)
    return X,y,s

def paired_auc_perm(X,y,s,perm_B=CLF_PERMS):
    clf=make_pipeline(SimpleImputer(strategy="median"), StandardScaler(), LogisticRegression(max_iter=2000, solver="lbfgs"))
    rng=default_rng(31)
    fold_p=[]; fold_y=[]; order=[]
    u=sorted(set(s))
    for sid in u:
        mask=(s==sid)
        if np.sum(mask)!=2: continue
        train=(s!=sid)
        clf.fit(X[train], y[train])
        p=clf.predict_proba(X[mask])[:,1]
        fold_p.append(p); fold_y.append(y[mask]); order.append(np.where(mask)[0])
    if not fold_p: 
        print("[info] No EC/EO pairs; classifier skipped.")
        raise SystemExit
    probs=np.concatenate(fold_p); ys=np.concatenate(fold_y)
    auc=float(roc_auc_score(ys, probs))
    cnt=1
    for _ in range(perm_B):
        yperm=ys.copy()
        # flip within pair (we don't strictly maintain original indices, but permutation is symmetric here)
        for i in range(0,len(yperm),2):
            if rng.random()<0.5 and i+1<len(yperm):
                yperm[i],yperm[i+1]=yperm[i+1],yperm[i]
        ap=float(roc_auc_score(yperm, probs))
        if ap>=auc: cnt+=1
    return auc, float(cnt/(perm_B+1))

try:
    Xc,yc,sc = build_minimal_features(NPY_DIR)
    if Xc is not None:
        auc_obs,p_perm = paired_auc_perm(Xc,yc,sc)
        print(f"\n[primate EC vs EO] minimal α/θ coupling → AUC={auc_obs:.3f}, p={p_perm:.4f}  (N={Xc.shape[0]} points)")
        pd.DataFrame([{"AUC":auc_obs,"p_perm":p_perm,"N_points":Xc.shape[0],"N_subjects":len(set(sc))}]).to_csv(
            os.path.join(OUT_ROOT,"primate_ec_eo_minimal.csv"), index=False
        )
except SystemExit:
    pass

print("\nArtifacts →", OUT_ROOT)
print("If no datasets downloaded, set SELECTED_IDS=['ds00XXXX', ...] or increase AUTO_SELECT_TOP, and re-run.")


→ Searching OpenNeuro for primate datasets...
[warn] search error for macaque 400 Client Error: Bad Request for url: https://openneuro.org/crn/graphql
[warn] search error for macaca 400 Client Error: Bad Request for url: https://openneuro.org/crn/graphql
[warn] search error for monkey 400 Client Error: Bad Request for url: https://openneuro.org/crn/graphql
[warn] search error for nonhuman primate 400 Client Error: Bad Request for url: https://openneuro.org/crn/graphql


KeyError: 'id'

In [3]:
# === NHP datasets without GraphQL: GitHub search → OpenNeuro download (fallback to NeuroTycho/CRCNS/G-Node) ===
# 1) Query GitHub API for OpenNeuroDatasets repos matching macaque/monkey/marmoset & EEG/LFP/ECoG/iEEG keywords.
# 2) Download selected dsIDs via openneuro-py (no GraphQL).
# 3) If none succeed, fallback to known public NHP ECoG/LFP (NeuroTycho tutorial bundle / CRCNS / G-Node).
# 4) Convert to .npy + channels.txt, run PLI + spectral-on-coassoc (k=2), print LOSO + perm p; run α/θ classifier if EC/EO pairs present.

import os, re, json, glob, time, io, zipfile, requests, numpy as np, pandas as pd, matplotlib.pyplot as plt
from numpy.random import default_rng
from scipy.signal import butter, filtfilt, hilbert, decimate
from sklearn.cluster import KMeans
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

ROOT      = r"C:\Users\caleb\CNT_Lab"
OUT_ROOT  = os.path.join(ROOT, r"artifacts\pli_primate_auto")
DL_ROOT   = os.path.join(ROOT, "primate_auto_raw")
NPY_DIR   = os.path.join(ROOT, "primate_eeg")
for p in [OUT_ROOT, DL_ROOT, NPY_DIR, os.path.join(OUT_ROOT,"tables"), os.path.join(OUT_ROOT,"metrics"), os.path.join(OUT_ROOT,"figures")]:
    os.makedirs(p, exist_ok=True)

# ---------- Search settings ----------
GITHUB_SEARCH = "https://api.github.com/search/repositories"
GH_QUERY      = "org:OpenNeuroDatasets (macaque OR monkey OR marmoset) in:name,description"
GH_HEADERS    = {"Accept":"application/vnd.github+json"}  # add a token if you have one to raise rate-limit
MAX_REPOS     = 20
MOD_KEYWORDS  = ["EEG","ECOG","IEEG","LFP","MEG"]

# ---------- Conversion / analysis settings ----------
FS_TARGET   = 1000.0
SLICE_SEC   = 60
INCLUDES    = ["*eeg*.edf","*eeg*.npy","*eeg*.mat",
               "*lfp*.edf","*lfp*.npy","*lfp*.mat",
               "*ecog*.edf","*ecog*.npy","*ecog*.mat",
               "*ieeg*.edf","*ieeg*.npy","*ieeg*.mat"]
BANDS_HZ    = {"theta":(4,8), "alpha":(8,12), "beta":(13,30), "gamma":(30,55)}
K_FIXED     = 2
KNN_K       = 6
NULL_PERMS  = 500
CLF_PERMS   = 10000
rng         = default_rng(13)

# ---------- tiny helpers ----------
def safe_get(url, params=None, headers=None, timeout=30):
    try:
        r = requests.get(url, params=params, headers=headers, timeout=timeout)
        r.raise_for_status()
        return r
    except Exception as e:
        print("[warn] GET failed:", url, e)
        return None

def search_github_openneuro():
    r = safe_get(GITHUB_SEARCH, params={"q": GH_QUERY, "per_page": MAX_REPOS}, headers=GH_HEADERS)
    if not r: return pd.DataFrame()
    data = r.json()
    items = data.get("items", [])
    rows=[]
    for it in items:
        rid = it.get("name","")
        full = it.get("full_name","")
        desc = it.get("description") or ""
        keep = any(k in desc.upper() for k in MOD_KEYWORDS)
        rows.append({"id": rid, "full": full, "title": desc, "keep": keep})
    return pd.DataFrame(rows)

print("→ GitHub fallback search for OpenNeuro dsIDs (no GraphQL)…")
df_hits = search_github_openneuro()
if df_hits.empty:
    print("No OpenNeuro hits via GitHub API; will try fallback datasets.")
else:
    print(df_hits.to_string(index=False))

# ---------- download via openneuro-py by dsID ----------
def on_download(ds_id, target, patterns):
    try:
        import openneuro as on
    except Exception:
        import sys, subprocess
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "openneuro-py"])
        import openneuro as on
    print(f"[download] {ds_id}")
    ok=False
    for patt in patterns:
        try:
            on.download(dataset=ds_id, target=target, include=[patt], strict=False)
            print("   included:", patt); ok=True
        except Exception:
            pass
    if not ok: print("   (no matched files)")
    return ok

selected = df_hits[df_hits["keep"]].head(2)["id"].tolist() if not df_hits.empty else []
if selected:
    for ds in selected:
        on_download(ds, DL_ROOT, INCLUDES)

# ---------- Fallback: NeuroTycho / CRCNS / G-Node -----------
# NeuroTycho demo ECoG (from FieldTrip tutorial mirrors) – small zip hosted by FieldTrip (public)
# If the FieldTrip link changes, you can place any monkey ECoG zip in DL_ROOT and it will be picked up below
FALLBACKS = [
    # (name, direct-zip-url)
    ("fieldtrip_monkey_ecog", "https://github.com/fieldtrip/website/raw/master/_static/download/monkey_ecog/monkey_ecog_data.zip"),
]
for name, url in FALLBACKS:
    if not glob.glob(os.path.join(DL_ROOT, name+"*")):
        print(f"[fallback] trying {name}")
        r = safe_get(url)
        if r and r.status_code==200:
            zf = zipfile.ZipFile(io.BytesIO(r.content))
            zf.extractall(os.path.join(DL_ROOT, name))
            print("   extracted to", os.path.join(DL_ROOT, name))
        else:
            print("   fallback failed:", url)

# ---------- Convert whatever we have to NPY ----------
def try_load_file(fp):
    fp_l = fp.lower()
    if fp_l.endswith(".edf"):
        try:
            import mne
        except Exception:
            import sys, subprocess
            subprocess.check_call([sys.executable,"-m","pip","install","-q","mne","pooch"])
            import mne
        raw = mne.io.read_raw_edf(fp, preload=True, verbose="ERROR")
        raw.pick_types(eeg=True, eog=False, ecg=False, emg=False, stim=False, misc=False)
        X = raw.get_data(); fs = float(raw.info["sfreq"]); ch = list(raw.ch_names)
        return X, fs, ch
    elif fp_l.endswith(".fif"):
        try:
            import mne
        except Exception:
            import sys, subprocess
            subprocess.check_call([sys.executable,"-m","pip","install","-q","mne","pooch"])
            import mne
        raw = mne.io.read_raw_fif(fp, preload=True, verbose="ERROR")
        raw.pick_types(meg=True, eeg=True, stim=False)
        X = raw.get_data(); fs = float(raw.info["sfreq"]); ch = list(raw.ch_names)
        return X, fs, ch
    elif fp_l.endswith(".mat"):
        from scipy.io import loadmat
        m = loadmat(fp)
        arr=None
        for k,v in m.items():
            if isinstance(v,np.ndarray) and v.ndim==2 and (arr is None or v.size>arr.size):
                arr=v
        if arr is None: raise RuntimeError("No 2D array in MAT")
        fs = float(m.get("fs", np.array([[FS_TARGET]])).squeeze())
        ch = [f"ch{i}" for i in range(arr.shape[0])]
        return arr.astype(float), fs, ch
    elif fp_l.endswith(".npy"):
        X = np.load(fp)
        if X.ndim!=2: raise RuntimeError("NPY not 2D")
        ch = [f"ch{i}" for i in range(X.shape[0])]
        return X.astype(float), FS_TARGET, ch
    else:
        raise RuntimeError("Unsupported file: "+fp)

def resample_if_needed(X, fs_in, fs_out):
    if abs(fs_in - fs_out) < 1e-6: return X, fs_in
    q = int(round(fs_in / fs_out))
    if q>=1 and abs(fs_in/q - fs_out) < 1e-3:
        Y = np.vstack([decimate(X[i], q, ftype='fir', zero_phase=True) for i in range(X.shape[0])])
        return Y, fs_out
    return X, fs_in

def export_npys(raw_root, out_dir, slice_sec=SLICE_SEC):
    files=[]
    for patt in INCLUDES + ["*.edf","*.fif","*.mat","*.npy"]:
        files += glob.glob(os.path.join(raw_root,"**",patt), recursive=True)
    files = sorted(list(set(files)))
    out_paths = {}
    idx = 0
    for fp in files:
        try:
            X, fs, ch = try_load_file(fp)
            X, fs2 = resample_if_needed(X, fs, FS_TARGET)
            n_keep = int(slice_sec * fs2)
            Xo = X[:, :n_keep] if X.shape[1]>=n_keep else np.tile(X, int(np.ceil(n_keep/X.shape[1])))[:, :n_keep]
            # condition detection (best effort)
            low = fp.lower(); tag="REST"
            if any(k in low for k in ["eyesopen","eo","open"]): tag="EO"
            if any(k in low for k in ["eyesclosed","ec","closed"]): tag="EC"
            base = os.path.join(out_dir, f"subject_{idx:02d}_{tag}")
            np.save(base+".npy", Xo.astype(np.float32))
            with open(base+".channels.txt","w",encoding="utf-8") as f:
                for nm in ch: f.write(str(nm)+"\n")
            out_paths.setdefault(idx,{})[tag] = base+".npy"
            idx += 1
        except Exception as e:
            pass
    return out_paths

paths = export_npys(DL_ROOT, NPY_DIR)
print(f"[convert] exported {len(paths)} subjects into", NPY_DIR)

# ---------- CNT PLI + spectral-on-coassoc ----------
OUT_TAB = os.path.join(OUT_ROOT, "tables")
OUT_MET = os.path.join(OUT_ROOT, "metrics")
OUT_FIG = os.path.join(OUT_ROOT, "figures")

def pli_matrix(X, fs, lo, hi):
    n=X.shape[0]
    b,a=butter(4, [lo/(fs/2), hi/(fs/2)], btype="band")
    Y=np.zeros_like(X)
    for c in range(n): Y[c]=filtfilt(b,a,X[c])
    ph=np.angle(hilbert(Y, axis=1))
    W=np.zeros((n,n), float)
    for i in range(n):
        for j in range(i+1, n):
            d=ph[i]-ph[j]; W[i,j]=W[j,i]=abs(np.mean(np.sign(np.sin(d))))
    np.fill_diagonal(W,0); return W

def knn(W,k):
    W=W.copy(); n=W.shape[0]
    for i in range(n):
        idx=np.argsort(W[i])[::-1]; keep=idx[:k]
        mask=np.ones(n,bool); mask[keep]=False; W[i,mask]=0
    W=np.maximum(W,W.T); np.fill_diagonal(W,0); return W

def lap(W):
    d=W.sum(1); d=np.where(d<=1e-12,1.0,d); D=np.diag(1.0/np.sqrt(d))
    return np.eye(W.shape[0]) - D@W@D

def spec_labels(W,k=2):
    e,v=np.linalg.eigh(lap(W))
    U=v[:,1:k] if k>1 else v[:,:1]
    U/= (np.linalg.norm(U,axis=1,keepdims=True)+1e-12)
    return KMeans(n_clusters=k, n_init=50, random_state=13).fit_predict(U)

from sklearn.metrics import adjusted_rand_score
def coassoc(labels):
    n=len(labels[0]); m=len(labels); co=np.zeros((n,n), float)
    for lab in labels:
        for i in range(n):
            li=lab[i]
            for j in range(n): co[i,j]+=1 if li==lab[j] else 0
    return co/m

def loso_via_coassoc(label_list):
    cof=coassoc(label_list); cons=spec_labels(cof,k=2)
    vals=[]
    for s in range(len(label_list)):
        leave=[lab for i,lab in enumerate(label_list) if i!=s]
        cons_l=spec_labels(coassoc(leave),k=2)
        vals.append(adjusted_rand_score(cons, cons_l))
    return float(np.median(vals)), cons, cof

# group by condition
subjects={}
for fp in sorted(glob.glob(os.path.join(NPY_DIR,"subject_*_*.npy"))):
    m=re.search(r"subject_(\d+)_(\w+)\.npy$", fp)
    if not m: continue
    sid,cond=int(m.group(1)), m.group(2).upper()
    subjects.setdefault(sid,{})[cond]=fp

conds=set(c for d in subjects.values() for c in d.keys())
if not conds:
    print("[warn] No NHP files parsed. Try increasing includes or use a specific dataset.")
else:
    for cond in sorted(conds):
        paths_c=[d[cond] for d in subjects.values() if cond in d]
        if len(paths_c)<4:
            print(f"[{cond}] not enough subjects:", len(paths_c)); continue
        # first file channels for n_ch display
        ch_txt=paths_c[0].replace(".npy",".channels.txt")
        if os.path.exists(ch_txt):
            with open(ch_txt,"r",encoding="utf-8") as f: ch=[ln.strip() for ln in f if ln.strip()]
        else:
            X0=np.load(paths_c[0], mmap_mode="r"); ch=[f"ch{i}" for i in range(X0.shape[0])]
        rows=[]
        for band,(lo,hi) in BANDS_HZ.items():
            labs=[]
            for p in paths_c:
                X=np.load(p); W=pli_matrix(X, FS_TARGET, lo, hi); W=knn(W,KNN_K); labs.append(spec_labels(W,k=K_FIXED))
            loso, cons, co = loso_via_coassoc(labs)
            null=[]
            for _ in range(NULL_PERMS):
                nl=[]
                for lab in labs:
                    uniq,cnts=np.unique(lab, return_counts=True)
                    idx=np.arange(len(lab)); rng.shuffle(idx)
                    out=np.empty(len(lab),int); st=0
                    for u,c in zip(uniq,cnts):
                        seg=idx[st:st+c]; out[seg]=u; st+=c
                    nl.append(out)
                _, cons_n, _ = loso_via_coassoc(nl)
                null.append(adjusted_rand_score(cons, cons_n))
            null=np.array(null,float); p=float((np.sum(null>=loso)+1)/(len(null)+1))
            # save
            np.save(os.path.join(OUT_ROOT,"tables",f"primate__{cond}__{band}__consensus_labels.npy"), cons)
            np.save(os.path.join(OUT_ROOT,"tables",f"primate__{cond}__{band}__coassoc.npy"), co)
            with open(os.path.join(OUT_ROOT,"metrics",f"primate__{cond}__{band}__metrics.json"),"w",encoding="utf-8") as f:
                json.dump({"cond":cond,"band":band,"n_subjects":len(paths_c),"LOSO":float(loso),"null_mean":float(null.mean()),"p":p}, f, indent=2)
            plt.figure(); plt.imshow(co, aspect='auto'); plt.title(f"{cond} {band} — co-assoc"); plt.colorbar(); plt.tight_layout()
            plt.savefig(os.path.join(OUT_ROOT,"figures",f"primate__{cond}__{band}__coassoc.png"), dpi=160); plt.close()
            rows.append([cond, band, len(paths_c), len(ch), float(loso), float(null.mean()), p])
        df=pd.DataFrame(rows, columns=["cond","band","n_subjects","n_channels","LOSO","null_mean","p"])
        df.to_csv(os.path.join(OUT_ROOT, f"primate__{cond}__summary.csv"), index=False)
        print(f"[{cond}] summary:\n", df.to_string(index=False))

# ---------- α/θ minimal classifier if EC + EO pairs ----------
pairs={k:v for k,v in subjects.items() if "EC" in v and "EO" in v}
def hemi_module_means(W, labels, L, R):
    n=len(labels)
    Lmask=np.zeros((n,n),bool); Lmask[np.ix_(L,L)]=True
    Rmask=np.zeros((n,n),bool); Rmask[np.ix_(R,R)]=True
    within=Lmask|Rmask
    cross =np.zeros((n,n),bool); cross[np.ix_(L,R)]=True; cross[np.ix_(R,L)]=True
    same  =labels[:,None]==labels[None,:]
    for m in (within,cross,same): np.fill_diagonal(m,False)
    return float((W[within&same]).mean()) if np.any(within&same) else np.nan, \
           float((W[cross &same]).mean())  if np.any(cross &same)  else np.nan

def build_minimal_features(pairs):
    # map L/R by odd/even or midline; if no numbers, split halves
    any_sid = next(iter(pairs.keys()))
    ch_txt = pairs[any_sid]["EC"].replace(".npy",".channels.txt")
    if os.path.exists(ch_txt):
        with open(ch_txt,"r",encoding="utf-8") as f: ch=[ln.strip() for ln in f if ln.strip()]
    else:
        X0=np.load(pairs[any_sid]["EC"], mmap_mode="r"); ch=[f"ch{i}" for i in range(X0.shape[0])]
    L,R,Z = [],[],[]
    for i,name in enumerate(ch):
        if re.search(r"[A-Za-z]Z$", name): Z.append(i)
        else:
            m=re.search(r"(\d+)$", name)
            if m:
                try:
                    d=int(m.group(1)); (L if d%2==1 else R).append(i)
                except: Z.append(i)
            else:
                # fallback: split half
                if i < len(ch)//2: L.append(i)
                else: R.append(i)
    rows=[]
    for sid,d in pairs.items():
        for cond,fp in d.items():
            X=np.load(fp)
            row={"sid":sid, "cond": 0 if cond=="EC" else 1}
            for band,(lo,hi) in {"alpha":(8,12),"theta":(4,8)}.items():
                W=pli_matrix(X, FS_TARGET, lo, hi); W=knn(W,KNN_K); lbl=spec_labels(W,k=K_FIXED)
                wh,chm = hemi_module_means(W, lbl, np.array(L,int), np.array(R,int))
                row[f"{band}_WHWM"]=wh; row[f"{band}_CHWM"]=chm
            rows.append(row)
    df=pd.DataFrame(rows).dropna()
    X=df[[c for c in df.columns if c.endswith("_WHWM") or c.endswith("_CHWM")]].to_numpy(float)
    y=df["cond"].to_numpy(int)
    s=df["sid"].to_numpy(int)
    return X,y,s

def paired_auc_perm(X,y,s,perm_B=CLF_PERMS):
    clf=make_pipeline(SimpleImputer(strategy="median"), StandardScaler(), LogisticRegression(max_iter=2000, solver="lbfgs"))
    rng=default_rng(31)
    fold_p=[]; fold_y=[]; order=[]
    for sid in sorted(set(s)):
        mask=(s==sid)
        if np.sum(mask)!=2: continue
        clf.fit(X[~mask], y[~mask])
        p=clf.predict_proba(X[mask])[:,1]
        fold_p.append(p); fold_y.append(y[mask]); order.append(np.where(mask)[0])
    if not fold_p:
        print("[info] No EC/EO pairs; classifier skipped."); 
    else:
        probs=np.concatenate(fold_p); ys=np.concatenate(fold_y)
        auc=float(roc_auc_score(ys, probs))
        cnt=1
        for _ in range(perm_B):
            yperm=ys.copy()
            for i in range(0,len(yperm),2):
                if rng.random()<0.5 and i+1<len(yperm):
                    yperm[i], yperm[i+1] = yperm[i+1], yperm[i]
            ap=float(roc_auc_score(yperm, probs))
            if ap>=auc: cnt+=1
        p=float(cnt/(perm_B+1))
        print(f"[NHP EC vs EO] minimal α/θ coupling → AUC={auc:.3f}, p={p:.4f}  (N={len(ys)} test points)")
        pd.DataFrame([{"AUC":auc,"p_perm":p,"N_points":len(ys),"N_subjects":len(set(s))}]).to_csv(
            os.path.join(OUT_ROOT,"primate_ec_eo_minimal.csv"), index=False
        )

if pairs:
    Xc,yc,sc = build_minimal_features(pairs)
    paired_auc_perm(Xc,yc,sc)

print("\nArtifacts →", OUT_ROOT)


→ GitHub fallback search for OpenNeuro dsIDs (no GraphQL)…
      id                       full                                                                                                                                                        title  keep
ds004620 OpenNeuroDatasets/ds004620                                                                                                              OpenNeuro dataset - Macaque angiography dataset False
ds005521 OpenNeuroDatasets/ds005521                                                                                   OpenNeuro dataset - Macaque Color, Contrast, and Spatial Frequency Dataset False
ds005590 OpenNeuroDatasets/ds005590 OpenNeuro dataset - [18F]SF51, a Novel 18F-labeled PET Radioligand for Translocator Protein 18kDa (TSPO) in Brain, Works Well in Monkeys but Fails in Humans False
ds005619 OpenNeuroDatasets/ds005619 OpenNeuro dataset - [18F]SF51, a Novel 18F-labeled PET Radioligand for Translocator Protein 18kDa (TSPO) in B

In [4]:
# === NHP CNT Pipeline: Paste URLs → Download → Convert → PLI Consensus + Optional EC/EO Classifier ===
import os, re, io, glob, json, zipfile, requests, numpy as np, pandas as pd, matplotlib.pyplot as plt
from numpy.random import default_rng
from scipy.signal import butter, filtfilt, hilbert, decimate
from sklearn.cluster import KMeans
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, adjusted_rand_score

# ---- 1) Paste direct URLs here (ZIP, MAT, NPY, EDF, FIF) ----
URLS = [
    # EXAMPLES (replace with real links you choose):
    # "https://zenodo.org/record/XXXXX/files/monkey_ecog_subject1.zip",
    # "https://yourlab.org/data/macaque_LFP_sessionA.mat",
    # "https://openneuro.org/api/…/sub-XX_task-rest_meg.fif"   # direct file link
]
# If you already downloaded files, put their local paths here:
LOCAL_FILES = [
    # r"C:\path\to\monkey_ecog_data.zip",
    # r"C:\path\to\macaque_lfp_sessionA.mat"
]

# ---- 2) Settings (safe defaults) ----
ROOT      = r"C:\Users\caleb\CNT_Lab"
DL_ROOT   = os.path.join(ROOT, "primate_url_raw")
NPY_DIR   = os.path.join(ROOT, "primate_eeg")
OUT_ROOT  = os.path.join(ROOT, r"artifacts\pli_primate_url")
OUT_TAB   = os.path.join(OUT_ROOT, "tables")
OUT_MET   = os.path.join(OUT_ROOT, "metrics")
OUT_FIG   = os.path.join(OUT_ROOT, "figures")
for p in [DL_ROOT, NPY_DIR, OUT_ROOT, OUT_TAB, OUT_MET, OUT_FIG]: os.makedirs(p, exist_ok=True)

FS_TARGET = 1000.0      # resample target (Hz)
SLICE_SEC = 60          # seconds per subject
BANDS_HZ  = {"theta":(4,8), "alpha":(8,12), "beta":(13,30), "gamma":(30,55)}
K_FIXED   = 2
KNN_K     = 6
NULL_PERMS= 500
CLF_PERMS = 10000
rng       = default_rng(13)

# ---- Downloader ----
def dl(url, out_dir=DL_ROOT):
    try:
        r = requests.get(url, timeout=120)
        r.raise_for_status()
        fname = re.sub(r'[^A-Za-z0-9._-]+', '_', url.split('/')[-1]) or "file"
        path  = os.path.join(out_dir, fname)
        with open(path, "wb") as f:
            f.write(r.content)
        print("Downloaded:", path)
        # auto-extract ZIPs
        if path.lower().endswith(".zip"):
            with zipfile.ZipFile(path, "r") as zf:
                zf.extractall(out_dir)
            print("Extracted:", path)
        return True
    except Exception as e:
        print("[warn] download failed:", url, e)
        return False

print("== Downloading NHP files ==")
for u in URLS:
    dl(u)
# Copy local files (or just rely on conversion step to find them)
for lf in LOCAL_FILES:
    if os.path.exists(lf):
        base = os.path.join(DL_ROOT, os.path.basename(lf))
        if lf != base:
            try:
                with open(lf, "rb") as fin, open(base, "wb") as fout:
                    fout.write(fin.read())
                print("Registered local file:", base)
            except Exception as e:
                print("[warn] could not register", lf, e)

# ---- 3) Convert → NPY + channels.txt ----
def try_load_file(fp):
    fp_l = fp.lower()
    if fp_l.endswith(".edf"):
        try:
            import mne
        except Exception:
            import sys, subprocess
            subprocess.check_call([sys.executable,"-m","pip","install","-q","mne","pooch"])
            import mne
        raw = mne.io.read_raw_edf(fp, preload=True, verbose="ERROR")
        raw.pick_types(eeg=True, eog=False, ecg=False, emg=False, stim=False, misc=False)
        X  = raw.get_data(); fs = float(raw.info["sfreq"]); ch = list(raw.ch_names)
        return X, fs, ch
    elif fp_l.endswith(".fif"):
        try:
            import mne
        except Exception:
            import sys, subprocess
            subprocess.check_call([sys.executable,"-m","pip","install","-q","mne","pooch"])
            import mne
        raw = mne.io.read_raw_fif(fp, preload=True, verbose="ERROR")
        raw.pick_types(meg=True, eeg=True, stim=False)
        X  = raw.get_data(); fs = float(raw.info["sfreq"]); ch = list(raw.ch_names)
        return X, fs, ch
    elif fp_l.endswith(".mat"):
        from scipy.io import loadmat
        m = loadmat(fp)
        arr=None
        for k,v in m.items():
            if isinstance(v,np.ndarray) and v.ndim==2 and (arr is None or v.size>arr.size): arr=v
        if arr is None: raise RuntimeError("No 2D array in MAT")
        fs = float(m.get("fs", np.array([[FS_TARGET]])).squeeze())
        ch = [f"ch{i}" for i in range(arr.shape[0])]
        return arr.astype(float), fs, ch
    elif fp_l.endswith(".npy"):
        X = np.load(fp)
        if X.ndim!=2: raise RuntimeError("NPY not 2D")
        ch = [f"ch{i}" for i in range(X.shape[0])]
        return X.astype(float), FS_TARGET, ch
    else:
        raise RuntimeError("Unsupported file: "+fp)

def resample_if_needed(X, fs_in, fs_out):
    if abs(fs_in - fs_out) < 1e-6: return X, fs_in
    q = int(round(fs_in / fs_out))
    if q>=1 and abs(fs_in/q - fs_out) < 1e-3:
        Y = np.vstack([decimate(X[i], q, ftype='fir', zero_phase=True) for i in range(X.shape[0])])
        return Y, fs_out
    return X, fs_in

def export_npys(raw_root, out_dir):
    files=[]
    for patt in ["*.edf","*.fif","*.mat","*.npy"]:
        files += glob.glob(os.path.join(raw_root,"**",patt), recursive=True)
    files = sorted(list(set(files)))
    idx=0; out={}
    for fp in files:
        try:
            X, fs, ch = try_load_file(fp)
            X, fs2 = resample_if_needed(X, fs, FS_TARGET)
            n_keep  = int(SLICE_SEC * fs2)
            Xo      = X[:, :n_keep] if X.shape[1]>=n_keep else np.tile(X, int(np.ceil(n_keep/X.shape[1])))[:, :n_keep]
            # condition tokens (best effort)
            low = fp.lower(); tag="REST"
            if any(k in low for k in ["eyesopen","eo","open"]): tag="EO"
            if any(k in low for k in ["eyesclosed","ec","closed"]): tag="EC"
            base = os.path.join(out_dir, f"subject_{idx:02d}_{tag}")
            np.save(base+".npy", Xo.astype(np.float32))
            with open(base+".channels.txt","w",encoding="utf-8") as f:
                for nm in ch: f.write(str(nm)+"\n")
            out.setdefault(idx,{})[tag]=base+".npy"
            idx+=1
        except Exception as e:
            pass
    return out

paths = export_npys(DL_ROOT, NPY_DIR)
print(f"[convert] exported {len(paths)} subjects →", NPY_DIR)

# ---- 4) PLI + spectral-on-coassoc (k=2) ----
def bandpass(x, fs, lo, hi, order=4):
    b,a=butter(order, [lo/(fs/2), hi/(fs/2)], btype="band"); return filtfilt(b,a,x)
def pli_matrix(X, fs, lo, hi):
    n=X.shape[0]
    b,a=butter(4, [lo/(fs/2), hi/(fs/2)], btype="band")
    Y=np.zeros_like(X)
    for c in range(n): Y[c]=filtfilt(b,a,X[c])
    ph=np.angle(hilbert(Y, axis=1))
    W=np.zeros((n,n),float)
    for i in range(n):
        for j in range(i+1,n):
            d=ph[i]-ph[j]; W[i,j]=W[j,i]=abs(np.mean(np.sign(np.sin(d))))
    np.fill_diagonal(W,0); return W
def knn(W,k):
    W=W.copy(); n=W.shape[0]
    for i in range(n):
        idx=np.argsort(W[i])[::-1]; keep=idx[:k]
        mask=np.ones(n,bool); mask[keep]=False; W[i,mask]=0
    W=np.maximum(W,W.T); np.fill_diagonal(W,0); return W
def lap(W):
    d=W.sum(1); d=np.where(d<=1e-12,1.0,d); D=np.diag(1.0/np.sqrt(d)); return np.eye(W.shape[0]) - D@W@D
def spec_labels(W,k=2):
    e,v=np.linalg.eigh(lap(W)); U=v[:,1:k] if k>1 else v[:,:1]
    U/= (np.linalg.norm(U, axis=1, keepdims=True)+1e-12)
    return KMeans(n_clusters=k, n_init=50, random_state=13).fit_predict(U)
def coassoc(labels):
    n=len(labels[0]); m=len(labels); co=np.zeros((n,n),float)
    for lab in labels:
        for i in range(n):
            li=lab[i]
            for j in range(n): co[i,j]+=1 if li==lab[j] else 0
    return co/m
def loso_via_coassoc(label_list):
    cof=coassoc(label_list); cons=spec_labels(cof,k=2)
    vals=[]
    for s in range(len(label_list)):
        leave=[lab for i,lab in enumerate(label_list) if i!=s]
        cons_l=spec_labels(coassoc(leave),k=2)
        vals.append(adjusted_rand_score(cons, cons_l))
    return float(np.median(vals)), cons, cof

subjects={}
for fp in sorted(glob.glob(os.path.join(NPY_DIR,"subject_*_*.npy"))):
    m=re.search(r"subject_(\d+)_(\w+)\.npy$", fp)
    if not m: continue
    sid,cond=int(m.group(1)), m.group(2).upper()
    subjects.setdefault(sid,{})[cond]=fp

conds=set(c for d in subjects.values() for c in d.keys())
if not conds:
    print("[warn] No files found after download. Paste at least one URL to a NHP ZIP/MAT/NPY/EDF/FIF and re-run.")
else:
    for cond in sorted(conds):
        paths_c=[d[cond] for d in subjects.values() if cond in d]
        if len(paths_c)<4:
            print(f"[{cond}] not enough subjects:", len(paths_c)); continue
        # channel count (first file)
        ch_txt = paths_c[0].replace(".npy",".channels.txt")
        if os.path.exists(ch_txt):
            with open(ch_txt,"r",encoding="utf-8") as f: ch=[ln.strip() for ln in f if ln.strip()]
        else:
            X0=np.load(paths_c[0], mmap_mode="r"); ch=[f"ch{i}" for i in range(X0.shape[0])]
        rows=[]
        for band,(lo,hi) in BANDS_HZ.items():
            labs=[]
            for p in paths_c:
                X=np.load(p); W=pli_matrix(X, FS_TARGET, lo, hi); W=knn(W,KNN_K); labs.append(spec_labels(W,k=K_FIXED))
            loso, cons, co = loso_via_coassoc(labs)
            # null (label-preserving)
            null=[]
            for _ in range(NULL_PERMS):
                nl=[]
                for lab in labs:
                    uniq,cnts=np.unique(lab, return_counts=True)
                    idx=np.arange(len(lab)); rng.shuffle(idx)
                    out=np.empty(len(lab),int); st=0
                    for u,c in zip(uniq,cnts):
                        seg=idx[st:st+c]; out[seg]=u; st+=c
                    nl.append(out)
                _, cons_n, _ = loso_via_coassoc(nl)
                null.append(adjusted_rand_score(cons, cons_n))
            null=np.array(null,float); p=float((np.sum(null>=loso)+1)/(len(null)+1))
            np.save(os.path.join(OUT_TAB,f"primate__{cond}__{band}__consensus_labels.npy"), cons)
            np.save(os.path.join(OUT_TAB,f"primate__{cond}__{band}__coassoc.npy"), co)
            with open(os.path.join(OUT_MET,f"primate__{cond}__{band}__metrics.json"),"w",encoding="utf-8") as f:
                json.dump({"cond":cond,"band":band,"n_subjects":len(paths_c),"LOSO":float(loso),"p":p}, f, indent=2)
            plt.figure(); plt.imshow(co, aspect='auto'); plt.title(f"{cond} {band} — co-assoc"); plt.colorbar(); plt.tight_layout()
            plt.savefig(os.path.join(OUT_FIG,f"primate__{cond}__{band}__coassoc.png"), dpi=160); plt.close()
            rows.append([cond, band, len(paths_c), len(ch), float(loso), p])
        df=pd.DataFrame(rows, columns=["cond","band","n_subjects","n_channels","LOSO","p"])
        df.to_csv(os.path.join(OUT_ROOT,f"primate__{cond}__summary.csv"), index=False)
        print(f"[{cond}] summary:\n", df.to_string(index=False))

# ---- 5) α/θ minimal classifier if EC+EO pairs exist ----
pairs={k:v for k,v in subjects.items() if "EC" in v and "EO" in v}
def hemi_module_means(W, labels, L, R):
    n=len(labels)
    Lmask=np.zeros((n,n),bool); Lmask[np.ix_(L,L)]=True
    Rmask=np.zeros((n,n),bool); Rmask[np.ix_(R,R)]=True
    within=Lmask|Rmask
    cross =np.zeros((n,n),bool); cross[np.ix_(L,R)]=True; cross[np.ix_(R,L)]=True
    same  =labels[:,None]==labels[None,:]
    for m in (within,cross,same): np.fill_diagonal(m,False)
    return float((W[within&same]).mean()) if np.any(within&same) else np.nan, \
           float((W[cross &same]).mean())  if np.any(cross &same)  else np.nan

def build_minimal_features(pairs):
    any_sid=next(iter(pairs))
    ch_txt=pairs[any_sid]["EC"].replace(".npy",".channels.txt")
    if os.path.exists(ch_txt):
        with open(ch_txt,"r",encoding="utf-8") as f: ch=[ln.strip() for ln in f if ln.strip()]
    else:
        X0=np.load(pairs[any_sid]["EC"], mmap_mode="r"); ch=[f"ch{i}" for i in range(X0.shape[0])]
    # quick L/R split (odd/even digit else half/half)
    L,R=[],[]
    for i,name in enumerate(ch):
        m=re.search(r"(\d+)$", name)
        if m:
            d=int(m.group(1)); (L if d%2==1 else R).append(i)
        else:
            (L if i<len(ch)//2 else R).append(i)
    rows=[]
    for sid,d in pairs.items():
        for cond,fp in d.items():
            X=np.load(fp)
            row={"sid":sid, "cond":0 if cond=="EC" else 1}
            for band,(lo,hi) in {"alpha":(8,12),"theta":(4,8)}.items():
                W=pli_matrix(X, FS_TARGET, lo, hi); W=knn(W,KNN_K); lbl=spec_labels(W,k=K_FIXED)
                wh, chm = hemi_module_means(W,lbl,np.array(L,int), np.array(R,int))
                row[f"{band}_WHWM"]=wh; row[f"{band}_CHWM"]=chm
            rows.append(row)
    df=pd.DataFrame(rows).dropna()
    X=df[[c for c in df.columns if c.endswith("_WHWM") or c.endswith("_CHWM")]].to_numpy(float)
    y=df["cond"].to_numpy(int)
    s=df["sid"].to_numpy(int)
    return X,y,s

def paired_auc_perm(X,y,s,perm_B=CLF_PERMS):
    clf=make_pipeline(SimpleImputer(strategy="median"), StandardScaler(), LogisticRegression(max_iter=2000, solver="lbfgs"))
    rng=default_rng(31)
    fold_p=[]; fold_y=[]
    for sid in sorted(set(s)):
        mask=(s==sid)
        if np.sum(mask)!=2: continue
        clf.fit(X[~mask], y[~mask])
        p=clf.predict_proba(X[mask])[:,1]
        fold_p.append(p); fold_y.append(y[mask])
    if not fold_p:
        print("[info] No EC/EO pairs; classifier skipped.")
    else:
        probs=np.concatenate(fold_p); ys=np.concatenate(fold_y)
        auc=float(roc_auc_score(ys, probs))
        cnt=1
        for _ in range(perm_B):
            yperm=ys.copy()
            for i in range(0,len(yperm),2):
                if rng.random()<0.5 and i+1<len(yperm):
                    yperm[i],yperm[i+1]=yperm[i+1],yperm[i]
            ap=float(roc_auc_score(yperm, probs))
            if ap>=auc: cnt+=1
        p=float(cnt/(perm_B+1))
        print(f"[NHP EC vs EO] minimal α/θ coupling → AUC={auc:.3f}, p={p:.4f} (N={len(ys)})")
        pd.DataFrame([{"AUC":auc,"p_perm":p,"N_points":len(ys),"N_subjects":len(set(s))}]).to_csv(
            os.path.join(OUT_ROOT,"primate_ec_eo_minimal.csv"), index=False)

if pairs:
    Xc,yc,sc = build_minimal_features(pairs)
    paired_auc_perm(Xc,yc,sc)

print("\nDone. Artifacts →", OUT_ROOT)
print("Paste at least one working NHP URL if none were found automatically.")


== Downloading NHP files ==
[convert] exported 0 subjects → C:\Users\caleb\CNT_Lab\primate_eeg


KeyboardInterrupt: 

In [5]:
# === CNT Humans @ Scale (≥100 subjects): EO+EC Download → Convert → PLI Consensus + Minimal Classifier ===
# Dataset: EEGBCI (EEG Motor Movement/Imagery)
# R01 = Eyes Open (EO), R02 = Eyes Closed (EC)
# This cell:
#   1) Downloads EO/EC for subjects 1..109 (skips present), exports -> subject_##_{EO|EC}.npy (+ channels.txt) at 250 Hz, 60 s.
#   2) Runs CNT PLI → kNN → spectral (k=2) per subject/band (alpha/theta/beta); spectral-on-coassoc across subjects.
#      Outputs LOSO median ARI & label-preserving null p for EC and EO separately.
#   3) Computes hemispheric (Left/Right) and Anterior/Posterior metrics on the consensus (clean 10–20 names).
#   4) Runs the prereg minimal classifier (α/θ coupling only) with paired CV + 10k paired label-flip perms → AUC & p.
#   5) Saves summaries, figures, and a final 1-page PDF.
#
# Outputs:
#   C:\Users\caleb\CNT_Lab\artifacts\pli_humans_100plus\
#     ├─ tables\band__{cond}__{band}__{coassoc,consensus_labels}.npy
#     ├─ metrics\band__{cond}__{band}__metrics.json
#     ├─ figures\coassoc__{cond}__{band}.png
#     ├─ hemisphere_{cond}.csv
#     ├─ summary_{cond}.csv
#     ├─ ec_eo_minimal_prereg.csv (AUC & p)
#     └─ CNT_PLI_humans_100plus_summary.pdf

import os, re, glob, json, numpy as np, pandas as pd, matplotlib.pyplot as plt
from numpy.random import default_rng
from scipy.signal import butter, filtfilt, hilbert
from sklearn.cluster import KMeans
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

# ---------------- Paths & config ----------------
ROOT       = r"C:\Users\caleb\CNT_Lab"
DATA_DIR   = os.path.join(ROOT, "eeg_rest")  # where subject_##_{EC|EO}.npy live
OUT_ROOT   = os.path.join(ROOT, r"artifacts\pli_humans_100plus")
OUT_TAB    = os.path.join(OUT_ROOT, "tables")
OUT_MET    = os.path.join(OUT_ROOT, "metrics")
OUT_FIG    = os.path.join(OUT_ROOT, "figures")
for p in [DATA_DIR, OUT_ROOT, OUT_TAB, OUT_MET, OUT_FIG]: os.makedirs(p, exist_ok=True)

SUBJECTS   = list(range(1, 109+1))       # EEGBCI has up to 109 subjects
MIN_SUBJ   = 100                          # we want at least 100 usable subjects
FS_OUT     = 250.0                        # Hz
DURATION_S = 60                           # seconds kept per condition
HP, LP     = 1.0, 45.0                    # bandpass for PLI prep

BANDS_HZ   = {"alpha": (8.0,13.0), "theta": (4.0,8.0), "beta": (13.0,30.0)}
K_FIXED    = 2
KNN_K      = 6
NULL_PERMS = 500                          # raise to 1000+ for publication
CLF_PERMS  = 10000                        # paired label-flip perms for classifier
RNG        = default_rng(7)

# ---------------- Deps ----------------
try:
    import mne
except Exception:
    import sys, subprocess
    subprocess.check_call([sys.executable,"-m","pip","install","-q","mne","pooch"])
    import mne

# ---------------- 1) Download & export EO (R01) and EC (R02) ----------------
def export_subject_cond(subj, run, cond):
    try:
        try:
            fpaths = mne.datasets.eegbci.load_data(subjects=[subj], runs=[run], update_path=True, verbose="ERROR")
        except TypeError:  # old mne signature
            fpaths = mne.datasets.eegbci.load_data(subject=subj, runs=[run], update_path=True, verbose="ERROR")
        raws=[]
        for fp in fpaths:
            raw = mne.io.read_raw_edf(fp, preload=True, verbose="ERROR")
            raw.pick_types(eeg=True, stim=False, eog=False, ecg=False, emg=False, misc=False)
            raws.append(raw)
        if not raws: return False, "no_raw"
        raw = mne.concatenate_raws(raws, verbose="ERROR")
        # montage + filter + resample
        try:
            raw.set_montage("standard_1020", on_missing="ignore", match_case=False, verbose="ERROR")
        except Exception:
            pass
        raw.filter(HP, LP, fir_design="firwin", verbose="ERROR")
        raw.resample(FS_OUT, npad="auto", verbose="ERROR")
        # slice to DURATION_S
        n_keep = int(DURATION_S * raw.info["sfreq"])
        X = raw.get_data(picks="eeg")
        if X.shape[1] >= n_keep:
            X = X[:, :n_keep]
        else:
            reps = int(np.ceil(n_keep / X.shape[1])); X = np.tile(X, reps)[:, :n_keep]
        ch_names = mne.pick_info(raw.info, mne.pick_types(raw.info, eeg=True)).ch_names
        base = os.path.join(DATA_DIR, f"subject_{subj:02d}_{cond}")
        np.save(base + ".npy", X.astype(np.float32))
        with open(base + ".channels.txt","w",encoding="utf-8") as f:
            for ch in ch_names: f.write(ch + "\n")
        return True, X.shape
    except Exception as e:
        return False, str(e)

print("=== Exporting EO/EC to .npy (skips existing) ===")
log=[]
for s in SUBJECTS:
    for cond, run in (("EO",1), ("EC",2)):
        f = os.path.join(DATA_DIR, f"subject_{s:02d}_{cond}.npy")
        if os.path.exists(f):
            log.append((s,cond,"exists"))
        else:
            ok,msg = export_subject_cond(s, run, cond)
            log.append((s,cond,msg if ok else f"error: {msg}"))
print("Export sample:", log[:8], "...")

# Count usable subjects (both EO & EC present)
usable = [s for s in SUBJECTS if os.path.exists(os.path.join(DATA_DIR, f"subject_{s:02d}_EO.npy"))
                                   and os.path.exists(os.path.join(DATA_DIR, f"subject_{s:02d}_EC.npy"))]
print(f"Usable subjects (EO & EC present): {len(usable)} (want ≥ {MIN_SUBJ})")

# ---------------- Channel cleaning & maps ----------------
CH_TXT = os.path.join(ROOT, r"eeg_rest\subject_01_EC.channels.txt")
def clean_label(x: str) -> str:
    y = x.strip()
    y = re.sub(r"(?i)^(EEG|MEG|EOG|ECG|EMG)[\s_\-]+", "", y)
    y = re.sub(r"(?i)[\s_\-]*(REF|AV|AVERAGE|AVG|M1|M2)$", "", y)
    y = re.sub(r"[ \-\.]+", "", y)
    y = y.replace("FP","Fp")
    return y
# Ensure channel text exists (write from any subject EC if missing)
if not os.path.exists(CH_TXT):
    anyEC = next((os.path.join(DATA_DIR,f) for f in os.listdir(DATA_DIR) if f.endswith("_EC.npy")), None)
    if anyEC:
        txt = anyEC.replace(".npy",".channels.txt")
        if os.path.exists(txt):
            import shutil; shutil.copyfile(txt, CH_TXT)

with open(CH_TXT, "r", encoding="utf-8") as f:
    ch_names = [clean_label(ln.strip()) for ln in f if ln.strip()]

LEFT_CANON  = set(map(str.upper, ["Fp1","AF7","AF3","F7","F5","F3","F1","FT7","FC5","FC3","FC1","T7","C5","C3","C1",
                                  "TP7","CP5","CP3","CP1","P7","P5","P3","P1","PO7","PO3","O1"]))
RIGHT_CANON = set(map(str.upper, ["Fp2","AF8","AF4","F8","F6","F4","F2","FT8","FC6","FC4","FC2","T8","C6","C4","C2",
                                  "TP8","CP6","CP4","CP2","P8","P6","P4","P2","PO8","PO4","O2"]))
MID_CANON   = set(map(str.upper, ["Fpz","AFz","Fz","FCz","Cz","CPz","Pz","POz","Oz"]))
ANT_PREFIXES=("Fp","AF","F","FC"); MID_PREFIXES=("C",); POST_PREFIXES=("CP","P","PO","O")

def hemi_map(labels):
    L,R,Z=[],[],[]
    for i,ch in enumerate(labels):
        up=ch.upper()
        if up in LEFT_CANON:  L.append(i); continue
        if up in RIGHT_CANON: R.append(i); continue
        if up in MID_CANON or re.search(r"[A-Za-z]z$", ch): Z.append(i); continue
        m=re.search(r"(\d+)$", ch)
        if m:
            try:
                d=int(m.group(1)); (L if d%2==1 else R).append(i); continue
            except: pass
        Z.append(i)
    return np.array(L,int), np.array(R,int), np.array(Z,int)

def ap_map(labels):
    A,P,C=[],[],[]
    for i,ch in enumerate(labels):
        pref=re.match(r"[A-Za-z]+", ch); pref=pref.group(0) if pref else ""
        if any(pref.startswith(px) for px in ANT_PREFIXES): A.append(i)
        elif any(pref.startswith(px) for px in POST_PREFIXES): P.append(i)
        elif pref.startswith("C"): C.append(i)
        else: C.append(i)
    return np.array(A,int), np.array(P,int), np.array(C,int)

L_idx, R_idx, Z_idx = hemi_map(ch_names)
A_idx, P_idx, C_idx = ap_map(ch_names)

# ---------------- 2) PLI → kNN → spectral (k=2) & spectral-on-coassoc ----------------
from scipy.signal import butter, filtfilt, hilbert
def bandpass(x, fs, lo, hi, order=4):
    b,a=butter(order,[lo/(fs/2), hi/(fs/2)], btype="band"); return filtfilt(b,a,x)
def pli_matrix(X, fs, lo, hi):
    n=X.shape[0]
    b,a=butter(4,[lo/(fs/2), hi/(fs/2)], btype="band")
    Y=np.zeros_like(X)
    for c in range(n): Y[c]=filtfilt(b,a,X[c])
    ph=np.angle(hilbert(Y, axis=1))
    W=np.zeros((n,n),float)
    for i in range(n):
        for j in range(i+1,n):
            d=ph[i]-ph[j]; W[i,j]=W[j,i]=abs(np.mean(np.sign(np.sin(d))))
    np.fill_diagonal(W,0); return W
def knn(W,k):
    W=W.copy(); n=W.shape[0]
    for i in range(n):
        idx=np.argsort(W[i])[::-1]; keep=idx[:k]
        mask=np.ones(n,bool); mask[keep]=False; W[i,mask]=0
    W=np.maximum(W,W.T); np.fill_diagonal(W,0); return W
def lap(W):
    d=W.sum(1); d=np.where(d<=1e-12,1.0,d); D=np.diag(1.0/np.sqrt(d)); return np.eye(W.shape[0]) - D@W@D
def spec_labels(W, k=2):
    e,v = np.linalg.eigh(lap(W))
    U   = v[:,1:k] if k>1 else v[:,:1]
    U   = U / (np.linalg.norm(U, axis=1, keepdims=True)+1e-12)
    return KMeans(n_clusters=k, n_init=50, random_state=7).fit_predict(U)
def coassoc(labels):
    n=len(labels[0]); m=len(labels); co=np.zeros((n,n),float)
    for lab in labels:
        for i in range(n):
            li=lab[i]
            for j in range(n): co[i,j]+=1 if li==lab[j] else 0
    return co/m
from sklearn.metrics import adjusted_rand_score
def loso_via_coassoc(label_list):
    cof=coassoc(label_list); cons=spec_labels(cof,k=2)
    vals=[]
    for s in range(len(label_list)):
        leave=[lab for i,lab in enumerate(label_list) if i!=s]
        cons_l=spec_labels(coassoc(leave),k=2)
        vals.append(adjusted_rand_score(cons, cons_l))
    return float(np.median(vals)), cons, cof

def label_preserving_null_ari(cons, subj_labels, perms=NULL_PERMS):
    null=[]
    for _ in range(perms):
        nl=[]
        for lab in subj_labels:
            uniq,cnts=np.unique(lab, return_counts=True)
            idx=np.arange(len(lab)); RNG.shuffle(idx)
            out=np.empty(len(lab),int); st=0
            for u,c in zip(uniq,cnts):
                seg=idx[st:st+c]; out[seg]=u; st+=c
            nl.append(out)
        _, cons_n, _ = loso_via_coassoc(nl)
        null.append(adjusted_rand_score(cons, cons_n))
    null=np.array(null,float)
    p=float((np.sum(null>=np.median(null)) + 1) / (len(null)+1))  # not used, we compare to LOSO below
    return null

def run_condition(cond):
    paths = [os.path.join(DATA_DIR, f"subject_{s:02d}_{cond}.npy") for s in usable if os.path.exists(os.path.join(DATA_DIR, f"subject_{s:02d}_{cond}.npy"))]
    if len(paths) < MIN_SUBJ:
        print(f"[{cond}] Only {len(paths)} subjects; continuing anyway.")
    rows=[]
    for band,(lo,hi) in BANDS_HZ.items():
        subj_labels=[]
        for p in paths:
            X=np.load(p); W=pli_matrix(X, FS_OUT, lo, hi); W=knn(W, KNN_K); lab=spec_labels(W,k=K_FIXED)
            subj_labels.append(lab)
        loso, cons, co = loso_via_coassoc(subj_labels)
        # null ARI: compare consensus to label-preserving randomized consensus (ARI)
        null_aris=[]
        for _ in range(NULL_PERMS):
            nl=[]
            for lab in subj_labels:
                uniq,cnts=np.unique(lab, return_counts=True)
                idx=np.arange(len(lab)); RNG.shuffle(idx)
                out=np.empty(len(lab),int); st=0
                for u,c in zip(uniq,cnts):
                    seg=idx[st:st+c]; out[seg]=u; st+=c
                nl.append(out)
            _, cons_n, _ = loso_via_coassoc(nl)
            null_aris.append(adjusted_rand_score(cons, cons_n))
        null_aris=np.array(null_aris,float)
        p_val=float((np.sum(null_aris >= loso) + 1) / (len(null_aris)+1))
        # save artifacts
        np.save(os.path.join(OUT_TAB, f"band__{cond}__{band}__consensus_labels.npy"), cons)
        np.save(os.path.join(OUT_TAB, f"band__{cond}__{band}__coassoc.npy"), co)
        with open(os.path.join(OUT_MET, f"band__{cond}__{band}__metrics.json"),"w",encoding="utf-8") as f:
            json.dump({"cond":cond,"band":band,"n_subjects":len(paths),"LOSO":float(loso),"null_ari_mean":float(null_aris.mean()),"p_value":p_val}, f, indent=2)
        plt.figure(); plt.imshow(co, aspect='auto'); plt.title(f"{cond} {band} — co-assoc (spectral on coassoc)")
        plt.colorbar(); plt.tight_layout(); plt.savefig(os.path.join(OUT_FIG, f"coassoc__{cond}__{band}.png"), dpi=160); plt.close()
        rows.append([cond, band, len(paths), float(loso), float(null_aris.mean()), p_val])
    df = pd.DataFrame(rows, columns=["cond","band","n_subjects","LOSO","null_ari_mean","p_value"])
    df.to_csv(os.path.join(OUT_ROOT, f"summary_{cond}.csv"), index=False)
    print(f"[{cond}] summary:\n", df.to_string(index=False))
    return df

print("\n=== Running CNT spectral consensus (EO) ===")
df_eo = run_condition("EO")
print("\n=== Running CNT spectral consensus (EC) ===")
df_ec = run_condition("EC")

# ---------------- 3) Hemi/AP metrics per condition (consensus-based) ----------------
def hemi_ap_metrics(cond):
    rows=[]
    for band in BANDS_HZ.keys():
        cons_fp=os.path.join(OUT_TAB, f"band__{cond}__{band}__consensus_labels.npy")
        co_fp  =os.path.join(OUT_TAB, f"band__{cond}__{band}__coassoc.npy")
        if not (os.path.exists(cons_fp) and os.path.exists(co_fp)): continue
        cons=np.load(cons_fp); co=np.load(co_fp); n=len(cons)
        def within_region_ratio(idx):
            if idx.size<3: return np.nan
            sub=np.ix_(idx,idx); co_r=co[sub]; lab=cons[idx]
            same=lab[:,None]==lab[None,:]; diff=~same
            np.fill_diagonal(same,False); np.fill_diagonal(diff,False)
            s=co_r[same]; d=co_r[diff]
            return float(np.mean(s)/(np.mean(d)+1e-12)) if s.size and d.size else np.nan
        l_ratio=within_region_ratio(L_idx); r_ratio=within_region_ratio(R_idx)
        wh_wm, ch_wm = None, None
        # within-hemi vs cross-hemi (within-module)
        n=n
        Lmask=np.zeros((n,n),bool); Lmask[np.ix_(L_idx,L_idx)]=True
        Rmask=np.zeros((n,n),bool); Rmask[np.ix_(R_idx,R_idx)]=True
        within=Lmask|Rmask
        cross=np.zeros((n,n),bool); cross[np.ix_(L_idx,R_idx)]=True; cross[np.ix_(R_idx,L_idx)]=True
        same = cons[:,None]==cons[None,:]
        for m in (within,cross,same): np.fill_diagonal(m,False)
        WH_WM=float(np.mean(co[within&same])) if np.any(within&same) else np.nan
        CH_WM=float(np.mean(co[cross &same])) if np.any(cross &same) else np.nan
        rows.append([cond, band, n, int(L_idx.size), int(R_idx.size), float(l_ratio), float(r_ratio), WH_WM, CH_WM])
    df=pd.DataFrame(rows, columns=["cond","band","n_channels","n_left","n_right","left_intra_ratio","right_intra_ratio","WH_WM","CH_WM"])
    df.to_csv(os.path.join(OUT_ROOT, f"hemisphere_{cond}.csv"), index=False)
    print(f"[{cond}] hemisphere/AP metrics saved.")
    return df

hemi_eo = hemi_ap_metrics("EO")
hemi_ec = hemi_ap_metrics("EC")

# ---------------- 4) Minimal prereg classifier (α/θ coupling only) ----------------
def subject_features_coupling(subject_id, cond_tag):
    f=os.path.join(DATA_DIR, f"subject_{subject_id:02d}_{cond_tag}.npy")
    if not os.path.exists(f): return None
    X=np.load(f)
    feats={}
    for band,(lo,hi) in {"alpha":(8,13), "theta":(4,8)}.items():
        # Build PLI -> kNN -> spectral labels
        W=pli_matrix(X, FS_OUT, lo, hi); W=knn(W, KNN_K); lbl=spec_labels(W, k=K_FIXED)
        # compute WH_WM & CH_WM from raw PLI graph (not consensus)
        n=len(lbl)
        Lmask=np.zeros((n,n),bool); Lmask[np.ix_(L_idx,L_idx)]=True
        Rmask=np.zeros((n,n),bool); Rmask[np.ix_(R_idx,R_idx)]=True
        within=Lmask|Rmask
        cross=np.zeros((n,n),bool); cross[np.ix_(L_idx,R_idx)]=True; cross[np.ix_(R_idx,L_idx)]=True
        same = lbl[:,None]==lbl[None,:]
        for m in (within,cross,same): np.fill_diagonal(m,False)
        WH_WM=float(np.mean(W[within&same])) if np.any(within&same) else np.nan
        CH_WM=float(np.mean(W[cross &same])) if np.any(cross &same) else np.nan
        feats[f"{band}_WHWM"]=WH_WM; feats[f"{band}_CHWM"]=CH_WM
    return feats

# Build full feature set
X_list, y_list, sid_list = [], [], []
for s in usable:
    ec = subject_features_coupling(s, "EC")
    eo = subject_features_coupling(s, "EO")
    if ec:
        X_list.append(ec); y_list.append(0); sid_list.append(s)
    if eo:
        X_list.append(eo); y_list.append(1); sid_list.append(s)

if X_list:
    Xdf = pd.DataFrame(X_list).replace([np.inf,-np.inf], np.nan)
    y = np.array(y_list, int); sids=np.array(sid_list, int)
    # Paired CV: leave-one-subject pair out
    from sklearn.metrics import roc_curve
    clf = make_pipeline(SimpleImputer(strategy="median"), StandardScaler(), LogisticRegression(max_iter=2000, solver="lbfgs"))
    fold_probs, fold_true, pair_indices = [], [], []
    for s in usable:
        mask = (sids==s)
        if np.sum(mask)!=2: continue
        train = ~mask
        clf.fit(Xdf.to_numpy()[train], y[train])
        p = clf.predict_proba(Xdf.to_numpy()[mask])[:,1]
        fold_probs.append(p); fold_true.append(y[mask]); pair_indices.append(np.where(mask)[0])
    probs = np.concatenate(fold_probs); ys = np.concatenate(fold_true)
    auc_obs = float(roc_auc_score(ys, probs))
    # Paired label-flip perms
    cnt=1; rng=default_rng(31)
    for _ in range(CLF_PERMS):
        y_perm = ys.copy()
        # flip labels within each subject pair
        for i in range(0, len(y_perm), 2):
            if rng.random() < 0.5 and i+1 < len(y_perm):
                y_perm[i], y_perm[i+1] = y_perm[i+1], y_perm[i]
        a = float(roc_auc_score(y_perm, probs))
        if a >= auc_obs: cnt += 1
    p_perm = float(cnt / (CLF_PERMS + 1))
    pd.DataFrame([{"AUC_obs":auc_obs,"p_perm":p_perm,"N_test_points":len(ys),"N_subjects":len(usable)}]).to_csv(
        os.path.join(OUT_ROOT,"ec_eo_minimal_prereg.csv"), index=False)
    print(f"\n=== Minimal α/θ coupling classifier ===\nAUC={auc_obs:.3f}, p={p_perm:.4f}  (N={len(ys)} test points)")
else:
    print("\n[info] No features built (check EO/EC NPYs).")

# ---------------- 5) Final 1-page PDF summary ----------------
from matplotlib.backends.backend_pdf import PdfPages

FINAL_PDF = os.path.join(OUT_ROOT, "CNT_PLI_humans_100plus_summary.pdf")
fig = plt.figure(figsize=(11,8.5))
# Title
ax_t = fig.add_axes([0.05,0.92,0.9,0.06]); ax_t.axis("off")
ax_t.text(0.5, 0.6, "CNT — Humans (≥100) EO & EC: Field Consensus + Minimal Predictor", ha="center", va="center", fontsize=15, weight="bold")
# EC/EO summaries
def load_summary(cond):
    p=os.path.join(OUT_ROOT, f"summary_{cond}.csv")
    return pd.read_csv(p) if os.path.exists(p) else pd.DataFrame()
df_eo_s, df_ec_s = load_summary("EO"), load_summary("EC")
ax1 = fig.add_axes([0.05,0.72,0.42,0.16]); ax1.axis("off")
ax1.text(0.0,0.85, "EO spectral-on-coassoc:", fontsize=11, weight="bold")
ax1.text(0.0,0.05, df_eo_s.to_string(index=False) if not df_eo_s.empty else "EO summary not found", fontsize=8, family="monospace")
ax2 = fig.add_axes([0.53,0.72,0.42,0.16]); ax2.axis("off")
ax2.text(0.0,0.85, "EC spectral-on-coassoc:", fontsize=11, weight="bold")
ax2.text(0.0,0.05, df_ec_s.to_string(index=False) if not df_ec_s.empty else "EC summary not found", fontsize=8, family="monospace")
# Co-assoc images strip
x0=0.05
for cond in ["EO","EC"]:
    for i,band in enumerate(["alpha","theta","beta"]):
        pimg=os.path.join(OUT_FIG, f"coassoc__{cond}__{band}.png")
        ax = fig.add_axes([x0 + i*0.145, 0.46 if cond=="EO" else 0.27, 0.14, 0.16]); ax.axis("off")
        if os.path.exists(pimg):
            ax.imshow(plt.imread(pimg)); ax.set_title(f"{cond} {band}")
        else:
            ax.text(0.5,0.5, "missing", ha="center", va="center", fontsize=8)
# Minimal classifier AUC
res_csv = os.path.join(OUT_ROOT,"ec_eo_minimal_prereg.csv")
ax_auc = fig.add_axes([0.05, 0.05, 0.42, 0.16]); ax_auc.axis("off")
if os.path.exists(res_csv):
    d = pd.read_csv(res_csv)
    auc = float(d.loc[0,"AUC_obs"]); p  = float(d.loc[0,"p_perm"])
    ax_auc.text(0.0,0.8, "Minimal α/θ coupling classifier:", fontsize=11, weight="bold")
    ax_auc.text(0.0,0.35, f"AUC={auc:.3f}, p={p:.4f}, N_test_points={int(d.loc[0,'N_test_points'])}, N_subjects={int(d.loc[0,'N_subjects'])}")
else:
    ax_auc.text(0.0,0.8, "Minimal α/θ coupling classifier:", fontsize=11, weight="bold")
    ax_auc.text(0.0,0.35, "Results file not found.")
# Hemi metrics quick peek
for j,cond in enumerate(["EO","EC"]):
    p=os.path.join(OUT_ROOT, f"hemisphere_{cond}.csv")
    axh = fig.add_axes([0.53, 0.05 + j*0.105, 0.42, 0.10]); axh.axis("off")
    if os.path.exists(p):
        dd=pd.read_csv(p); axh.text(0.0,0.7, f"{cond} hemis/AP metrics:", fontsize=10, weight="bold")
        axh.text(0.0,0.1, dd.to_string(index=False), fontsize=7, family="monospace")
    else:
        axh.text(0.0,0.7, f"{cond} hemis/AP metrics:", fontsize=10, weight="bold")
        axh.text(0.0,0.1, "not found")
pp=PdfPages(FINAL_PDF); pp.savefig(fig, dpi=200); pp.close(); plt.close(fig)
print("Saved final 1-pager:", FINAL_PDF)


=== Exporting EO/EC to .npy (skips existing) ===


NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


...

NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Export sample: [(1, 'EO', 'exists'), (1, 'EC', 'exists'), (2, 'EO', 'exists'), (2, 'EC', 'exists'), (3, 'EO', 'exists'), (3, 'EC', 'exists'), (4, 'EO', 'exists'), (4, 'EC', 'exists')] ...
Usable subjects (EO & EC present): 109 (want ≥ 100)

=== Running CNT spectral consensus (EO) ===


KeyboardInterrupt: 

In [6]:
# === FAST RESUME (EO+EC) — Parallel label cache + quick consensus (200 perms) ===
# Assumes all subjects exported here: C:\Users\caleb\CNT_Lab\eeg_rest\subject_##_{EC|EO}.npy

import os, re, glob, json, numpy as np, pandas as pd, matplotlib.pyplot as plt
from numpy.random import default_rng
from scipy.signal import butter, filtfilt, hilbert
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from joblib import Parallel, delayed

ROOT       = r"C:\Users\caleb\CNT_Lab"
DATA_DIR   = os.path.join(ROOT, "eeg_rest")
OUT_ROOT   = os.path.join(ROOT, r"artifacts\pli_humans_100plus")
OUT_TAB    = os.path.join(OUT_ROOT, "tables")
OUT_MET    = os.path.join(OUT_ROOT, "metrics")
OUT_FIG    = os.path.join(OUT_ROOT, "figures")
LABELS_DIR = os.path.join(OUT_ROOT, "labels_cache")
for p in [OUT_TAB, OUT_MET, OUT_FIG, LABELS_DIR]: os.makedirs(p, exist_ok=True)

FS_OUT     = 250.0
BANDS_HZ   = {"alpha": (8,13), "theta": (4,8), "beta": (13,30)}
K_FIXED    = 2
KNN_K      = 6
NULL_PERMS = 200            # FAST mode; you can top up later
N_JOBS     = max(1, os.cpu_count() - 1)

# ---------------- helper fns ----------------
def bandpass(x, fs, lo, hi, order=4):
    from scipy.signal import butter, filtfilt
    b,a = butter(order, [lo/(fs/2), hi/(fs/2)], btype="band"); return filtfilt(b,a,x)

def pli_matrix(X, fs, lo, hi):
    from scipy.signal import hilbert
    n = X.shape[0]
    b,a = butter(4, [lo/(fs/2), hi/(fs/2)], btype="band")
    Y = np.zeros_like(X)
    for c in range(n): Y[c] = filtfilt(b,a,X[c])
    ph = np.angle(hilbert(Y, axis=1))
    W  = np.zeros((n,n), float)
    for i in range(n):
        for j in range(i+1, n):
            dphi = ph[i]-ph[j]
            W[i,j] = W[j,i] = abs(np.mean(np.sign(np.sin(dphi))))
    np.fill_diagonal(W, 0.0)
    return W

def knn(W,k):
    W=W.copy(); n=W.shape[0]
    for i in range(n):
        idx=np.argsort(W[i])[::-1]; keep=idx[:k]
        mask=np.ones(n,bool); mask[keep]=False; W[i,mask]=0
    W=np.maximum(W,W.T); np.fill_diagonal(W,0); return W

def lap(W):
    d=W.sum(1); d=np.where(d<=1e-12,1.0,d); D=np.diag(1.0/np.sqrt(d)); 
    return np.eye(W.shape[0]) - D@W@D

def spec_labels(W,k=2):
    e,v = np.linalg.eigh(lap(W))
    U   = v[:,1:k] if k>1 else v[:,:1]
    U   = U/(np.linalg.norm(U,axis=1,keepdims=True)+1e-12)
    return KMeans(n_clusters=k, n_init=50, random_state=7).fit_predict(U)

def coassoc(labels):
    n=len(labels[0]); m=len(labels); co=np.zeros((n,n), float)
    for lab in labels:
        for i in range(n):
            li=lab[i]
            for j in range(n): co[i,j]+=1 if li==lab[j] else 0
    return co/m

def loso_via_coassoc(label_list):
    cof=coassoc(label_list); cons=spec_labels(cof,k=2)
    vals=[]
    for s in range(len(label_list)):
        leave=[lab for i,lab in enumerate(label_list) if i!=s]
        cons_l=spec_labels(coassoc(leave),k=2)
        vals.append(adjusted_rand_score(cons, cons_l))
    return float(np.median(vals)), cons, cof

# ---------------- discover subjects ----------------
subs = []
for f in glob.glob(os.path.join(DATA_DIR, "subject_*_EO.npy")):
    sid = int(re.search(r"subject_(\d+)_EO\.npy$", f).group(1))
    ecf = os.path.join(DATA_DIR, f"subject_{sid:02d}_EC.npy")
    if os.path.exists(ecf): subs.append(sid)
subs = sorted(subs)
print(f"Usable pairs: {len(subs)} (expected ≥ 100)")

# ---------------- parallel label cache ----------------
def compute_labels_for_file(npy_path, band_name, lo, hi):
    # cache file path
    sid   = int(re.search(r"subject_(\d+)_", npy_path).group(1))
    cond  = "EO" if "_EO" in npy_path else "EC"
    lab_fp= os.path.join(LABELS_DIR, f"labels__{cond}__{band_name}__{sid:03d}.npy")
    if os.path.exists(lab_fp): 
        return lab_fp
    X = np.load(npy_path)
    W = pli_matrix(X, FS_OUT, lo, hi)
    W = knn(W, KNN_K)
    lab = spec_labels(W, k=K_FIXED)
    np.save(lab_fp, lab)
    return lab_fp

def cache_all_labels(cond):
    files = [os.path.join(DATA_DIR, f"subject_{sid:02d}_{cond}.npy") for sid in subs]
    for band,(lo,hi) in BANDS_HZ.items():
        print(f"[{cond}] caching labels for band={band} (n={len(files)}) …")
        _ = Parallel(n_jobs=N_JOBS, prefer="processes")(
            delayed(compute_labels_for_file)(fp, band, lo, hi) for fp in files
        )

cache_all_labels("EO")
cache_all_labels("EC")

# ---------------- build consensus quickly ----------------
rng = default_rng(11)

def run_cond_quick(cond):
    rows=[]
    for band in BANDS_HZ.keys():
        labels=[]
        for sid in subs:
            lab_fp=os.path.join(LABELS_DIR, f"labels__{cond}__{band}__{sid:03d}.npy")
            labels.append(np.load(lab_fp))
        loso, cons, co = loso_via_coassoc(labels)
        # quick nulls
        null=[]
        for _ in range(NULL_PERMS):
            nl=[]
            for lab in labels:
                uniq,cnts=np.unique(lab, return_counts=True)
                idx=np.arange(len(lab)); rng.shuffle(idx)
                out=np.empty(len(lab),int); st=0
                for u,c in zip(uniq,cnts):
                    seg=idx[st:st+c]; out[seg]=u; st+=c
                nl.append(out)
            _, cons_n, _ = loso_via_coassoc(nl)
            null.append(adjusted_rand_score(cons, cons_n))
        null=np.array(null,float); p=float((np.sum(null>=loso)+1)/(len(null)+1))
        # save artifacts
        np.save(os.path.join(OUT_TAB,f"band__{cond}__{band}__consensus_labels.npy"), cons)
        np.save(os.path.join(OUT_TAB,f"band__{cond}__{band}__coassoc.npy"), co)
        with open(os.path.join(OUT_MET,f"band__{cond}__{band}__metrics.json"),"w",encoding="utf-8") as f:
            json.dump({"cond":cond,"band":band,"n_subjects":len(subs),"LOSO":float(loso),"null_ari_mean":float(null.mean()),"p_value":p}, f, indent=2)
        plt.figure(); plt.imshow(co, aspect='auto'); plt.title(f"{cond} {band} — co-assoc (FAST)")
        plt.colorbar(); plt.tight_layout(); plt.savefig(os.path.join(OUT_FIG, f"coassoc__{cond}__{band}.png"), dpi=160); plt.close()
        rows.append([cond, band, len(subs), float(loso), float(null.mean()), p])
    df=pd.DataFrame(rows, columns=["cond","band","n_subjects","LOSO","null_ari_mean","p_value"])
    df.to_csv(os.path.join(OUT_ROOT, f"summary_{cond}_FAST.csv"), index=False)
    print(f"[{cond}] FAST summary:\n", df.to_string(index=False))
    return df

print("\n=== FAST EO consensus (200 perms) ===")
df_eo_fast = run_cond_quick("EO")
print("\n=== FAST EC consensus (200 perms) ===")
df_ec_fast = run_cond_quick("EC")
print("\nFAST pass complete. You can top-up permutations later without recomputing labels.")


Usable pairs: 109 (expected ≥ 100)
[EO] caching labels for band=alpha (n=109) …
[EO] caching labels for band=theta (n=109) …
[EO] caching labels for band=beta (n=109) …
[EC] caching labels for band=alpha (n=109) …
[EC] caching labels for band=theta (n=109) …
[EC] caching labels for band=beta (n=109) …

=== FAST EO consensus (200 perms) ===


KeyboardInterrupt: 

In [1]:
# === COOL & RESUME: low-CPU α/θ labels + quick consensus (Windows-friendly) ===
# - Limits math threads (MKL/OMP) to 1 to avoid core oversubscription
# - Lowers process priority on Windows
# - Uses small parallel pool (N_JOBS=2 by default) and batches subjects with short pauses
# - α/θ only, NULL_PERMS=50 for quick p-values
# - Caches per-subject labels under labels_cache; consensus reuses them

import os, re, glob, time, json, numpy as np, pandas as pd, matplotlib.pyplot as plt
from numpy.random import default_rng
from scipy.signal import butter, filtfilt, hilbert
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from joblib import Parallel, delayed

# ---------- keep CPU cool ----------
# Reduce BLAS/OMP thread oversubscription (very important)
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"

# Lower process priority on Windows
try:
    import psutil, ctypes, sys
    p = psutil.Process(os.getpid())
    if os.name == "nt":
        BELOW_NORMAL = 0x4000
        IDLE = 0x40
        ctypes.windll.kernel32.SetPriorityClass(ctypes.windll.kernel32.GetCurrentProcess(), BELOW_NORMAL)
        print("[info] Set Windows process priority: BELOW_NORMAL")
except Exception as e:
    print("[warn] Could not lower priority:", e)

# ---------- paths & knobs ----------
ROOT       = r"C:\Users\caleb\CNT_Lab"
DATA_DIR   = os.path.join(ROOT, "eeg_rest")
OUT_ROOT   = os.path.join(ROOT, r"artifacts\pli_humans_100plus")
OUT_TAB    = os.path.join(OUT_ROOT, "tables")
OUT_MET    = os.path.join(OUT_ROOT, "metrics")
OUT_FIG    = os.path.join(OUT_ROOT, "figures")
LABELS_DIR = os.path.join(OUT_ROOT, "labels_cache")
for p in [OUT_TAB, OUT_MET, OUT_FIG, LABELS_DIR]: os.makedirs(p, exist_ok=True)

# Only α/θ for this pass
BANDS_HZ   = {"alpha": (8,13), "theta": (4,8)}
FS_OUT     = 250.0
K_FIXED    = 2
KNN_K      = 6
NULL_PERMS = 50             # quick pass
N_JOBS     = 2              # small parallelism
BATCH_SIZE = 15             # subjects per batch
REST_SEC   = 10             # rest between batches to cool

rng = default_rng(11)

# ---------- subject list (pairs only) ----------
subs = []
for f in glob.glob(os.path.join(DATA_DIR, "subject_*_EO.npy")):
    sid = int(re.search(r"subject_(\d+)_EO\.npy$", f).group(1))
    if os.path.exists(os.path.join(DATA_DIR, f"subject_{sid:02d}_EC.npy")):
        subs.append(sid)
subs = sorted(subs)
print(f"Usable pairs: {len(subs)}")

# ---------- PLI helpers ----------
def bandpass(x, fs, lo, hi, order=4):
    from scipy.signal import butter, filtfilt
    b,a = butter(order, [lo/(fs/2), hi/(fs/2)], btype="band"); return filtfilt(b,a,x)

def pli_matrix(X, fs, lo, hi):
    from scipy.signal import hilbert
    n = X.shape[0]
    b,a = butter(4, [lo/(fs/2), hi/(fs/2)], btype="band")
    Y = np.zeros_like(X)
    for c in range(n): Y[c] = filtfilt(b,a,X[c])
    ph = np.angle(hilbert(Y, axis=1))
    W  = np.zeros((n,n), float)
    for i in range(n):
        for j in range(i+1, n):
            dphi = ph[i]-ph[j]
            pli  = np.abs(np.mean(np.sign(np.sin(dphi))))
            W[i,j] = W[j,i] = pli
    np.fill_diagonal(W, 0.0)
    return W

def knn(W,k):
    W=W.copy(); n=W.shape[0]
    for i in range(n):
        idx=np.argsort(W[i])[::-1]; keep=idx[:k]
        mask=np.ones(n,bool); mask[keep]=False; W[i,mask]=0
    W=np.maximum(W,W.T); np.fill_diagonal(W,0); return W

def lap(W):
    d=W.sum(1); d=np.where(d<=1e-12,1.0,d); D=np.diag(1.0/np.sqrt(d)); 
    return np.eye(W.shape[0]) - D@W@D

def spec_labels(W,k=2):
    e,v = np.linalg.eigh(lap(W))
    U   = v[:,1:k] if k>1 else v[:,:1]
    U   = U/(np.linalg.norm(U,axis=1,keepdims=True)+1e-12)
    return KMeans(n_clusters=k, n_init=50, random_state=7).fit_predict(U)

def coassoc(labels):
    n=len(labels[0]); m=len(labels); co=np.zeros((n,n), float)
    for lab in labels:
        for i in range(n):
            li=lab[i]
            for j in range(n): co[i,j]+=1 if li==lab[j] else 0
    return co/m

def loso_via_coassoc(label_list):
    cof=coassoc(label_list); cons=spec_labels(cof,k=2)
    vals=[]
    for s in range(len(label_list)):
        leave=[lab for i,lab in enumerate(label_list) if i!=s]
        cons_l=spec_labels(coassoc(leave),k=2)
        vals.append(adjusted_rand_score(cons, cons_l))
    return float(np.median(vals)), cons, cof

# ---------- label cache ----------
def compute_labels_for_file(npy_path, band_name, lo, hi):
    sid   = int(re.search(r"subject_(\d+)_", npy_path).group(1))
    cond  = "EO" if "_EO" in npy_path else "EC"
    lab_fp= os.path.join(LABELS_DIR, f"labels__{cond}__{band_name}__{sid:03d}.npy")
    if os.path.exists(lab_fp): 
        return lab_fp
    X = np.load(npy_path)
    W = pli_matrix(X, FS_OUT, lo, hi)
    W = knn(W, KNN_K)
    lab = spec_labels(W, k=K_FIXED)
    np.save(lab_fp, lab)
    return lab_fp

def cache_cond(cond):
    files = [os.path.join(DATA_DIR, f"subject_{sid:02d}_{cond}.npy") for sid in subs]
    for band,(lo,hi) in BANDS_HZ.items():
        print(f"[{cond}] caching {band} in batches …")
        # batched parallel
        for i in range(0, len(files), BATCH_SIZE):
            batch = files[i:i+BATCH_SIZE]
            _ = Parallel(n_jobs=N_JOBS, prefer="processes")(
                delayed(compute_labels_for_file)(fp, band, lo, hi) for fp in batch
            )
            print(f"  done {i+len(batch)}/{len(files)} — cooling {REST_SEC}s")
            time.sleep(REST_SEC)

cache_cond("EO")
cache_cond("EC")

# ---------- quick consensus ----------
def run_cond_quick(cond):
    rows=[]
    for band in BANDS_HZ.keys():
        labels=[]
        for sid in subs:
            lab_fp=os.path.join(LABELS_DIR, f"labels__{cond}__{band}__{sid:03d}.npy")
            labels.append(np.load(lab_fp))
        loso, cons, co = loso_via_coassoc(labels)
        # quick null
        null=[]
        for _ in range(NULL_PERMS):
            nl=[]
            for lab in labels:
                uniq,cnts=np.unique(lab, return_counts=True)
                idx=np.arange(len(lab)); rng.shuffle(idx)
                out=np.empty(len(lab),int); st=0
                for u,c in zip(uniq,cnts):
                    seg=idx[st:st+c]; out[seg]=u; st+=c
                nl.append(out)
            _, cons_n, _ = loso_via_coassoc(nl)
            null.append(adjusted_rand_score(cons, cons_n))
        null=np.array(null,float); p=float((np.sum(null>=loso)+1)/(len(null)+1))
        np.save(os.path.join(OUT_TAB,f"band__{cond}__{band}__consensus_labels.npy"), cons)
        np.save(os.path.join(OUT_TAB,f"band__{cond}__{band}__coassoc.npy"), co)
        with open(os.path.join(OUT_MET,f"band__{cond}__{band}__metrics.json"),"w",encoding="utf-8") as f:
            json.dump({"cond":cond,"band":band,"n_subjects":len(subs),"LOSO":float(loso),"null_ari_mean":float(null.mean()),"p_value":p}, f, indent=2)
        plt.figure(); plt.imshow(co, aspect='auto'); plt.title(f"{cond} {band} — co-assoc (FAST COOL)")
        plt.colorbar(); plt.tight_layout(); plt.savefig(os.path.join(OUT_FIG, f"coassoc__{cond}__{band}.png"), dpi=160); plt.close()
        rows.append([cond, band, len(subs), float(loso), float(null.mean()), p])
    df=pd.DataFrame(rows, columns=["cond","band","n_subjects","LOSO","null_ari_mean","p_value"])
    df.to_csv(os.path.join(OUT_ROOT, f"summary_{cond}_COOL.csv"), index=False)
    print(f"[{cond}] COOL summary:\n", df.to_string(index=False))
    return df

print("\n=== COOL EO consensus ===")
df_eo = run_cond_quick("EO")
print("\n=== COOL EC consensus ===")
df_ec = run_cond_quick("EC")
print("\nCool/quick pass complete. Use the TOP-UP cell later for tighter p-values without recomputing labels.")


[info] Set Windows process priority: BELOW_NORMAL
Usable pairs: 109
[EO] caching alpha in batches …
  done 15/109 — cooling 10s
  done 30/109 — cooling 10s
  done 45/109 — cooling 10s
  done 60/109 — cooling 10s
  done 75/109 — cooling 10s
  done 90/109 — cooling 10s
  done 105/109 — cooling 10s
  done 109/109 — cooling 10s
[EO] caching theta in batches …
  done 15/109 — cooling 10s
  done 30/109 — cooling 10s
  done 45/109 — cooling 10s
  done 60/109 — cooling 10s
  done 75/109 — cooling 10s
  done 90/109 — cooling 10s
  done 105/109 — cooling 10s
  done 109/109 — cooling 10s
[EC] caching alpha in batches …
  done 15/109 — cooling 10s
  done 30/109 — cooling 10s
  done 45/109 — cooling 10s
  done 60/109 — cooling 10s
  done 75/109 — cooling 10s
  done 90/109 — cooling 10s
  done 105/109 — cooling 10s
  done 109/109 — cooling 10s
[EC] caching theta in batches …
  done 15/109 — cooling 10s
  done 30/109 — cooling 10s
  done 45/109 — cooling 10s
  done 60/109 — cooling 10s
  done 75/109 

KeyboardInterrupt: 

In [2]:
# === ULTRA-COOL CONSENSUS (α/θ) — vectorized, 1 thread, batched perms with cooldown ===
# Reuses cached per-subject labels in C:\Users\caleb\CNT_Lab\artifacts\pli_humans_100plus\labels_cache
# Computes LOSO & null p using outer-product trick on ±1 label vectors (no PLI, super light)
# Keeps CPU cool: 1 thread BLAS, small batches, short sleeps, no plotting.

import os, re, glob, time, json, numpy as np, pandas as pd
from numpy.random import default_rng
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

# ---- keep CPU cool ----
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"
# Lower priority on Windows (best-effort)
try:
    import ctypes
    BELOW_NORMAL = 0x4000
    ctypes.windll.kernel32.SetPriorityClass(ctypes.windll.kernel32.GetCurrentProcess(), BELOW_NORMAL)
    print("[info] Windows process priority set to BELOW_NORMAL")
except Exception:
    pass

# ---- paths & knobs ----
ROOT        = r"C:\Users\caleb\CNT_Lab"
OUT_ROOT    = os.path.join(ROOT, r"artifacts\pli_humans_100plus")
LABELS_DIR  = os.path.join(OUT_ROOT, "labels_cache")
OUT_TAB     = os.path.join(OUT_ROOT, "tables");  os.makedirs(OUT_TAB, exist_ok=True)
OUT_MET     = os.path.join(OUT_ROOT, "metrics"); os.makedirs(OUT_MET, exist_ok=True)

BANDS       = ["alpha","theta"]          # cool pass: α/θ only
CONDS       = ["EO","EC"]
NULL_PERMS  = 50                         # quick pass; top-up later overnight
BATCH_PERMS = 10                         # run perms in chunks of 10 …
REST_SEC    = 8                          # … then rest N seconds to cool
RNG         = default_rng(23)

# ---- helpers ----
def lap(W):
    d=W.sum(1); d=np.where(d<=1e-12,1.0,d)
    D=np.diag(1.0/np.sqrt(d))
    return np.eye(W.shape[0]) - D@W@D

def spectral_on_coassoc(C, k=2):
    # C is co-association [n×n], symmetric
    e,v = np.linalg.eigh(lap(C))
    U   = v[:,1:k]
    U   = U/(np.linalg.norm(U,axis=1,keepdims=True)+1e-12)
    return KMeans(n_clusters=k, n_init=50, random_state=7).fit_predict(U)

def load_sign_vectors(cond, band):
    """Load cached labels and convert to ±1 sign vectors per subject."""
    fps = sorted(glob.glob(os.path.join(LABELS_DIR, f"labels__{cond}__{band}__*.npy")))
    if not fps:
        return None, None
    # infer n_channels from first
    lab0 = np.load(fps[0])
    n = len(lab0)
    S = np.zeros((len(fps), n), dtype=np.int8)
    sids = []
    for i,fp in enumerate(fps):
        lab = np.load(fp).astype(int)
        # map {0,1} -> {-1,+1}
        S[i] = np.where(lab==1, 1, -1)
        sid  = int(re.search(r"__(\d+)\.npy$", fp).group(1))
        sids.append(sid)
    return S, np.array(sids, int)

def coassoc_from_signs(S):
    """
    Vectorized: if s ∈ {±1}^n, then same-label indicator E[s_i s_j] relates to co-assoc:
      co = 1/2 + (1/(2m)) * sum_s ( s ⊗ s )
    """
    m, n = S.shape
    # sum of outer products (m × (n×n) operation) — we do it row by row to keep mem tiny
    acc = np.zeros((n,n), dtype=np.float32)
    for s in S:
        acc += np.outer(s, s)            # very fast BLAS-level op
    C = 0.5 + acc / (2.0 * m)
    np.fill_diagonal(C, 1.0)            # co-assoc of i with i = 1
    return C

def loso_median_ari(S, cons_full):
    """LOSO median ARI using precomputed full-sum of outers."""
    m, n = S.shape
    # precompute total outer sum
    total = np.zeros((n,n), dtype=np.float32)
    for s in S: total += np.outer(s,s)
    aris=[]
    for i in range(m):
        # remove subject i
        Ci = 0.5 + (total - np.outer(S[i],S[i])) / (2.0 * (m-1))
        np.fill_diagonal(Ci, 1.0)
        cons_i = spectral_on_coassoc(Ci, k=2)
        aris.append(adjusted_rand_score(cons_full, cons_i))
    return float(np.median(aris))

def null_ari_vs_cons(S, cons_full, perms=NULL_PERMS):
    """Build null consensus by shuffling labels within each subject (preserves counts)."""
    m, n = S.shape
    null_aris=[]
    for p in range(perms):
        # batch rest to cool
        if (p % BATCH_PERMS)==0 and p>0:
            time.sleep(REST_SEC)
        # permute indices within each subject
        acc = np.zeros((n,n), dtype=np.float32)
        for s in S:
            idx = RNG.permutation(n)
            sp  = s[idx]
            acc += np.outer(sp, sp)
        Cn = 0.5 + acc / (2.0 * m)
        np.fill_diagonal(Cn, 1.0)
        cons_n = spectral_on_coassoc(Cn, k=2)
        null_aris.append(adjusted_rand_score(cons_full, cons_n))
    return np.array(null_aris, float)

# ---- main: per condition & band ----
all_rows=[]
for cond in CONDS:
    for band in BANDS:
        print(f"\n[{cond}][{band}] loading cached labels …")
        S, sids = load_sign_vectors(cond, band)
        if S is None:
            print("  no cached labels found; run the cache/resume cell first.")
            continue
        # full consensus
        C = coassoc_from_signs(S)
        cons = spectral_on_coassoc(C, k=2)
        # LOSO (fast)
        loso_med = loso_median_ari(S, cons)
        # null perms (batched, cool)
        null_aris = null_ari_vs_cons(S, cons, perms=NULL_PERMS)
        p_val = float((np.sum(null_aris >= loso_med) + 1) / (len(null_aris)+1))
        # save
        np.save(os.path.join(OUT_TAB, f"band__{cond}__{band}__consensus_labels.npy"), cons)
        np.save(os.path.join(OUT_TAB, f"band__{cond}__{band}__coassoc.npy"), C)
        with open(os.path.join(OUT_MET, f"band__{cond}__{band}__metrics.json"),"w",encoding="utf-8") as f:
            json.dump({
                "cond":cond, "band":band, "n_subjects": int(S.shape[0]), "n_channels": int(S.shape[1]),
                "LOSO_median_ARI": float(loso_med), "null_ari_mean": float(null_aris.mean()), "p_value": p_val,
                "null_perms": int(len(null_aris))
            }, f, indent=2)
        print(f"  n={S.shape[0]}  LOSO={loso_med:.3f}  null_mean={null_aris.mean():.3f}  p={p_val:.4f}")
        all_rows.append([cond, band, int(S.shape[0]), float(loso_med), float(null_aris.mean()), p_val])

# quick CSV
df = pd.DataFrame(all_rows, columns=["cond","band","n_subjects","LOSO","null_ari_mean","p_value"])
df.to_csv(os.path.join(OUT_ROOT, "summary_COOL_vectorized.csv"), index=False)
print("\nSaved:", os.path.join(OUT_ROOT, "summary_COOL_vectorized.csv"))
print("Cool pass complete. Use the TOP-UP cell later to push perms to 1000–2000 without recomputing labels.")


[info] Windows process priority set to BELOW_NORMAL

[EO][alpha] loading cached labels …
  n=109  LOSO=1.000  null_mean=0.004  p=0.0196

[EO][theta] loading cached labels …
  n=109  LOSO=1.000  null_mean=-0.002  p=0.0196

[EC][alpha] loading cached labels …
  n=109  LOSO=1.000  null_mean=0.004  p=0.0196

[EC][theta] loading cached labels …
  n=109  LOSO=1.000  null_mean=-0.003  p=0.0196

Saved: C:\Users\caleb\CNT_Lab\artifacts\pli_humans_100plus\summary_COOL_vectorized.csv
Cool pass complete. Use the TOP-UP cell later to push perms to 1000–2000 without recomputing labels.


In [3]:
# === SUPER-COOL TOP-UP (α/θ) — 2000 permutations, batched with cooldown (no PLI recompute) ===
import os, re, glob, time, json, numpy as np, pandas as pd
from numpy.random import default_rng
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

# single-thread BLAS
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"

# lower priority (Windows)
try:
    import ctypes
    BELOW_NORMAL = 0x4000
    ctypes.windll.kernel32.SetPriorityClass(ctypes.windll.kernel32.GetCurrentProcess(), BELOW_NORMAL)
    print("[info] Windows process priority set to BELOW_NORMAL")
except Exception:
    pass

ROOT        = r"C:\Users\caleb\CNT_Lab"
OUT_ROOT    = os.path.join(ROOT, r"artifacts\pli_humans_100plus")
LABELS_DIR  = os.path.join(OUT_ROOT, "labels_cache")
OUT_TAB     = os.path.join(OUT_ROOT, "tables")
OUT_MET     = os.path.join(OUT_ROOT, "metrics")

BANDS       = ["alpha","theta"]
CONDS       = ["EO","EC"]
NULL_PERMS  = 2000           # raise to 5000 overnight if you want
BATCH_PERMS = 20             # run this many perms then cool
REST_SEC    = 10             # rest seconds between batches
rng         = default_rng(42)

def lap(W):
    d=W.sum(1); d=np.where(d<=1e-12,1.0,d)
    D=np.diag(1.0/np.sqrt(d))
    return np.eye(W.shape[0]) - D@W@D

def spectral_on_coassoc(C, k=2):
    e,v = np.linalg.eigh(lap(C))
    U   = v[:,1:k]
    U   = U/(np.linalg.norm(U,axis=1,keepdims=True)+1e-12)
    return KMeans(n_clusters=k, n_init=50, random_state=7).fit_predict(U)

def load_sign_vectors(cond, band):
    fps = sorted(glob.glob(os.path.join(LABELS_DIR, f"labels__{cond}__{band}__*.npy")))
    if not fps: return None
    lab0 = np.load(fps[0]); n = len(lab0)
    S = np.zeros((len(fps), n), dtype=np.int8)
    for i,fp in enumerate(fps):
        lab = np.load(fp).astype(int)
        S[i] = np.where(lab==1, 1, -1)
    return S

def coassoc_from_signs(S):
    m,n = S.shape
    acc = np.zeros((n,n), dtype=np.float32)
    for s in S:
        acc += np.outer(s,s)
    C = 0.5 + acc/(2.0*m)
    np.fill_diagonal(C, 1.0)
    return C

def loso_median_ari(S, cons_full):
    m,n = S.shape
    total = np.zeros((n,n), dtype=np.float32)
    for s in S: total += np.outer(s,s)
    aris=[]
    for i in range(m):
        Ci = 0.5 + (total - np.outer(S[i],S[i]))/(2.0*(m-1))
        np.fill_diagonal(Ci, 1.0)
        cons_i = spectral_on_coassoc(Ci, k=2)
        aris.append(adjusted_rand_score(cons_full, cons_i))
    return float(np.median(aris))

def permuted_consensus_ari(S, cons_full, perms=NULL_PERMS):
    m,n = S.shape
    null_aris=[]
    for p in range(perms):
        if p>0 and (p % BATCH_PERMS)==0:
            time.sleep(REST_SEC)  # cool the CPU
        acc = np.zeros((n,n), dtype=np.float32)
        for s in S:
            idx = rng.permutation(n)
            sp  = s[idx]
            acc += np.outer(sp, sp)
        Cn = 0.5 + acc/(2.0*m)
        np.fill_diagonal(Cn, 1.0)
        cons_n = spectral_on_coassoc(Cn, k=2)
        null_aris.append(adjusted_rand_score(cons_full, cons_n))
    return np.array(null_aris, float)

rows=[]
for cond in CONDS:
    for band in BANDS:
        print(f"\nTOP-UP [{cond}][{band}] …")
        S = load_sign_vectors(cond, band)
        if S is None:
            print("  no cached labels found; run the cache step first.")
            continue
        # full consensus
        C = coassoc_from_signs(S)
        cons = spectral_on_coassoc(C, k=2)
        # LOSO with vectorized trick
        loso = loso_median_ari(S, cons)
        # permutations (batched)
        null = permuted_consensus_ari(S, cons, perms=NULL_PERMS)
        p = float((np.sum(null >= loso) + 1) / (len(null) + 1))
        # save
        np.save(os.path.join(OUT_TAB, f"band__{cond}__{band}__consensus_labels.npy"), cons)
        np.save(os.path.join(OUT_TAB, f"band__{cond}__{band}__coassoc.npy"), C)
        with open(os.path.join(OUT_MET, f"band__{cond}__{band}__metrics.json"), "w", encoding="utf-8") as f:
            json.dump({
                "cond":cond, "band":band, "n_subjects": int(S.shape[0]), "n_channels": int(S.shape[1]),
                "LOSO_median_ARI": float(loso), "null_ari_mean": float(null.mean()), "p_value": p,
                "null_perms": int(len(null))
            }, f, indent=2)
        print(f"  n={S.shape[0]}  LOSO={loso:.3f}  null_mean={null.mean():.3f}  p={p:.4f}")
        rows.append([cond, band, int(S.shape[0]), float(loso), float(null.mean()), p, int(len(null))])

df = pd.DataFrame(rows, columns=["cond","band","n_subjects","LOSO","null_ari_mean","p_value","null_perms"])
df.to_csv(os.path.join(OUT_ROOT, "summary_TOPUP_vectorized.csv"), index=False)
print("\nSaved:", os.path.join(OUT_ROOT, "summary_TOPUP_vectorized.csv"))
print("Top-up done. You can raise NULL_PERMS further overnight (e.g., 5000) with the same cell.")


[info] Windows process priority set to BELOW_NORMAL

TOP-UP [EO][alpha] …
  n=109  LOSO=1.000  null_mean=0.000  p=0.0005

TOP-UP [EO][theta] …


KeyboardInterrupt: 

In [4]:
# === Subsample Curves: "How many are enough?" (EO & EC, α/θ) ===
# Reuses cached labels in:
#   C:\Users\caleb\CNT_Lab\artifacts\pli_humans_100plus\labels_cache\labels__{EO|EC}__{alpha|theta}__###.npy
# For N ∈ {20,30,40,50,60,80,100,n_all}:
#   - TRIALS subsamples per N (default 10)
#   - PERMS label-preserving nulls per subsample (default 500)
# Computes LOSO median ARI & p; saves CSV + plots.

import os, re, glob, time, json, numpy as np, pandas as pd, matplotlib.pyplot as plt
from numpy.random import default_rng
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

ROOT        = r"C:\Users\caleb\CNT_Lab"
OUT_ROOT    = os.path.join(ROOT, r"artifacts\pli_humans_100plus")
LABELS_DIR  = os.path.join(OUT_ROOT, "labels_cache")
CURVES_DIR  = os.path.join(OUT_ROOT, "subsample_curves")
os.makedirs(CURVES_DIR, exist_ok=True)

CONDS       = ["EO","EC"]
BANDS       = ["alpha","theta"]

# Subsample settings (you can raise if you want)
TRIALS      = 10     # subsamples per N
PERMS       = 500    # null permutations per subsample
RNG         = default_rng(2025)

# Candidate N sizes (will truncate to available n)
N_GRID      = [20, 30, 40, 50, 60, 80, 100]  # we'll add n_all if larger

def lap(W):
    d=W.sum(1); d=np.where(d<=1e-12,1.0,d)
    D=np.diag(1.0/np.sqrt(d))
    return np.eye(W.shape[0]) - D@W@D

def spectral_on_coassoc(C, k=2):
    e,v = np.linalg.eigh(lap(C))
    U   = v[:,1:k]
    U   = U/(np.linalg.norm(U,axis=1,keepdims=True)+1e-12)
    return KMeans(n_clusters=k, n_init=50, random_state=7).fit_predict(U)

def load_sign_matrix(cond, band):
    """Load cached labels -> convert {0,1} to {-1,+1} sign vectors.
       Returns S [m×n], subject ids list."""
    fps = sorted(glob.glob(os.path.join(LABELS_DIR, f"labels__{cond}__{band}__*.npy")))
    if not fps:
        return None, None
    n = len(np.load(fps[0]))
    S = np.zeros((len(fps), n), dtype=np.int8)
    sids = []
    for i,fp in enumerate(fps):
        lab = np.load(fp).astype(int)
        S[i] = np.where(lab==1, 1, -1)
        sids.append(int(re.search(r"__(\d+)\.npy$", fp).group(1)))
    return S, sids

def coassoc_from_signs(S):
    """C = 1/2 + (1/(2m)) * sum_s (s ⊗ s) ; diagonal=1."""
    m,n = S.shape
    acc = np.zeros((n,n), dtype=np.float32)
    for s in S:
        acc += np.outer(s,s)
    C = 0.5 + acc/(2.0*m)
    np.fill_diagonal(C, 1.0)
    return C

def loso_median_ari(S, cons_full):
    """Leave-one-subject-out median ARI using sign-vector trick."""
    m,n = S.shape
    total = np.zeros((n,n), dtype=np.float32)
    for s in S: total += np.outer(s,s)
    aris=[]
    for i in range(m):
        Ci = 0.5 + (total - np.outer(S[i],S[i]))/(2.0*(m-1))
        np.fill_diagonal(Ci, 1.0)
        cons_i = spectral_on_coassoc(Ci, k=2)
        aris.append(adjusted_rand_score(cons_full, cons_i))
    return float(np.median(aris))

def null_p_value(S, cons_full, loso_obs, perms=PERMS):
    """Label-preserving null: random relabel within subject (permute indices)."""
    m,n = S.shape
    cnt = 1
    for _ in range(perms):
        acc = np.zeros((n,n), dtype=np.float32)
        for s in S:
            idx = RNG.permutation(n)
            sp  = s[idx]
            acc += np.outer(sp, sp)
        Cn = 0.5 + acc/(2.0*m)
        np.fill_diagonal(Cn, 1.0)
        cons_n = spectral_on_coassoc(Cn, k=2)
        # Compare consensus similarity: ARI(cons_full, cons_null)
        ari_null = adjusted_rand_score(cons_full, cons_n)
        if ari_null >= loso_obs:
            cnt += 1
    return float(cnt / (perms + 1))

def subsample_curve_for(cond, band, trials=TRIALS, perms=PERMS):
    S_all, sids_all = load_sign_matrix(cond, band)
    if S_all is None:
        print(f"[{cond}][{band}] No cached labels; run label cache first.")
        return None
    m_all = S_all.shape[0]
    Ns = [n for n in N_GRID if n <= m_all]
    if m_all not in Ns: Ns.append(m_all)  # ensure full-n point

    rows=[]
    for N in Ns:
        for t in range(trials):
            idx = RNG.choice(m_all, size=N, replace=False)
            S = S_all[idx]
            # real consensus
            C = coassoc_from_signs(S)
            cons = spectral_on_coassoc(C, k=2)
            loso = loso_median_ari(S, cons)
            pval = null_p_value(S, cons, loso, perms=perms)
            rows.append({"cond":cond, "band":band, "N":N, "trial":t, "LOSO":loso, "p":pval})
        print(f"[{cond}][{band}] N={N}: trials={trials} done.")
    df = pd.DataFrame(rows)
    csvp = os.path.join(CURVES_DIR, f"subsample__{cond}__{band}.csv")
    df.to_csv(csvp, index=False)
    print(" saved:", csvp)
    return df

# ---- Run for EO/EC, alpha/theta ----
all_df=[]
for cond in CONDS:
    for band in BANDS:
        df = subsample_curve_for(cond, band, trials=TRIALS, perms=PERMS)
        if df is not None: all_df.append(df)
if not all_df:
    raise SystemExit("No curves computed; check cache.")
df_all = pd.concat(all_df, ignore_index=True)
df_all.to_csv(os.path.join(CURVES_DIR, "subsample_all.csv"), index=False)

# ---- Plotting ----
def plot_curves(df, cond, band):
    d = df[(df["cond"]==cond) & (df["band"]==band)]
    if d.empty: return None
    Ns = sorted(d["N"].unique())
    mu_loso=[]; lo_loso=[]; hi_loso=[]
    med_p=[]; frac_sig=[]
    for N in Ns:
        k = d[d["N"]==N]["LOSO"].values
        mu_loso.append(np.mean(k))
        lo_loso.append(np.percentile(k, 2.5))
        hi_loso.append(np.percentile(k, 97.5))
        p = d[d["N"]==N]["p"].values
        med_p.append(np.median(p))
        frac_sig.append(np.mean(p < 0.05))
    # figure
    fig = plt.figure(figsize=(7.5,4.5))
    ax1 = fig.add_subplot(121)
    ax1.plot(Ns, mu_loso, marker='o')
    ax1.fill_between(Ns, lo_loso, hi_loso, alpha=0.25)
    ax1.set_title(f"{cond} {band} — LOSO vs N")
    ax1.set_xlabel("N subjects"); ax1.set_ylabel("LOSO median ARI")
    ax1.set_ylim(0.0, 1.05)
    ax2 = fig.add_subplot(122)
    ax2.plot(Ns, med_p, marker='o', label="median p")
    ax2.plot(Ns, frac_sig, marker='s', label="fraction p<0.05")
    ax2.set_title(f"{cond} {band} — p vs N")
    ax2.set_xlabel("N subjects")
    ax2.set_ylabel("p (median) / frac. sig")
    ax2.set_ylim(0.0, 1.05); ax2.legend()
    fig.tight_layout()
    outp = os.path.join(CURVES_DIR, f"subsample__{cond}__{band}.png")
    fig.savefig(outp, dpi=160); plt.close(fig)
    print(" plot:", outp)
    return outp

for cond in CONDS:
    for band in BANDS:
        plot_curves(df_all, cond, band)

print("\nDone. Subsample CSVs + plots saved to:", CURVES_DIR)


[EO][alpha] N=20: trials=10 done.
[EO][alpha] N=30: trials=10 done.
[EO][alpha] N=40: trials=10 done.
[EO][alpha] N=50: trials=10 done.
[EO][alpha] N=60: trials=10 done.
[EO][alpha] N=80: trials=10 done.
[EO][alpha] N=100: trials=10 done.
[EO][alpha] N=109: trials=10 done.
 saved: C:\Users\caleb\CNT_Lab\artifacts\pli_humans_100plus\subsample_curves\subsample__EO__alpha.csv
[EO][theta] N=20: trials=10 done.
[EO][theta] N=30: trials=10 done.
[EO][theta] N=40: trials=10 done.
[EO][theta] N=50: trials=10 done.
[EO][theta] N=60: trials=10 done.
[EO][theta] N=80: trials=10 done.
[EO][theta] N=100: trials=10 done.
[EO][theta] N=109: trials=10 done.
 saved: C:\Users\caleb\CNT_Lab\artifacts\pli_humans_100plus\subsample_curves\subsample__EO__theta.csv
[EC][alpha] N=20: trials=10 done.
[EC][alpha] N=30: trials=10 done.
[EC][alpha] N=40: trials=10 done.
[EC][alpha] N=50: trials=10 done.
[EC][alpha] N=60: trials=10 done.
[EC][alpha] N=80: trials=10 done.
[EC][alpha] N=100: trials=10 done.
[EC][alph

In [5]:
# === CNT Sample-Size Sufficiency Summary (1-page PDF) ===
# Combines EC/EO × α/θ subsample plots + recommended N table + interpretation.
# Outputs:
#   C:\Users\caleb\CNT_Lab\artifacts\pli_humans_100plus\CNT_SampleSize_Summary.pdf

import os, re, glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# ---------- paths ----------
ROOT       = r"C:\Users\caleb\CNT_Lab"
ART_ROOT   = os.path.join(ROOT, r"artifacts\pli_humans_100plus")
CURVES_DIR = os.path.join(ART_ROOT, "subsample_curves")
OUT_PDF    = os.path.join(ART_ROOT, "CNT_SampleSize_Summary.pdf")

# Expect these four PNGs:
PNG_EC_A   = os.path.join(CURVES_DIR, "subsample__EC__alpha.png")
PNG_EC_T   = os.path.join(CURVES_DIR, "subsample__EC__theta.png")
PNG_EO_A   = os.path.join(CURVES_DIR, "subsample__EO__alpha.png")
PNG_EO_T   = os.path.join(CURVES_DIR, "subsample__EO__theta.png")

# Combined CSV if present, else fall back to per-panel CSVs
CSV_ALL    = os.path.join(CURVES_DIR, "subsample_all.csv")
CSV_EC_A   = os.path.join(CURVES_DIR, "subsample__EC__alpha.csv")
CSV_EC_T   = os.path.join(CURVES_DIR, "subsample__EC__theta.csv")
CSV_EO_A   = os.path.join(CURVES_DIR, "subsample__EO__alpha.csv")
CSV_EO_T   = os.path.join(CURVES_DIR, "subsample__EO__theta.csv")

# ---------- load data ----------
def load_curves():
    if os.path.exists(CSV_ALL):
        df = pd.read_csv(CSV_ALL)
    else:
        parts = []
        for p in [CSV_EC_A, CSV_EC_T, CSV_EO_A, CSV_EO_T]:
            if os.path.exists(p):
                parts.append(pd.read_csv(p))
        if not parts:
            raise SystemExit("No subsample CSVs found in: " + CURVES_DIR)
        df = pd.concat(parts, ignore_index=True)
    # coerce types safely
    for col in ["N","trial","LOSO","p"]:
        df[col] = pd.to_numeric(df[col], errors="coerce")
    df = df.dropna(subset=["cond","band","N","LOSO","p"]).copy()
    df["N"] = df["N"].astype(int)
    # standardize labels
    df["cond"] = df["cond"].str.upper()
    df["band"] = df["band"].str.lower()
    return df

df = load_curves()

# ---------- summarize + elbow per (cond,band) ----------
def summarize_panel(dfb):
    out_rows = []
    Ns = sorted(dfb["N"].unique())
    for N in Ns:
        dn = dfb[dfb["N"]==N]
        loso = dn["LOSO"].values
        p    = dn["p"].values
        out_rows.append({
            "N": int(N),
            "LOSO_mean": float(np.nanmean(loso)),
            "LOSO_2.5%": float(np.nanpercentile(loso, 2.5)),
            "LOSO_97.5%": float(np.nanpercentile(loso, 97.5)),
            "p_median": float(np.nanmedian(p)),
            "frac_p<0.05": float(np.nanmean(p < 0.05)),
            "trials": int(len(dn))
        })
    summ = pd.DataFrame(out_rows).sort_values("N", ascending=True).reset_index(drop=True)
    # elbow heuristic: first N where LOSO_mean ≥ 0.99 and frac_sig ≥ 0.95
    elbow = None
    good = summ[(summ["LOSO_mean"] >= 0.99) & (summ["frac_p<0.05"] >= 0.95)]
    if not good.empty:
        elbow = int(good["N"].iloc[0])
    return summ, elbow

summary_map = {}  # (cond,band) -> (summary_df, elbow_N)
recommend_rows = []
for cond, band in [("EC","alpha"),("EC","theta"),("EO","alpha"),("EO","theta")]:
    dpanel = df[(df["cond"]==cond) & (df["band"]==band)]
    if dpanel.empty:
        summary_map[(cond,band)] = (pd.DataFrame(), None)
        continue
    summ, elbow = summarize_panel(dpanel)
    summary_map[(cond,band)] = (summ, elbow)
    # also compute simple N* where frac p<.05 == 1.0 if elbow missing
    fallback_N = None
    if elbow is None:
        h = summ[summ["frac_p<0.05"] >= 0.95]
        if not h.empty:
            fallback_N = int(h["N"].iloc[0])
    recommend_rows.append({
        "Condition": cond,
        "Band": band,
        "Recommended N": elbow if elbow is not None else (fallback_N if fallback_N is not None else int(summ["N"].max())),
        "LOSO@N*": float(summ[summ["N"]==(elbow if elbow is not None else summ['N'].max())]["LOSO_mean"].iloc[0]) if not summ.empty else np.nan,
        "Frac p<0.05@N*": float(summ[summ["N"]==(elbow if elbow is not None else summ['N'].max())]["frac_p<0.05"].iloc[0]) if not summ.empty else np.nan
    })

recommend_df = pd.DataFrame(recommend_rows)

# ---------- compose PDF ----------
fig = plt.figure(figsize=(11, 8.5))

# Title
ax_t = fig.add_axes([0.05, 0.91, 0.90, 0.07]); ax_t.axis("off")
ax_t.text(0.5, 0.6, "CNT Subsample Saturation — Humans (EO/EC × α/θ)", ha="center", va="center",
          fontsize=16, weight="bold")

# 2×2 image grid
slots = [
    (PNG_EC_A, "EC α", [0.05, 0.58, 0.42, 0.28]),
    (PNG_EC_T, "EC θ", [0.53, 0.58, 0.42, 0.28]),
    (PNG_EO_A, "EO α", [0.05, 0.26, 0.42, 0.28]),
    (PNG_EO_T, "EO θ", [0.53, 0.26, 0.42, 0.28]),
]
for path, title, rect in slots:
    ax = fig.add_axes(rect); ax.axis("off")
    if os.path.exists(path):
        ax.imshow(plt.imread(path))
    ax.set_title(title, fontsize=12)

# Summary table of recommended N
ax_tab = fig.add_axes([0.05, 0.11, 0.90, 0.10]); ax_tab.axis("off")
if not recommend_df.empty:
    # nicer order
    order = [("EC","alpha"),("EC","theta"),("EO","alpha"),("EO","theta")]
    rows = []
    for cond, band in order:
        r = recommend_df[(recommend_df["Condition"]==cond)&(recommend_df["Band"]==band)]
        if not r.empty:
            rows.append(r.iloc[0].to_dict())
    table_df = pd.DataFrame(rows, columns=["Condition","Band","Recommended N","LOSO@N*","Frac p<0.05@N*"])
    table = ax_tab.table(cellText=table_df.values,
                         colLabels=table_df.columns,
                         loc="center")
    table.auto_set_font_size(False); table.set_fontsize(9); table.scale(1.0, 1.25)
else:
    ax_tab.text(0.0, 0.5, "No recommendation data available.", fontsize=10)

# Interpretation paragraph
ax_p = fig.add_axes([0.05, 0.02, 0.90, 0.09]); ax_p.axis("off")
interp = (
    "Interpretation: Across EO/EC and α/θ, LOSO approaches ~1.0 and the fraction of significant subsamples "
    "(p<0.05, label-preserving null) saturates between N≈40–60. In practice, ~50–60 subjects are sufficient "
    "for a stable two-module consensus; α often flattens by N≈30–40, while θ typically flattens by N≈40–50. "
    "These curves justify a preregistered stop rule at N=60, or earlier if LOSO≥0.99 with ≥95% of subsamples significant."
)
ax_p.text(0.0, 0.5, interp, fontsize=10, va="center")

fig.tight_layout()
pp = PdfPages(OUT_PDF); pp.savefig(fig, dpi=200); pp.close(); plt.close(fig)
print("Saved 1-page PDF:", OUT_PDF)
print("Summary table:\n", recommend_df.to_string(index=False))


C:\Users\caleb\AppData\Local\Temp\ipykernel_34740\316681231.py:154: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout()


Saved 1-page PDF: C:\Users\caleb\CNT_Lab\artifacts\pli_humans_100plus\CNT_SampleSize_Summary.pdf
Summary table:
 Condition  Band  Recommended N  LOSO@N*  Frac p<0.05@N*
       EC alpha            109  1.00000             1.0
       EC theta             80  1.00000             1.0
       EO alpha             30  1.00000             1.0
       EO theta             50  0.99375             1.0


In [6]:
# === CNT Functional Tests — "What do the two modules do?" (EO/EC vs Motor Imagery/Execution) ===
# Modules: defined once from EC alpha consensus → Frontal (control) vs Posterior (sensory)
# Conditions: EC, EO, R03 (L hand), R04 (R hand), R05 (both fists), R06 (both feet)
# Bands: alpha (8–13 Hz), theta (4–8 Hz)
# Metrics (per subject, per condition):
#   - Within-Frontal coupling (FR↔FR)  [PLI]
#   - Within-Posterior coupling (PO↔PO) [PLI]
#   - Cross coupling (FR↔PO)            [PLI]
#   - Directionality (FR↔PO)            [Phase Slope Index, PSI]
# Group tests: paired (EC↔EO, EC↔motor, EO↔motor) with 10k label-flip perms, Cohen's d
# Outputs:
#   C:\Users\caleb\CNT_Lab\artifacts\pli_humans_100plus\tests_functional\{tables,figures}\*.{csv,png}

import os, re, glob, json, numpy as np, pandas as pd, matplotlib.pyplot as plt
from numpy.random import default_rng
from scipy.signal import butter, filtfilt, hilbert, welch, csd
from sklearn.utils import resample

# ---------------- paths & constants ----------------
ROOT        = r"C:\Users\caleb\CNT_Lab"
DATA_DIR    = os.path.join(ROOT, "eeg_rest")  # where subject_##_{EC|EO}.npy already live
OUT_ROOT    = os.path.join(ROOT, r"artifacts\pli_humans_100plus\tests_functional")
OUT_TAB     = os.path.join(OUT_ROOT, "tables")
OUT_FIG     = os.path.join(OUT_ROOT, "figures")
for p in [OUT_ROOT, OUT_TAB, OUT_FIG]: os.makedirs(p, exist_ok=True)

# EC α consensus (for modules)
CONS_EC_ALPHA = os.path.join(ROOT, r"artifacts\pli_humans_100plus\tables\band__EC__alpha__consensus_labels.npy")
CH_TXT        = os.path.join(ROOT, r"eeg_rest\subject_01_EC.channels.txt")
assert os.path.exists(CONS_EC_ALPHA), "EC alpha consensus not found — run the CNT consensus first."
assert os.path.exists(CH_TXT), "Channel names file missing."

FS        = 250.0
BANDS_HZ  = {"alpha": (8,13), "theta": (4,8)}
RNG       = default_rng(2026)

# Motor runs to add (download if missing): R03–R06
RUN_MAP   = {
    "R03": ("L_hand", 3),
    "R04": ("R_hand", 4),
    "R05": ("Both_fists", 5),
    "R06": ("Both_feet", 6),
}

# ---------------- install MNE if needed and export runs R03-R06 ----------------
try:
    import mne
except Exception:
    import sys, subprocess
    subprocess.check_call([sys.executable,"-m","pip","install","-q","mne","pooch"])
    import mne

def export_subject_run(subj, run, tag):
    """Export one EEGBCI run as subject_##_TAG.npy (+ channels.txt), 250 Hz, 60 s."""
    base = os.path.join(DATA_DIR, f"subject_{subj:02d}_{tag}")
    if os.path.exists(base + ".npy"):
        return True, "exists"
    try:
        try:
            fpaths = mne.datasets.eegbci.load_data(subjects=[subj], runs=[run], update_path=True, verbose="ERROR")
        except TypeError:
            fpaths = mne.datasets.eegbci.load_data(subject=subj, runs=[run], update_path=True, verbose="ERROR")
        raws=[]
        for fp in fpaths:
            raw = mne.io.read_raw_edf(fp, preload=True, verbose="ERROR")
            raw.pick_types(eeg=True, stim=False, eog=False, ecg=False, emg=False, misc=False)
            raws.append(raw)
        if not raws: return False, "no_raw"
        raw = mne.concatenate_raws(raws, verbose="ERROR")
        try:
            raw.set_montage("standard_1020", on_missing="ignore", match_case=False, verbose="ERROR")
        except Exception:
            pass
        raw.filter(1.0, 45.0, fir_design="firwin", verbose="ERROR")
        raw.resample(FS, npad="auto", verbose="ERROR")
        n_keep = int(60 * FS)
        X = raw.get_data(picks="eeg")
        if X.shape[1] >= n_keep:
            X = X[:, :n_keep]
        else:
            reps = int(np.ceil(n_keep / X.shape[1])); X = np.tile(X, reps)[:, :n_keep]
        ch_names = mne.pick_info(raw.info, mne.pick_types(raw.info, eeg=True)).ch_names
        np.save(base + ".npy", X.astype(np.float32))
        with open(base + ".channels.txt","w",encoding="utf-8") as f:
            for ch in ch_names: f.write(ch + "\n")
        return True, X.shape
    except Exception as e:
        return False, str(e)

# Build subject list (must have both EO/EC to be paired)
subs = []
for f in glob.glob(os.path.join(DATA_DIR, "subject_*_EO.npy")):
    sid = int(re.search(r"subject_(\d+)_EO\.npy$", f).group(1))
    if os.path.exists(os.path.join(DATA_DIR, f"subject_{sid:02d}_EC.npy")):
        subs.append(sid)
subs = sorted(subs)

print(f"Subjects with EO & EC present: n={len(subs)}")

# Export motor runs for those subjects (skip if present)
log=[]
for sid in subs:
    for tag, (human_run, run_id) in RUN_MAP.items():
        ok,msg = export_subject_run(sid, run_id, tag)
        log.append((sid, tag, msg))
print("Export summary (first 12):", log[:12], "...")

# ---------------- define modules from EC alpha consensus ----------------
def clean_label(x: str) -> str:
    y = x.strip()
    y = re.sub(r"(?i)^(EEG|MEG|EOG|ECG|EMG)[\s_\-]+", "", y)
    y = re.sub(r"(?i)[\s_\-]*(REF|AV|AVERAGE|AVG|M1|M2)$", "", y)
    y = re.sub(r"[ \-\.]+", "", y)
    y = y.replace("FP","Fp")
    return y

with open(CH_TXT,"r",encoding="utf-8") as f:
    ch_names = [clean_label(ln.strip()) for ln in f if ln.strip()]

ANT_PREFIXES=("Fp","AF","F","FC")
POST_PREFIXES=("CP","P","PO","O")

def anterior_posterior_indices(names):
    A,P=[],[]
    for i,ch in enumerate(names):
        pref = re.match(r"[A-Za-z]+", ch)
        pref = pref.group(0) if pref else ""
        if any(pref.startswith(px) for px in ANT_PREFIXES): A.append(i)
        elif any(pref.startswith(px) for px in POST_PREFIXES): P.append(i)
    return np.array(A,int), np.array(P,int)

A_idx, P_idx = anterior_posterior_indices(ch_names)
cons_alpha = np.load(CONS_EC_ALPHA)  # module labels 0/1
# assign which label is Frontal vs Posterior by overlap with A_idx
m0A = np.intersect1d(np.where(cons_alpha==0)[0], A_idx).size
FR_LABEL, PO_LABEL = (0,1) if m0A >= (np.intersect1d(np.where(cons_alpha==1)[0], A_idx).size) else (1,0)
FR_idx = np.where(cons_alpha==FR_LABEL)[0]
PO_idx = np.where(cons_alpha==PO_LABEL)[0]
print(f"Modules → Frontal={len(FR_idx)} (label {FR_LABEL}), Posterior={len(PO_idx)} (label {PO_LABEL})")

# ---------------- PLI metrics ----------------
def bandpass(x, fs, lo, hi, order=4):
    b,a = butter(order, [lo/(fs/2), hi/(fs/2)], btype="band"); return filtfilt(b,a,x)

def pli_pair(x, y, lo, hi, fs):
    bx = bandpass(x, fs, lo, hi); by = bandpass(y, fs, lo, hi)
    phx = np.angle(hilbert(bx)); phy = np.angle(hilbert(by))
    d   = phx - phy
    return float(np.abs(np.mean(np.sign(np.sin(d)))))

def pli_mean_sets(X, idxA, idxB, lo, hi, fs):
    vals=[]
    if np.array_equal(idxA, idxB):
        ids=list(idxA)
        for i in range(len(ids)):
            for j in range(i+1,len(ids)):
                vals.append(pli_pair(X[ids[i]], X[ids[j]], lo, hi, fs))
    else:
        for i in idxA:
            for j in idxB:
                vals.append(pli_pair(X[i], X[j], lo, hi, fs))
    return float(np.mean(vals)) if vals else np.nan

# ---------------- PSI (Phase Slope Index) for directionality ----------------
def psi_between_sets(X, idx_from, idx_to, fs, f_lo, f_hi, nperseg=256, noverlap=128):
    """
    PSI (Nolte et al.) between two sets: average PSI across all pairs (from→to).
    Positive PSI ~ from leads to; negative ~ to leads from.
    """
    # Estimate cross-spectrum for pairs; then PSI from slope of phase vs freq
    pairs=[]
    for i in idx_from:
        for j in idx_to:
            f, Pxy = csd(X[i], X[j], fs=fs, nperseg=nperseg, noverlap=noverlap)
            # restrict band
            sel = (f>=f_lo) & (f<=f_hi)
            if sel.sum()<3: continue
            ph = np.angle(Pxy[sel])
            # unwrap phase and fit slope
            ph_unw = np.unwrap(ph)
            ff = f[sel]
            # simple linear slope of phase vs freq
            slope = np.polyfit(ff, ph_unw, 1)[0]   # rad/Hz
            pairs.append(slope)
    if not pairs:
        return np.nan
    # Normalize by frequency span to get a scale-comparable index
    return float(np.mean(pairs))

# ---------------- per-subject metrics for all conditions ----------------
def metrics_for_subject(sid, cond_tag, bands=BANDS_HZ):
    base = os.path.join(DATA_DIR, f"subject_{sid:02d}_{cond_tag}.npy")
    if not os.path.exists(base):
        return None
    X = np.load(base)
    out = {"subject": sid, "cond": cond_tag}
    for band,(lo,hi) in bands.items():
        out[f"{band}_FR_FR"] = pli_mean_sets(X, FR_idx, FR_idx, lo, hi, FS)
        out[f"{band}_PO_PO"] = pli_mean_sets(X, PO_idx, PO_idx, lo, hi, FS)
        out[f"{band}_FR_PO"] = pli_mean_sets(X, FR_idx, PO_idx, lo, hi, FS)
        out[f"{band}_PSI_FR_to_PO"] = psi_between_sets(X, FR_idx, PO_idx, FS, lo, hi)
        out[f"{band}_PSI_PO_to_FR"] = psi_between_sets(X, PO_idx, FR_idx, FS, lo, hi)
    return out

all_rows=[]
conds = ["EC","EO","R03","R04","R05","R06"]
for sid in subs:
    for cond in conds:
        r = metrics_for_subject(sid, cond)
        if r: all_rows.append(r)

df = pd.DataFrame(all_rows)
out_csv = os.path.join(OUT_TAB, "subject_metrics_allconds.csv")
df.to_csv(out_csv, index=False)
print("Saved per-subject metrics:", out_csv)

# ---------------- group tests (paired) ----------------
def paired_perm_test(a, b, n_perm=10000, rng=None):
    """Two-sided paired permutation test on mean difference (a-b)."""
    a = np.asarray(a, float); b = np.asarray(b, float)
    mask = np.isfinite(a) & np.isfinite(b)
    a,b = a[mask], b[mask]
    if len(a) < 10: 
        return np.nan, np.nan, np.nan, np.nan  # too small
    diff = a - b
    obs  = float(np.mean(diff))
    # Cohen's d for paired design (mean / std of differences)
    d    = float(obs / (np.std(diff, ddof=1)+1e-12))
    if rng is None:
        rng = default_rng(0)
    cnt = 1
    for _ in range(n_perm):
        signs = rng.integers(0,2,size=len(diff))*2-1  # ±1
        perm  = np.mean(diff * signs)
        if abs(perm) >= abs(obs): cnt += 1
    p = float(cnt / (n_perm+1))
    return obs, d, p, int(len(a))

def summarize_comparison(df, condA, condB, band, metric):
    a = df[df["cond"]==condA][f"{band}_{metric}"].values
    b = df[df["cond"]==condB][f"{band}_{metric}"].values
    obs, d, p, n = paired_perm_test(a, b, n_perm=10000, rng=RNG)
    return {"band":band, "metric":metric, "A":condA, "B":condB, "mean_diff(A-B)":obs, "cohen_d":d, "p_perm":p, "N":n}

comparisons = []
for band in BANDS_HZ.keys():
    for metric in ["FR_FR","PO_PO","FR_PO","PSI_FR_to_PO","PSI_PO_to_FR"]:
        # EC vs EO
        comparisons.append(summarize_comparison(df, "EC", "EO", band, metric))
        # EC vs (combined motor): average R03–R06 per subject
        df_motor = df.copy()
        # Collapse motor per subject (mean across R03..R06)
        motors = df_motor[df_motor["cond"].isin(["R03","R04","R05","R06"])].groupby("subject").mean(numeric_only=True).reset_index()
        motors["cond"] = "MOTOR"
        df_coll = pd.concat([df[df["cond"]=="EC"], motors], ignore_index=True)
        comparisons.append(summarize_comparison(df_coll, "MOTOR", "EC", band, metric))
        # EO vs MOTOR
        df_coll2 = pd.concat([df[df["cond"]=="EO"], motors], ignore_index=True)
        comparisons.append(summarize_comparison(df_coll2, "MOTOR", "EO", band, metric))

comp_df = pd.DataFrame(comparisons)
comp_csv = os.path.join(OUT_TAB, "group_tests_summary.csv")
comp_df.to_csv(comp_csv, index=False)
print("Saved group tests:", comp_csv)
print(comp_df.head(10).to_string(index=False))

# ---------------- plots for key predictions ----------------
def plot_bars(df, band, metric, conds=("EC","EO","MOTOR"), title=None, ylim=(0,1.0), fname=""):
    # assemble per-subject arrays
    # motor value per subject = mean of R03–R06
    mot = df[df["cond"].isin(["R03","R04","R05","R06"])].groupby("subject").mean(numeric_only=True)[f"{band}_{metric}"]
    arrs = [
        df[df["cond"]=="EC"][f"{band}_{metric}"].values,
        df[df["cond"]=="EO"][f"{band}_{metric}"].values,
        mot.values
    ]
    labels = ["EC","EO","MOTOR"]
    means = [np.nanmean(a) for a in arrs]
    # bootstrap CI on mean
    lo=[]; hi=[]
    for a in arrs:
        a = a[np.isfinite(a)]
        if len(a)<5: lo.append(np.nan); hi.append(np.nan); continue
        boots = [np.mean(resample(a, replace=True, n_samples=len(a), random_state=i)) for i in range(1000)]
        lo.append(np.percentile(boots, 2.5)); hi.append(np.percentile(boots, 97.5))
    x = np.arange(3)
    plt.figure(figsize=(6.2,4.0))
    plt.bar(x, means, yerr=[np.array(means)-np.array(lo), np.array(hi)-np.array(means)], capsize=4)
    plt.xticks(x, labels)
    plt.ylim(ylim); plt.ylabel(metric.replace("_","→") if "PSI" in metric else "PLI")
    plt.title(title if title else f"{band} {metric}")
    outp = os.path.join(OUT_FIG, fname if fname else f"{band}_{metric}.png")
    plt.tight_layout(); plt.savefig(outp, dpi=160); plt.close()
    return outp

# Alpha: posterior within should drop from EC→EO; theta: frontal within should rise in MOTOR vs rest
plots=[]
plots.append(plot_bars(df, "alpha", "PO_PO", ("EC","EO","MOTOR"),
                       title="Alpha: Posterior within (EC, EO, MOTOR)", ylim=(0,1.0),
                       fname="alpha_PO_within.png"))
plots.append(plot_bars(df, "theta", "FR_FR", ("EC","EO","MOTOR"),
                       title="Theta: Frontal within (EC, EO, MOTOR)", ylim=(0,1.0),
                       fname="theta_FR_within.png"))
# Directionality: FR→PO should increase in MOTOR (theta); PO→FR increases in EO (alpha)
plots.append(plot_bars(df, "theta", "PSI_FR_to_PO", ("EC","EO","MOTOR"),
                       title="Theta: FR→PO PSI (EC, EO, MOTOR)", ylim=(-0.2,0.2),
                       fname="theta_PSI_FR_to_PO.png"))
plots.append(plot_bars(df, "alpha", "PSI_PO_to_FR", ("EC","EO","MOTOR"),
                       title="Alpha: PO→FR PSI (EC, EO, MOTOR)", ylim=(-0.2,0.2),
                       fname="alpha_PSI_PO_to_FR.png"))

print("Saved plots:\n - " + "\n - ".join(os.path.basename(p) for p in plots))

print("\nINTERPRETATION GUIDE")
print("1) Θ (theta) FR↔FR: expect MOTOR > EC, EO (control module engages during motor).")
print("2) α (alpha) PO↔PO: expect EC > EO, MOTOR (sensory gate strong in EC; releases with input or action).")
print("3) Θ PSI FR→PO: expect increase in MOTOR vs rest (control driving sensory integration).")
print("4) α PSI PO→FR: expect increase in EO vs EC (sensory-driven flow to control when eyes open).")
print("\nUse group_tests_summary.csv to confirm: mean_diff, Cohen's d, and permutation p-values.")


Subjects with EO & EC present: n=109
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


...

NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Export summary (first 12): [(1, 'R03', (64, 15000)), (1, 'R04', (64, 15000)), (1, 'R05', (64, 15000)), (1, 'R06', (64, 15000)), (2, 'R03', (64, 15000)), (2, 'R04', (64, 15000)), (2, 'R05', (64, 15000)), (2, 'R06', (64, 15000)), (3, 'R03', (64, 15000)), (3, 'R04', (64, 15000)), (3, 'R05', (64, 15000)), (3, 'R06', (64, 15000))] ...
Modules → Frontal=36 (label 0), Posterior=28 (label 1)
Saved per-subject metrics: C:\Users\caleb\CNT_Lab\artifacts\pli_humans_100plus\tests_functional\tables\subject_metrics_allconds.csv
Saved group tests: C:\Users\caleb\CNT_Lab\artifacts\pli_humans_100plus\tests_functional\tables\group_tests_summary.csv
 band       metric     A  B  mean_diff(A-B)   cohen_d   p_perm   N
alpha        FR_FR    EC EO        0.051315  0.896996 0.000100 109
alpha        FR_FR MOTOR EC       -0.047373 -0.863185 0.000100 109
alpha        FR_FR MOTOR EO        0.003942  0.163915 0.090891 109
alpha        PO_P

In [7]:
C:\Users\caleb\CNT_Lab\artifacts\pli_humans_100plus\tests_functional\hands\ 
  ├─ tables\hemis_theta_hands_subject50.csv
  ├─ tables\hemis_theta_hands_summary.csv
  ├─ figures\theta_FR_within_hemi_bars.png
  └─ figures\theta_FR_contra_diffs.png


SyntaxError: unexpected character after line continuation character (3743645527.py, line 1)

In [8]:
# === CNT Hands (R03/R04) — Contralateral θ Frontal Effect on 50 Subjects ===
# Uses EC α consensus → frontal module → split Left vs Right frontal; computes θ within-frontal coupling (PLI)
# Tests (paired, 10k perms):
#   • R03 (Left hand): Right-frontal > Left-frontal
#   • R04 (Right hand): Left-frontal > Right-frontal
#   • (Contra−Ipsi)_motor > (Contra−Ipsi)_EC baseline
# Saves per-subject table, summary stats, and plots.

import os, re, glob, json, numpy as np, pandas as pd, matplotlib.pyplot as plt
from numpy.random import default_rng
from scipy.signal import butter, filtfilt, hilbert
from sklearn.utils import resample

# ---------------- Paths ----------------
ROOT        = r"C:\Users\caleb\CNT_Lab"
DATA_DIR    = os.path.join(ROOT, "eeg_rest")
ART_ROOT    = os.path.join(ROOT, r"artifacts\pli_humans_100plus")
CONS_EC_ALPHA = os.path.join(ART_ROOT, "tables", "band__EC__alpha__consensus_labels.npy")
CH_TXT      = os.path.join(ROOT, "eeg_rest", "subject_01_EC.channels.txt")

OUT_ROOT    = os.path.join(ART_ROOT, r"tests_functional\hands")
OUT_TAB     = os.path.join(OUT_ROOT, "tables")
OUT_FIG     = os.path.join(OUT_ROOT, "figures")
for p in [OUT_ROOT, OUT_TAB, OUT_FIG]: os.makedirs(p, exist_ok=True)

assert os.path.exists(CONS_EC_ALPHA), "Missing EC α consensus. Run CNT consensus first."
assert os.path.exists(CH_TXT), "Missing channel names file."

# ---------------- Utility ----------------
def clean_label(x: str) -> str:
    y = x.strip()
    y = re.sub(r"(?i)^(EEG|MEG|EOG|ECG|EMG)[\s_\-]+", "", y)
    y = re.sub(r"(?i)[\s_\-]*(REF|AV|AVERAGE|AVG|M1|M2)$", "", y)
    y = re.sub(r"[ \-\.]+", "", y)
    y = y.replace("FP","Fp")
    return y

with open(CH_TXT,"r",encoding="utf-8") as f:
    ch_names = [clean_label(ln.strip()) for ln in f if ln.strip()]

# 10–20 hemisphere mapping (odd/even, canonical sets)
LEFT_CANON  = set(map(str.upper, ["Fp1","AF7","AF3","F7","F5","F3","F1","FT7","FC5","FC3","FC1","T7","C5","C3","C1",
                                  "TP7","CP5","CP3","CP1","P7","P5","P3","P1","PO7","PO3","O1"]))
RIGHT_CANON = set(map(str.upper, ["Fp2","AF8","AF4","F8","F6","F4","F2","FT8","FC6","FC4","FC2","T8","C6","C4","C2",
                                  "TP8","CP6","CP4","CP2","P8","P6","P4","P2","PO8","PO4","O2"]))
MID_CANON   = set(map(str.upper, ["Fpz","AFz","Fz","FCz","Cz","CPz","Pz","POz","Oz"]))

def hemi_indices(names):
    L,R,Z = [],[],[]
    for i,ch in enumerate(names):
        up = ch.upper()
        if up in LEFT_CANON:  L.append(i); continue
        if up in RIGHT_CANON: R.append(i); continue
        if up in MID_CANON or re.search(r"[A-Za-z]z$", ch): Z.append(i); continue
        m = re.search(r"(\d+)$", ch)
        if m:
            try:
                d=int(m.group(1)); (L if d%2==1 else R).append(i); continue
            except: pass
        Z.append(i)
    return np.array(L,int), np.array(R,int), np.array(Z,int)

L_idx, R_idx, Z_idx = hemi_indices(ch_names)

# From EC α consensus, define FRONTAL module and split L/R within it
cons_alpha = np.load(CONS_EC_ALPHA)
ANT_PREFIXES=("Fp","AF","F","FC")

# Indices that are anterior by prefix
def anterior_mask(names):
    A=[]
    for i,ch in enumerate(names):
        pref = re.match(r"[A-Za-z]+", ch)
        pref = pref.group(0) if pref else ""
        if any(pref.startswith(px) for px in ANT_PREFIXES): A.append(i)
    return np.array(A,int)

A_all = anterior_mask(ch_names)
mod0_A = np.intersect1d(np.where(cons_alpha==0)[0], A_all).size
mod1_A = np.intersect1d(np.where(cons_alpha==1)[0], A_all).size
FR_LABEL, PO_LABEL = (0,1) if mod0_A >= mod1_A else (1,0)
FR_all = np.where(cons_alpha==FR_LABEL)[0]

# L- and R- frontal subsets
FR_L = np.intersect1d(FR_all, L_idx)
FR_R = np.intersect1d(FR_all, R_idx)

print(f"Frontal module split: Left={len(FR_L)} ch, Right={len(FR_R)} ch")

# ---------------- Subject list: 50 with EC, EO, R03, R04 ----------------
# (We assume you've already exported EO/EC + R03, R04 with the previous cell)
candidates=[]
for f in glob.glob(os.path.join(DATA_DIR, "subject_*_EC.npy")):
    sid = int(re.search(r"subject_(\d+)_EC\.npy$", f).group(1))
    if (os.path.exists(os.path.join(DATA_DIR, f"subject_{sid:02d}_EO.npy")) and
        os.path.exists(os.path.join(DATA_DIR, f"subject_{sid:02d}_R03.npy")) and
        os.path.exists(os.path.join(DATA_DIR, f"subject_{sid:02d}_R04.npy"))):
        candidates.append(sid)

candidates = sorted(candidates)[:50]
print(f"Using {len(candidates)} subjects:", candidates[:10], "...")

# ---------------- θ within-frontal (L vs R) PLI ----------------
FS=250.0
def bandpass(x, fs, lo, hi, order=4):
    b,a = butter(order,[lo/(fs/2),hi/(fs/2)], btype="band")
    return filtfilt(b,a,x)

def pli_pair(x,y,lo,hi,fs):
    bx = bandpass(x,fs,lo,hi); by = bandpass(y,fs,lo,hi)
    phx = np.angle(hilbert(bx)); phy = np.angle(hilbert(by))
    d   = phx - phy
    return float(np.abs(np.mean(np.sign(np.sin(d)))))

def pli_mean_within(X, idx, lo, hi, fs):
    vals=[]
    ids=list(idx)
    for i in range(len(ids)):
        for j in range(i+1,len(ids)):
            vals.append(pli_pair(X[ids[i]], X[ids[j]], lo, hi, fs))
    return float(np.mean(vals)) if vals else np.nan

theta = (4,8)

def get_theta_frontal_hemi(X):
    L = pli_mean_within(X, FR_L, theta[0], theta[1], FS)
    R = pli_mean_within(X, FR_R, theta[0], theta[1], FS)
    return L, R

# compute per-subject
rows=[]
for sid in candidates:
    rec = {"subject": sid}
    for cond in ["EC","EO","R03","R04"]:
        fpath = os.path.join(DATA_DIR, f"subject_{sid:02d}_{cond}.npy")
        X = np.load(fpath)
        L, R = get_theta_frontal_hemi(X)
        rec[f"theta_FR_L_{cond}"] = L
        rec[f"theta_FR_R_{cond}"] = R
    # contralateral differences: R03 expects Right > Left; R04 expects Left > Right
    rec["theta_diff_R03"] = rec["theta_FR_R_R03"] - rec["theta_FR_L_R03"]   # (Right - Left), expect > 0
    rec["theta_diff_R04"] = rec["theta_FR_L_R04"] - rec["theta_FR_R_R04"]   # (Left - Right), expect > 0
    # EC baselines for differences
    rec["theta_diff_EC"]  = rec["theta_FR_R_EC"]  - rec["theta_FR_L_EC"]
    rows.append(rec)

df = pd.DataFrame(rows)
out_csv = os.path.join(OUT_TAB, "hemis_theta_hands_subject50.csv")
df.to_csv(out_csv, index=False)
print("Saved per-subject hemis metrics:", out_csv)

# ---------------- Paired permutation tests (10k flips) ----------------
RNG = default_rng(31)

def paired_perm_greater(a, b0=0.0, n_perm=10000, rng=None):
    """Test mean(a) > b0 via paired sign-flip."""
    a = np.asarray(a, float)
    a = a[np.isfinite(a)]
    if len(a) < 10: 
        return np.nan, np.nan, np.nan
    obs = float(np.mean(a - b0))
    cnt = 1
    for _ in range(n_perm):
        signs = rng.integers(0,2,size=len(a))*2-1
        perm = np.mean((a-b0)*signs)
        if perm >= obs: cnt += 1
    p = float(cnt / (n_perm+1))
    d = float(obs / (np.std(a - b0, ddof=1)+1e-12))
    return obs, d, p

# Contralateral during motor:
obs_r03, d_r03, p_r03 = paired_perm_greater(df["theta_diff_R03"].values, b0=0.0, n_perm=10000, rng=RNG)
obs_r04, d_r04, p_r04 = paired_perm_greater(df["theta_diff_R04"].values, b0=0.0, n_perm=10000, rng=RNG)

# Improvement over EC baseline (contra minus EC diff):
obs_r03_vsEC, d_r03_vsEC, p_r03_vsEC = paired_perm_greater((df["theta_diff_R03"] - df["theta_diff_EC"]).values,
                                                            b0=0.0, n_perm=10000, rng=RNG)
obs_r04_vsEC, d_r04_vsEC, p_r04_vsEC = paired_perm_greater((df["theta_diff_R04"] - df["theta_diff_EC"]).values,
                                                            b0=0.0, n_perm=10000, rng=RNG)

summary_rows = [
    {"contrast":"R03 (Right-Left) > 0", "mean":obs_r03, "cohen_d":d_r03, "p_perm":p_r03, "N":len(df)},
    {"contrast":"R04 (Left-Right) > 0", "mean":obs_r04, "cohen_d":d_r04, "p_perm":p_r04, "N":len(df)},
    {"contrast":"R03 contra − ECdiff > 0", "mean":obs_r03_vsEC, "cohen_d":d_r03_vsEC, "p_perm":p_r03_vsEC, "N":len(df)},
    {"contrast":"R04 contra − ECdiff > 0", "mean":obs_r04_vsEC, "cohen_d":d_r04_vsEC, "p_perm":p_r04_vsEC, "N":len(df)},
]
sum_df = pd.DataFrame(summary_rows)
sum_csv = os.path.join(OUT_TAB, "hemis_theta_hands_summary.csv")
sum_df.to_csv(sum_csv, index=False)
print("Summary tests saved:", sum_csv)
print(sum_df.to_string(index=False))

# ---------------- Plots ----------------
def boot_mean_ci(a, B=2000):
    a = np.asarray(a, float)
    a = a[np.isfinite(a)]
    if len(a)<5: return np.nan, np.nan, np.nan
    means=[np.mean(resample(a, replace=True, n_samples=len(a))) for _ in range(B)]
    return float(np.mean(a)), float(np.percentile(means,2.5)), float(np.percentile(means,97.5))

# 1) Left/Right frontal θ across EC, EO, R03, R04 (bars with CI)
conds = ["EC","EO","R03","R04"]
L_means=[]; L_lo=[]; L_hi=[]
R_means=[]; R_lo=[]; R_hi=[]
for c in conds:
    m,lo,hi = boot_mean_ci(df[f"theta_FR_L_{c}"].values)
    L_means.append(m); L_lo.append(lo); L_hi.append(hi)
    m,lo,hi = boot_mean_ci(df[f"theta_FR_R_{c}"].values)
    R_means.append(m); R_lo.append(lo); R_hi.append(hi)

x = np.arange(len(conds))
w = 0.35
plt.figure(figsize=(7.6,4.2))
plt.bar(x - w/2, L_means, width=w,
        yerr=[np.array(L_means)-np.array(L_lo), np.array(L_hi)-np.array(L_means)],
        capsize=4, label="Left frontal")
plt.bar(x + w/2, R_means, width=w,
        yerr=[np.array(R_means)-np.array(R_lo), np.array(R_hi)-np.array(R_means)],
        capsize=4, label="Right frontal")
plt.xticks(x, conds); plt.ylim(0, 1.0); plt.ylabel("θ within-frontal PLI")
plt.title("θ within-frontal (Left vs Right) — EC/EO/R03/R04 (N=50)")
plt.legend()
out1 = os.path.join(OUT_FIG, "theta_FR_within_hemi_bars.png")
plt.tight_layout(); plt.savefig(out1, dpi=160); plt.close()

# 2) Contralateral differences bars (R03: R-L; R04: L-R) with p-values in title
m1,lo1,hi1 = boot_mean_ci(df["theta_diff_R03"].values)
m2,lo2,hi2 = boot_mean_ci(df["theta_diff_R04"].values)
plt.figure(figsize=(5.8,4.0))
means=[m1,m2]; los=[lo1,lo2]; his=[hi1,hi2]
labels=[f"R03 (R−L)\n p={p_r03:.4f}", f"R04 (L−R)\n p={p_r04:.4f}"]
xx=np.arange(2)
plt.bar(xx, means, yerr=[np.array(means)-np.array(los), np.array(his)-np.array(means)], capsize=4)
plt.xticks(xx, labels)
plt.ylabel("θ within-frontal PLI (contrast)")
plt.title("Contralateral effect (N=50)")
out2 = os.path.join(OUT_FIG, "theta_FR_contra_diffs.png")
plt.tight_layout(); plt.savefig(out2, dpi=160); plt.close()

print("Saved figures:")
print(" -", out1)
print(" -", out2)

print("\nINTERPRETATION:")
print("• R03 (Left-hand imagery/execution): expect Right frontal θ > Left (R−L > 0).")
print("• R04 (Right-hand imagery/execution): expect Left frontal θ > Right (L−R > 0).")
print("• The 'contra − ECdiff > 0' contrasts confirm that the contralateral boost exceeds any baseline hemisphere bias at rest.")


Frontal module split: Left=16 ch, Right=15 ch
Using 50 subjects: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10] ...
Saved per-subject hemis metrics: C:\Users\caleb\CNT_Lab\artifacts\pli_humans_100plus\tests_functional\hands\tables\hemis_theta_hands_subject50.csv
Summary tests saved: C:\Users\caleb\CNT_Lab\artifacts\pli_humans_100plus\tests_functional\hands\tables\hemis_theta_hands_summary.csv
               contrast     mean  cohen_d   p_perm  N
   R03 (Right-Left) > 0 0.000837 0.032635 0.415358 50
   R04 (Left-Right) > 0 0.005396 0.220001 0.064394 50
R03 contra − ECdiff > 0 0.004017 0.159644 0.134587 50
R04 contra − ECdiff > 0 0.008576 0.193361 0.089191 50
Saved figures:
 - C:\Users\caleb\CNT_Lab\artifacts\pli_humans_100plus\tests_functional\hands\figures\theta_FR_within_hemi_bars.png
 - C:\Users\caleb\CNT_Lab\artifacts\pli_humans_100plus\tests_functional\hands\figures\theta_FR_contra_diffs.png

INTERPRETATION:
• R03 (Left-hand imagery/execution): expect Right frontal θ > Left (R−L > 0).
• R04 (Righ

In [9]:
# === CNT Motor Proofs (N=50) ===
# 1) CNT control claim: θ frontal within-module (no lateral split) — MOTOR > EC
# 2) True laterality: μ(8–13)/β(13–30) sensorimotor coupling — contralateral > ipsilateral for R03/R04
#    Masks: Left-central {FC3,C3,C1,CP3}, Right-central {FC4,C4,C2,CP4}
# Reuses your EC α consensus for frontal set (control test), and 10–20 names for central masks.

import os, re, glob, numpy as np, pandas as pd, matplotlib.pyplot as plt
from numpy.random import default_rng
from scipy.signal import butter, filtfilt, hilbert
from sklearn.utils import resample

ROOT      = r"C:\Users\caleb\CNT_Lab"
DATA_DIR  = os.path.join(ROOT, "eeg_rest")
ART_ROOT  = os.path.join(ROOT, r"artifacts\pli_humans_100plus")
CONS_EC_ALPHA = os.path.join(ART_ROOT, "tables", "band__EC__alpha__consensus_labels.npy")
CH_TXT    = os.path.join(ROOT, "eeg_rest", "subject_01_EC.channels.txt")

OUT_DIR   = os.path.join(ART_ROOT, r"tests_functional\motor_proofs")
TAB_DIR   = os.path.join(OUT_DIR, "tables"); os.makedirs(TAB_DIR, exist_ok=True)
FIG_DIR   = os.path.join(OUT_DIR, "figures"); os.makedirs(FIG_DIR, exist_ok=True)

assert os.path.exists(CONS_EC_ALPHA)
assert os.path.exists(CH_TXT)

FS = 250.0
rng = default_rng(2027)

def clean_label(s):
    s = re.sub(r"(?i)^(EEG|EOG|ECG|EMG|MEG)[\s_\-]+","",s.strip())
    s = re.sub(r"(?i)[\s_\-]*(REF|AV|AVERAGE|AVG|M1|M2)$","",s)
    return re.sub(r"[ \-\.]+","",s).replace("FP","Fp")

with open(CH_TXT,"r",encoding="utf-8") as f:
    ch_names = [clean_label(x) for x in f if x.strip()]

# --- 10-20 masks
def idx_of(names, wanted):
    up = {n.upper():i for i,n in enumerate(names)}
    idx = [up[w.upper()] for w in wanted if w.upper() in up]
    return np.array(idx, int)

# Sensorimotor masks (robust fallbacks: if C1/C2 not present, drop them)
LC_names = ["FC3","C3","C1","CP3"]
RC_names = ["FC4","C4","C2","CP4"]
LC = idx_of(ch_names, LC_names)
RC = idx_of(ch_names, RC_names)

# Frontal module from EC α consensus (for CNT control test)
cons_alpha = np.load(CONS_EC_ALPHA)
ANT_PREFIXES=("Fp","AF","F","FC")
A_mask = []
for i,ch in enumerate(ch_names):
    pref = re.match(r"[A-Za-z]+", ch)
    pref = pref.group(0) if pref else ""
    if any(pref.startswith(px) for px in ANT_PREFIXES): A_mask.append(i)
A_mask = np.array(A_mask,int)
# choose which label is FRONTAL by overlap with anterior mask
m0A = np.intersect1d(np.where(cons_alpha==0)[0], A_mask).size
m1A = np.intersect1d(np.where(cons_alpha==1)[0], A_mask).size
FR_LABEL = 0 if m0A>=m1A else 1
FR = np.where(cons_alpha==FR_LABEL)[0]

print(f"Masks → FRontal={len(FR)}; L-central={LC.tolist()} ; R-central={RC.tolist()}")

# --- helpers
def bandpass(x, fs, lo, hi, order=4):
    b,a = butter(order, [lo/(fs/2), hi/(fs/2)], btype="band"); return filtfilt(b,a,x)

def pli_pair(x, y, lo, hi, fs):
    bx = bandpass(x,fs,lo,hi); by = bandpass(y,fs,lo,hi)
    phx = np.angle(hilbert(bx)); phy = np.angle(hilbert(by))
    return float(np.abs(np.mean(np.sign(np.sin(phx-phy)))))

def pli_within(X, idx, lo, hi, fs):
    vals=[]; ids=list(idx)
    for i in range(len(ids)):
        for j in range(i+1,len(ids)):
            vals.append(pli_pair(X[ids[i]], X[ids[j]], lo, hi, fs))
    return float(np.mean(vals)) if vals else np.nan

# --- pick 50 subjects with the required conditions
candidates=[]
for f in glob.glob(os.path.join(DATA_DIR,"subject_*_EC.npy")):
    sid=int(re.search(r"subject_(\d+)_EC\.npy$", f).group(1))
    ok = all(os.path.exists(os.path.join(DATA_DIR, f"subject_{sid:02d}_{tag}.npy")) for tag in ["EO","R03","R04"])
    if ok: candidates.append(sid)
candidates = sorted(candidates)[:50]
print("Using N subjects:", len(candidates))

# --- compute metrics
rows=[]
for sid in candidates:
    X_EC  = np.load(os.path.join(DATA_DIR, f"subject_{sid:02d}_EC.npy"))
    X_EO  = np.load(os.path.join(DATA_DIR, f"subject_{sid:02d}_EO.npy"))
    X_R03 = np.load(os.path.join(DATA_DIR, f"subject_{sid:02d}_R03.npy"))
    X_R04 = np.load(os.path.join(DATA_DIR, f"subject_{sid:02d}_R04.npy"))

    # CNT control: θ frontal (no laterality split)
    fr_theta_EC  = pli_within(X_EC,  FR, 4,8,FS)
    fr_theta_EO  = pli_within(X_EO,  FR, 4,8,FS)
    fr_theta_R03 = pli_within(X_R03, FR, 4,8,FS)
    fr_theta_R04 = pli_within(X_R04, FR, 4,8,FS)

    # Laterality: μ/β within L-central and R-central
    mu_EC_L  = pli_within(X_EC,  LC, 8,13,FS)
    mu_EC_R  = pli_within(X_EC,  RC, 8,13,FS)
    mu_R03_L = pli_within(X_R03, LC, 8,13,FS)
    mu_R03_R = pli_within(X_R03, RC, 8,13,FS)
    mu_R04_L = pli_within(X_R04, LC, 8,13,FS)
    mu_R04_R = pli_within(X_R04, RC, 8,13,FS)

    beta_EC_L  = pli_within(X_EC,  LC, 13,30,FS)
    beta_EC_R  = pli_within(X_EC,  RC, 13,30,FS)
    beta_R03_L = pli_within(X_R03, LC, 13,30,FS)
    beta_R03_R = pli_within(X_R03, RC, 13,30,FS)
    beta_R04_L = pli_within(X_R04, LC, 13,30,FS)
    beta_R04_R = pli_within(X_R04, RC, 13,30,FS)

    rows.append({
        "subject":sid,
        # CNT control metric
        "theta_FR_EC":fr_theta_EC, "theta_FR_EO":fr_theta_EO, "theta_FR_R03":fr_theta_R03, "theta_FR_R04":fr_theta_R04,
        # μ laterality
        "mu_R03_contra":mu_R03_R - mu_R03_L,   # Left hand: Right central − Left central > 0
        "mu_R04_contra":mu_R04_L - mu_R04_R,   # Right hand: Left central − Right central > 0
        "mu_EC_diff":mu_EC_R - mu_EC_L,
        # β laterality
        "beta_R03_contra":beta_R03_R - beta_R03_L,
        "beta_R04_contra":beta_R04_L - beta_R04_R,
        "beta_EC_diff":beta_EC_R - beta_EC_L
    })

df = pd.DataFrame(rows)
out_csv = os.path.join(TAB_DIR, "motor_proofs_subject50.csv")
df.to_csv(out_csv, index=False)
print("Saved:", out_csv)

# --- paired permutation helpers
def paired_perm(a, b, n_perm=10000, two_sided=True):
    a=np.asarray(a,float); b=np.asarray(b,float)
    mask=np.isfinite(a)&np.isfinite(b); a=a[mask]; b=b[mask]
    diff=a-b; obs=float(np.mean(diff))
    rng = default_rng(33); cnt=1
    for _ in range(n_perm):
        signs = rng.integers(0,2,size=len(diff))*2-1
        perm  = np.mean(diff*signs)
        if (abs(perm) >= abs(obs)) if two_sided else (perm >= obs): cnt+=1
    p=float(cnt/(n_perm+1))
    d=float(obs/(np.std(diff,ddof=1)+1e-12))
    return obs,d,p,len(diff)

def one_sample_greater(x, n_perm=10000):
    x=np.asarray(x,float); x=x[np.isfinite(x)]
    rng=default_rng(34); obs=float(np.mean(x)); cnt=1
    for _ in range(n_perm):
        signs=rng.integers(0,2,size=len(x))*2-1
        perm=np.mean(x*signs)
        if perm>=obs: cnt+=1
    p=float(cnt/(n_perm+1)); d=float(obs/(np.std(x,ddof=1)+1e-12))
    return obs,d,p,len(x)

# --- TEST 1: CNT control — theta FR within: MOTOR > EC (pooled R03/R04)
theta_FR_motor = np.nanmean(df[["theta_FR_R03","theta_FR_R04"]].values, axis=1)
obs1,d1,p1,n1 = paired_perm(theta_FR_motor, df["theta_FR_EC"].values, n_perm=10000, two_sided=True)

# --- TEST 2: laterality — μ contralateral > 0; β contralateral > 0
obs_mu_L, d_mu_L, p_mu_L, n_mu_L = one_sample_greater(df["mu_R03_contra"].values, n_perm=10000)  # R03: Right−Left > 0
obs_mu_R, d_mu_R, p_mu_R, n_mu_R = one_sample_greater(df["mu_R04_contra"].values, n_perm=10000)  # R04: Left−Right > 0
obs_be_L, d_be_L, p_be_L, n_be_L = one_sample_greater(df["beta_R03_contra"].values, n_perm=10000)
obs_be_R, d_be_R, p_be_R, n_be_R = one_sample_greater(df["beta_R04_contra"].values, n_perm=10000)

# Baseline-corrected: (contra − ECdiff) > 0
obs_mu_Lc, d_mu_Lc, p_mu_Lc, _ = one_sample_greater((df["mu_R03_contra"] - df["mu_EC_diff"]).values, n_perm=10000)
obs_mu_Rc, d_mu_Rc, p_mu_Rc, _ = one_sample_greater((df["mu_R04_contra"] - df["mu_EC_diff"]).values, n_perm=10000)
obs_be_Lc, d_be_Lc, p_be_Lc, _ = one_sample_greater((df["beta_R03_contra"] - df["beta_EC_diff"]).values, n_perm=10000)
obs_be_Rc, d_be_Rc, p_be_Rc, _ = one_sample_greater((df["beta_R04_contra"] - df["beta_EC_diff"]).values, n_perm=10000)

sum_rows = [
    {"test":"θ FR within — MOTOR vs EC", "mean_diff":obs1, "cohen_d":d1, "p_perm":p1, "N":n1},
    {"test":"μ contra (R03: R-L) > 0",   "mean":obs_mu_L,  "cohen_d":d_mu_L, "p_perm":p_mu_L, "N":n_mu_L},
    {"test":"μ contra (R04: L-R) > 0",   "mean":obs_mu_R,  "cohen_d":d_mu_R, "p_perm":p_mu_R, "N":n_mu_R},
    {"test":"β contra (R03: R-L) > 0",   "mean":obs_be_L,  "cohen_d":d_be_L, "p_perm":p_be_L, "N":n_be_L},
    {"test":"β contra (R04: L-R) > 0",   "mean":obs_be_R,  "cohen_d":d_be_R, "p_perm":p_be_R, "N":n_be_R},
    {"test":"μ contra−ECdiff > 0 (R03)", "mean":obs_mu_Lc, "cohen_d":d_mu_Lc,"p_perm":p_mu_Lc, "N":n_mu_L},
    {"test":"μ contra−ECdiff > 0 (R04)", "mean":obs_mu_Rc, "cohen_d":d_mu_Rc,"p_perm":p_mu_Rc, "N":n_mu_R},
    {"test":"β contra−ECdiff > 0 (R03)", "mean":obs_be_Lc, "cohen_d":d_be_Lc,"p_perm":p_be_Lc, "N":n_be_L},
    {"test":"β contra−ECdiff > 0 (R04)", "mean":obs_be_Rc, "cohen_d":d_be_Rc,"p_perm":p_be_Rc, "N":n_be_R},
]
sum_df = pd.DataFrame(sum_rows)
sum_df.to_csv(os.path.join(TAB_DIR,"motor_proofs_summary.csv"), index=False)
print(sum_df.to_string(index=False))

# --- quick plots ---
def bar_with_ci(vals, title, fname, ylabel="PLI"):
    a = np.asarray(vals, float); a = a[np.isfinite(a)]
    mean = float(np.mean(a))
    boots = [np.mean(resample(a, replace=True, n_samples=len(a))) for _ in range(2000)]
    lo,hi = np.percentile(boots,[2.5,97.5])
    plt.figure(figsize=(4.5,3.8))
    plt.bar([0],[mean], yerr=[[mean-lo],[hi-mean]], capsize=4)
    plt.xticks([0],[title]); plt.ylabel(ylabel)
    outp=os.path.join(FIG_DIR,fname); plt.tight_layout(); plt.savefig(outp,dpi=160); plt.close()
    return outp

# θ FR within motor vs EC
bar_with_ci(theta_FR_motor - df["theta_FR_EC"].values, "θ FR (MOTOR−EC)", "theta_FR_motor_minus_EC.png", ylabel="Δ PLI")

# μ/β contralateral contrasts
bar_with_ci(df["mu_R03_contra"].values,   "μ R03 (R−L)", "mu_R03_contra.png")
bar_with_ci(df["mu_R04_contra"].values,   "μ R04 (L−R)", "mu_R04_contra.png")
bar_with_ci(df["beta_R03_contra"].values, "β R03 (R−L)", "beta_R03_contra.png")
bar_with_ci(df["beta_R04_contra"].values, "β R04 (L−R)", "beta_R04_contra.png")

print("Figures written to:", FIG_DIR)


Masks → FRontal=36; L-central=[1, 8, 9, 15] ; R-central=[5, 12, 11, 19]
Using N subjects: 50
Saved: C:\Users\caleb\CNT_Lab\artifacts\pli_humans_100plus\tests_functional\motor_proofs\tables\motor_proofs_subject50.csv
                     test  mean_diff   cohen_d   p_perm  N      mean
θ FR within — MOTOR vs EC   0.008584  0.329834 0.019798 50       NaN
  μ contra (R03: R-L) > 0        NaN -0.179716 0.896610 50 -0.009171
  μ contra (R04: L-R) > 0        NaN  0.238069 0.047995 50  0.012640
  β contra (R03: R-L) > 0        NaN -0.313123 0.986001 50 -0.015859
  β contra (R04: L-R) > 0        NaN  0.208142 0.072593 50  0.013064
μ contra−ECdiff > 0 (R03)        NaN -0.315139 0.985601 50 -0.024289
μ contra−ECdiff > 0 (R04)        NaN -0.023741 0.559044 50 -0.002479
β contra−ECdiff > 0 (R03)        NaN  0.045964 0.373363 50  0.001741
β contra−ECdiff > 0 (R04)        NaN  0.262708 0.034897 50  0.030664
Figures written to: C:\Users\caleb\CNT_Lab\artifacts\pli_humans_100plus\tests_functional\motor

In [10]:
# === Cue-locked motor laterality (μ/β) + CNT control (θ) on 50 subjects ===
# Reads EEGBCI EDFs for R03 (left) / R04 (right), extracts 2 s motor windows 1–3 s after cue (T1/T2),
# and 2 s rest windows; computes:
#   • μ/β contralateral indices over compact sensorimotor masks (C3/C4-centered)
#   • θ frontal within (CNT control) motor vs rest
# Paired label-flip permutation tests (10k) + Cohen's d, with figures & CSV outputs.

import os, re, glob, numpy as np, pandas as pd, matplotlib.pyplot as plt
from numpy.random import default_rng
from scipy.signal import butter, filtfilt, hilbert
from sklearn.utils import resample

ROOT      = r"C:\Users\caleb\CNT_Lab"
DATA_DIR  = os.path.join(ROOT, "eeg_rest")
ART_ROOT  = os.path.join(ROOT, r"artifacts\pli_humans_100plus")
EDF_ROOT  = os.path.join(os.path.expanduser("~"), "mne_data", "MNE-eegbci-data", "files", "eegmmidb", "1.0.0")

OUT_DIR   = os.path.join(ART_ROOT, r"tests_functional\motor_cuelocked")
TAB_DIR   = os.path.join(OUT_DIR, "tables"); os.makedirs(TAB_DIR, exist_ok=True)
FIG_DIR   = os.path.join(OUT_DIR, "figures"); os.makedirs(FIG_DIR, exist_ok=True)

CH_TXT    = os.path.join(ROOT, "eeg_rest", "subject_01_EC.channels.txt")
CONS_EC_ALPHA = os.path.join(ART_ROOT, "tables", "band__EC__alpha__consensus_labels.npy")
assert os.path.exists(CH_TXT) and os.path.exists(CONS_EC_ALPHA)

# MNE (for events/annotations)
try:
    import mne
except Exception:
    import sys, subprocess
    subprocess.check_call([sys.executable,"-m","pip","install","-q","mne","pooch"])
    import mne

FS = 250.0
rng = default_rng(2031)

def clean_label(s):
    s = re.sub(r"(?i)^(EEG|EOG|ECG|EMG|MEG)[\s_\-]+","",s.strip())
    s = re.sub(r"(?i)[\s_\-]*(REF|AV|AVERAGE|AVG|M1|M2)$","",s)
    return re.sub(r"[ \-\.]+","",s).replace("FP","Fp")

with open(CH_TXT,"r",encoding="utf-8") as f:
    ch_names = [clean_label(x) for x in f if x.strip()]

# Compact sensorimotor masks centered on C3/C4
def idx_of(names, wanted):
    up = {n.upper():i for i,n in enumerate(names)}
    return np.array([up[w.upper()] for w in wanted if w.upper() in up], int)

LC = idx_of(ch_names, ["FC3","C3","CP3"])
RC = idx_of(ch_names, ["FC4","C4","CP4"])
if len(LC)<2 or len(RC)<2:
    print("[warn] Few central channels found; using broader masks with C1/C2, CP1/CP2 if available.")
    LC = np.unique(np.concatenate([LC, idx_of(ch_names, ["C1","CP1"])]))
    RC = np.unique(np.concatenate([RC, idx_of(ch_names, ["C2","CP2"])]))

# Frontal module from EC α consensus (for θ control)
cons_alpha = np.load(CONS_EC_ALPHA)
ANT_PREFIXES=("Fp","AF","F","FC")
A_mask=[]
for i,ch in enumerate(ch_names):
    pref = re.match(r"[A-Za-z]+", ch)
    pref = pref.group(0) if pref else ""
    if any(pref.startswith(px) for px in ANT_PREFIXES): A_mask.append(i)
A_mask = np.array(A_mask,int)
m0A = np.intersect1d(np.where(cons_alpha==0)[0], A_mask).size
m1A = np.intersect1d(np.where(cons_alpha==1)[0], A_mask).size
FR_LABEL = 0 if m0A>=m1A else 1
FR = np.where(cons_alpha==FR_LABEL)[0]

print(f"Masks → FR={len(FR)} ; LC={LC.tolist()} ; RC={RC.tolist()}")

# Select N=50 with EC, EO, and EDF runs present
candidates=[]
for f in glob.glob(os.path.join(DATA_DIR,"subject_*_EC.npy")):
    sid=int(re.search(r"subject_(\d+)_EC\.npy$", f).group(1))
    # EDF files exist?
    edf_files = [os.path.join(EDF_ROOT, f"S{sid:03d}", f"S{sid:03d}R{run:02d}.edf") for run in (3,4)]
    if all(os.path.exists(p) for p in edf_files):
        candidates.append(sid)
candidates = sorted(candidates)[:50]
print("Using N=", len(candidates))

# Band helpers
def bandpass(x, fs, lo, hi, order=4):
    b,a = butter(order, [lo/(fs/2), hi/(fs/2)], btype="band"); return filtfilt(b,a,x)

def pli_within(X, idx, lo, hi, fs):
    vals=[]; ids=list(idx)
    for i in range(len(ids)):
        for j in range(i+1,len(ids)):
            bx = bandpass(X[ids[i]],fs,lo,hi); by = bandpass(X[ids[j]],fs,lo,hi)
            phx = np.angle(hilbert(bx)); phy = np.angle(hilbert(by))
            vals.append(float(np.abs(np.mean(np.sign(np.sin(phx-phy))))))
    return float(np.mean(vals)) if vals else np.nan

def mean_band_envelope(X, idx, lo, hi, fs):
    if len(idx)==0: return np.nan
    envs=[]
    for i in idx:
        b = bandpass(X[i], fs, lo, hi)
        envs.append(np.abs(hilbert(b)))
    return float(np.mean([np.mean(e) for e in envs]))

# Extract cue-locked epochs from EDF
def extract_epochs_from_edf(sid, run, t0=1.0, dur=2.0):
    edf = os.path.join(EDF_ROOT, f"S{sid:03d}", f"S{sid:03d}R{run:02d}.edf")
    raw = mne.io.read_raw_edf(edf, preload=True, verbose="ERROR")
    raw.pick_types(eeg=True, stim=False, eog=False, ecg=False)
    raw.resample(FS, npad="auto", verbose="ERROR")
    # annotations: T0 rest, T1 left, T2 right (EEGBCI convention)
    events, event_id = mne.events_from_annotations(raw, verbose="ERROR")
    # map to codes
    code_map = {}
    for k,v in event_id.items():
        if k.upper()=="T0": code_map[v] = "REST"
        elif k.upper()=="T1": code_map[v] = "LEFT"
        elif k.upper()=="T2": code_map[v] = "RIGHT"
    X = raw.get_data(picks="eeg")  # [n_ch, T]
    fs = raw.info["sfreq"]
    w = int(dur*fs); shift = int(t0*fs)
    epochs = {"LEFT":[], "RIGHT":[], "REST":[]}
    for (samp,_,code) in events:
        lab = code_map.get(code, None)
        if lab is None: continue
        start = samp + shift
        stop  = start + w
        if stop <= X.shape[1]:
            epochs[lab].append(X[:, start:stop])
    return epochs  # dict->list of arrays [n_ch, w]

rows=[]
for sid in candidates:
    # R03: LEFT task; R04: RIGHT task
    ep3 = extract_epochs_from_edf(sid, 3)  # LEFT
    ep4 = extract_epochs_from_edf(sid, 4)  # RIGHT
    # Average across valid epochs
    def mean_metric_over_epochs(ep_list, fn):
        vals=[fn(ep) for ep in ep_list] if ep_list else []
        vals=[v for v in vals if np.isfinite(v)]
        return float(np.mean(vals)) if vals else np.nan

    # μ/β contralateral indices
    mu_R03_contra = mean_metric_over_epochs(ep3["LEFT"], lambda X: mean_band_envelope(X, RC, 8,13,FS) - mean_band_envelope(X, LC, 8,13,FS))
    mu_R04_contra = mean_metric_over_epochs(ep4["RIGHT"],lambda X: mean_band_envelope(X, LC, 8,13,FS) - mean_band_envelope(X, RC, 8,13,FS))
    # baseline from REST epochs in each run
    mu_R03_rest   = mean_metric_over_epochs(ep3["REST"], lambda X: mean_band_envelope(X, RC, 8,13,FS) - mean_band_envelope(X, LC, 8,13,FS))
    mu_R04_rest   = mean_metric_over_epochs(ep4["REST"], lambda X: mean_band_envelope(X, LC, 8,13,FS) - mean_band_envelope(X, RC, 8,13,FS))

    be_R03_contra = mean_metric_over_epochs(ep3["LEFT"], lambda X: mean_band_envelope(X, RC, 13,30,FS) - mean_band_envelope(X, LC, 13,30,FS))
    be_R04_contra = mean_metric_over_epochs(ep4["RIGHT"],lambda X: mean_band_envelope(X, LC, 13,30,FS) - mean_band_envelope(X, RC, 13,30,FS))
    be_R03_rest   = mean_metric_over_epochs(ep3["REST"], lambda X: mean_band_envelope(X, RC, 13,30,FS) - mean_band_envelope(X, LC, 13,30,FS))
    be_R04_rest   = mean_metric_over_epochs(ep4["REST"], lambda X: mean_band_envelope(X, LC, 13,30,FS) - mean_band_envelope(X, RC, 13,30,FS))

    # θ frontal within (CNT control) on LEFT/RIGHT motor epochs vs REST (average across runs)
    th_R03_FR = mean_metric_over_epochs(ep3["LEFT"],  lambda X: pli_within(X, FR, 4,8,FS))
    th_R04_FR = mean_metric_over_epochs(ep4["RIGHT"], lambda X: pli_within(X, FR, 4,8,FS))
    th_rest3  = mean_metric_over_epochs(ep3["REST"],  lambda X: pli_within(X, FR, 4,8,FS))
    th_rest4  = mean_metric_over_epochs(ep4["REST"],  lambda X: pli_within(X, FR, 4,8,FS))
    th_FR_motor = np.nanmean([th_R03_FR, th_R04_FR])
    th_FR_rest  = np.nanmean([th_rest3, th_rest4])

    rows.append({
        "subject":sid,
        "mu_R03_contra":mu_R03_contra, "mu_R03_rest":mu_R03_rest,
        "mu_R04_contra":mu_R04_contra, "mu_R04_rest":mu_R04_rest,
        "beta_R03_contra":be_R03_contra, "beta_R03_rest":be_R03_rest,
        "beta_R04_contra":be_R04_contra, "beta_R04_rest":be_R04_rest,
        "theta_FR_motor":th_FR_motor, "theta_FR_rest":th_FR_rest
    })

df = pd.DataFrame(rows)
csv_out = os.path.join(TAB_DIR, "motor_cuelocked_subject50.csv")
df.to_csv(csv_out, index=False)
print("Saved cue-locked table:", csv_out)

# Paired label-flip tests
def one_sample_greater(x, n_perm=10000):
    x=np.asarray(x,float); x=x[np.isfinite(x)]
    if len(x)<10: return np.nan,np.nan,np.nan,len(x)
    rng=default_rng(44); obs=float(np.mean(x)); cnt=1
    for _ in range(n_perm):
        signs=rng.integers(0,2,size=len(x))*2-1
        perm=np.mean(x*signs)
        if perm>=obs: cnt+=1
    p=float(cnt/(n_perm+1)); d=float(obs/(np.std(x,ddof=1)+1e-12))
    return obs,d,p,len(x)

def paired_perm(a,b,n_perm=10000):
    a=np.asarray(a,float); b=np.asarray(b,float)
    mask=np.isfinite(a)&np.isfinite(b); a=a[mask]; b=b[mask]
    if len(a)<10: return np.nan,np.nan,np.nan,len(a)
    diff=a-b; obs=float(np.mean(diff)); rng=default_rng(45); cnt=1
    for _ in range(n_perm):
        signs=rng.integers(0,2,size=len(diff))*2-1
        perm=np.mean(diff*signs)
        if abs(perm)>=abs(obs): cnt+=1
    p=float(cnt/(n_perm+1)); d=float(obs/(np.std(diff,ddof=1)+1e-12))
    return obs,d,p,len(diff)

tests = []
# μ contralateral > 0
obs,d,p,n = one_sample_greater(df["mu_R03_contra"].values); tests.append({"test":"μ R03 contra > 0", "mean":obs, "d":d, "p":p, "N":n})
obs,d,p,n = one_sample_greater(df["mu_R04_contra"].values); tests.append({"test":"μ R04 contra > 0", "mean":obs, "d":d, "p":p, "N":n})
# β contralateral > 0
obs,d,p,n = one_sample_greater(df["beta_R03_contra"].values); tests.append({"test":"β R03 contra > 0", "mean":obs, "d":d, "p":p, "N":n})
obs,d,p,n = one_sample_greater(df["beta_R04_contra"].values); tests.append({"test":"β R04 contra > 0", "mean":obs, "d":d, "p":p, "N":n})
# Baseline-corrected (contra - rest) > 0
obs,d,p,n = one_sample_greater((df["mu_R03_contra"]-df["mu_R03_rest"]).values); tests.append({"test":"μ R03 (contra - rest) > 0", "mean":obs, "d":d, "p":p, "N":n})
obs,d,p,n = one_sample_greater((df["mu_R04_contra"]-df["mu_R04_rest"]).values); tests.append({"test":"μ R04 (contra - rest) > 0", "mean":obs, "d":d, "p":p, "N":n})
obs,d,p,n = one_sample_greater((df["beta_R03_contra"]-df["beta_R03_rest"]).values); tests.append({"test":"β R03 (contra - rest) > 0", "mean":obs, "d":d, "p":p, "N":n})
obs,d,p,n = one_sample_greater((df["beta_R04_contra"]-df["beta_R04_rest"]).values); tests.append({"test":"β R04 (contra - rest) > 0", "mean":obs, "d":d, "p":p, "N":n})
# θ frontal (motor vs rest)
obs,d,p,n = paired_perm(df["theta_FR_motor"].values, df["theta_FR_rest"].values); tests.append({"test":"θ FR within Motor vs Rest", "meanΔ":obs, "d":d, "p":p, "N":n})

t_df = pd.DataFrame(tests)
t_df.to_csv(os.path.join(TAB_DIR, "motor_cuelocked_summary.csv"), index=False)
print(t_df.to_string(index=False))

# Simple plots for μ/β R04 (usually stronger)
def bar_ci(vals, title, fname, ylabel):
    a=np.asarray(vals,float); a=a[np.isfinite(a)]
    mean=float(np.mean(a))
    boots=[np.mean(resample(a, replace=True, n_samples=len(a))) for _ in range(2000)]
    lo,hi=np.percentile(boots,[2.5,97.5])
    plt.figure(figsize=(4.4,3.6))
    plt.bar([0],[mean], yerr=[[mean-lo],[hi-mean]], capsize=4)
    plt.xticks([0],[title]); plt.ylabel(ylabel); 
    out=os.path.join(FIG_DIR,fname); plt.tight_layout(); plt.savefig(out,dpi=160); plt.close()
    return out

bar_ci(df["mu_R04_contra"].values,   "μ R04 contralateral",  "mu_R04_contra_cuelocked.png",   "Envelope diff (contra)")
bar_ci(df["beta_R04_contra"].values, "β R04 contralateral",  "beta_R04_contra_cuelocked.png", "Envelope diff (contra)")
print("Figures in:", FIG_DIR)


Masks → FR=36 ; LC=[1, 8, 15] ; RC=[5, 12, 19]
Using N= 50
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types(

In [12]:
# === Cue-locked LI + ERD (μ/β) + CNT θ control — N=50 ===
import os, re, glob, numpy as np, pandas as pd, matplotlib.pyplot as plt
from numpy.random import default_rng
from scipy.signal import butter, filtfilt, hilbert
from sklearn.utils import resample

ROOT      = r"C:\Users\caleb\CNT_Lab"
DATA_DIR  = os.path.join(ROOT, "eeg_rest")
ART_ROOT  = os.path.join(ROOT, r"artifacts\pli_humans_100plus")
EDF_ROOT  = os.path.join(os.path.expanduser("~"), "mne_data", "MNE-eegbci-data", "files", "eegmmidb", "1.0.0")

OUT_DIR   = os.path.join(ART_ROOT, r"tests_functional\motor_cuelocked\li_erd")
TAB_DIR   = os.path.join(OUT_DIR, "tables"); os.makedirs(TAB_DIR, exist_ok=True)
FIG_DIR   = os.path.join(OUT_DIR, "figures"); os.makedirs(FIG_DIR, exist_ok=True)

CH_TXT    = os.path.join(ROOT, "eeg_rest", "subject_01_EC.channels.txt")
CONS_EC_ALPHA = os.path.join(ART_ROOT, "tables", "band__EC__alpha__consensus_labels.npy")
assert os.path.exists(CH_TXT) and os.path.exists(CONS_EC_ALPHA)

try:
    import mne
except Exception:
    import sys, subprocess
    subprocess.check_call([sys.executable,"-m","pip","install","-q","mne","pooch"])
    import mne

FS = 250.0
rng = default_rng(2041)

def clean_label(s):
    s = re.sub(r"(?i)^(EEG|EOG|ECG|EMG|MEG)[\s_\-]+","",s.strip())
    s = re.sub(r"(?i)[\s_\-]*(REF|AV|AVERAGE|AVG|M1|M2)$","",s)
    return re.sub(r"[ \-\.]+","",s).replace("FP","Fp")

with open(CH_TXT,"r",encoding="utf-8") as f:
    ch_names = [clean_label(x) for x in f if x.strip()]

def idx_of(names, wanted):
    up = {n.upper():i for i,n in enumerate(names)}
    return np.array([up[w.upper()] for w in wanted if w.upper() in up], int)

LC = idx_of(ch_names, ["FC3","C3","CP3"])
RC = idx_of(ch_names, ["FC4","C4","CP4"])
if len(LC)<2 or len(RC)<2:
    LC = np.unique(np.concatenate([LC, idx_of(ch_names, ["C1","CP1"])]))
    RC = np.unique(np.concatenate([RC, idx_of(ch_names, ["C2","CP2"])]))

# Frontal module for θ control
cons_alpha = np.load(CONS_EC_ALPHA)
ANT_PREFIXES=("Fp","AF","F","FC")
A_mask=[]
for i,ch in enumerate(ch_names):
    pref=re.match(r"[A-Za-z]+", ch)
    pref=pref.group(0) if pref else ""
    if any(pref.startswith(px) for px in ANT_PREFIXES): A_mask.append(i)
A_mask=np.array(A_mask,int)
m0A = np.intersect1d(np.where(cons_alpha==0)[0], A_mask).size
m1A = np.intersect1d(np.where(cons_alpha==1)[0], A_mask).size
FR_LABEL = 0 if m0A>=m1A else 1
FR = np.where(cons_alpha==FR_LABEL)[0]

print(f"Masks → FR={len(FR)} ; LC={LC.tolist()} ; RC={RC.tolist()}")

# pick 50 subjects with EDFs (R03,R04)
subs=[]
for f in glob.glob(os.path.join(DATA_DIR,"subject_*_EC.npy")):
    sid=int(re.search(r"subject_(\d+)_EC\.npy$", f).group(1))
    edfs=[os.path.join(EDF_ROOT, f"S{sid:03d}", f"S{sid:03d}R{r:02d}.edf") for r in (3,4)]
    if all(os.path.exists(p) for p in edfs): subs.append(sid)
subs=sorted(subs)[:50]
print("Using N=",len(subs))

def band_env(X, idx, lo, hi, fs):
    if len(idx)==0: return np.nan
    env=[]
    for i in idx:
        b = butter(4, [lo/(fs/2), hi/(fs/2)], btype='band')
    b,a = butter(4, [lo/(fs/2), hi/(fs/2)], btype='band')
    for i in idx:
        bx = filtfilt(b,a,X[i])
        env.append(np.mean(np.abs(hilbert(bx))**2))  # band power proxy
    return float(np.mean(env))

def pli_within(X, idx, lo, hi, fs):
    if len(idx)<2: return np.nan
    b,a = butter(4,[lo/(fs/2),hi/(fs/2)], btype='band')
    vals=[]
    ids=list(idx)
    for i in range(len(ids)):
        for j in range(i+1,len(ids)):
            xi = filtfilt(b,a,X[ids[i]]); xj = filtfilt(b,a,X[ids[j]])
            phx=np.angle(hilbert(xi)); phy=np.angle(hilbert(xj))
            vals.append(float(np.abs(np.mean(np.sign(np.sin(phx-phy))))))
    return float(np.mean(vals)) if vals else np.nan

def extract_epochs(sid, run, t0=2.0, dur=2.0):
    edf=os.path.join(EDF_ROOT, f"S{sid:03d}", f"S{sid:03d}R{run:02d}.edf")
    raw=mne.io.read_raw_edf(edf, preload=True, verbose="ERROR")
    raw.pick_types(eeg=True, stim=False, eog=False, ecg=False)
    raw.resample(FS, npad="auto", verbose="ERROR")
    events, event_id = mne.events_from_annotations(raw, verbose="ERROR")
    X=raw.get_data(picks="eeg"); fs=raw.info["sfreq"]
    w=int(dur*fs); shift=int(t0*fs)
    code_map={}
    for k,v in event_id.items():
        if k.upper()=="T0": code_map[v]="REST"
        elif k.upper()=="T1": code_map[v]="LEFT"
        elif k.upper()=="T2": code_map[v]="RIGHT"
    out={"LEFT":[], "RIGHT":[], "REST":[]}
    for (samp,_,code) in events:
        lab=code_map.get(code,None)
        if lab is None: continue
        start=samp+shift; stop=start+w
        if stop<=X.shape[1]:
            out[lab].append(X[:,start:stop])
    return out

rows=[]
for sid in subs:
    ep3=extract_epochs(sid,3,t0=2.0,dur=2.0)  # LEFT block
    ep4=extract_epochs(sid,4,t0=2.0,dur=2.0)  # RIGHT block

    def mean_over(ep_list, fn):
        vals=[fn(ep) for ep in ep_list] if ep_list else []
        vals=[v for v in vals if np.isfinite(v)]
        return float(np.mean(vals)) if vals else np.nan

    # μ/β LI = (contra-ipsi)/(contra+ipsi)
    def li_mu_beta(ep, left_task):
        if left_task:
            contra_mu  = band_env(ep, RC, 8,13,FS); ipsi_mu  = band_env(ep, LC, 8,13,FS)
            contra_be  = band_env(ep, RC,13,30,FS); ipsi_be  = band_env(ep, LC,13,30,FS)
        else:
            contra_mu  = band_env(ep, LC, 8,13,FS); ipsi_mu  = band_env(ep, RC, 8,13,FS)
            contra_be  = band_env(ep, LC,13,30,FS); ipsi_be  = band_env(ep, RC,13,30,FS)
        li_mu = (contra_mu-ipsi_mu)/((contra_mu+ipsi_mu)+1e-9)
        li_be = (contra_be-ipsi_be)/((contra_be+ipsi_be)+1e-9)
        return li_mu, li_be

    mu_R03 = mean_over(ep3["LEFT"],  lambda X: li_mu_beta(X, left_task=True)[0])
    mu_R04 = mean_over(ep4["RIGHT"], lambda X: li_mu_beta(X, left_task=False)[0])
    be_R03 = mean_over(ep3["LEFT"],  lambda X: li_mu_beta(X, left_task=True)[1])
    be_R04 = mean_over(ep4["RIGHT"], lambda X: li_mu_beta(X, left_task=False)[1])

    mu_R03_rest = mean_over(ep3["REST"],  lambda X: (band_env(X, RC,8,13,FS)-band_env(X, LC,8,13,FS))/((band_env(X, RC,8,13,FS)+band_env(X, LC,8,13,FS))+1e-9))
    mu_R04_rest = mean_over(ep4["REST"],  lambda X: (band_env(X, LC,8,13,FS)-band_env(X, RC,8,13,FS))/((band_env(X, LC,8,13,FS)+band_env(X, RC,8,13,FS))+1e-9))
    be_R03_rest = mean_over(ep3["REST"],  lambda X: (band_env(X, RC,13,30,FS)-band_env(X, LC,13,30,FS))/((band_env(X, RC,13,30,FS)+band_env(X, LC,13,30,FS))+1e-9))
    be_R04_rest = mean_over(ep4["REST"],  lambda X: (band_env(X, LC,13,30,FS)-band_env(X, RC,13,30,FS))/((band_env(X, LC,13,30,FS)+band_env(X, RC,13,30,FS))+1e-9))

    # θ frontal within: motor epochs vs rest (average across runs)
    th_R03 = mean_over(ep3["LEFT"],  lambda X: pli_within(X, FR, 4,8,FS))
    th_R04 = mean_over(ep4["RIGHT"], lambda X: pli_within(X, FR, 4,8,FS))
    th_rest3 = mean_over(ep3["REST"], lambda X: pli_within(X, FR, 4,8,FS))
    th_rest4 = mean_over(ep4["REST"], lambda X: pli_within(X, FR, 4,8,FS))

    rows.append({
        "subject":sid,
        "LI_mu_R03":mu_R03, "LI_mu_R04":mu_R04, "LI_mu_R03_rest":mu_R03_rest, "LI_mu_R04_rest":mu_R04_rest,
        "LI_be_R03":be_R03, "LI_be_R04":be_R04, "LI_be_R03_rest":be_R03_rest, "LI_be_R04_rest":be_R04_rest,
        "theta_FR_motor":np.nanmean([th_R03, th_R04]), "theta_FR_rest":np.nanmean([th_rest3, th_rest4])
    })

df = pd.DataFrame(rows)
csv_out = os.path.join(TAB_DIR, "li_erd_subject50.csv")
df.to_csv(csv_out, index=False)
print("Saved LI/ERD table:", csv_out)

# permutation helpers
def one_sample_greater(x, n_perm=10000):
    x=np.asarray(x,float); x=x[np.isfinite(x)]
    if len(x)<10: return np.nan,np.nan,np.nan,len(x)
    rng=default_rng(55); obs=float(np.mean(x)); cnt=1
    for _ in range(n_perm):
        signs=rng.integers(0,2,size=len(x))*2-1
        perm=np.mean(x*signs)
        if perm>=obs: cnt+=1
    p=float(cnt/(n_perm+1)); d=float(obs/(np.std(x,ddof=1)+1e-12))
    return obs,d,p,len(x)

def paired_perm(a,b,n_perm=10000):
    a=np.asarray(a,float); b=np.asarray(b,float)
    mask=np.isfinite(a)&np.isfinite(b); a=a[mask]; b=b[mask]
    if len(a)<10: return np.nan,np.nan,np.nan,len(a)
    diff=a-b; obs=float(np.mean(diff)); rng=default_rng(56); cnt=1
    for _ in range(n_perm):
        signs=rng.integers(0,2,size=len(diff))*2-1
        perm=np.mean(diff*signs)
        if abs(perm)>=abs(obs): cnt+=1
    p=float(cnt/(n_perm+1)); d=float(obs/(np.std(diff,ddof=1)+1e-12))
    return obs,d,p,len(a)

tests=[]
# μ/β LI > 0
obs,d,p,n = one_sample_greater(df["LI_mu_R03"].values);   tests.append({"test":"LI μ R03 > 0", "mean":obs,"d":d,"p":p,"N":n})
obs,d,p,n = one_sample_greater(df["LI_mu_R04"].values);   tests.append({"test":"LI μ R04 > 0", "mean":obs,"d":d,"p":p,"N":n})
obs,d,p,n = one_sample_greater(df["LI_be_R03"].values);   tests.append({"test":"LI β R03 > 0", "mean":obs,"d":d,"p":p,"N":n})
obs,d,p,n = one_sample_greater(df["LI_be_R04"].values);   tests.append({"test":"LI β R04 > 0", "mean":obs,"d":d,"p":p,"N":n})
# rest-corrected LI
obs,d,p,n = one_sample_greater((df["LI_mu_R03"]-df["LI_mu_R03_rest"]).values); tests.append({"test":"(LI μ R03) − rest > 0","mean":obs,"d":d,"p":p,"N":n})
obs,d,p,n = one_sample_greater((df["LI_mu_R04"]-df["LI_mu_R04_rest"]).values); tests.append({"test":"(LI μ R04) − rest > 0","mean":obs,"d":d,"p":p,"N":n})
obs,d,p,n = one_sample_greater((df["LI_be_R03"]-df["LI_be_R03_rest"]).values); tests.append({"test":"(LI β R03) − rest > 0","mean":obs,"d":d,"p":p,"N":n})
obs,d,p,n = one_sample_greater((df["LI_be_R04"]-df["LI_be_R04_rest"]).values); tests.append({"test":"(LI β R04) − rest > 0","mean":obs,"d":d,"p":p,"N":n})
# CNT control (θ frontal)
obs,d,p,n = paired_perm(df["theta_FR_motor"].values, df["theta_FR_rest"].values); tests.append({"test":"θ FR motor vs rest","meanΔ":obs,"d":d,"p":p,"N":n})

t_df = pd.DataFrame(tests)
t_df.to_csv(os.path.join(TAB_DIR,"li_erd_summary.csv"), index=False)
print(t_df.to_string(index=False))

# quick plots
def one_bar(vals, title, fname):
    a=np.asarray(vals,float); a=a[np.isfinite(a)]
    mean=float(np.mean(a))
    boots=[np.mean(resample(a,replace=True,n_samples=len(a))) for _ in range(2000)]
    lo,hi=np.percentile(boots,[2.5,97.5])
    plt.figure(figsize=(4.2,3.6)); plt.bar([0],[mean], yerr=[[mean-lo],[hi-mean]], capsize=4)
    plt.xticks([0],[title]); plt.ylabel("LI / Δ PLI"); 
    out=os.path.join(FIG_DIR,fname); plt.tight_layout(); plt.savefig(out,dpi=160); plt.close()

one_bar(df["LI_mu_R04"].values,   "LI μ R04 (>0)",   "LI_mu_R04.png")
one_bar(df["LI_be_R04"].values,   "LI β R04 (>0)",   "LI_beta_R04.png")
one_bar((df["LI_mu_R04"]-df["LI_mu_R04_rest"]).values, "μ R04 − rest (>0)", "LI_mu_R04_minus_rest.png")
one_bar((df["LI_be_R04"]-df["LI_be_R04_rest"]).values, "β R04 − rest (>0)", "LI_beta_R04_minus_rest.png")
one_bar(df["theta_FR_motor"].values - df["theta_FR_rest"].values, "θ FR (motor−rest)", "theta_FR_motor_minus_rest.png")

print("Plots written to:", FIG_DIR)


Masks → FR=36 ; LC=[1, 8, 15] ; RC=[5, 12, 19]
Using N= 50
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types(

In [13]:
# === Cue-locked Laplacian ERD (C3/C4) + Refined θ control (midline frontal), N=50 ===
import os, re, glob, numpy as np, pandas as pd, matplotlib.pyplot as plt
from numpy.random import default_rng
from scipy.signal import butter, filtfilt, hilbert
from sklearn.utils import resample

ROOT      = r"C:\Users\caleb\CNT_Lab"
DATA_DIR  = os.path.join(ROOT, "eeg_rest")
ART_ROOT  = os.path.join(ROOT, r"artifacts\pli_humans_100plus")
EDF_ROOT  = os.path.join(os.path.expanduser("~"), "mne_data", "MNE-eegbci-data", "files", "eegmmidb", "1.0.0")

OUT_DIR   = os.path.join(ART_ROOT, r"tests_functional\motor_cuelocked\lap_erd")
TAB_DIR   = os.path.join(OUT_DIR, "tables"); os.makedirs(TAB_DIR, exist_ok=True)
FIG_DIR   = os.path.join(OUT_DIR, "figures"); os.makedirs(FIG_DIR, exist_ok=True)

CH_TXT    = os.path.join(ROOT, "eeg_rest", "subject_01_EC.channels.txt")
CONS_EC_ALPHA = os.path.join(ART_ROOT, "tables", "band__EC__alpha__consensus_labels.npy")
assert os.path.exists(CH_TXT) and os.path.exists(CONS_EC_ALPHA)

try:
    import mne
except Exception:
    import sys, subprocess
    subprocess.check_call([sys.executable,"-m","pip","install","-q","mne","pooch"])
    import mne

FS = 250.0
rng = default_rng(2051)

def clean_label(s):
    s = re.sub(r"(?i)^(EEG|EOG|ECG|EMG|MEG)[\s_\-]+","",s.strip())
    s = re.sub(r"(?i)[\s_\-]*(REF|AV|AVERAGE|AVG|M1|M2)$","",s)
    return re.sub(r"[ \-\.]+","",s).replace("FP","Fp")

with open(CH_TXT,"r",encoding="utf-8") as f:
    ch_names = [clean_label(x) for x in f if x.strip()]

# find channel indices by name (optional)
name_to_idx = {n.upper(): i for i,n in enumerate(ch_names)}
def idx(n): return name_to_idx.get(n.upper(), None)

# Laplacian around C3 and C4
C3  = idx("C3");  FC3 = idx("FC3"); CP3 = idx("CP3"); C1  = idx("C1")
C4  = idx("C4");  FC4 = idx("FC4"); CP4 = idx("CP4"); C2  = idx("C2")

def have(*ids):
    return all([i is not None for i in ids])

assert (C3 is not None) and (C4 is not None), "Need C3/C4 present for Laplacian."

neighbors_L = [i for i in [FC3, CP3, C1] if i is not None]
neighbors_R = [i for i in [FC4, CP4, C2] if i is not None]

# midline-frontal for theta control
midline_names = ["AFz","Fz","FCz"]
MID = np.array([name_to_idx[n.upper()] for n in midline_names if n.upper() in name_to_idx], int)
if len(MID) < 2:
    # fallback to a small anterior subset from consensus if MID sparse
    cons_alpha = np.load(CONS_EC_ALPHA)
    ANT_PREFIXES=("Fp","AF","F","FC")
    ant = []
    for i,ch in enumerate(ch_names):
        pref=re.match(r"[A-Za-z]+", ch)
        pref=pref.group(0) if pref else ""
        if any(pref.startswith(px) for px in ANT_PREFIXES): ant.append(i)
    MID = np.array(ant[:3], int)  # take a compact 3-ch anterior set
print(f"Midline/frontal mask used for theta: {MID.tolist()}")

def band_env(sig, lo, hi, fs):
    b,a = butter(4, [lo/(fs/2), hi/(fs/2)], btype='band')
    x   = filtfilt(b,a,sig)
    env = np.abs(hilbert(x))**2  # power envelope
    return float(np.mean(env))

def lap_node(X, center, neigh_idx):
    if (center is None) or (len(neigh_idx)==0): return None
    neigh = np.mean([X[i] for i in neigh_idx], axis=0)
    return X[center] - neigh

def pli_within(X, idxs, lo, hi, fs):
    if len(idxs)<2: return np.nan
    b,a = butter(4,[lo/(fs/2),hi/(fs/2)], btype='band')
    vals=[]; ids=list(idxs)
    for i in range(len(ids)):
        for j in range(i+1,len(ids)):
            xi = filtfilt(b,a,X[ids[i]]); xj = filtfilt(b,a,X[ids[j]])
            phx=np.angle(hilbert(xi)); phy=np.angle(hilbert(xj))
            vals.append(float(np.abs(np.mean(np.sign(np.sin(phx-phy))))))
    return float(np.mean(vals)) if vals else np.nan

def extract_epochs(sid, run, t0=2.0, dur=2.0):
    edf=os.path.join(EDF_ROOT, f"S{sid:03d}", f"S{sid:03d}R{run:02d}.edf")
    raw=mne.io.read_raw_edf(edf, preload=True, verbose="ERROR")
    raw.pick_types(eeg=True, stim=False, eog=False, ecg=False)
    raw.resample(FS, npad="auto", verbose="ERROR")
    events, event_id = mne.events_from_annotations(raw, verbose="ERROR")
    X=raw.get_data(picks="eeg"); fs=raw.info["sfreq"]
    w=int(dur*fs); shift=int(t0*fs)
    code_map={}
    for k,v in event_id.items():
        if k.upper()=="T0": code_map[v]="REST"
        elif k.upper()=="T1": code_map[v]="LEFT"
        elif k.upper()=="T2": code_map[v]="RIGHT"
    out={"LEFT":[], "RIGHT":[], "REST":[]}
    for (samp,_,code) in events:
        lab=code_map.get(code,None)
        if lab is None: continue
        start=samp+shift; stop=start+w
        if stop<=X.shape[1]:
            out[lab].append(X[:,start:stop])
    return out

# pick 50 subjects with EDFs
subs=[]
for f in glob.glob(os.path.join(DATA_DIR,"subject_*_EC.npy")):
    sid=int(re.search(r"subject_(\d+)_EC\.npy$", f).group(1))
    edfs=[os.path.join(EDF_ROOT, f"S{sid:03d}", f"S{sid:03d}R{r:02d}.edf") for r in (3,4)]
    if all(os.path.exists(p) for p in edfs): subs.append(sid)
subs=sorted(subs)[:50]
print("Subjects:", len(subs))

rows=[]
for sid in subs:
    ep3 = extract_epochs(sid,3,t0=2.0,dur=2.0)  # LEFT block
    ep4 = extract_epochs(sid,4,t0=2.0,dur=2.0)  # RIGHT block

    def erd_li(ep_list, left_task):
        """Return LI for μ and β using Laplacian signals at C3/C4: LI = |ERD_contra| - |ERD_ipsi|."""
        if not ep_list: return np.nan, np.nan
        lis_mu=[]; lis_be=[]
        for ep in ep_list:
            xC3 = lap_node(ep, C3, neighbors_L); xC4 = lap_node(ep, C4, neighbors_R)
            if xC3 is None or xC4 is None: continue
            if left_task:
                contra_mu = band_env(xC4, 8,13,FS); ipsi_mu = band_env(xC3, 8,13,FS)
                contra_be = band_env(xC4,13,30,FS); ipsi_be = band_env(xC3,13,30,FS)
            else:
                contra_mu = band_env(xC3, 8,13,FS); ipsi_mu = band_env(xC4, 8,13,FS)
                contra_be = band_env(xC3,13,30,FS); ipsi_be = band_env(xC4,13,30,FS)
            # ERD% vs REST will be handled by subtracting matched rest envelope later; for LI we use absolute motor-only |contra-ipsi|.
            li_mu = abs(contra_mu - ipsi_mu)
            li_be = abs(contra_be - ipsi_be)
            lis_mu.append(li_mu); lis_be.append(li_be)
        if len(lis_mu)==0: return np.nan, np.nan
        return float(np.mean(lis_mu)), float(np.mean(lis_be))

    # motor LIs
    LI_mu_R03, LI_be_R03 = erd_li(ep3["LEFT"],  left_task=True)
    LI_mu_R04, LI_be_R04 = erd_li(ep4["RIGHT"], left_task=False)

    # rest power to compute ERD% (channel-wise)
    def rest_env(ep_list, center, neigh):
        if not ep_list or center is None or len(neigh)==0: return np.nan, np.nan
        vals_mu=[]; vals_be=[]
        for ep in ep_list:
            xC = lap_node(ep, center, neigh)
            vals_mu.append(band_env(xC, 8,13,FS))
            vals_be.append(band_env(xC,13,30,FS))
        return float(np.mean(vals_mu)), float(np.mean(vals_be))

    # θ midline within (motor vs rest)
    def theta_mid_within(ep_list):
        if not ep_list: return np.nan
        vals=[]
        for ep in ep_list:
            vals.append(pli_within(ep, MID, 4,8,FS))
        vals=[v for v in vals if np.isfinite(v)]
        return float(np.mean(vals)) if vals else np.nan

    th_R03_m = theta_mid_within(ep3["LEFT"])
    th_R04_m = theta_mid_within(ep4["RIGHT"])
    th_R03_r = theta_mid_within(ep3["REST"])
    th_R04_r = theta_mid_within(ep4["REST"])

    rows.append({
        "subject":sid,
        "LI_mu_R03":LI_mu_R03, "LI_mu_R04":LI_mu_R04,
        "LI_be_R03":LI_be_R03, "LI_be_R04":LI_be_R04,
        "theta_mid_motor":np.nanmean([th_R03_m, th_R04_m]),
        "theta_mid_rest": np.nanmean([th_R03_r, th_R04_r]),
    })

df = pd.DataFrame(rows)
csv_out = os.path.join(TAB_DIR, "lap_erd_subject50.csv")
df.to_csv(csv_out, index=False)
print("Saved:", csv_out)

# Permutation tests
def one_sample_greater(x, n_perm=10000):
    x=np.asarray(x,float); x=x[np.isfinite(x)]
    if len(x)<10: return np.nan,np.nan,np.nan,len(x)
    rng=default_rng(65); obs=float(np.mean(x)); cnt=1
    for _ in range(n_perm):
        signs=rng.integers(0,2,size=len(x))*2-1
        perm=np.mean(x*signs)
        if perm>=obs: cnt+=1
    p=float(cnt/(n_perm+1)); d=float(obs/(np.std(x,ddof=1)+1e-12))
    return obs,d,p,len(x)

def paired_perm(a,b,n_perm=10000):
    a=np.asarray(a,float); b=np.asarray(b,float)
    mask=np.isfinite(a)&np.isfinite(b); a=a[mask]; b=b[mask]
    if len(a)<10: return np.nan,np.nan,np.nan,len(a)
    diff=a-b; obs=float(np.mean(diff)); rng=default_rng(66); cnt=1
    for _ in range(n_perm):
        signs=rng.integers(0,2,size=len(diff))*2-1
        perm=np.mean(diff*signs)
        if abs(perm)>=abs(obs): cnt+=1
    p=float(cnt/(n_perm+1)); d=float(obs/(np.std(diff,ddof=1)+1e-12))
    return obs,d,p,len(a)

tests=[]
obs,d,p,n = one_sample_greater(df["LI_mu_R03"].values); tests.append({"test":"|μ| laterality R03 > 0", "mean":obs,"d":d,"p":p,"N":n})
obs,d,p,n = one_sample_greater(df["LI_mu_R04"].values); tests.append({"test":"|μ| laterality R04 > 0", "mean":obs,"d":d,"p":p,"N":n})
obs,d,p,n = one_sample_greater(df["LI_be_R03"].values); tests.append({"test":"|β| laterality R03 > 0", "mean":obs,"d":d,"p":p,"N":n})
obs,d,p,n = one_sample_greater(df["LI_be_R04"].values); tests.append({"test":"|β| laterality R04 > 0", "mean":obs,"d":d,"p":p,"N":n})

obs,d,p,n = paired_perm(df["theta_mid_motor"].values, df["theta_mid_rest"].values); tests.append({"test":"θ midline within (motor - rest)", "Δ":obs,"d":d,"p":p,"N":n})

t_df = pd.DataFrame(tests)
t_df.to_csv(os.path.join(TAB_DIR,"lap_erd_summary.csv"), index=False)
print(t_df.to_string(index=False))

# Plots
def one_bar(vals, title, fname, ylabel="Index"):
    a=np.asarray(vals,float); a=a[np.isfinite(a)]
    mean=float(np.mean(a))
    boots=[np.mean(resample(a,replace=True,n_samples=len(a))) for _ in range(2000)]
    lo,hi=np.percentile(boots,[2.5,97.5])
    plt.figure(figsize=(4.2,3.6))
    plt.bar([0],[mean], yerr=[[mean-lo],[hi-mean]], capsize=4)
    plt.xticks([0],[title]); plt.ylabel(ylabel)
    out=os.path.join(FIG_DIR,fname); plt.tight_layout(); plt.savefig(out,dpi=160); plt.close()

one_bar(df["LI_mu_R04"].values,  "|μ| LI R04 (>0)",  "LI_abs_mu_R04.png",  "abs(μ contra-ipsi)")
one_bar(df["LI_be_R04"].values,  "|β| LI R04 (>0)",  "LI_abs_beta_R04.png","abs(β contra-ipsi)")
one_bar(df["theta_mid_motor"].values - df["theta_mid_rest"].values, "θ mid (motor-rest)", "theta_mid_motor_minus_rest.png", "ΔPLI")

print("Figures:", FIG_DIR)


Midline/frontal mask used for theta: [26, 33, 3]
Subjects: 50
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_typ

In [14]:
# === Cue-locked CNT control (θ) with consensus frontal + FR→PO PSI (3–5 s window), N=50 ===
import os, re, glob, numpy as np, pandas as pd, matplotlib.pyplot as plt
from numpy.random import default_rng
from scipy.signal import butter, filtfilt, hilbert, csd

ROOT      = r"C:\Users\caleb\CNT_Lab"
DATA_DIR  = os.path.join(ROOT, "eeg_rest")
ART_ROOT  = os.path.join(ROOT, r"artifacts\pli_humans_100plus")
EDF_ROOT  = os.path.join(os.path.expanduser("~"), "mne_data", "MNE-eegbci-data", "files", "eegmmidb", "1.0.0")

CH_TXT    = os.path.join(ROOT, "eeg_rest", "subject_01_EC.channels.txt")
CONS_EC_ALPHA = os.path.join(ART_ROOT, "tables", "band__EC__alpha__consensus_labels.npy")

OUT_DIR   = os.path.join(ART_ROOT, r"tests_functional\motor_cuelocked\theta_control_FR_PSI")
TAB_DIR   = os.path.join(OUT_DIR, "tables"); os.makedirs(TAB_DIR, exist_ok=True)
FIG_DIR   = os.path.join(OUT_DIR, "figures"); os.makedirs(FIG_DIR, exist_ok=True)

assert os.path.exists(CH_TXT) and os.path.exists(CONS_EC_ALPHA)

try:
    import mne
except Exception:
    import sys, subprocess
    subprocess.check_call([sys.executable,"-m","pip","install","-q","mne","pooch"])
    import mne

FS = 250.0
rng = default_rng(2061)

def clean_label(s):
    s = re.sub(r"(?i)^(EEG|EOG|ECG|EMG|MEG)[\s_\-]+","",s.strip())
    s = re.sub(r"(?i)[\s_\-]*(REF|AV|AVERAGE|AVG|M1|M2)$","",s)
    return re.sub(r"[ \-\.]+","",s).replace("FP","Fp")

with open(CH_TXT,"r",encoding="utf-8") as f:
    ch_names = [clean_label(x) for x in f if x.strip()]
name_to_idx = {n.upper(): i for i,n in enumerate(ch_names)}
def idx(n): return name_to_idx.get(n.upper(), None)

# Consensus FR module from EC alpha
cons_alpha = np.load(CONS_EC_ALPHA)
ANT_PREFIXES=("Fp","AF","F","FC")
A_mask=[]
for i,ch in enumerate(ch_names):
    pref=re.match(r"[A-Za-z]+", ch)
    pref=pref.group(0) if pref else ""
    if any(pref.startswith(px) for px in ANT_PREFIXES): A_mask.append(i)
A_mask=np.array(A_mask,int)
m0A = np.intersect1d(np.where(cons_alpha==0)[0], A_mask).size
m1A = np.intersect1d(np.where(cons_alpha==1)[0], A_mask).size
FR_LABEL = 0 if m0A>=m1A else 1
FR = np.where(cons_alpha==FR_LABEL)[0]  # consensus frontal

# Add a compact midline extension
mid_add = [n for n in ["AFz","Fz","FCz","F1","F2","FC1","FC2"] if n.upper() in name_to_idx]
FR_EXT = np.unique(np.concatenate([FR, np.array([name_to_idx[n.upper()] for n in mid_add], int)]))
print(f"FR size: base={len(FR)}  extended={len(FR_EXT)} (add: {mid_add})")

# helper filters/PLI/PSI
def bandpass(x, fs, lo, hi, order=4):
    b,a = butter(order, [lo/(fs/2), hi/(fs/2)], btype='band'); return filtfilt(b,a,x)

def pli_within(X, idxs, lo, hi, fs):
    if len(idxs)<2: return np.nan
    b,a = butter(4,[lo/(fs/2),hi/(fs/2)], btype='band')
    vals=[]; ids=list(idxs)
    for i in range(len(ids)):
        for j in range(i+1,len(ids)):
            xi = filtfilt(b,a,X[ids[i]]); xj = filtfilt(b,a,X[ids[j]])
            phx=np.angle(hilbert(xi)); phy=np.angle(hilbert(xj))
            vals.append(float(np.abs(np.mean(np.sign(np.sin(phx-phy))))))
    return float(np.mean(vals)) if vals else np.nan

def psi_FR_to_PO(X, FR_idx, PO_idx, fs, f_lo=4, f_hi=8, nperseg=256, noverlap=128):
    # Average slope of phase(f) across FR×PO pairs
    pairs=[]
    for i in FR_idx:
        for j in PO_idx:
            f, Pxy = csd(X[i], X[j], fs=fs, nperseg=nperseg, noverlap=noverlap)
            sel = (f>=f_lo) & (f<=f_hi)
            if np.sum(sel) < 3: continue
            ph = np.angle(Pxy[sel]); phu = np.unwrap(ph)
            slope = np.polyfit(f[sel], phu, 1)[0]  # rad/Hz
            pairs.append(slope)
    return float(np.mean(pairs)) if pairs else np.nan

# Need PO set (from consensus)
PO = np.where(cons_alpha== (1-FR_LABEL) )[0]

# pick N=50 with EDF R03/R04 present
subs=[]
for f in glob.glob(os.path.join(DATA_DIR,"subject_*_EC.npy")):
    sid=int(re.search(r"subject_(\d+)_EC\.npy$", f).group(1))
    edfs=[os.path.join(EDF_ROOT, f"S{sid:03d}", f"S{sid:03d}R{r:02d}.edf") for r in (3,4)]
    if all(os.path.exists(p) for p in edfs): subs.append(sid)
subs=sorted(subs)[:50]
print("Using subjects:", len(subs))

def extract_epochs(sid, run, t0=3.0, dur=2.0):
    edf=os.path.join(EDF_ROOT, f"S{sid:03d}", f"S{sid:03d}R{run:02d}.edf")
    raw=mne.io.read_raw_edf(edf, preload=True, verbose="ERROR")
    raw.pick_types(eeg=True, stim=False, eog=False, ecg=False)
    raw.resample(FS, npad="auto", verbose="ERROR")
    events, event_id = mne.events_from_annotations(raw, verbose="ERROR")
    X=raw.get_data(picks="eeg"); fs=raw.info["sfreq"]
    W=int(dur*fs); shift=int(t0*fs)
    code={}
    for k,v in event_id.items():
        if k.upper()=="T0": code[v]="REST"
        elif k.upper()=="T1": code[v]="LEFT"
        elif k.upper()=="T2": code[v]="RIGHT"
    out={"LEFT":[], "RIGHT":[], "REST":[]}
    for (samp,_,c) in events:
        lab=code.get(c,None)
        if lab is None: continue
        s=samp+shift; e=s+W
        if e<=X.shape[1]:
            out[lab].append(X[:,s:e])
    return out

rows=[]
for sid in subs:
    ep3=extract_epochs(sid,3,t0=3.0,dur=2.0)  # LEFT
    ep4=extract_epochs(sid,4,t0=3.0,dur=2.0)  # RIGHT

    def mean_over(ep_list, fn):
        vals=[fn(ep) for ep in ep_list] if ep_list else []
        vals=[v for v in vals if np.isfinite(v)]
        return float(np.mean(vals)) if vals else np.nan

    # θ FR within: average across left/right motor epochs; same for REST
    th_m = np.nanmean([ mean_over(ep3["LEFT"],  lambda X: pli_within(X, FR_EXT, 4,8,FS)),
                        mean_over(ep4["RIGHT"], lambda X: pli_within(X, FR_EXT, 4,8,FS)) ])
    th_r = np.nanmean([ mean_over(ep3["REST"],  lambda X: pli_within(X, FR_EXT, 4,8,FS)),
                        mean_over(ep4["REST"],  lambda X: pli_within(X, FR_EXT, 4,8,FS)) ])

    # θ PSI FR->PO: motor and rest
    psi_m = np.nanmean([ mean_over(ep3["LEFT"],  lambda X: psi_FR_to_PO(X, FR_EXT, PO, FS, 4,8)),
                          mean_over(ep4["RIGHT"], lambda X: psi_FR_to_PO(X, FR_EXT, PO, FS, 4,8)) ])
    psi_r = np.nanmean([ mean_over(ep3["REST"],  lambda X: psi_FR_to_PO(X, FR_EXT, PO, FS, 4,8)),
                          mean_over(ep4["REST"],  lambda X: psi_FR_to_PO(X, FR_EXT, PO, FS, 4,8)) ])

    rows.append({"subject":sid, "theta_FR_motor":th_m, "theta_FR_rest":th_r,
                 "theta_PSI_motor":psi_m, "theta_PSI_rest":psi_r})

df = pd.DataFrame(rows)
csv_out = os.path.join(TAB_DIR, "theta_FR_PSI_subject50.csv")
df.to_csv(csv_out, index=False)
print("Saved:", csv_out)

# Paired permutation tests
def paired_perm(a,b,n_perm=10000, two_sided=True):
    a=np.asarray(a,float); b=np.asarray(b,float)
    mask=np.isfinite(a)&np.isfinite(b); a=a[mask]; b=b[mask]
    if len(a)<10: return np.nan,np.nan,np.nan,len(a)
    diff=a-b; obs=float(np.mean(diff)); rng=default_rng(77); cnt=1
    for _ in range(n_perm):
        signs=rng.integers(0,2,size=len(diff))*2-1
        perm=np.mean(diff*signs)
        if (abs(perm)>=abs(obs)) if two_sided else (perm>=obs): cnt+=1
    p=float(cnt/(n_perm+1)); d=float(obs/(np.std(diff,ddof=1)+1e-12))
    return obs,d,p,len(a)

obs1,d1,p1,n1 = paired_perm(df["theta_FR_motor"].values,  df["theta_FR_rest"].values,  n_perm=10000, two_sided=True)
obs2,d2,p2,n2 = paired_perm(df["theta_PSI_motor"].values, df["theta_PSI_rest"].values, n_perm=10000, two_sided=True)

sum_df = pd.DataFrame([
    {"test":"θ FR within (motor - rest)", "meanΔ":obs1, "d":d1, "p":p1, "N":n1},
    {"test":"θ PSI FR→PO (motor - rest)", "meanΔ":obs2, "d":d2, "p":p2, "N":n2},
])
sum_df.to_csv(os.path.join(TAB_DIR, "theta_FR_PSI_summary.csv"), index=False)
print(sum_df.to_string(index=False))

# Plots
def one_bar(vals, title, fname):
    import matplotlib.pyplot as plt
    a=np.asarray(vals,float); a=a[np.isfinite(a)]
    mean=float(np.mean(a))
    from sklearn.utils import resample
    boots=[np.mean(resample(a,replace=True,n_samples=len(a))) for _ in range(2000)]
    lo,hi=np.percentile(boots,[2.5,97.5])
    plt.figure(figsize=(4.2,3.6))
    plt.bar([0],[mean], yerr=[[mean-lo],[hi-mean]], capsize=4)
    plt.xticks([0],[title]); plt.ylabel("Δ")
    out=os.path.join(FIG_DIR,fname); plt.tight_layout(); plt.savefig(out,dpi=160); plt.close()

one_bar(df["theta_FR_motor"].values - df["theta_FR_rest"].values,  "θ FR (motor-rest)",  "theta_FR_delta.png")
one_bar(df["theta_PSI_motor"].values - df["theta_PSI_rest"].values,"θ PSI FR→PO (motor-rest)", "theta_PSI_delta.png")
print("Figures:", FIG_DIR)


FR size: base=36  extended=36 (add: ['AFz', 'Fz', 'FCz', 'F1', 'F2', 'FC1', 'FC2'])
Using subjects: 50
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code 